In [1]:
import requests
import pandas as pd
import os
import datetime
import numpy as np
import time
#import requests_cache


In [2]:
# Retirar algumas críticas e definir alguns paramentros
pd.set_option('float_format', '{:,.2f}'.format)
pd.set_option('mode.chained_assignment', None)
#pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_colwidth', None) # Definir a largura máxima das colunas para None (automático o ajuste)
# Configurar pandas para alinhar à esquerda
pd.set_option('display.colheader_justify', 'left')

In [3]:
# Função para colorir as células com "ERRO" de vermelho
def colorir_celulas(val):
    color = 'red' if val == 'ERRO' else 'white'
    return f'color: {color}'

In [4]:
ano = '2024'
ente = '33'
cod = 'RJ'
# 5 primeiros em 2023 >> GO 52 / RO 11 / TO 17 / ES 32 / MS 50  
# Norte >> AC 12 / AM 13 / RR 14 / PA 15 / AP 16 / MA 21 / PI 22 
# Nordeste >> CE 23 / RN 24 / PB 25 / PE 26 / AL 27 / SE 28 / BA 29 
# Sudeste >> MG 31 / SP 35 
# Sul >> PR 41 / SC 42 / RS 43 
# Centro-Oeste >> MS 50 / MT 51 / DF 53


In [5]:
# Criação do diretório
pasta_destino = os.path.join(ano, f"{ente}_{cod}")
os.makedirs(pasta_destino, exist_ok=True)
print(f"Diretório criado ou já existente: {pasta_destino}")

Diretório criado ou já existente: 2024\33_RJ


### Buscar os Dados da API ??

### Defina "S" (de "sim" )para buscar os dados da API, ou "N" (de "não") se quiser usar pelos arquivos já baixados!!!

In [6]:
buscar_api = "S" 

# Código Assíncrono

In [7]:
# pip install -U httpx[http2]
import asyncio
import os
import pandas as pd
import httpx

API_ROOT = "https://apidatalake.tesouro.gov.br/ords/siconfi/tt"

async def _request_json(client, url, params, sem, retries=3, backoff=0.5, timeout=120):
    for attempt in range(retries):
        if sem: await sem.acquire()
        try:
            resp = await client.get(url, params=params, timeout=timeout)
            if resp.status_code in (429, 500, 502, 503, 504):
                # backoff e retry para sobrecarga/limite
                await asyncio.sleep(backoff * (2 ** attempt))
                continue
            resp.raise_for_status()
            data = resp.json()
            return data.get("items", [])
        finally:
            if sem: sem.release()
    # última tentativa
    resp.raise_for_status()

async def fetch_paginated(client, path, params, sem=None, page_size=5000, delay=0.0):
    frames, offset = [], 0
    while True:
        q = dict(params)
        q.update({"offset": offset, "limit": page_size})
        items = await _request_json(client, f"{API_ROOT}/{path}", q, sem)
        if not items:
            break
        frames.append(pd.DataFrame(items))
        offset += page_size
        if delay:  # para ser gentil com a API
            await asyncio.sleep(delay)
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

async def fetch_once(client, path, params, sem=None):
    items = await _request_json(client, f"{API_ROOT}/{path}", params, sem)
    return pd.DataFrame(items)

async def load_msc_group(client, path, classes, co_tipo_matriz, tipos_balanco, meses, ente, ano, sem, delay=0.0):
    tasks = []
    for classe in map(str, classes):
        for tipo in tipos_balanco:
            for mes in meses:
                params = {
                    "id_ente": ente,
                    "an_referencia": ano,
                    "me_referencia": mes,
                    "co_tipo_matriz": co_tipo_matriz,
                    "classe_conta": classe,
                    "id_tv": tipo,
                }
                tasks.append(fetch_paginated(client, path, params, sem=sem, delay=delay))
    dfs = await asyncio.gather(*tasks)
    return pd.concat([df for df in dfs if not df.empty], ignore_index=True) if dfs else pd.DataFrame()

async def load_msc_all(ente, ano, meses, tipos_balanco, co_tipo_matriz="MSCC", concurrency=8, delay=0.05):
    sem = asyncio.Semaphore(concurrency)
    async with httpx.AsyncClient(http2=True) as client:
        msc_patrimonial = await load_msc_group(client, "msc_patrimonial", [1,2,3,4], co_tipo_matriz, tipos_balanco, meses, ente, ano, sem, delay)
        msc_orcam       = await load_msc_group(client, "msc_orcamentaria", [5,6],   co_tipo_matriz, tipos_balanco, meses, ente, ano, sem, delay)
        msc_ctr         = await load_msc_group(client, "msc_controle",     [7,8],   co_tipo_matriz, tipos_balanco, meses, ente, ano, sem, delay)
    return msc_patrimonial, msc_orcam, msc_ctr

async def load_dca(ente, ano, concurrency=8):
    sem = asyncio.Semaphore(concurrency)
    anexos = {
        "ab": "DCA-Anexo I-AB",
        "c":  "DCA-Anexo I-C",
        "d":  "DCA-Anexo I-D",
        "e":  "DCA-Anexo I-E",
        "f":  "DCA-Anexo I-F",
        "g":  "DCA-Anexo I-G",
        "hi": "DCA-Anexo I-HI",
    }
    async with httpx.AsyncClient(http2=True) as client:
        tasks = {
            k: fetch_once(client, "dca", {"an_exercicio": ano, "no_anexo": v, "id_ente": ente}, sem=sem)
            for k, v in anexos.items()
        }
        results = await asyncio.gather(*tasks.values())
    return dict(zip(tasks.keys(), results))

async def load_rreo(ente, ano, concurrency=8):
    sem = asyncio.Semaphore(concurrency)
    anexos = {
        "1":"RREO-Anexo 01","2":"RREO-Anexo 02","3":"RREO-Anexo 03","4":"RREO-Anexo 04",
        "4_rpps":"RREO-Anexo 04 - RPPS","4_rgps":"RREO-Anexo 04 - RGPS",
        "6":"RREO-Anexo 06","7":"RREO-Anexo 07","9":"RREO-Anexo 09","11":"RREO-Anexo 11","14":"RREO-Anexo 14"
    }
    base = {"an_exercicio": ano, "nr_periodo": 6, "co_tipo_demonstrativo": "RREO", "id_ente": ente}
    async with httpx.AsyncClient(http2=True) as client:
        tasks = {k: fetch_once(client, "rreo", base | {"no_anexo": v}, sem=sem) for k, v in anexos.items()}
        results = await asyncio.gather(*tasks.values())
    return dict(zip(tasks.keys(), results))

async def load_rgf(ente, ano, concurrency=8):
    sem = asyncio.Semaphore(concurrency)
    # Anexo 5 por poder
    a5_poderes = {"5e":"E","5l":"L","5j":"J","5m":"M","5d":"D"}
    # Anexo 1 por poder
    a1_poderes = {"1e":"E","1l":"L","1j":"J","1m":"M","1d":"D"}
    # Anexos 2/3/4 somente Executivo
    outros = {"2e":("RGF-Anexo 02","E"), "3e":("RGF-Anexo 03","E"), "4e":("RGF-Anexo 04","E")}
    async with httpx.AsyncClient(http2=True) as client:
        tasks = {}
        for k,poder in a5_poderes.items():
            tasks[k] = fetch_once(client, "rgf",
                                  {"an_exercicio": ano, "in_periodicidade":"Q","nr_periodo":3,
                                   "co_tipo_demonstrativo":"RGF","no_anexo":"RGF-Anexo 05",
                                   "co_poder":poder,"id_ente":ente}, sem=sem)
        for k,poder in a1_poderes.items():
            tasks[k] = fetch_once(client, "rgf",
                                  {"an_exercicio": ano, "in_periodicidade":"Q","nr_periodo":3,
                                   "co_tipo_demonstrativo":"RGF","no_anexo":"RGF-Anexo 01",
                                   "co_poder":poder,"id_ente":ente}, sem=sem)
        for k,(anexo,poder) in outros.items():
            tasks[k] = fetch_once(client, "rgf",
                                  {"an_exercicio": ano, "in_periodicidade":"Q","nr_periodo":3,
                                   "co_tipo_demonstrativo":"RGF","no_anexo":anexo,
                                   "co_poder":poder,"id_ente":ente}, sem=sem)
        results = await asyncio.gather(*tasks.values())
    return dict(zip(tasks.keys(), results))


### Geração

In [8]:
if buscar_api == "S":
    tipos_balanco = ['ending_balance', 'beginning_balance', 'period_change']
    mes_selecionado = 12
    meses = list(range(1, mes_selecionado + 1))

    # MSC corrente (MSCC)
    msc_patrimonial, msc_orcam, msc_ctr = await load_msc_all(ente, ano, meses, tipos_balanco, co_tipo_matriz="MSCC", concurrency=8, delay=0.05)
    msc_patrimonial_orig = msc_patrimonial.copy()
    msc_orcam_orig = msc_orcam.copy()
    msc_ctr_orig = msc_ctr.copy()

    # Ajustes de sinal (iguais ao seu código)
    mascara_retificadora1 = (
        ((msc_patrimonial['conta_contabil'].astype(str).str[0] == '1') & (msc_patrimonial['natureza_conta'] == 'C')) |
        ((msc_patrimonial['conta_contabil'].astype(str).str[0] == '2') & (msc_patrimonial['natureza_conta'] == 'D')) |
        ((msc_patrimonial['conta_contabil'].astype(str).str[0] == '3') & (msc_patrimonial['natureza_conta'] == 'C')) |
        ((msc_patrimonial['conta_contabil'].astype(str).str[0] == '4') & (msc_patrimonial['natureza_conta'] == 'D'))
    )
    if not (msc_patrimonial.loc[mascara_retificadora1, 'valor'] < 0).any():
        msc_patrimonial.loc[mascara_retificadora1, 'valor'] *= -1

    mascara_retificadora2 = (
        ((msc_orcam['conta_contabil'].astype(str).str[0] == '5') & (msc_orcam['natureza_conta'] == 'C') & (~msc_orcam['tipo_valor'].eq('period_change'))) |
        ((msc_orcam['conta_contabil'].astype(str).str[0] == '6') & (msc_orcam['natureza_conta'] == 'D') & (~msc_orcam['tipo_valor'].eq('period_change')))
    )
    if not (msc_orcam.loc[mascara_retificadora2, 'valor'] < 0).any():
        msc_orcam.loc[mascara_retificadora2, 'valor'] *= -1

    mascara_retificadora3 = (
        ((msc_ctr['conta_contabil'].astype(str).str[0] == '7') & (msc_ctr['natureza_conta'] == 'C') & (~msc_ctr['tipo_valor'].eq('period_change'))) |
        ((msc_ctr['conta_contabil'].astype(str).str[0] == '8') & (msc_ctr['natureza_conta'] == 'D') & (~msc_ctr['tipo_valor'].eq('period_change')))
    )
    if not (msc_ctr.loc[mascara_retificadora3, 'valor'] < 0).any():
        msc_ctr.loc[mascara_retificadora3, 'valor'] *= -1

    msc = pd.concat([msc_patrimonial, msc_orcam, msc_ctr])

    # MSC de Encerramento (MSCE, mês 12)
    msc_patrimonial_encerr, msc_orcam_encerr, msc_ctr_encerr = await load_msc_all(
        ente, ano, [12], tipos_balanco, co_tipo_matriz="MSCE", concurrency=8, delay=0.05
    )
    msc_patr_encerr_orig = msc_patrimonial_encerr.copy()
    msc_orcam_encerr_orig = msc_orcam_encerr.copy()
    msc_ctr_encerr_orig = msc_ctr_encerr.copy()

    mascara_retificadora7 = (
        ((msc_patrimonial_encerr['conta_contabil'].astype(str).str[0] == '1') & (msc_patrimonial_encerr['natureza_conta'] == 'C') & (~msc_patrimonial_encerr['tipo_valor'].eq('period_change'))) |
        ((msc_patrimonial_encerr['conta_contabil'].astype(str).str[0] == '2') & (msc_patrimonial_encerr['natureza_conta'] == 'D') & (~msc_patrimonial_encerr['tipo_valor'].eq('period_change'))) |
        ((msc_patrimonial_encerr['conta_contabil'].astype(str).str[0] == '3') & (msc_patrimonial_encerr['natureza_conta'] == 'C') & (~msc_patrimonial_encerr['tipo_valor'].eq('period_change'))) |
        ((msc_patrimonial_encerr['conta_contabil'].astype(str).str[0] == '4') & (msc_patrimonial_encerr['natureza_conta'] == 'D') & (~msc_patrimonial_encerr['tipo_valor'].eq('period_change')))
    )
    if not (msc_patrimonial_encerr.loc[mascara_retificadora7, 'valor'] < 0).any():
        msc_patrimonial_encerr.loc[mascara_retificadora7, 'valor'] *= -1

    mascara_retificadora8 = (
        ((msc_orcam_encerr['conta_contabil'].astype(str).str[0] == '5') & (msc_orcam_encerr['natureza_conta'] == 'C') & (~msc_orcam_encerr['tipo_valor'].eq('period_change'))) |
        ((msc_orcam_encerr['conta_contabil'].astype(str).str[0] == '6') & (msc_orcam_encerr['natureza_conta'] == 'D') & (~msc_orcam_encerr['tipo_valor'].eq('period_change')))
    )
    if not (msc_orcam_encerr.loc[mascara_retificadora8, 'valor'] < 0).any():
        msc_orcam_encerr.loc[mascara_retificadora8, 'valor'] *= -1

    mascara_retificadora9 = (
        ((msc_ctr_encerr['conta_contabil'].astype(str).str[0] == '7') & (msc_ctr_encerr['natureza_conta'] == 'C') & (~msc_ctr_encerr['tipo_valor'].eq('period_change'))) |
        ((msc_ctr_encerr['conta_contabil'].astype(str).str[0] == '8') & (msc_ctr_encerr['natureza_conta'] == 'D') & (~msc_ctr_encerr['tipo_valor'].eq('period_change')))
    )
    if not (msc_ctr_encerr.loc[mascara_retificadora9, 'valor'] < 0).any():
        msc_ctr_encerr.loc[mascara_retificadora9, 'valor'] *= -1

    msc_encerr = pd.concat([msc_patrimonial_encerr, msc_orcam_encerr, msc_ctr_encerr])
    msc_consolidada = pd.concat([msc, msc_encerr])

    msc_orig = pd.concat([msc_patrimonial_orig, msc_orcam_orig, msc_ctr_orig])
    msc_orig_encerr = pd.concat([msc_patr_encerr_orig, msc_orcam_encerr_orig, msc_ctr_encerr_orig])
    msc_orig_consolidada = pd.concat([msc_orig, msc_orig_encerr])

    # Salvar CSVs
    msc_consolidada.to_csv(os.path.join(pasta_destino, f'MSC_Consolidada_{ano}_{cod}_{ente}.csv'), index=False)
    msc_orig_consolidada.to_csv(os.path.join(pasta_destino, f'MSC_Consolidada_Original_{ano}_{cod}_{ente}.csv'), index=False)
    msc_orig.to_csv(os.path.join(pasta_destino, f'MSC_Original_{ano}_{cod}_{ente}.csv'), index=False)
    msc_orig_encerr.to_csv(os.path.join(pasta_destino, f'MSC_ENCERR_Original_{ano}_{cod}_{ente}.csv'), index=False)
    msc.to_csv(os.path.join(pasta_destino, f'MSC_{ano}_{cod}_{ente}.csv'), index=False)
    msc_encerr.to_csv(os.path.join(pasta_destino, f'MSC_ENCERR_{ano}_{cod}_{ente}.csv'), index=False)

    # DCA (assíncrono)
    dca = await load_dca(ente, ano)
    df_dca_ab = dca["ab"]; df_dca_c = dca["c"]; df_dca_d = dca["d"]; df_dca_e = dca["e"]
    df_dca_f = dca["f"]; df_dca_g = dca["g"]; df_dca_hi = dca["hi"]
    df_dca_ab_orig = df_dca_ab.copy(); df_dca_c_orig = df_dca_c.copy()
    # Ajustes DCA
    if not df_dca_c.empty and "coluna" in df_dca_c and "valor" in df_dca_c:
        df_dca_c['valor'] = df_dca_c.apply(lambda row: -row['valor'] if 'Deduções' in str(row.get('coluna', '')) else row['valor'], axis=1)
    if not df_dca_ab.empty and "conta" in df_dca_ab and "valor" in df_dca_ab:
        df_dca_ab['valor'] = df_dca_ab.apply(lambda row: -row['valor'] if '(-)' in str(row.get('conta', '')) else row['valor'], axis=1)
    # Salvar DCA
    df_dca_ab.to_excel(os.path.join(pasta_destino, f'DCA_Anexo_AB_{ano}_{cod}_{ente}.xlsx'), index=False)
    df_dca_ab_orig.to_excel(os.path.join(pasta_destino, f'DCA_Anexo_AB_ORIGINAL_{ano}_{cod}_{ente}.xlsx'), index=False)
    df_dca_c.to_excel(os.path.join(pasta_destino, f'DCA_Anexo_C_{ano}_{cod}_{ente}.xlsx'), index=False)
    df_dca_c_orig.to_excel(os.path.join(pasta_destino, f'DCA_Anexo_C_ORIGINAL_{ano}_{cod}_{ente}.xlsx'), index=False)
    df_dca_d.to_excel(os.path.join(pasta_destino, f'DCA_Anexo_D_{ano}_{cod}_{ente}.xlsx'), index=False)
    df_dca_e.to_excel(os.path.join(pasta_destino, f'DCA_Anexo_E_{ano}_{cod}_{ente}.xlsx'), index=False)
    df_dca_f.to_excel(os.path.join(pasta_destino, f'DCA_Anexo_F_{ano}_{cod}_{ente}.xlsx'), index=False)
    df_dca_g.to_excel(os.path.join(pasta_destino, f'DCA_Anexo_G_{ano}_{cod}_{ente}.xlsx'), index=False)
    df_dca_hi.to_excel(os.path.join(pasta_destino, f'DCA_Anexo_HI_{ano}_{cod}_{ente}.xlsx'), index=False)

    # RREO (assíncrono)
    rreo = await load_rreo(ente, ano)
    rreo["1"].to_excel(os.path.join(pasta_destino, f'RREO_Anexo_1_{ano}_{cod}_{ente}.xlsx'), index=False)
    rreo["2"].to_excel(os.path.join(pasta_destino, f'RREO_Anexo_2_{ano}_{cod}_{ente}.xlsx'), index=False)
    rreo["3"].to_excel(os.path.join(pasta_destino, f'RREO_Anexo_3_{ano}_{cod}_{ente}.xlsx'), index=False)
    rreo["4"].to_excel(os.path.join(pasta_destino, f'RREO_Anexo_4_{ano}_{cod}_{ente}.xlsx'), index=False)
    rreo["4_rpps"].to_excel(os.path.join(pasta_destino, f'RREO_Anexo_4_RPPS_{ano}_{cod}_{ente}.xlsx'), index=False)
    rreo["4_rgps"].to_excel(os.path.join(pasta_destino, f'RREO_Anexo_4_RGPS_{ano}_{cod}_{ente}.xlsx'), index=False)
    rreo["6"].to_excel(os.path.join(pasta_destino, f'RREO_Anexo_6_{ano}_{cod}_{ente}.xlsx'), index=False)
    rreo["7"].to_excel(os.path.join(pasta_destino, f'RREO_Anexo_7_{ano}_{cod}_{ente}.xlsx'), index=False)
    rreo["9"].to_excel(os.path.join(pasta_destino, f'RREO_Anexo_9_{ano}_{cod}_{ente}.xlsx'), index=False)
    rreo["11"].to_excel(os.path.join(pasta_destino, f'RREO_Anexo_11_{ano}_{cod}_{ente}.xlsx'), index=False)
    rreo["14"].to_excel(os.path.join(pasta_destino, f'RREO_Anexo_14_{ano}_{cod}_{ente}.xlsx'), index=False)

    # RGF (assíncrono)
    rgf = await load_rgf(ente, ano)
    rgf["5e"].to_excel(os.path.join(pasta_destino, f'RGF_EXECUTIVO_Anexo_5_{ano}_{cod}_{ente}.xlsx'), index=False)
    rgf["5l"].to_excel(os.path.join(pasta_destino, f'RGF_LEGISLATIVO_Anexo_5_{ano}_{cod}_{ente}.xlsx'), index=False)
    rgf["5j"].to_excel(os.path.join(pasta_destino, f'RGF_JUDICIÁRIO_Anexo_5_{ano}_{cod}_{ente}.xlsx'), index=False)
    rgf["5m"].to_excel(os.path.join(pasta_destino, f'RGF_MP_Anexo_5_{ano}_{cod}_{ente}.xlsx'), index=False)
    rgf["5d"].to_excel(os.path.join(pasta_destino, f'RGF_DP_Anexo_5_{ano}_{cod}_{ente}.xlsx'), index=False)
    rgf["1e"].to_excel(os.path.join(pasta_destino, f'RGF_EXECUTIVO_Anexo_1_{ano}_{cod}_{ente}.xlsx'), index=False)
    rgf["1l"].to_excel(os.path.join(pasta_destino, f'RGF_LEGISLATIVO_Anexo_1_{ano}_{cod}_{ente}.xlsx'), index=False)
    rgf["1j"].to_excel(os.path.join(pasta_destino, f'RGF_JUDICIÁRIO_Anexo_1_{ano}_{cod}_{ente}.xlsx'), index=False)
    rgf["1m"].to_excel(os.path.join(pasta_destino, f'RGF_MP_Anexo_1_{ano}_{cod}_{ente}.xlsx'), index=False)
    rgf["1d"].to_excel(os.path.join(pasta_destino, f'RGF_DP_Anexo_1_{ano}_{cod}_{ente}.xlsx'), index=False)
    rgf["2e"].to_excel(os.path.join(pasta_destino, f'RGF_EXECUTIVO_Anexo_2_{ano}_{cod}_{ente}.xlsx'), index=False)
    rgf["3e"].to_excel(os.path.join(pasta_destino, f'RGF_EXECUTIVO_Anexo_3_{ano}_{cod}_{ente}.xlsx'), index=False)
    rgf["4e"].to_excel(os.path.join(pasta_destino, f'RGF_EXECUTIVO_Anexo_4_{ano}_{cod}_{ente}.xlsx'), index=False)

    # Agregações RGF iguais às suas
    rgf_total = pd.concat([rgf["5e"], rgf["5l"], rgf["5j"], rgf["5m"], rgf["5d"]], ignore_index=True)
    df_rgf_5 = pd.concat([rgf["5e"], rgf["5l"], rgf["5j"], rgf["5m"], rgf["5d"]], ignore_index=True)
    rgf_o = pd.concat([rgf["5l"], rgf["5j"], rgf["5m"], rgf["5d"]], ignore_index=True)


C:\Users\mjandussi\AppData\Local\Temp\ipykernel_16960\3433640510.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df for df in dfs if not df.empty], ignore_index=True) if dfs else pd.DataFrame()
C:\Users\mjandussi\AppData\Local\Temp\ipykernel_16960\3433640510.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
C:\Users\mjandussi\AppData\Local\Temp\ipykernel_16960\2886278998.py:36: FutureWarning: The 

### Variáveis para a Matriz

In [9]:
##############################################################################################################################
##############################################################################################################################
# Criando matrizes Específicas para as análises

msc_dez = msc.query('mes_referencia == 12')

msc_consolidada_e = msc_consolidada.query('tipo_valor == "ending_balance"')
msc_consolidada_b = msc_consolidada.query('tipo_valor == "beginning_balance"')

msc_orig_e = msc_orig.query('tipo_valor == "ending_balance"')
msc_orig_b = msc_orig.query('tipo_valor == "beginning_balance"')

msc_orig_consolidada_e = msc_orig_consolidada.query('tipo_valor == "ending_balance"')
msc_orig_consolidada_b = msc_orig_consolidada.query('tipo_valor == "beginning_balance"')


##########################################################################################################
# Criando AJUSTES
# Função para extrair o terceiro e quarto dígito com verificação de comprimento
def extrair_terceiro_quarto_digito(valor):
    valor_str = str(valor)
    if len(valor_str) >= 4:
        return valor_str[2:4]  # Extrai do índice 2 até o índice 3 (terceiro e quarto dígito)
    else:
        return None  # Ou qualquer valor padrão que você queira usar
    
# Criando Variáveis da MSC
# Variáveis da MSC       
#receita = msc_dez.query('conta_contabil == "621200000" or conta_contabil == "621310100" or conta_contabil == "621310200" or conta_contabil == "621390000"')
#receita = msc_consolidada_e.query('conta_contabil == "621200000" or conta_contabil == "621310100" or conta_contabil == "621310200" or conta_contabil == "621390000"')
receita = msc_consolidada_e[msc_consolidada_e['conta_contabil'].str.match(r"^(6212|6213)")]


receita['cat_receita'] = receita['natureza_receita'].astype(str).str[0]
receita_corr = receita.query('cat_receita == "1"')
receita_capi = receita.query('cat_receita == "2"')

## Criando Despesa Corrente e de Capital 
despesa = msc_dez[msc_dez['conta_contabil'].str.match(r"^(6221)")]

# Aplicando a função ao DataFrame
despesa['DIGITO_INTRA'] = despesa['natureza_despesa'].apply(extrair_terceiro_quarto_digito)

#Pegando a Depesa Empenhada na Matriz e quebrando a Informação da Modalidade em Não Intra e Intra
emp_msc_dez = despesa.query('tipo_valor == "ending_balance" and (conta_contabil == "622130500" or conta_contabil == "622130600" or conta_contabil == "622130700" or conta_contabil == "622130400")')

despesa_corr = despesa[despesa['natureza_despesa'].str.match(r"^(3)")]
despesa_capi = despesa[despesa['natureza_despesa'].str.match(r"^(4)")]

#################################################################################

#Pegando a Depesa Empenhada na Matriz e quebrando a Informação da Modalidade em Não Intra e Intra
emp_msc_encerr = msc_encerr.query('tipo_valor == "beginning_balance" and (conta_contabil == "622130500" or conta_contabil == "622130600" or conta_contabil == "622130700" or conta_contabil == "622130400")')
# q = emp_msc_encerr['IC5'].str.split("", n=5, expand=True)
# emp_msc_encerr['intra'] = q[4]
emp_msc_encerr['DIGITO_INTRA'] = emp_msc_encerr['natureza_despesa'].apply(extrair_terceiro_quarto_digito)


msc_orig_encerr_b = msc_orig_encerr.query('tipo_valor == "beginning_balance"')
msc_orig_encerr_e = msc_orig_encerr.query('tipo_valor == "ending_balance"')


# Criando uma junção dos Saldos Finais das matrizes de JAN a DEZ e a de Saldo Inicial da matriz de encerramento
msc_original_e_b_p_13 = pd.concat([msc_orig_e, msc_orig_encerr_b], ignore_index=True)

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

## D1_00017 >> Análise se existe Saldo Negativo na Matriz. Cada MSC sem movimentação incorreta vale 1/13 da pontuação.

In [10]:
#D1_00017
d1_00017_t = msc_orig_consolidada.query('valor < 0')

########################### Verifica se a coluna de Saldo possui campos Com Valores NEGATIVOS
if (msc_orig_consolidada['valor'] < 0).any():
    resposta_d1_00017 = 'ERRO'
else:
    resposta_d1_00017 = 'OK'

d1_00017_t = msc_orig_consolidada.query('valor < 0')

contagem = d1_00017_t.mes_referencia.unique()
erros = len(contagem)
nota = (100/13) * (13-erros)
nota = round(nota)/100   

# Cria um DataFrame com a resposta
d1_00017 = pd.DataFrame([resposta_d1_00017], columns=['Resposta'])
# Insere novas colunas
d1_00017.insert(0, 'Dimensão', 'D1_00017')
d1_00017.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com valores negativos')
d1_00017.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros} - Pontos: {nota}')
d1_00017

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00017,OK,Verifica a quantidade de matrizes com valores negativos,Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0


In [11]:
d1_00017_t

,tipo_matriz,cod_ibge,classe_conta,conta_contabil,poder_orgao,financeiro_permanente,ano_fonte_recursos,fonte_recursos,exercicio,mes_referencia,...,valor,natureza_conta,tipo_valor,complemento_fonte,funcao,subfuncao,educacao_saude,natureza_despesa,ano_inscricao,natureza_receita


# D1_00018 >>  Movimentação inconsistente: saldo inicial + movimentação <> saldo final. Cada MSC sem movimentação incorreta vale 1/13 da pontuação.

In [12]:
msc_base = msc_orig_consolidada.groupby(['tipo_matriz','conta_contabil', 'mes_referencia', 'tipo_valor', 'natureza_conta'])['valor'].sum().reset_index()
msc_base['conta_contabil'] = msc_base['conta_contabil'].astype(str)
# Pegar só o primeiro elemento de uma coluna
msc_base["Grupo_Contas"] = msc_base["conta_contabil"].str[0]

# Aplicando a fórmula para trocar o sinal das contas retificadoras
msc_base['valor'] = msc_base.apply(lambda x: x['valor'] * -1
if (x['Grupo_Contas'] == '1' and x['natureza_conta'] == 'C' and not x['tipo_valor'] == 'period_change')
or (x['Grupo_Contas'] == '2' and x['natureza_conta'] == 'D' and not x['tipo_valor'] == 'period_change')
or (x['Grupo_Contas'] == '4' and x['natureza_conta'] == 'D' and not x['tipo_valor'] == 'period_change')
or (x['Grupo_Contas'] == '5' and x['natureza_conta'] == 'C' and not x['tipo_valor'] == 'period_change')
or (x['Grupo_Contas'] == '6' and x['natureza_conta'] == 'D' and not x['tipo_valor'] == 'period_change')
or (x['Grupo_Contas'] == '7' and x['natureza_conta'] == 'C' and not x['tipo_valor'] == 'period_change')
or (x['Grupo_Contas'] == '8' and x['natureza_conta'] == 'D' and not x['tipo_valor'] == 'period_change')
else x['valor'], axis=1)

# Aplicando a fórmula para trocar o sinal do period_change
msc_base['valor'] = msc_base.apply(lambda x: x['valor'] * -1
if (x['Grupo_Contas'] == '1' and x['natureza_conta'] == 'C' and x['tipo_valor'] == 'period_change')
or (x['Grupo_Contas'] == '2' and x['natureza_conta'] == 'D' and x['tipo_valor'] == 'period_change')
or (x['Grupo_Contas'] == '3' and x['natureza_conta'] == 'C' and x['tipo_valor'] == 'period_change')
or (x['Grupo_Contas'] == '4' and x['natureza_conta'] == 'D' and x['tipo_valor'] == 'period_change')
or (x['Grupo_Contas'] == '5' and x['natureza_conta'] == 'C' and x['tipo_valor'] == 'period_change')
or (x['Grupo_Contas'] == '6' and x['natureza_conta'] == 'D' and x['tipo_valor'] == 'period_change')
or (x['Grupo_Contas'] == '7' and x['natureza_conta'] == 'C' and x['tipo_valor'] == 'period_change')
or (x['Grupo_Contas'] == '8' and x['natureza_conta'] == 'D' and x['tipo_valor'] == 'period_change')
else x['valor'], axis=1)

analise_b = msc_base.query('tipo_valor != "ending_balance"')
analise_b = analise_b.groupby(['tipo_matriz', 'mes_referencia','conta_contabil'])['valor'].sum().reset_index()
analise_b['mes_referencia'] = analise_b['mes_referencia'].astype(str)
analise_b['chave'] = analise_b['tipo_matriz'] + analise_b['mes_referencia'] + analise_b['conta_contabil']

analise_e = msc_base.query('tipo_valor == "ending_balance"')
analise_e = analise_e.groupby(['tipo_matriz','mes_referencia', 'conta_contabil'])['valor'].sum().reset_index()
analise_e['mes_referencia'] = analise_e['mes_referencia'].astype(str)
# CRIAÇÃO DA CHAVE
analise_e['chave'] = analise_e['tipo_matriz'] + analise_e['mes_referencia'] + analise_e['conta_contabil']

analise = analise_b.merge(analise_e, on="chave")
analise['DIF'] = analise['valor_x'] - analise['valor_y']

# Definir um limite muito pequeno para considerar como zero (por conta de quando dá notação científica)
limite_zero = 1e-2  # Por exemplo, considerando valores menores que X (exemplo, se quiser menor que 0.00001 como 4 zeros)

# Filtro para valores maiores que zero ou menores que zero, mas não tão próximos de zero quanto o 'limite_zero'
filtro_valores = (analise['DIF'] > limite_zero) | (analise['DIF'] < -limite_zero)

# FAzendo a NOta e a ocorrencia
#d1_00018_t = analise.groupby(['tipo_matriz_x','mes_referencia_x'])['DIF'].sum().reset_index()
#ocorrencias_incorretas = (f'{mes_selecionado}' - analise.query('DIF != 0').value_counts().sum())
#ocorrencias = analise.query('DIF != 0').value_counts().sum()

d1_00018_t = analise[filtro_valores]
contagem = d1_00018_t.mes_referencia_x.unique()
erros = len(contagem)
nota = (100/13) * (13-erros)
nota = round(nota)/100

if filtro_valores.any():
    resposta_d1_00018 = 'ERRO'
else:
    resposta_d1_00018 = 'OK'

# Cria um DataFrame com a resposta
d1_00018 = pd.DataFrame([resposta_d1_00018], columns=['Resposta'])
# Insere novas colunas
d1_00018.insert(0, 'Dimensão', 'D1_00018')
d1_00018.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com diferenças na movimentação (SI+ MOV <> SF)')
d1_00018.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros} - Pontos: {nota}')
d1_00018

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00018,OK,Verifica a quantidade de matrizes com diferenças na movimentação (SI+ MOV <> SF),Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0


In [13]:
d1_00018_t

,tipo_matriz_x,mes_referencia_x,conta_contabil_x,valor_x,chave,tipo_matriz_y,mes_referencia_y,conta_contabil_y,valor_y,DIF


# D1_00019 >> Códigos de poderes incorretos. Cada MSC sem PO incorreto vale 1/13 da pontuação.

In [14]:
po_stn = pd.read_excel(f'{ano}_Anexo_II_Portaria_STN_642_Leiaute_MSC.xlsx', sheet_name="PO", header=4)
po_stn['Código'] = po_stn['Código'].astype(int)
po_stn = po_stn.rename(columns={'Código': 'poder_orgao'})

codigos_na_msc = msc_orig_consolidada.groupby(['poder_orgao'])['valor'].sum().reset_index()
codigos_na_msc['poder_orgao'] = codigos_na_msc['poder_orgao'].astype(int)

d1_00019_t = codigos_na_msc.merge(po_stn, how="left", on="poder_orgao")

d1_00019_ta = msc_orig_consolidada.groupby(['mes_referencia', 'poder_orgao'])['valor'].sum().reset_index()

# Sua lista de poderes de referência
lista_poderes = ['10111', '10112', '20211', '20212', '20213', '30390', '50511', '60611']

# 1. Criar um conjunto dos códigos de referência
set_poderes_referencia = set(lista_poderes)

# 2. Obter os meses únicos na tabela
meses_unicos = d1_00019_ta['mes_referencia'].unique()

# Lista para armazenar os meses com códigos diferentes
meses_com_codigos_diferentes = []

# 3. Iterar sobre os meses
for mes in meses_unicos:
    # 4. Filtrar a tabela para o mês atual
    df_mes_filtrado = d1_00019_ta[d1_00019_ta['mes_referencia'] == mes]

    # Obter os códigos de poder únicos para este mês
    codigos_mes = set(df_mes_filtrado['poder_orgao'].astype(str).unique())

    # Verificar se algum código deste mês não está no conjunto de referência
    codigos_diferentes_mes = codigos_mes - set_poderes_referencia

    if codigos_diferentes_mes:
        meses_com_codigos_diferentes.append(mes)
        
# Contar o número de meses com códigos diferentes
erro = len(meses_com_codigos_diferentes)
nota = (100/13) * (13-erros)
nota = round(nota)/100

if erro == 0:
    resposta_d1_00019 = 'OK'
else:
    resposta_d1_00019 = 'ERRO'

# Cria um DataFrame com a resposta
d1_00019 = pd.DataFrame([resposta_d1_00019], columns=['Resposta'])
# Insere novas colunas
d1_00019.insert(0, 'Dimensão', 'D1_00019')
d1_00019.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com códigos de Poderes incorretos')
d1_00019.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros} - Pontos: {nota}')
d1_00019

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00019,OK,Verifica a quantidade de matrizes com códigos de Poderes incorretos,Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0


In [15]:
d1_00019_t

,poder_orgao,valor,Nomenclatura
0,10111,"140,487,225,658,600.77",Poder Executivo – Governo Estadual
1,10112,"38,674,471,050,175.86",Poder Executivo – RPPS Estadual
2,20211,"907,665,256,120.58",Poder Legislativo - Assembleia Legislativa
3,20212,"758,369,371,876.08",Poder Legislativo - Tribunal de Contas do Estado
4,30390,"5,468,867,712,749.54",Poder Judiciário - Tribunal de Justiça
5,50511,"1,891,742,260,912.58",Ministério Público do Estado
6,60611,"741,212,269,163.04",Defensoria Pública do Estado


# D1_00020 >> Contas cujo saldo inicial é diferente do saldo final da MSC do mês anterior. Cada MSC sem divergência vale 1/12 da pontuação.

In [16]:
msc_consolidada_dif = msc_orig.sort_values(by=["conta_contabil", "mes_referencia", "tipo_valor"])
msc_consolidada_dif = msc_consolidada_dif.groupby(["conta_contabil", "mes_referencia", "tipo_valor"])['valor'].sum().reset_index()
# msc_consolidada_dif = msc_consolidada_anual.sort_values(by=["conta_contabil", "mes_referencia"])
# msc_consolidada_dif = msc_consolidada_dif.groupby(['exercicio', 'tipo_matriz', 'conta_contabil', 'mes_referencia', 'tipo_valor'])['valor'].sum().reset_index()

msc_dif = msc_consolidada_dif[msc_consolidada_dif['tipo_valor'] != 'period_change']

msc_dif['diferenca_valor'] = msc_dif.groupby(['conta_contabil'])['valor'].diff()
msc_dif = msc_dif[msc_dif['tipo_valor'] == 'beginning_balance']

msc_dif = msc_dif.dropna()

d1_00020_t = msc_dif.query('diferenca_valor != 0')

contagem = d1_00020_t.mes_referencia.unique()
erros = len(contagem)
nota = (100/12) * (12-erros)
nota = round(nota)/100

# Ajuste a tolerância para até 3 casas decimais
tolerancia = 1e-3  
# Verifica se há algum valor diferente de zero na coluna 'DIF'
condicao = ((abs(msc_dif['diferenca_valor']) > tolerancia) | (abs(msc_dif['diferenca_valor']) < -tolerancia))

if condicao.any():
    resposta_d1_00020 = 'ERRO'
else:
    resposta_d1_00020 = 'OK'

# Cria um DataFrame com a resposta
d1_00020 = pd.DataFrame([resposta_d1_00020], columns=['Resposta'])
# Insere novas colunas
d1_00020.insert(0, 'Dimensão', 'D1_00020')
d1_00020.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com SI <> do SF da MSC do mês anterior')
d1_00020.insert(3, 'OBS', f'Cada MSC vale 1/12 - Erros: {erros} - Pontos: {nota}')
d1_00020

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00020,OK,Verifica a quantidade de matrizes com SI <> do SF da MSC do mês anterior,Cada MSC vale 1/12 - Erros: 0 - Pontos: 1.0


In [17]:
d1_00020_t

,conta_contabil,mes_referencia,tipo_valor,valor,diferenca_valor


# D1_00021 >> Contas dos grupos 1111, 1121, 1125, 1231 e 1232 cujo saldo final está com natureza diferente da padrão do PCASP Estendido. Cada MSC vale 1/13 da pontuação.

In [18]:
pc_estendido = pd.read_excel(f'{ano}_Anexo_II_Portaria_STN_642_Leiaute_MSC.xlsx', sheet_name=f'PcaspEstendido{ano}', header=3)
pc_estendido['CONTA'] = pc_estendido['CONTA'].astype(str)
# Pegar os 4 primeiros dígitos da coluna 'codigo'
pc_estendido['conta_4'] = pc_estendido['CONTA'].str.slice(stop=4)

# Groupby (Tabela Dinamica)
ativo_pcasp = pc_estendido.query('conta_4 == "1111" or conta_4 == "1121" or conta_4 == "1125" or conta_4 == "1231" or conta_4 == "1232"')
ativo_pcasp = ativo_pcasp.groupby(['conta_4', 'CONTA', 'TÍTULO.1', 'NATUREZA DO SALDO', 'STATUS']).sum().reset_index()
ativo_pcasp = ativo_pcasp.rename(columns={"CONTA": "conta_contabil"})

#################### ATIVO na MSC com Saldo Devedor
msc_consolidada['conta_contabil'] = msc_consolidada['conta_contabil'].astype(str)

ativo_msc = msc_consolidada[msc_consolidada['tipo_valor'] != 'period_change']

ativo_msc = ativo_msc[(ativo_msc['conta_contabil'].str.startswith('1111')) | (ativo_msc['conta_contabil'].str.startswith('1121')) | (ativo_msc['conta_contabil'].str.startswith('1125'))\
                    | (ativo_msc['conta_contabil'].str.startswith('1231')) | (ativo_msc['conta_contabil'].str.startswith('1232'))]

# Agrupar e somar os valores pelaqs linhas negativas com valores baixos
ativo_msc = ativo_msc.groupby(['mes_referencia', 'tipo_matriz', 'conta_contabil'], as_index=False)['valor'].sum()
# Determinar a nova natureza: 'D' se positivo, 'C' se negativo
ativo_msc['natureza_conta'] = ativo_msc['valor'].apply(lambda x: 'D' if x >= 0 else 'C')

# MERGE
erro_ativo = ativo_msc.merge(ativo_pcasp, on='conta_contabil', how="left")

erro_ativo = erro_ativo[(erro_ativo['valor'] != 0)]
erro_ativo = erro_ativo[['mes_referencia', 'tipo_matriz','natureza_conta', 'NATUREZA DO SALDO', 'valor']]
erro_ativo['chave'] = erro_ativo['natureza_conta'] + erro_ativo['NATUREZA DO SALDO']
d1_00021_ta = erro_ativo.groupby(['mes_referencia','tipo_matriz','chave'])['valor'].sum().reset_index()
d1_00021_t = d1_00021_ta.query('chave == "CDevedora" or chave == "DCredora"')

condicao = d1_00021_ta.query('chave == "CDevedora" or chave == "DCredora"').value_counts().sum()

# Para obter o número total de ocorrências (linhas) no DataFrame
contagem = d1_00021_t.mes_referencia.unique()
erros = len(contagem)   
nota = (100/13) * (13-erros)
nota = round(nota)/100

if condicao != 0:
    resposta_d1_00021 = 'ERRO'
else:
    resposta_d1_00021 = 'OK'

# Cria um DataFrame com a resposta
d1_00021 = pd.DataFrame([resposta_d1_00021], columns=['Resposta'])
# Insere novas colunas
d1_00021.insert(0, 'Dimensão', 'D1_00021')
d1_00021.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com contas dos grupos de ATIVO com natureza diferente do PCASP')
d1_00021.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros} - Pontos: {nota}')
d1_00021

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00021,OK,Verifica a quantidade de matrizes com contas dos grupos de ATIVO com natureza diferente do PCASP,Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0


In [19]:
d1_00021_ta

,mes_referencia,tipo_matriz,chave,valor
0,1,MSCC,DDevedora,"165,497,537,196.65"
1,2,MSCC,DDevedora,"174,178,617,645.95"
2,3,MSCC,DDevedora,"180,374,836,146.25"
3,4,MSCC,DDevedora,"177,501,299,384.91"
4,5,MSCC,DDevedora,"179,448,468,180.26"
5,6,MSCC,DDevedora,"182,307,266,563.89"
6,7,MSCC,DDevedora,"175,279,381,379.32"
7,8,MSCC,DDevedora,"175,545,173,762.70"
8,9,MSCC,DDevedora,"181,335,515,066.14"
9,10,MSCC,DDevedora,"185,237,581,617.24"


# D1_00022 >> Todos os códigos de poder/órgão informados. Cada MSC completa vale 1/13 da pontuação.

In [20]:
# Verifica se a coluna "PO" possui campos vazios
if msc_consolidada['poder_orgao'].isna().any() or (msc_consolidada['poder_orgao'] == '').any():
    resposta_d1_00022 = 'ERRO'
else:
    resposta_d1_00022 = 'OK'

# Criando uma coluna auxiliar para identificar linhas com IC1 nulo ou vazio
msc_consolidada['msc_consolidada_null_or_empty'] = msc_consolidada['poder_orgao'].isna() | (msc_consolidada['poder_orgao'] == '')
# Usando query para filtrar o DataFrame com base na coluna auxiliar
d1_00022_t = msc_consolidada.query('msc_consolidada_null_or_empty == True')
# Removendo a coluna auxiliar após o filtro, se desejado
d1_00022_t = d1_00022_t.drop(columns=['msc_consolidada_null_or_empty'])

d1_00022_ta = msc_consolidada.groupby(['mes_referencia', 'tipo_matriz'])['valor'].sum().reset_index()

contagem = d1_00022_t.mes_referencia.unique()
erros = len(contagem) 
nota = (100/13) * (13-erros)
nota = round(nota)/100
    
# Cria um DataFrame com a resposta
d1_00022 = pd.DataFrame([resposta_d1_00022], columns=['Resposta'])
# Insere novas colunas
d1_00022.insert(0, 'Dimensão', 'D1_00022')
d1_00022.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com todos os códigos de poder/órgão informados')
d1_00022.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros} - Pontos: {nota}')
d1_00022

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00022,OK,Verifica a quantidade de matrizes com todos os códigos de poder/órgão informados,Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0


In [21]:
d1_00022_ta

,mes_referencia,tipo_matriz,valor
0,1,MSCC,"7,488,935,711,692.82"
1,2,MSCC,"7,309,551,068,649.94"
2,3,MSCC,"7,415,214,404,122.86"
3,4,MSCC,"8,012,976,496,909.18"
4,5,MSCC,"7,976,249,695,553.90"
5,6,MSCC,"8,505,964,542,700.38"
6,7,MSCC,"9,253,178,753,019.56"
7,8,MSCC,"9,396,761,296,155.80"
8,9,MSCC,"9,844,503,369,701.46"
9,10,MSCC,"10,575,302,909,783.62"


# D1_00023 >> Dados do poder executivo repetida em mais de um mês. Cada MSC única vale 1/13 da pontuação.

In [22]:
msc_consolidada_anual_e = msc_consolidada.query('poder_orgao == 10111 or poder_orgao == 10112')
# msc_consolidada_anual_e['conta_contabil'] = msc_consolidada_anual_e['conta_contabil'].astype(str)
# # Abrir também pelo Grupo de Contas (OBS: pode dar errado pois na Matriz de Encerramento alguns grupos de Contas, como Ativo, não tem diferenças mesmo)
# msc_consolidada_anual_e["Grupo_Contas"] = msc_consolidada_anual_e["conta_contabil"].str[0]
#d1_00023_t = msc_consolidada_anual_e.groupby(['Grupo_Contas', 'mes_referencia', 'tipo_matriz'])['valor'].sum().reset_index()
d1_00023_ta = msc_consolidada_anual_e.groupby(['mes_referencia', 'tipo_matriz'])['valor'].sum().reset_index()

d1_00023_ta['diferenca'] = d1_00023_ta['valor'].diff()

d1_00023_t = d1_00023_ta.query('diferenca == 0')
contagem = d1_00023_t.mes_referencia.unique()
erros = len(contagem)
nota = (100/13) * (13-erros)
nota = round(nota)/100

# Verifica se existe algum valor igual a zero na coluna 'diferenca'
condicao = (d1_00023_ta['diferenca'] == 0).any()

if condicao.any():
    resposta_d1_00023 = 'ERRO'
else:
    resposta_d1_00023 = 'OK'

# Cria um DataFrame com a resposta
d1_00023 = pd.DataFrame([resposta_d1_00023], columns=['Resposta'])
# Insere novas colunas
d1_00023.insert(0, 'Dimensão', 'D1_00023')
d1_00023.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade matrizes com dados do poder Executivo repetida em mais de um mês')
d1_00023.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros} - Pontos: {nota}')
d1_00023

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00023,OK,Verifica a quantidade matrizes com dados do poder Executivo repetida em mais de um mês,Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0


In [23]:
d1_00023_ta

,mes_referencia,tipo_matriz,valor,diferenca


# D1_00024 >> Dados do poder legislativo repetida em mais de um mês. Cada MSC única vale 1/13 da pontuação.

In [24]:
msc_consolidada_anual_l = msc_consolidada.query('poder_orgao == 20211 or poder_orgao == 20212')
d1_00024_ta = msc_consolidada_anual_l.groupby(['mes_referencia', 'tipo_matriz'])['valor'].sum().reset_index()
d1_00024_ta['diferenca'] = d1_00024_ta['valor'].diff()

d1_00024_t = d1_00024_ta.query('diferenca == 0')
contagem = d1_00024_t.mes_referencia.unique()
erros = len(contagem)
nota = (100/13) * (13-erros)
nota = round(nota)/100

# Verifica se existe algum valor igual a zero na coluna 'diferenca'
condicao = (d1_00024_ta['diferenca'] == 0).any()

if condicao.any():
    resposta_d1_00024 = 'ERRO'
else:
    resposta_d1_00024 = 'OK'

# Cria um DataFrame com a resposta
d1_00024 = pd.DataFrame([resposta_d1_00024], columns=['Resposta'])
# Insere novas colunas
d1_00024.insert(0, 'Dimensão', 'D1_00024')
d1_00024.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade matrizes com dados do poder Legislativo repetida em mais de um mês')
d1_00024.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros} - Pontos: {nota}')
d1_00024

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00024,OK,Verifica a quantidade matrizes com dados do poder Legislativo repetida em mais de um mês,Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0


In [25]:
d1_00024_t

,mes_referencia,tipo_matriz,valor,diferenca


# D1_00025 >> Matrizes com contas dos grupos 2111, 2112, 2113,2114, 2121, 2122, 2123, 2124, 2125,2126, 213, 214, 215, 221, 222, 223 cujo saldo final está com natureza diferente da padrão do PCASP Estendido. Cada MSC vale 1/13 da pontuação.

In [26]:
filtro_1 = pc_estendido[pc_estendido['CONTA'].str.match(r"^(2111|2112|2113|2114|2121|2122|2123|2124|2125|2126|213|214|215|221|222|223)")]
pass_pcasp = filtro_1.groupby(['CONTA', 'TÍTULO.1', 'NATUREZA DO SALDO', 'STATUS']).sum().reset_index()
pass_pcasp = pass_pcasp.rename(columns={"CONTA": "conta_contabil"})

msc_base_e = msc_consolidada.query('tipo_valor == "ending_balance"')
pass_msc = msc_base_e[msc_base_e['conta_contabil'].str.match(r"^(2111|2112|2113|2114|2121|2122|2123|2124|2125|2126|213|214|215|221|222|223)")]
#pass_msc = pass_msc.groupby(['conta_contabil', 'natureza_conta', 'mes_referencia'])['valor'].sum().reset_index()

# Agrupar e somar os valores pelaqs linhas negativas com valores baixos
pass_msc = pass_msc.groupby(['mes_referencia', 'tipo_matriz', 'conta_contabil'], as_index=False)['valor'].sum()
# Determinar a nova natureza: 'C' se positivo, 'D' se negativo
pass_msc['natureza_conta'] = pass_msc['valor'].apply(lambda x: 'C' if x >= 0 else 'D')

### MERGE
erro_pass = pass_msc.merge(pass_pcasp, on='conta_contabil', how="left")

erro_pass = erro_pass[(erro_pass['valor'] != 0)]

d1_00025_ta = erro_pass[['conta_contabil','natureza_conta', 'NATUREZA DO SALDO', 'valor', 'mes_referencia']]

d1_00025_ta['chave'] = d1_00025_ta['natureza_conta'] + d1_00025_ta['NATUREZA DO SALDO']

d1_00025_t = d1_00025_ta.query('chave == "CDevedora" or chave == "DCredora"')

condicao = d1_00025_ta.query('chave == "CDevedora" or chave == "DCredora"').value_counts().sum()

d1_00025_te = d1_00025_t.groupby(['mes_referencia'])['valor'].sum().reset_index()

contagem = d1_00025_t.mes_referencia.unique()
erros = len(contagem)   
nota = (100/13) * (13-erros)
nota = round(nota)/100

# Verifica se a condição é maior que zero
if condicao > 0:
    resposta_d1_00025 = 'ERRO'
else:
    resposta_d1_00025 = 'OK'

# # Criando umacoluna com um grupo de iniciado com 3
pc_estendido['conta_3'] = pc_estendido['CONTA'].str.slice(stop=3)

# Cria um DataFrame com a resposta
d1_00025 = pd.DataFrame([resposta_d1_00025], columns=['Resposta'])
# Insere novas colunas
d1_00025.insert(0, 'Dimensão', 'D1_00025')
d1_00025.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com contas dos grupos de PASSIVO com natureza diferente do PCASP')
d1_00025.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros} - Pontos: {nota}')
d1_00025

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00025,ERRO,Verifica a quantidade de matrizes com contas dos grupos de PASSIVO com natureza diferente do PCASP,Cada MSC vale 1/13 - Erros: 1 - Pontos: 0.92


In [27]:
d1_00025_t

,conta_contabil,natureza_conta,NATUREZA DO SALDO,valor,mes_referencia,chave
935,215150100,D,Credora,"-31,075,702.96",12,DCredora
936,215150200,D,Credora,"-377,834.89",12,DCredora
1017,215150100,D,Credora,"-31,075,702.96",12,DCredora
1018,215150200,D,Credora,"-377,834.89",12,DCredora


# D1_00026 >> Matrizes com contas dos grupos 2311, 2312, 232, 233, 234, 235, 236 cujo saldo final está com natureza diferente da padrão do PCASP Estendido. Cada MSC vale 1/13 da pontuação.

In [28]:
# # Criando umacoluna com um grupo de iniciado com 3
pc_estendido['conta_3'] = pc_estendido['CONTA'].str.slice(stop=3)

pl_pcasp1 = pc_estendido.query('conta_4 == "2311" or conta_4 == "2321"')
pl_pcasp2 = pc_estendido.query('conta_3 == "232" or conta_3 == "233" or conta_3 == "234" or conta_3 == "235" or conta_3 == "236"')
pl_pcasp = pd.concat([pl_pcasp1, pl_pcasp2])

pl_pcasp = pl_pcasp.groupby(['CONTA', 'TÍTULO.1', 'NATUREZA DO SALDO', 'STATUS']).sum().reset_index()
pl_pcasp = pl_pcasp.rename(columns={"CONTA": "conta_contabil"})

#################### ATIVO na MSC com Saldo Devedor
pl_msc = msc_consolidada[msc_consolidada['tipo_valor'] == 'ending_balance']
pl_msc = pl_msc[(pl_msc['conta_contabil'].str.startswith('2311')) | (pl_msc['conta_contabil'].str.startswith('2312')) | (pl_msc['conta_contabil'].str.startswith('232'))\
                    | (pl_msc['conta_contabil'].str.startswith('233')) | (pl_msc['conta_contabil'].str.startswith('234')) | (pl_msc['conta_contabil'].str.startswith('235')) \
                        | (pl_msc['conta_contabil'].str.startswith('236'))]

pl_msc = pl_msc.groupby(['tipo_matriz', 'conta_contabil', 'natureza_conta', 'mes_referencia'])['valor'].sum().reset_index()

### MERGE
erro_pl = pl_msc.merge(pl_pcasp, on='conta_contabil', how="left")

erro_pl = erro_pl[(erro_pl['valor'] != 0)]
erro_pl = erro_pl[['tipo_matriz','conta_contabil', 'natureza_conta', 'NATUREZA DO SALDO', 'TÍTULO.1', 'mes_referencia',  'valor']]
erro_pl['chave'] = erro_pl['natureza_conta'] + erro_pl['NATUREZA DO SALDO']
d1_00026_ta = erro_pl.groupby(['chave', 'mes_referencia', 'tipo_matriz'])['valor'].sum().reset_index()

d1_00026_t = d1_00026_ta.query('chave == "CDevedora" or chave == "DCredora"')

condicao = d1_00026_ta.query('chave == "CDevedora" or chave == "DCredora"').value_counts().sum()

contagem = d1_00026_t.mes_referencia.unique()
erros = len(contagem)   
nota = (100/13) * (13-erros)
nota = round(nota)/100

if condicao.any():
    resposta_d1_00026 = 'ERRO'
else:
    resposta_d1_00026 = 'OK'

# Cria um DataFrame com a resposta
d1_00026 = pd.DataFrame([resposta_d1_00026], columns=['Resposta'])
# Insere novas colunas
d1_00026.insert(0, 'Dimensão', 'D1_00026')
d1_00026.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com contas dos grupos de PL com natureza diferente do PCASP')
d1_00026.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros} - Pontos: {nota}')
d1_00026


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00026,OK,Verifica a quantidade de matrizes com contas dos grupos de PL com natureza diferente do PCASP,Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0


In [29]:
d1_00026_ta

,chave,mes_referencia,tipo_matriz,valor
0,CCredora,1,MSCC,"8,130,519,439.14"
1,CCredora,2,MSCC,"8,130,517,917.80"
2,CCredora,3,MSCC,"8,130,516,396.46"
3,CCredora,4,MSCC,"8,131,437,169.02"
4,CCredora,5,MSCC,"8,133,795,782.01"
5,CCredora,6,MSCC,"8,136,172,491.60"
6,CCredora,7,MSCC,"8,138,069,990.81"
7,CCredora,8,MSCC,"8,138,071,796.02"
8,CCredora,9,MSCC,"8,138,066,948.13"
9,CCredora,10,MSCC,"8,138,065,426.79"


# D1_00027 >> Matrizes com contas contábeis com informação complementar de atributo do superávit financeiro igual a F que não tenham informação complementar de FR - fonte ou destinação de recurso - associada. Cada MSC vale 1/13 da pontuação.

In [30]:
########################### Verifica se Existe Conta F (1) sem a informação de Fonte de Recursos
condicao = (msc_consolidada['financeiro_permanente'] == 1.0) & (msc_consolidada['fonte_recursos'].isnull())

d1_00027_t = msc_consolidada.query('financeiro_permanente == 1.0 and fonte_recursos.isnull()', engine='python')
d1_00027_t = d1_00027_t.groupby(['tipo_matriz','cod_ibge', 'mes_referencia'])['valor'].sum().reset_index()

contagem = d1_00027_t.mes_referencia.unique()
erros = len(contagem)
#ptos = ((erros/13)-1)*-1
nota = (100/13) * (13-erros)
nota = round(nota)/100

if condicao.any():
    resposta_d1_00027 = 'ERRO'
else:
    resposta_d1_00027 = 'OK'
    
d1_00027 = pd.DataFrame([resposta_d1_00027], columns=['Resposta'])
# Insere novas colunas
d1_00027.insert(0, 'Dimensão', 'D1_00027')
d1_00027.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com contas F e sem Fonte de Recursos')
d1_00027.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros} - Pontos: {nota}')
d1_00027

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00027,OK,Verifica a quantidade de matrizes com contas F e sem Fonte de Recursos,Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0


In [31]:
d1_00027_t

,tipo_matriz,cod_ibge,mes_referencia,valor


In [32]:
# d1_27_sem_contas_f = msc_consolidada.query('financeiro_permanente == 1.0 and fonte_recursos.isnull()')
# d1_27_sem_contas_f.to_excel('d1_27_sem_contas_f.xlsx')
# d1_27_sem_contas_f

# D1_00028 >> Valores nas Matrizes de Saldos Contábeis, diferentes de zero, em todas as classes de contas da MSC: patrimonial (1, 2, 3 e 4), orçamentária (5 e 6) e controle (7 e 8). Cada MSC vale 1/13 da pontuação.

In [33]:
d1_00028_t = msc_base.groupby(['Grupo_Contas', 'mes_referencia', 'tipo_valor', 'tipo_matriz'])['valor'].sum().reset_index()
d1_00028_t = d1_00028_t.query('(tipo_valor == "ending_balance" and mes_referencia == 1) or \
                  (tipo_valor == "beginning_balance" and mes_referencia != 1)')

# Condição para selecionar as linhas onde 'mes_referencia' é 12 e 'tipo_matriz' é 'MSCE'
condicao_alt = (d1_00028_t['mes_referencia'] == 12) & (d1_00028_t['tipo_matriz'] == 'MSCE')

# Substituir o valor de 'mes_referencia' para 13 nas linhas selecionadas
d1_00028_t.loc[condicao_alt, 'mes_referencia'] = 13
d1_00028_t = d1_00028_t.reset_index(drop=True)

contagem_total = d1_00028_t.value_counts().sum()

contagem_meses_a = d1_00028_t['mes_referencia'].unique()
contagem_meses = len(contagem_meses_a)

condicao = (contagem_total/erros)

# Inicializar o contador de erros
erros = 0

for mes in contagem:
    # Filtrar o DataFrame para o mês atual
    df_mes = d1_00028_t[d1_00028_t['mes_referencia'] == mes]

    # Verificar se há algum grupo de contas com valor menor que zero neste mês
    contagem_negativa = df_mes[df_mes['valor'] < 0]
    erros_mes = contagem_negativa['mes_referencia'].unique()
    quantidade_erros_mes = len(erros_mes)
    # Adicionar a quantidade de erros do mês ao total
    erros += quantidade_erros_mes

nota = (100/13) * (13-erros)
nota = round(nota)/100

# # Verifica se todas as contas esperadas estão presentes
if condicao < 8 or erros != 0:
    resposta_d1_00028 = 'ERRO'
else:
    resposta_d1_00028 = 'OK'
    
d1_00028 = pd.DataFrame([resposta_d1_00028], columns=['Resposta'])
# Insere novas colunas
d1_00028.insert(0, 'Dimensão', 'D1_00028')
d1_00028.insert(2, 'Descrição da Dimensão', 'Verifica se foram enviados valores diferentes de zero em todas as classes de contas da MSC')
d1_00028.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros} - Pontos: {nota}')
d1_00028

C:\Users\mjandussi\AppData\Local\Temp\ipykernel_16960\4169932068.py:17: RuntimeWarning: divide by zero encountered in scalar divide
  condicao = (contagem_total/erros)


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00028,OK,Verifica se foram enviados valores diferentes de zero em todas as classes de contas da MSC,Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0


In [34]:
d1_00028_t

,Grupo_Contas,mes_referencia,tipo_valor,tipo_matriz,valor
0,1,1,ending_balance,MSCC,"137,124,637,698.22"
1,1,2,beginning_balance,MSCC,"137,124,637,698.22"
2,1,3,beginning_balance,MSCC,"143,171,713,769.95"
3,1,4,beginning_balance,MSCC,"143,114,961,417.14"
4,1,5,beginning_balance,MSCC,"141,689,265,073.36"
...,...,...,...,...,...
99,8,9,beginning_balance,MSCC,"1,558,665,456,764.26"
100,8,10,beginning_balance,MSCC,"1,626,972,306,296.81"
101,8,11,beginning_balance,MSCC,"1,693,883,522,807.10"
102,8,12,beginning_balance,MSCC,"1,764,648,132,740.93"


# D1_00029 >> MSC com contas dos grupos 6211, 6212, 6213 cujos registros não apresentam detalhamento de fonte ou destinação de recurso. Cada MSC vale 1/13 da pontuação

In [35]:
condicao = (msc_consolidada['conta_contabil'].str.startswith('621') | msc_consolidada['conta_contabil'].str.startswith('6212') | msc_consolidada['conta_contabil'].str.startswith('6213')) \
                & (msc_consolidada['fonte_recursos'].isnull())

d1_00029_t = msc_consolidada.query('(conta_contabil.str.startswith("6211") or conta_contabil.str.startswith("6212") or conta_contabil.str.startswith("6213")) and fonte_recursos.isnull()', engine='python')
d1_00029_t = d1_00029_t.groupby(['tipo_matriz','cod_ibge', 'mes_referencia'])['valor'].sum().reset_index()

contagem = d1_00029_t.mes_referencia.unique()
erros = len(contagem)   
nota = (100/13) * (13-erros)
nota = round(nota)/100

if condicao.any():
    resposta_d1_00029 = 'ERRO'
else:
    resposta_d1_00029 = 'OK'
    
d1_00029 = pd.DataFrame([resposta_d1_00029], columns=['Resposta'])
# Insere novas colunas
d1_00029.insert(0, 'Dimensão', 'D1_00029')
d1_00029.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com contas de Receita sem FR')
d1_00029.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros} - Pontos: {nota}')
d1_00029

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00029,OK,Verifica a quantidade de matrizes com contas de Receita sem FR,Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0


In [36]:
d1_00029_t

,tipo_matriz,cod_ibge,mes_referencia,valor


# D1_00030 >> Envio de MSCs com contas de receita orçamentária e deduções sem o detalhamento de natureza de receita

In [37]:
condicao = (msc_consolidada['conta_contabil'].str.startswith('6211') | msc_consolidada['conta_contabil'].str.startswith('6212') | msc_consolidada['conta_contabil'].str.startswith('6213')) \
                & (msc_consolidada['natureza_receita'].isnull())

d1_00030_t = msc_consolidada.query('(conta_contabil.str.startswith("6211") or conta_contabil.str.startswith("6212") or conta_contabil.str.startswith("6213")) and natureza_receita.isnull()', engine='python')
d1_00030_t = d1_00030_t.groupby(['tipo_matriz', 'mes_referencia'])['valor'].sum().reset_index()

contagem = d1_00030_t.mes_referencia.unique()
erros = len(contagem)   
nota = (100/13) * (13-erros)
nota = round(nota)/100

if condicao.any():
    resposta_d1_00030 = 'ERRO'
else:
    resposta_d1_00030 = 'OK'
    
d1_00030 = pd.DataFrame([resposta_d1_00030], columns=['Resposta'])
# Insere novas colunas
d1_00030.insert(0, 'Dimensão', 'D1_00030')
d1_00030.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com contas de Receita sem NR')
d1_00030.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros} - Pontos: {nota}')
d1_00030

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00030,OK,Verifica a quantidade de matrizes com contas de Receita sem NR,Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0


In [38]:
d1_00030_t

,tipo_matriz,mes_referencia,valor


# D1_00031 >> Envio de MSCs com contas de despesa orçamentária sem o detalhamento de natureza de despesa

In [39]:
condicao = (msc_consolidada['conta_contabil'].str.startswith('62213')) & msc_consolidada['natureza_despesa'].isnull()

d1_00031_t = msc_consolidada.query('conta_contabil.str.startswith("62213") and natureza_despesa.isnull()', engine='python')
d1_00031_t = d1_00031_t.groupby(['tipo_matriz', 'mes_referencia'])['valor'].sum().reset_index()

contagem = d1_00031_t.mes_referencia.unique()
erros = len(contagem)   
nota = (100/13) * (13-erros)
nota = round(nota)/100

if condicao.any():
    resposta_d1_00031 = 'ERRO'
else:
    resposta_d1_00031 = 'OK'
    
d1_00031 = pd.DataFrame([resposta_d1_00031], columns=['Resposta'])
# Insere novas colunas
d1_00031.insert(0, 'Dimensão', 'D1_00031')
d1_00031.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com contas de Despesa sem ND')
d1_00031.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros} - Pontos: {nota}')
d1_00031

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00031,OK,Verifica a quantidade de matrizes com contas de Despesa sem ND,Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0


In [40]:
d1_00031_t

,tipo_matriz,mes_referencia,valor


# D1_00032 >> Envio de MSCs com contas de despesa orçamentária sem o detalhamento de função/subfunção

In [41]:
msc_consolidada['funcao_subfuncao'] = msc_consolidada['funcao'] + msc_consolidada['subfuncao']

condicao = (msc_consolidada['conta_contabil'].str.startswith('62213')) & msc_consolidada['funcao_subfuncao'].isnull()

d1_00032_t = msc_consolidada.query('conta_contabil.str.startswith("62213") and funcao_subfuncao.isnull()', engine='python')
d1_00032_t = d1_00032_t.groupby(['tipo_matriz', 'mes_referencia'])['valor'].sum().reset_index()

contagem = d1_00032_t.mes_referencia.unique()
erros = len(contagem)   
nota = (100/13) * (13-erros)
nota = round(nota)/100

if condicao.any():
    resposta_d1_00032 = 'ERRO'
else:
    resposta_d1_00032 = 'OK'
    
d1_00032 = pd.DataFrame([resposta_d1_00032], columns=['Resposta'])
# Insere novas colunas
d1_00032.insert(0, 'Dimensão', 'D1_00032')
d1_00032.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com contas de Despesa sem Função ou Subfunção')
d1_00032.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros} - Pontos: {nota}')
d1_00032

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00032,OK,Verifica a quantidade de matrizes com contas de Despesa sem Função ou Subfunção,Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0


In [42]:
d1_00032_t

,tipo_matriz,mes_referencia,valor


# D1_00033 >> Envio de MSCs com contas de despesa orçamentária sem o detalhamento de fonte ou destinação de recursos

In [43]:
condicao = (msc_consolidada['conta_contabil'].str.startswith('62213')) & msc_consolidada['fonte_recursos'].isnull()

d1_00033_t = msc_consolidada.query('conta_contabil.str.startswith("62213") and fonte_recursos.isnull()', engine='python')
d1_00033_t = d1_00033_t.groupby(['tipo_matriz', 'mes_referencia'])['valor'].sum().reset_index()

contagem = d1_00033_t.mes_referencia.unique()
erros = len(contagem)   
nota = (100/13) * (13-erros)
nota = round(nota)/100

if condicao.any():
    resposta_d1_00033 = 'ERRO'
else:
    resposta_d1_00033 = 'OK'
    
# Cria um DataFrame com a resposta
d1_00033 = pd.DataFrame([resposta_d1_00033], columns=['Resposta'])
# Insere novas colunas
d1_00033.insert(0, 'Dimensão', 'D1_00033')
d1_00033.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de MSC com contas dos grupos 62213 cujos registros não apresentam a informação complementar de FR')
d1_00033.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros} - Pontos: {nota}')
d1_00033

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00033,OK,Verifica a quantidade de MSC com contas dos grupos 62213 cujos registros não apresentam a informação complementar de FR,Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0


In [44]:
d1_00033_t

,tipo_matriz,mes_referencia,valor


<hr style="border: none; border-top: 4px solid gray;">
<p style="color: lightgreen; font-size: 26px;"><strong>NOVAS 2023</strong></p>
<hr style="border: none; border-top: 4px solid gray;">


# D1_00034 >> Envio de MSC com contas de VPD com saldo invertido incorretamente

In [45]:
filtro_1 = pc_estendido[pc_estendido['CONTA'].str.match(r"^(311|312|313|321|322|323|331|332|333|351|352|353|361|362|363)")]
vpd_pcasp = filtro_1.groupby(['CONTA', 'TÍTULO.1', 'NATUREZA DO SALDO', 'STATUS']).sum().reset_index()

vpd_msc = msc_consolidada_e[msc_consolidada_e['conta_contabil'].str.match(r"^(311|312|313|321|322|323|331|332|333|351|352|353|361|362|363)")]
vpd_msc = vpd_msc.groupby(['conta_contabil', 'natureza_conta', 'mes_referencia'])['valor'].sum().reset_index()
# Renomeando 'coluna1' para 'nova_coluna1'
vpd_msc.rename(columns={'conta_contabil': 'CONTA', 'natureza_conta': 'NATUREZA_VALOR', 'valor': 'VALOR'}, inplace=True)

### MERGE
erro_vpd = vpd_msc.merge(vpd_pcasp, on='CONTA', how="left")

erro_vpd = erro_vpd[(erro_vpd['VALOR'] != 0)]

erro_vpd = erro_vpd[['CONTA', 'NATUREZA_VALOR', 'NATUREZA DO SALDO', 'TÍTULO.1', 'mes_referencia', 'VALOR']]

erro_vpd['chave'] = erro_vpd['NATUREZA_VALOR'] + erro_vpd['NATUREZA DO SALDO']

condicao = erro_vpd.query('chave == "CDevedora" or chave == "DCredora"').value_counts().sum()

d1_00034_ta = erro_vpd.query('chave == "CDevedora" or chave == "DCredora"')

d1_00034_t = erro_vpd.groupby(['chave', 'mes_referencia'])['VALOR'].sum().reset_index()

contagem = d1_00034_ta.mes_referencia.unique()
erros = len(contagem)   
nota = (100/12) * (12-erros)
nota = round(nota)/100

# Verifica se a condição é maior que zero
if condicao > 0:
    resposta_d1_00034 = 'ERRO'
else:
    resposta_d1_00034 = 'OK'

# Cria um DataFrame com a resposta
d1_00034 = pd.DataFrame([resposta_d1_00034], columns=['Resposta'])
# Insere novas colunas
d1_00034.insert(0, 'Dimensão', 'D1_00034')
d1_00034.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com contas dos grupos de VPD com natureza diferente do PCASP')
d1_00034.insert(3, 'OBS', f'Cada MSC vale 1/12 - Erros: {erros} - Pontos: {nota}')

d1_00034

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00034,OK,Verifica a quantidade de matrizes com contas dos grupos de VPD com natureza diferente do PCASP,Cada MSC vale 1/12 - Erros: 0 - Pontos: 1.0


In [46]:
d1_00034_t

,chave,mes_referencia,VALOR
0,DDevedora,1,"14,489,866,699.55"
1,DDevedora,2,"31,761,149,770.93"
2,DDevedora,3,"48,179,912,843.46"
3,DDevedora,4,"69,067,602,068.90"
4,DDevedora,5,"86,726,256,336.14"
5,DDevedora,6,"105,739,507,096.32"
6,DDevedora,7,"124,051,484,208.28"
7,DDevedora,8,"144,320,166,891.36"
8,DDevedora,9,"166,877,318,292.56"
9,DDevedora,10,"182,861,859,657.78"


# D1_00035 >> Envio de MSC com contas de VPA com saldo invertido incorretamente

In [47]:
filtro_1 = pc_estendido[pc_estendido['CONTA'].str.match(r"^(411|412|413|421|422|423|424)")]
vpa_pcasp = filtro_1.groupby(['CONTA', 'TÍTULO.1', 'NATUREZA DO SALDO', 'STATUS']).sum().reset_index()

vpa_msc = msc_consolidada_e[msc_consolidada_e['conta_contabil'].str.match(r"^(411|412|413|421|422|423|424)")]
vpa_msc = vpa_msc.groupby(['conta_contabil', 'natureza_conta', 'mes_referencia'])['valor'].sum().reset_index()


# Renomeando 'coluna1' para 'nova_coluna1'
vpa_msc.rename(columns={'conta_contabil': 'CONTA', 'natureza_conta': 'NATUREZA_VALOR', 'valor': 'VALOR'}, inplace=True)

### MERGE
erro_vpa = vpa_msc.merge(vpa_pcasp, on='CONTA', how="left")

erro_vpa = erro_vpa[(erro_vpa['VALOR'] != 0)]

erro_vpa = erro_vpa[['CONTA', 'NATUREZA_VALOR', 'NATUREZA DO SALDO', 'TÍTULO.1', 'mes_referencia', 'VALOR']]

erro_vpa['chave'] = erro_vpa['NATUREZA_VALOR'] + erro_vpa['NATUREZA DO SALDO']

condicao = erro_vpa.query('chave == "CDevedora" or chave == "DCredora"').value_counts().sum()

d1_00035_t = erro_vpa.groupby(['chave', 'mes_referencia'])['VALOR'].sum().reset_index()

d1_00035_ta = d1_00035_t.query('chave == "CDevedora" or chave == "DCredora"')

contagem = d1_00035_ta.mes_referencia.unique()
erros = len(contagem)   
nota = (100/12) * (12-erros)
nota = round(nota)/100

# Verifica se a condição é maior que zero
if condicao > 0:
    resposta_d1_00035 = 'ERRO'
else:
    resposta_d1_00035 = 'OK'

# Cria um DataFrame com a resposta
d1_00035 = pd.DataFrame([resposta_d1_00035], columns=['Resposta'])
# Insere novas colunas
d1_00035.insert(0, 'Dimensão', 'D1_00035')
d1_00035.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com contas dos grupos de VPA com natureza diferente do PCASP')
d1_00035.insert(3, 'OBS', f'Cada MSC vale 1/12 - Erros: {erros} - Pontos: {nota}')

d1_00035

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00035,OK,Verifica a quantidade de matrizes com contas dos grupos de VPA com natureza diferente do PCASP,Cada MSC vale 1/12 - Erros: 0 - Pontos: 1.0


In [48]:
d1_00035_t

,chave,mes_referencia,VALOR
0,CCredora,1,"8,231,562,699.53"
1,CCredora,2,"14,841,700,896.22"
2,CCredora,3,"21,453,571,326.80"
3,CCredora,4,"28,258,731,217.93"
4,CCredora,5,"34,883,141,381.26"
5,CCredora,6,"41,479,947,702.19"
6,CCredora,7,"48,197,699,705.66"
7,CCredora,8,"54,788,657,660.07"
8,CCredora,9,"61,237,911,765.71"
9,CCredora,10,"67,779,382,794.85"


# D1_00036 >> Envio de MSC encerramento com saldo final nas contas VPA e VPD

In [49]:
comeca_com_3 = msc_encerr['conta_contabil'].str.startswith('3')
comeca_com_4 = msc_encerr['conta_contabil'].str.startswith('4')
# Combinar as condições com o operador OR (|)
condicao_3_ou_4 = comeca_com_3 | comeca_com_4
# Filtrar o DataFrame com base na condição
d1_00036_t = msc_encerr[condicao_3_ou_4]
d1_00036_t = d1_00036_t.query('natureza_conta == "ending_balance"')

#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
condicao = d1_00036_t['valor'] == 0  
erros = (d1_00036_t['valor'] != 0).sum()   

contagem = d1_00036_t.mes_referencia.unique()
erros = len(contagem)   
nota = (100/12) * (12-erros)
nota = round(nota)/100

if condicao.all():
    resposta_d1_00036 = 'OK'
else:
    resposta_d1_00036 = 'ERRO'

d1_00036 = pd.DataFrame([resposta_d1_00036], columns=['Resposta'])
d1_00036.insert(0, 'Dimensão', 'D1_00036')
d1_00036.insert(2, 'Descrição da Dimensão', 'Verifica se a MSC de encerramento foi encaminhada com o correto encerramento das contas de VPA e VPD')
d1_00036.insert(3, 'OBS', f'MSC de encerramento - Erros: {erros} - Pontos: {nota}')
d1_00036

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00036,OK,Verifica se a MSC de encerramento foi encaminhada com o correto encerramento das contas de VPA e VPD,MSC de encerramento - Erros: 0 - Pontos: 1.0


In [50]:
d1_00036_t

,tipo_matriz,cod_ibge,classe_conta,conta_contabil,poder_orgao,financeiro_permanente,ano_fonte_recursos,fonte_recursos,exercicio,mes_referencia,...,valor,natureza_conta,tipo_valor,complemento_fonte,funcao,subfuncao,educacao_saude,natureza_despesa,ano_inscricao,natureza_receita


<hr style="border: none; border-top: 4px solid gray;">
<p style="color: orange; font-size: 26px;"><strong>NOVAS 2024</strong></p>
<hr style="border: none; border-top: 4px solid gray;">


# D1_00037 >> Verifica se estados e municípios enviaram informações em fontes de recursos da União (de 000 a 499). Cada MSC vale 1/13 da pontuação.

In [51]:
msc_consolidada_e['fonte'] = msc_consolidada_e['fonte_recursos'].str[-3:]
msc_consolidada_e['fonte'] = pd.to_numeric(msc_consolidada_e['fonte'], errors='coerce')
d1_00037_t = msc_consolidada_e.query('fonte < 500')

contagem = d1_00037_t.mes_referencia.unique()
erros = len(contagem)   
nota = (100/12) * (12-erros)
nota = round(nota)/100

condicao = (msc_consolidada_e['fonte'] < 500).any()
    
if condicao.any():
    resposta_d1_00037 = 'ERRO'
else:
    resposta_d1_00037 = 'OK'

d1_00037 = pd.DataFrame([resposta_d1_00037], columns=['Resposta'])
d1_00037.insert(0, 'Dimensão', 'D1_00037')
d1_00037.insert(2, 'Descrição da Dimensão', 'Verifica se estados e municípios enviaram informações em fontes de recursos da União (de 000 a 499).')
d1_00037.insert(3, 'OBS', f'Cada MSC vale 1/13 da pontuação - Erros: {erros} - Pontos: {nota}')
d1_00037


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00037,OK,Verifica se estados e municípios enviaram informações em fontes de recursos da União (de 000 a 499).,Cada MSC vale 1/13 da pontuação - Erros: 0 - Pontos: 1.0


In [52]:
d1_00037_t

,tipo_matriz,cod_ibge,classe_conta,conta_contabil,poder_orgao,financeiro_permanente,ano_fonte_recursos,fonte_recursos,exercicio,mes_referencia,...,natureza_conta,tipo_valor,complemento_fonte,funcao,subfuncao,educacao_saude,natureza_despesa,ano_inscricao,natureza_receita,fonte


# D1_00038 >> Verifica a quantidade de matrizes com contas de classe 5 e 6 cujo saldo final está com natureza diferente da padrão do PCASP Estendido. Cada MSC vale 1/13 da pontuação.

In [53]:

msc_original_e_b_p_13 = pd.concat([msc_orig_e, msc_orig_encerr_b], ignore_index=True)
msc_original_e_b_p_13

,tipo_matriz,cod_ibge,classe_conta,conta_contabil,poder_orgao,financeiro_permanente,ano_fonte_recursos,fonte_recursos,exercicio,mes_referencia,...,valor,natureza_conta,tipo_valor,complemento_fonte,funcao,subfuncao,educacao_saude,natureza_despesa,ano_inscricao,natureza_receita
0,MSCC,33,1,111110100,10111,1.00,1.00,1501,2024,1,...,"4,175.81",D,ending_balance,None,NaN,NaN,NaN,NaN,NaN,NaN
1,MSCC,33,1,111110200,10111,1.00,1.00,1898,2024,1,...,"332,080,708.24",D,ending_balance,None,NaN,NaN,NaN,NaN,NaN,NaN
2,MSCC,33,1,111110602,10112,1.00,1.00,1898,2024,1,...,"1,590,722,658.31",D,ending_balance,None,NaN,NaN,NaN,NaN,NaN,NaN
3,MSCC,33,1,111110603,10112,1.00,1.00,1898,2024,1,...,"1,902,162.55",D,ending_balance,None,NaN,NaN,NaN,NaN,NaN,NaN
4,MSCC,33,1,111110604,10112,1.00,1.00,1802,2024,1,...,"16,483,555.14",D,ending_balance,None,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358249,MSCE,33,8,899000000,20211,NaN,NaN,None,2024,12,...,"2,478,787,764.59",C,beginning_balance,None,None,None,NaN,None,NaN,NaN
358250,MSCE,33,8,899000000,20212,NaN,NaN,None,2024,12,...,"4,499,518,936.22",C,beginning_balance,None,None,None,NaN,None,NaN,NaN
358251,MSCE,33,8,899000000,30390,NaN,NaN,None,2024,12,...,"30,909,522,499.28",C,beginning_balance,None,None,None,NaN,None,NaN,NaN
358252,MSCE,33,8,899000000,50511,NaN,NaN,None,2024,12,...,"4,555,453,907.64",C,beginning_balance,None,None,None,NaN,None,NaN,NaN


In [54]:
# MSC
comeca_com_5 = msc_consolidada_e['conta_contabil'].str.startswith('5')
c_5_msc = msc_consolidada_e[comeca_com_5]

comeca_com_6 = msc_consolidada_e['conta_contabil'].str.startswith('6')
c_6_msc = msc_consolidada_e[comeca_com_6]

# PCASP
comeca_com_5_p = pc_estendido['CONTA'].str.startswith('5')
c_5_pcasp = pc_estendido[comeca_com_5_p]

comeca_com_6_p = pc_estendido['CONTA'].str.startswith('6')
c_6_pcasp = pc_estendido[comeca_com_6_p]


# Renomeando 'coluna1' para 'nova_coluna1'
c_5_msc.rename(columns={'conta_contabil': 'CONTA', 'natureza_conta': 'NATUREZA_VALOR', 'valor': 'VALOR'}, inplace=True)
c_6_msc.rename(columns={'conta_contabil': 'CONTA', 'natureza_conta': 'NATUREZA_VALOR', 'valor': 'VALOR'}, inplace=True)

### MERGE
erro_5 = c_5_msc.merge(c_5_pcasp, on='CONTA', how="left")
erro_6 = c_6_msc.merge(c_6_pcasp, on='CONTA', how="left")

# Retirando os valores iguais a zero
erro_5 = erro_5[(erro_5['VALOR'] != 0)]
erro_6 = erro_6[(erro_6['VALOR'] != 0)]

erro_5 = erro_5[['CONTA', 'NATUREZA_VALOR', 'NATUREZA DO SALDO', 'TÍTULO.1', 'mes_referencia', 'tipo_matriz', 'VALOR']]
erro_6 = erro_6[['CONTA', 'NATUREZA_VALOR', 'NATUREZA DO SALDO', 'TÍTULO.1', 'mes_referencia', 'tipo_matriz', 'VALOR']]

erro_5['chave'] = erro_5['NATUREZA_VALOR'] + erro_5['NATUREZA DO SALDO']
erro_6['chave'] = erro_6['NATUREZA_VALOR'] + erro_6['NATUREZA DO SALDO']

condicao_5 = erro_5.query('chave == "CDevedora" or chave == "DCredora"').value_counts().sum()
condicao_6 = erro_6.query('chave == "CDevedora" or chave == "DCredora"').value_counts().sum()

d1_00038_t_5 = erro_5.groupby(['chave', 'mes_referencia', 'tipo_matriz'])['VALOR'].sum().reset_index()
d1_00038_t_6 = erro_6.groupby(['chave', 'mes_referencia', 'tipo_matriz'])['VALOR'].sum().reset_index()

d1_00038_ta_5 = d1_00038_t_5.query('chave == "CDevedora" or chave == "DCredora"')
d1_00038_ta_6 = d1_00038_t_6.query('chave == "CDevedora" or chave == "DCredora"')

d1_00038_ta = pd.concat([d1_00038_ta_5, d1_00038_ta_6])

# Condição para selecionar as linhas onde 'mes_referencia' é 12 e 'tipo_matriz' é 'MSCE'
condicao_alt = (d1_00038_ta['mes_referencia'] == 12) & (d1_00038_ta['tipo_matriz'] == 'MSCE')

# Substituir o valor de 'mes_referencia' para 13 nas linhas selecionadas
d1_00038_ta.loc[condicao_alt, 'mes_referencia'] = 13
d1_00038_ta = d1_00038_ta.reset_index(drop=True)

contagem = d1_00038_ta.mes_referencia.unique()
erros = len(contagem)   
nota = (100/13) * (13-erros)
nota = round(nota)/100

# Soma das duas condições
condicao_total = condicao_5 + condicao_6

# Verifica se é maior que zero
if condicao_total > 0:
    resposta_d1_00038 = 'ERRO'
else:
    resposta_d1_00038 = 'OK'

# Cria um DataFrame com a resposta
d1_00038 = pd.DataFrame([resposta_d1_00038], columns=['Resposta'])
# Insere novas colunas
d1_00038.insert(0, 'Dimensão', 'D1_00038')
d1_00038.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com contas de classe 5 e 6 cujo saldo final está com natureza diferente da padrão do PCASP Estendido')
d1_00038.insert(3, 'OBS', f'Cada MSC vale 1/13 da pontuação - Erros: {erros} - Pontos: {nota}')
d1_00038

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00038,ERRO,Verifica a quantidade de matrizes com contas de classe 5 e 6 cujo saldo final está com natureza diferente da padrão do PCASP Estendido,Cada MSC vale 1/13 da pontuação - Erros: 13 - Pontos: 0.0


In [55]:
d1_00038_ta

,chave,mes_referencia,tipo_matriz,VALOR
0,CDevedora,1,MSCC,"-3,196,838.66"
1,CDevedora,2,MSCC,"-3,196,838.66"
2,CDevedora,3,MSCC,"-3,196,838.66"
3,CDevedora,4,MSCC,"-3,196,838.66"
4,CDevedora,5,MSCC,"-3,196,838.66"
5,CDevedora,6,MSCC,"-3,196,838.66"
6,CDevedora,7,MSCC,"-3,196,838.66"
7,CDevedora,8,MSCC,"-3,196,838.66"
8,CDevedora,9,MSCC,"-3,196,838.66"
9,CDevedora,10,MSCC,"-3,196,838.66"


In [56]:
d1_00038_t_5 = erro_5.groupby(['chave', 'mes_referencia', 'CONTA'])['VALOR'].sum().reset_index()
d1_00035_ta_5 = d1_00038_t_5.query('chave == "CDevedora" or chave == "DCredora"')
d1_00035_ta_5.CONTA.unique()

array(['532100000', '521110000', '522190209'], dtype=object)

In [57]:
#erro_5.query('CONTA == "521110000"')

In [58]:
d1_00038_t_6 = erro_6.groupby(['chave', 'mes_referencia', 'CONTA'])['VALOR'].sum().reset_index()
d1_00035_ta_6 = d1_00038_t_6.query('chave == "CDevedora" or chave == "DCredora"')
d1_00035_ta_6.CONTA.unique()

array(['632100000'], dtype=object)

In [59]:
d1 = pd.concat([d1_00017, d1_00018, d1_00019, d1_00020, d1_00021, d1_00022, d1_00023, d1_00024, d1_00025, d1_00026, d1_00027, d1_00028,
                  d1_00029, d1_00030, d1_00031, d1_00032, d1_00033, d1_00034, d1_00035, d1_00036, d1_00037, d1_00038])
# Remover a coluna de índice
d1.reset_index(drop=True, inplace=True)
# Aplicar a função de estilo
#styled_d1_m = d1_m.style.applymap(colorir_celulas, subset=['Resposta'])
print(f'Dados Extraídos da API - Ente: {ente} - Ano: {ano}')
d1

Dados Extraídos da API - Ente: 33 - Ano: 2024


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00017,OK,Verifica a quantidade de matrizes com valores negativos,Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0
1,D1_00018,OK,Verifica a quantidade de matrizes com diferenças na movimentação (SI+ MOV <> SF),Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0
2,D1_00019,OK,Verifica a quantidade de matrizes com códigos de Poderes incorretos,Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0
3,D1_00020,OK,Verifica a quantidade de matrizes com SI <> do SF da MSC do mês anterior,Cada MSC vale 1/12 - Erros: 0 - Pontos: 1.0
4,D1_00021,OK,Verifica a quantidade de matrizes com contas dos grupos de ATIVO com natureza diferente do PCASP,Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0
5,D1_00022,OK,Verifica a quantidade de matrizes com todos os códigos de poder/órgão informados,Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0
6,D1_00023,OK,Verifica a quantidade matrizes com dados do poder Executivo repetida em mais de um mês,Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0
7,D1_00024,OK,Verifica a quantidade matrizes com dados do poder Legislativo repetida em mais de um mês,Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0
8,D1_00025,ERRO,Verifica a quantidade de matrizes com contas dos grupos de PASSIVO com natureza diferente do PCASP,Cada MSC vale 1/13 - Erros: 1 - Pontos: 0.92
9,D1_00026,OK,Verifica a quantidade de matrizes com contas dos grupos de PL com natureza diferente do PCASP,Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0


In [60]:
d1.query('Resposta == "ERRO"')

,Dimensão,Resposta,Descrição da Dimensão,OBS
8,D1_00025,ERRO,Verifica a quantidade de matrizes com contas dos grupos de PASSIVO com natureza diferente do PCASP,Cada MSC vale 1/13 - Erros: 1 - Pontos: 0.92
21,D1_00038,ERRO,Verifica a quantidade de matrizes com contas de classe 5 e 6 cujo saldo final está com natureza diferente da padrão do PCASP Estendido,Cada MSC vale 1/13 da pontuação - Erros: 13 - Pontos: 0.0


In [61]:
contagem_total_dim = d1.Dimensão.unique()
contagem_total_dim = len(contagem_total_dim)
contagem_total_dim

22

In [62]:
total_dim = 20
dim_falta = total_dim - contagem_total_dim
dim_falta

-2

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

# D2_00002

In [63]:
#D2_00002_VPD FUNDEB >> VERIFICAR 3.5.2.2.4.00.00 - Transferências ao FUNDEB - Inter OFSS - Estado
vpd_fundeb = df_dca_hi.query('cod_conta == "P3.5.2.2.4.00.00"')
vpd_fundeb['dimensao'] = 'D2_00002_VPD FUNDEB'
vpd_fundeb = vpd_fundeb.groupby('dimensao').agg({'valor': 'sum'})
d2_00002_t = vpd_fundeb.reset_index()

#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
condicao = d2_00002_t['valor'] > 0

if condicao.any():
    resposta_d2_00002 = 'OK'
else:
    resposta_d2_00002 = 'ERRO'

d2_00002 = pd.DataFrame([resposta_d2_00002], columns=['Resposta'])
d2_00002.insert(0, 'Dimensão', 'D2_00002')
d2_00002.insert(2, 'Descrição da Dimensão', 'Verifica se foi informado o valor de VPD do FUNDEB')
d2_00002.insert(3, 'OBS', 'Anexo I-HI da DCA')
d2_00002

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00002,OK,Verifica se foi informado o valor de VPD do FUNDEB,Anexo I-HI da DCA


In [64]:
d2_00002_t

,dimensao,valor
0,D2_00002_VPD FUNDEB,"10,659,629,456.79"


# D2_00003

In [65]:
#D2_00003_Deduções FUNDEB
dedu_fundeb = df_dca_c.query('coluna == "Deduções - FUNDEB" & cod_conta == "TotalReceitas"')
dedu_fundeb['dimensao'] = 'D2_00003_Deduções FUNDEB'
dedu_fundeb = dedu_fundeb.filter(items=['dimensao', 'valor'])
d2_00003_t = dedu_fundeb.reset_index()

#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
condicao = d2_00003_t['valor'] < 0   #inverteu, pois o sinal é negativo

if condicao.any():
    resposta_d2_00003 = 'OK'
else:
    resposta_d2_00003 = 'ERRO'

d2_00003 = pd.DataFrame([resposta_d2_00003], columns=['Resposta'])
d2_00003.insert(0, 'Dimensão', 'D2_00003')
d2_00003.insert(2, 'Descrição da Dimensão', 'Verifica se foi informado o valor de Deduções de Receitas para formação do FUNDEB')
d2_00003.insert(3, 'OBS', 'Anexo I-C da DCA')
d2_00003

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00003,OK,Verifica se foi informado o valor de Deduções de Receitas para formação do FUNDEB,Anexo I-C da DCA


In [66]:
d2_00003_t

,index,dimensao,valor
0,322,D2_00003_Deduções FUNDEB,"-10,659,629,456.79"


# D2_00004

In [67]:
#D2_00004_Receitas FUNDEB
#rec_fundeb = df_dca_c.query('coluna == "Receitas Brutas Realizadas" & cod_conta == "RO1.7.5.8.01.0.0"') # Em 2021
rec_fundeb = df_dca_c.query('coluna == "Receitas Brutas Realizadas" & cod_conta == "RO1.7.5.1.00.0.0"') # Em 2022
rec_fundeb['dimensao'] = 'D2_00004_Receitas FUNDEB'
rec_fundeb = rec_fundeb.filter(items=['dimensao', 'valor'])
d2_00004_t = rec_fundeb.reset_index()

#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
condicao = d2_00004_t['valor'] > 0  

if condicao.any():
    resposta_d2_00004 = 'OK'
else:
    resposta_d2_00004 = 'ERRO'

d2_00004 = pd.DataFrame([resposta_d2_00004], columns=['Resposta'])
d2_00004.insert(0, 'Dimensão', 'D2_00004')
d2_00004.insert(2, 'Descrição da Dimensão', 'Verifica se foi informado o valor de Receitas Orçamentárias do FUNDEB')
d2_00004.insert(3, 'OBS', 'Anexo I-C da DCA')
d2_00004

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00004,OK,Verifica se foi informado o valor de Receitas Orçamentárias do FUNDEB,Anexo I-C da DCA


In [68]:
d2_00004_t

,index,dimensao,valor
0,201,D2_00004_Receitas FUNDEB,"4,080,033,051.89"


# D2_00005

In [69]:
#D2_00005_Obrigações Patronais
obrig_patr_emp = df_dca_d.query('coluna == "Despesas Empenhadas" and cod_conta == "DO3.1.90.13.00.00"')
obrig_patr_emp['dimensao'] = 'D2_00005_Despesas Empenhadas com Obrigações Patronais'
obrig_patr_emp = obrig_patr_emp.groupby(['dimensao', 'conta']).agg({'valor': 'sum'})
obrig_patr_emp = obrig_patr_emp.reset_index()

obrig_patr_liq = df_dca_d.query('coluna == "Despesas Liquidadas" and cod_conta == "DO3.1.90.13.00.00"')
obrig_patr_liq['dimensao'] = 'D2_00005_Despesas Liquidadas com Obrigações Patronais'
obrig_patr_liq = obrig_patr_liq.groupby(['dimensao', 'conta']).agg({'valor': 'sum'})
obrig_patr_liq = obrig_patr_liq.reset_index()

obrig_patr_pagas = df_dca_d.query('coluna == "Despesas Pagas" and cod_conta == "DO3.1.90.13.00.00"')
obrig_patr_pagas['dimensao'] = 'D2_00005_Despesas Pagas com Obrigações Patronais'
obrig_patr_pagas = obrig_patr_pagas.groupby(['dimensao', 'conta']).agg({'valor': 'sum'})
obrig_patr_pagas = obrig_patr_pagas.reset_index()

#D2_00005_Obrigações Patronais INTRA
obrig_patr_intra_emp = df_dca_d.query('coluna == "Despesas Empenhadas" and cod_conta == "DI3.1.91.13.00.00"')
obrig_patr_intra_emp['dimensao'] = 'D2_00005_Despesas Empenhadas com Obrigações Patronais INTRA'
obrig_patr_intra_emp = obrig_patr_intra_emp.groupby(['dimensao', 'conta']).agg({'valor': 'sum'})
obrig_patr_intra_emp = obrig_patr_intra_emp.reset_index()

obrig_patr_intra_liq = df_dca_d.query('coluna == "Despesas Liquidadas" and cod_conta == "DI3.1.91.13.00.00"')
obrig_patr_intra_liq['dimensao'] = 'D2_00005_Despesas Liquidadas com Obrigações Patronais INTRA'
obrig_patr_intra_liq = obrig_patr_intra_liq.groupby(['dimensao', 'conta']).agg({'valor': 'sum'})
obrig_patr_intra_liq = obrig_patr_intra_liq.reset_index()

obrig_patr_intra_pagas = df_dca_d.query('coluna == "Despesas Pagas" and cod_conta == "DI3.1.91.13.00.00"')
obrig_patr_intra_pagas['dimensao'] = 'D2_00005_Despesas Pagas com Obrigações Patronais INTRA'
obrig_patr_intra_pagas = obrig_patr_intra_pagas.groupby(['dimensao', 'conta']).agg({'valor': 'sum'})
obrig_patr_intra_pagas = obrig_patr_intra_pagas.reset_index()

d2_00005_t = pd.concat([obrig_patr_emp, obrig_patr_liq, obrig_patr_pagas, obrig_patr_intra_emp, obrig_patr_intra_liq, obrig_patr_intra_pagas])

#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
condicao = d2_00005_t['valor'] > 0  

if condicao.any():
    resposta_d2_00005 = 'OK'
else:
    resposta_d2_00005 = 'ERRO'

d2_00005 = pd.DataFrame([resposta_d2_00005], columns=['Resposta'])
d2_00005.insert(0, 'Dimensão', 'D2_00005')
d2_00005.insert(2, 'Descrição da Dimensão', 'Verifica se foi informado o valor de Despesas Orçamentárias com Encargos Patronais')
d2_00005.insert(3, 'OBS', 'Anexo I-D da DCA')
d2_00005

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00005,OK,Verifica se foi informado o valor de Despesas Orçamentárias com Encargos Patronais,Anexo I-D da DCA


In [70]:
d2_00005_t

,dimensao,conta,valor
0,D2_00005_Despesas Empenhadas com Obrigações Patronais,3.1.90.13.00 - Obrigações Patronais,"469,733,077.51"
0,D2_00005_Despesas Liquidadas com Obrigações Patronais,3.1.90.13.00 - Obrigações Patronais,"468,624,287.43"
0,D2_00005_Despesas Pagas com Obrigações Patronais,3.1.90.13.00 - Obrigações Patronais,"434,494,062.57"
0,D2_00005_Despesas Empenhadas com Obrigações Patronais INTRA,3.1.91.13.00 - Contribuições Patronais,"3,583,660,712.33"
0,D2_00005_Despesas Liquidadas com Obrigações Patronais INTRA,3.1.91.13.00 - Contribuições Patronais,"3,582,413,755.63"
0,D2_00005_Despesas Pagas com Obrigações Patronais INTRA,3.1.91.13.00 - Contribuições Patronais,"3,291,802,406.14"


# D2_00006

In [71]:
#D2_00006_Despesas com Pessoal
dps_pess_emp = df_dca_d.query('coluna == "Despesas Empenhadas" and cod_conta == "DO3.1.00.00.00.00"')
dps_pess_emp['dimensao'] = 'D2_00006_Despesas Empenhadas com Pessoal'
dps_pess_emp = dps_pess_emp.groupby(['dimensao', 'conta']).agg({'valor': 'sum'})
dps_pess_emp = dps_pess_emp.reset_index()

dps_pess_liq = df_dca_d.query('coluna == "Despesas Liquidadas" and cod_conta == "DO3.1.00.00.00.00"')
dps_pess_liq['dimensao'] = 'D2_00006_Despesas Liquidadas com Pessoal'
dps_pess_liq = dps_pess_liq.groupby(['dimensao', 'conta']).agg({'valor': 'sum'})
dps_pess_liq = dps_pess_liq.reset_index()

dps_pess_paga = df_dca_d.query('coluna == "Despesas Pagas" and cod_conta == "DO3.1.00.00.00.00"')
dps_pess_paga['dimensao'] = 'D2_00006_Despesas Pagas com Pessoal'
dps_pess_paga = dps_pess_paga.groupby(['dimensao', 'conta']).agg({'valor': 'sum'})
dps_pess_paga = dps_pess_paga.reset_index()

d2_00006_t = pd.concat([dps_pess_emp, dps_pess_liq, dps_pess_paga])

# Verifica a condição para dar a resposta
condicao = d2_00006_t['valor'] > 0  

if condicao.any():
    resposta_d2_00006 = 'OK'
else:
    resposta_d2_00006 = 'ERRO'

d2_00006 = pd.DataFrame([resposta_d2_00006], columns=['Resposta'])
d2_00006.insert(0, 'Dimensão', 'D2_00006')
d2_00006.insert(2, 'Descrição da Dimensão', 'Verifica se foi informado o valor de Despesas Orçamentárias com Pessoal')
d2_00006.insert(3, 'OBS', 'Anexo I-D da DCA')
d2_00006

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00006,OK,Verifica se foi informado o valor de Despesas Orçamentárias com Pessoal,Anexo I-D da DCA


In [72]:
d2_00006_t

,dimensao,conta,valor
0,D2_00006_Despesas Empenhadas com Pessoal,3.1.00.00.00 - Pessoal e Encargos Sociais,"64,662,017,177.31"
0,D2_00006_Despesas Liquidadas com Pessoal,3.1.00.00.00 - Pessoal e Encargos Sociais,"64,565,837,173.85"
0,D2_00006_Despesas Pagas com Pessoal,3.1.00.00.00 - Pessoal e Encargos Sociais,"63,796,283,005.41"


# D2_00007

In [73]:
#D2_00007_Despesas com Juros e Encargos da Dívida
dps_juros_emp = df_dca_d.query('coluna == "Despesas Empenhadas" and cod_conta == "DO3.2.00.00.00.00"')
dps_juros_emp['dimensao'] = 'D2_00007_Despesas Empenhadas com Juros e Encargos da Dívida'
dps_juros_emp = dps_juros_emp.groupby(['dimensao', 'conta']).agg({'valor': 'sum'})
dps_juros_emp = dps_juros_emp.reset_index()

dps_juros_liq = df_dca_d.query('coluna == "Despesas Liquidadas" and cod_conta == "DO3.2.00.00.00.00"')
dps_juros_liq['dimensao'] = 'D2_00007_Despesas Liquidadas com Juros e Encargos da Dívida'
dps_juros_liq = dps_juros_liq.groupby(['dimensao', 'conta']).agg({'valor': 'sum'})
dps_juros_liq = dps_juros_liq.reset_index()

dps_juros_paga = df_dca_d.query('coluna == "Despesas Pagas" and cod_conta == "DO3.2.00.00.00.00"')
dps_juros_paga['dimensao'] = 'D2_00007_Despesas Pagas com Juros e Encargos da Dívida'
dps_juros_paga = dps_juros_paga.groupby(['dimensao', 'conta']).agg({'valor': 'sum'})
dps_juros_paga = dps_juros_paga.reset_index()

#D2_00007_Despesas com Outras Despesas Correntes
dps_correntes_emp = df_dca_d.query('coluna == "Despesas Empenhadas" and cod_conta == "DO3.3.00.00.00.00"')
dps_correntes_emp['dimensao'] = 'D2_00007_Despesas Empenhadas com Outras Despesas Correntes'
dps_correntes_emp = dps_correntes_emp.groupby(['dimensao', 'conta']).agg({'valor': 'sum'})
dps_correntes_emp = dps_correntes_emp.reset_index()

dps_correntes_liq = df_dca_d.query('coluna == "Despesas Liquidadas" and cod_conta == "DO3.3.00.00.00.00"')
dps_correntes_liq['dimensao'] = 'D2_00007_Despesas Liquidadas com Outras Despesas Correntes'
dps_correntes_liq = dps_correntes_liq.groupby(['dimensao', 'conta']).agg({'valor': 'sum'})
dps_correntes_liq = dps_correntes_liq.reset_index()

dps_correntes_paga = df_dca_d.query('coluna == "Despesas Pagas" and cod_conta == "DO3.3.00.00.00.00"')
dps_correntes_paga['dimensao'] = 'D2_00007_Despesas Pagas com Outras Despesas Correntes'
dps_correntes_paga = dps_correntes_paga.groupby(['dimensao', 'conta']).agg({'valor': 'sum'})
dps_correntes_paga = dps_correntes_paga.reset_index()

d2_00007_t = pd.concat([dps_juros_emp, dps_juros_liq, dps_juros_paga, dps_correntes_emp, dps_correntes_liq, dps_correntes_paga])

#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
condicao = d2_00007_t['valor'] > 0  

if condicao.any():
    resposta_d2_00007 = 'OK'
else:
    resposta_d2_00007 = 'ERRO'

d2_00007 = pd.DataFrame([resposta_d2_00007], columns=['Resposta'])
d2_00007.insert(0, 'Dimensão', 'D2_00007')
d2_00007.insert(2, 'Descrição da Dimensão', 'Verifica se foi informado o valor Despesas Orçamentárias de Custeio')
d2_00007.insert(3, 'OBS', 'Anexo I-D da DCA')
d2_00007

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00007,OK,Verifica se foi informado o valor Despesas Orçamentárias de Custeio,Anexo I-D da DCA


In [74]:
d2_00007_t

,dimensao,conta,valor
0,D2_00007_Despesas Empenhadas com Juros e Encargos da Dívida,3.2.00.00.00 - Juros e Encargos da Dívida,"3,318,453,691.10"
0,D2_00007_Despesas Liquidadas com Juros e Encargos da Dívida,3.2.00.00.00 - Juros e Encargos da Dívida,"3,318,453,691.10"
0,D2_00007_Despesas Pagas com Juros e Encargos da Dívida,3.2.00.00.00 - Juros e Encargos da Dívida,"3,318,451,679.26"
0,D2_00007_Despesas Empenhadas com Outras Despesas Correntes,3.3.00.00.00 - Outras Despesas Correntes,"33,212,404,205.97"
0,D2_00007_Despesas Liquidadas com Outras Despesas Correntes,3.3.00.00.00 - Outras Despesas Correntes,"32,351,001,521.63"
0,D2_00007_Despesas Pagas com Outras Despesas Correntes,3.3.00.00.00 - Outras Despesas Correntes,"30,903,520,950.87"


# D2_00008

In [75]:
#D2_00008_Despesas por Função
dps_funcao = df_dca_e.query('cod_conta == "TotalDespesas" and coluna == "Despesas Empenhadas"')
dps_funcao['Funcao'] = dps_funcao['conta'].str[:2]
dps_funcao = dps_funcao.groupby('Funcao').agg({'valor': 'sum'})
dps_funcao = dps_funcao.reset_index()
dps_funcao.drop(dps_funcao.tail(2).index, inplace=True)
dps_funcao['dimensao'] = 'D2_00008_Despesas por Função'
d2_00008_t = dps_funcao.reindex(columns=['dimensao', 'Funcao', 'valor'])

#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
condicao = d2_00008_t['valor'] > 0  

if condicao.any():
    resposta_d2_00008 = 'OK'
else:
    resposta_d2_00008 = 'ERRO'

d2_00008 = pd.DataFrame([resposta_d2_00008], columns=['Resposta'])
d2_00008.insert(0, 'Dimensão', 'D2_00008')
d2_00008.insert(2, 'Descrição da Dimensão', 'Verifica se foi informado o valor Despesa Orçamentárias por Função')
d2_00008.insert(3, 'OBS', 'Anexo I-E da DCA')
d2_00008

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00008,OK,Verifica se foi informado o valor Despesa Orçamentárias por Função,Anexo I-E da DCA


In [76]:
d2_00008_t

,dimensao,Funcao,valor
0,D2_00008_Despesas por Função,01,"3,842,477,363.08"
1,D2_00008_Despesas por Função,02,"14,408,496,782.92"
2,D2_00008_Despesas por Função,03,"8,792,687,422.08"
3,D2_00008_Despesas por Função,04,"10,120,964,870.83"
4,D2_00008_Despesas por Função,06,"30,893,654,246.59"
5,D2_00008_Despesas por Função,08,"1,426,477,993.09"
6,D2_00008_Despesas por Função,09,"55,118,542,673.92"
7,D2_00008_Despesas por Função,10,"20,692,814,922.36"
8,D2_00008_Despesas por Função,11,"45,649,136.95"
9,D2_00008_Despesas por Função,12,"19,583,315,608.61"


# D2_00010

In [77]:
#D2_00010_Receitas de Transferência
rec_trasnf = df_dca_c.query('cod_conta == "RO1.7.1.0.00.0.0" or cod_conta == "RO1.7.2.0.00.0.0" or cod_conta == "RO1.7.3.0.00.0.0"')
rec_trasnf = rec_trasnf.groupby(['coluna', 'conta']).sum()
rec_trasnf = rec_trasnf.reset_index()
rec_trasnf['dimensao'] = 'D2_00010_Receitas de Transferência'
rec_trasnf = rec_trasnf.reindex(columns=['dimensao', 'conta', 'coluna', 'valor'])
rec_trasnf = rec_trasnf.sort_values(by=['conta', 'valor'], ascending=False)
rec_trasnf = rec_trasnf.groupby(['dimensao', 'conta']).agg({'valor': 'sum'})
d2_00010_t = rec_trasnf.reset_index()

#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
condicao = d2_00010_t['valor'] > 0  

if condicao.any():
    resposta_d2_00010 = 'OK'
else:
    resposta_d2_00010 = 'ERRO'

d2_00010 = pd.DataFrame([resposta_d2_00010], columns=['Resposta'])
d2_00010.insert(0, 'Dimensão', 'D2_00010')
d2_00010.insert(2, 'Descrição da Dimensão', 'Verifica se foi informado o valor das Receitas de Transferências Intergovernamentais')
d2_00010.insert(3, 'OBS', 'Anexo I-C da DCA')
d2_00010


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00010,OK,Verifica se foi informado o valor das Receitas de Transferências Intergovernamentais,Anexo I-C da DCA


In [78]:
d2_00010_t

,dimensao,conta,valor
0,D2_00010_Receitas de Transferência,1.7.1.0.00.0.0 - Transferências da União e de suas Entidades,"5,739,386,937.57"
1,D2_00010_Receitas de Transferência,1.7.2.0.00.0.0 - Transferências dos Estados e do Distrito Federal e de suas Entidades,"1,172,431.17"
2,D2_00010_Receitas de Transferência,1.7.3.0.00.0.0 - Transferências dos Municípios e de suas Entidades,"106,673,040.81"


# D2_00011

In [79]:
#D2_00011_Receitas de Tributos
rec_trib = df_dca_c.query('cod_conta == "RO1.1.0.0.00.0.0"') # Análise apenas do total do Grupo de Contas
rec_trib = rec_trib.groupby(['coluna', 'conta']).sum()
rec_trib = rec_trib.reset_index()
rec_trib['dimensao'] = 'D2_00011_Receitas de Tributos'
rec_trib = rec_trib.reindex(columns=['dimensao', 'conta', 'coluna', 'valor'])
rec_trib = rec_trib.sort_values(by=['conta', 'valor'], ascending=False)
rec_trib = rec_trib.groupby(['dimensao', 'conta']).agg({'valor': 'sum'})
d2_00011_t = rec_trib.reset_index()

#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
condicao = d2_00011_t['valor'] > 0  

if condicao.any():
    resposta_d2_00011 = 'OK'
else:
    resposta_d2_00011 = 'ERRO'

d2_00011 = pd.DataFrame([resposta_d2_00011], columns=['Resposta'])
d2_00011.insert(0, 'Dimensão', 'D2_00011')
d2_00011.insert(2, 'Descrição da Dimensão', 'Verifica se foi informado o valor das Receitas Orçamentárias Tributárias')
d2_00011.insert(3, 'OBS', 'Anexo I-C da DCA')
d2_00011

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00011,OK,Verifica se foi informado o valor das Receitas Orçamentárias Tributárias,Anexo I-C da DCA


In [80]:
d2_00011_t

,dimensao,conta,valor
0,D2_00011_Receitas de Tributos,"1.1.0.0.00.0.0 - Impostos, Taxas e Contribuições de Melhoria","50,559,729,848.71"


# D2_00012

In [81]:
d2_00012_t = df_dca_c.groupby(['cod_conta'])['valor'].sum().reset_index()
d2_00012_t = d2_00012_t[~d2_00012_t['cod_conta'].str.startswith('RO1.3.2')]

#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
d2_00012_ta = d2_00012_t[d2_00012_t['valor'] < 0]
condicao = d2_00012_ta['valor'] < 0

if condicao.any():
    resposta_d2_00012 = 'ERRO'
else:
    resposta_d2_00012 = 'OK'

d2_00012 = pd.DataFrame([resposta_d2_00012], columns=['Resposta'])
d2_00012.insert(0, 'Dimensão', 'D2_00012')
d2_00012.insert(2, 'Descrição da Dimensão', 'Verifica se o valor de deduções de receitas é superior às receitas brutas')
d2_00012.insert(3, 'OBS', 'Anexo I-C da DCA')
d2_00012

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00012,OK,Verifica se o valor de deduções de receitas é superior às receitas brutas,Anexo I-C da DCA


In [82]:
d2_00012_t

,cod_conta,valor
0,RI7.0.0.0.00.0.0,"7,816,001,453.09"
1,RI7.2.0.0.00.0.0,"3,382,016,591.11"
2,RI7.2.1.0.00.0.0,"3,382,016,591.11"
3,RI7.2.1.5.00.0.0,"3,363,761,734.63"
4,RI7.2.1.5.01.0.0,"3,363,761,734.63"
...,...,...
231,RO2.4.9.1.00.0.0,"850,959.36"
232,RO2.4.9.1.01.0.0,"850,959.36"
233,ReceitasExcetoIntraOrcamentarias,"98,296,331,940.65"
234,ReceitasIntraOrcamentarias,"7,816,122,712.75"


# D2_00013

In [83]:
# Como o Ajuste esta no 6o Dígito (99), podemos pegar até o Quitno nível (consolidação) que funciona e traz o valor liquido já com ajuste
cred_lp = df_dca_ab[df_dca_ab['cod_conta'].str.contains(r"P1\.2\.1\.1\.[1-5]\.00\.00")]
cred_lp = cred_lp.groupby(['cod_conta'])['valor'].sum().reset_index()
dif_cred_lp = cred_lp['valor'].sum()
dif_cred_lp

np.float64(25448575650.119995)

In [84]:
cred_cp_1of = df_dca_ab[df_dca_ab['cod_conta'].str.contains(r"P1\.1\.2\.[1-9]\.1\.00\.00")]
cred_cp_1of = cred_cp_1of.groupby(['cod_conta'])['valor'].sum().reset_index()
dif_cred_cp_1of = cred_cp_1of['valor'].sum()

cred_cp_2of = df_dca_ab[df_dca_ab['cod_conta'].str.contains(r"P1\.1\.2\.[1-9]\.2\.00\.00")]
cred_cp_2of = cred_cp_2of.groupby(['cod_conta'])['valor'].sum().reset_index()
dif_cred_cp_2of = cred_cp_2of['valor'].sum()

cred_cp_3of = df_dca_ab[df_dca_ab['cod_conta'].str.contains(r"P1\.1\.2\.[1-9]\.3\.00\.00")]
cred_cp_3of = cred_cp_3of.groupby(['cod_conta'])['valor'].sum().reset_index()
dif_cred_cp_3of = cred_cp_3of['valor'].sum()

cred_cp_4of = df_dca_ab[df_dca_ab['cod_conta'].str.contains(r"P1\.1\.2\.[1-9]\.4\.00\.00")]
cred_cp_4of = cred_cp_4of.groupby(['cod_conta'])['valor'].sum().reset_index()
dif_cred_cp_4of = cred_cp_4of['valor'].sum()

cred_cp_5of = df_dca_ab[df_dca_ab['cod_conta'].str.contains(r"P1\.1\.2\.[1-9]\.5\.00\.00")]
cred_cp_5of = cred_cp_5of.groupby(['cod_conta'])['valor'].sum().reset_index()
dif_cred_cp_5of = cred_cp_5of['valor'].sum()

cred_cp = pd.concat([cred_cp_1of, cred_cp_2of, cred_cp_3of, cred_cp_4of, cred_cp_5of])

# Aqui, exemplificar objetos de diferença para continuidade
diferencas = [
    dif_cred_cp_1of,
    dif_cred_cp_2of,
    dif_cred_cp_3of,
    dif_cred_cp_4of,
    dif_cred_cp_5of
]

# Condição para verificar se há algum valor negativo
condicao_negativa_cp = any(diferenca < 0 for diferenca in diferencas)

####################
# LONGO PRAZO
# Como o Ajuste esta no 6o Dígito (99), podemos pegar até o Quitno nível (consolidação) que funciona e traz o valor liquido já com ajuste
cred_lp = df_dca_ab[df_dca_ab['cod_conta'].str.contains(r"P1\.2\.1\.1\.[1-5]\.00\.00")]
cred_lp = cred_lp.groupby(['cod_conta'])['valor'].sum().reset_index()
dif_cred_lp = cred_lp['valor'].sum()

# Aqui, exemplificar objetos de diferença para continuidade
diferencas2 = [dif_cred_lp]

# Condição para verificar se há algum valor negativo
condicao_negativa_lp = any(diferenca < 0 for diferenca in diferencas2)

###############

d2_00013_t = pd.concat([cred_cp, cred_lp])

# Determina a resposta com base na condição
if condicao_negativa_cp or condicao_negativa_lp:
    resposta_d2_00013 = 'ERRO'
else:
    resposta_d2_00013 = 'OK'

# Criação do DataFrame de resultado
d2_00013 = pd.DataFrame([resposta_d2_00013], columns=['Resposta'])
d2_00013.insert(0, 'Dimensão', 'D2_00013')
d2_00013.insert(2, 'Descrição da Dimensão', 'Verifica a relação entre o ajuste para perdas dos "Créditos a curto e longo prazo" e suas contas principais. O valor dos ajustes não pode ser superior aos valores das contas originais')
d2_00013.insert(3, 'OBS', 'Anexo I-AB da DCA')
d2_00013

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00013,OK,"Verifica a relação entre o ajuste para perdas dos ""Créditos a curto e longo prazo"" e suas contas principais. O valor dos ajustes não pode ser superior aos valores das contas originais",Anexo I-AB da DCA


In [85]:
d2_00013_t


,cod_conta,valor
0,P1.1.2.1.1.00.00,"968,608,631.21"
1,P1.1.2.2.1.00.00,"585,947,497.49"
2,P1.1.2.4.1.00.00,"274,953,326.60"
3,P1.1.2.5.1.00.00,"919,460,958.00"
4,P1.1.2.6.1.00.00,"126,867,856.82"
5,P1.1.2.9.1.00.00,"-806,314,581.45"
0,P1.1.2.2.2.00.00,"210,459,159.78"
1,P1.1.2.4.2.00.00,"50,861,008.61"
0,P1.1.2.2.3.00.00,"1,438.10"
0,P1.1.2.2.5.00.00,"706,397.91"


# D2_00014

In [86]:
demais_cred_cp = df_dca_ab.query('cod_conta == "P1.1.3.0.0.00.00"')

demais_cred_lp = df_dca_ab.query('cod_conta == "P1.2.1.2.1.00.00" or cod_conta == "P1.2.1.2.2.00.00" or cod_conta == "P1.2.1.2.3.00.00" or cod_conta == "P1.2.1.2.4.00.00" or cod_conta == "P1.2.1.2.5.00.00"')

d2_00014_t = pd.concat([demais_cred_cp, demais_cred_lp])

d2_00014_t = d2_00014_t.groupby('conta').agg({'valor': 'sum'})


#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
# Verifica se existe algum valor negativo na coluna 'valor'
condicao_negativa = (d2_00014_t['valor'] < 0).any()

if condicao_negativa:
    # Se existe pelo menos um valor negativo, a resposta deve ser 'ERRO'
    resposta_d2_00014 = 'ERRO'
else:
    # Se não existem valores negativos, a resposta é 'OK'
    resposta_d2_00014 = 'OK'

d2_00014 = pd.DataFrame([resposta_d2_00014], columns=['Resposta'])
d2_00014.insert(0, 'Dimensão', 'D2_00014')
d2_00014.insert(2, 'Descrição da Dimensão', 'Verifica a relação entre o ajuste para perdas dos "Créditos a curto e longo prazo" e suas contas principais. O valor dos ajustes não pode ser superior aos valores das contas originais"')
d2_00014.insert(3, 'OBS', 'Anexo I-AB da DCA')
d2_00014

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00014,OK,"Verifica a relação entre o ajuste para perdas dos ""Créditos a curto e longo prazo"" e suas contas principais. O valor dos ajustes não pode ser superior aos valores das contas originais""",Anexo I-AB da DCA


In [87]:
d2_00014_t

,valor
conta,
1.1.3.0.0.00.00 - Demais Créditos e Valores a Curto Prazo,"17,009,797,523.44"
1.2.1.2.1.00.00 - Demais Créditos e Valores a Longo Prazo - Consolidação,"3,555,394,001.92"
1.2.1.2.2.00.00 - Demais Créditos e Valores a Longo Prazo - Intra OFSS,"199,086,086.23"
1.2.1.2.3.00.00 - Demais Créditos e Valores a Longo Prazo - Inter OFSS - União,"36,900,000.00"
1.2.1.2.5.00.00 - Demais Créditos e Valores a Longo Prazo - Inter OFSS - Município,"2,841,699.73"


## D2_00015

In [88]:
#D2_00015_Bens Móveis
bens_moveis = df_dca_ab.query('cod_conta == "P1.2.3.1.1.00.00"')
bens_moveis['dimensao'] = 'D2_00015_Bens Móveis'
bens_moveis = bens_moveis.groupby('dimensao').agg({'valor': 'sum'})
d2_00015_t = bens_moveis.reset_index()

#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
condicao = d2_00015_t['valor'] > 0  

if condicao.any():
    resposta_d2_00015 = 'OK'
else:
    resposta_d2_00015 = 'ERRO'

d2_00015 = pd.DataFrame([resposta_d2_00015], columns=['Resposta'])
d2_00015.insert(0, 'Dimensão', 'D2_00015')
d2_00015.insert(2, 'Descrição da Dimensão', 'Verifica a informação de valor patrimonial de bens móveis')
d2_00015.insert(3, 'OBS', 'Anexo I-AB da DCA')
d2_00015

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00015,OK,Verifica a informação de valor patrimonial de bens móveis,Anexo I-AB da DCA


In [89]:
d2_00015_t

,dimensao,valor
0,D2_00015_Bens Móveis,"10,458,408,592.64"


## D2_00016

In [90]:
#D2_00016_Depreciação de Bens Móveis
depr_bens_moveis = df_dca_ab.query('cod_conta == "P1.2.3.8.1.01.00"')
depr_bens_moveis['dimensao'] = 'D2_00016_Depreciação de Bens Móveis'
depr_bens_moveis = depr_bens_moveis.groupby('dimensao').agg({'valor': 'sum'})
d2_00016_t = depr_bens_moveis.reset_index()

#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
condicao = d2_00016_t['valor'] < 0  #como o sinal esta invertido da depreciação, para verificar precisei colocar menor que zero (para verificar que existe saldo)

if condicao.any():
    resposta_d2_00016 = 'OK'
else:
    resposta_d2_00016 = 'ERRO'

d2_00016 = pd.DataFrame([resposta_d2_00016], columns=['Resposta'])
d2_00016.insert(0, 'Dimensão', 'D2_00016')
d2_00016.insert(2, 'Descrição da Dimensão', 'Verifica a Informação de Depreciação acumulada de bens móveis')
d2_00016.insert(3, 'OBS', 'Anexo I-AB da DCA')
d2_00016

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00016,OK,Verifica a Informação de Depreciação acumulada de bens móveis,Anexo I-AB da DCA


In [91]:
d2_00016_t

,dimensao,valor
0,D2_00016_Depreciação de Bens Móveis,"-2,295,742,173.40"


## D2_00017

In [92]:
#D2_00017_VPD de Depreciação de Bens Móveis e Imóveis
vpd_depr_bens_moveis = df_dca_hi.query('cod_conta == "P3.3.3.1.1.00.00"').reset_index()
vpd_depr_bens_moveis['dimensao'] = 'D2_00017_VPD de Depreciação de Bens Móveis e Imóveis'
d2_00017_t = vpd_depr_bens_moveis.filter(items=['dimensao', 'valor'])

#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
condicao = d2_00017_t['valor'] > 0  

if condicao.any():
    resposta_d2_00017 = 'OK'
else:
    resposta_d2_00017 = 'ERRO'

d2_00017 = pd.DataFrame([resposta_d2_00017], columns=['Resposta'])
d2_00017.insert(0, 'Dimensão', 'D2_00017')
d2_00017.insert(2, 'Descrição da Dimensão', 'Verifica a informação de variação patrimonial diminutiva de depreciação de bens móveis e imóveis')
d2_00017.insert(3, 'OBS', 'Anexo I-HI da DCA')
d2_00017

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00017,OK,Verifica a informação de variação patrimonial diminutiva de depreciação de bens móveis e imóveis,Anexo I-HI da DCA


In [93]:
d2_00017_t

,dimensao,valor
0,D2_00017_VPD de Depreciação de Bens Móveis e Imóveis,"255,082,371.13"


## D2_00018

In [94]:
# D2_00018_DIF_Entre Depreciação e Bens Móveis
dif_depr_bens_mov = pd.concat([bens_moveis, depr_bens_moveis])
# Calcula a diferença
diff = dif_depr_bens_mov['valor'].iloc[-2] - dif_depr_bens_mov['valor'].iloc[-1]
# Cria uma linha no final com este cálculo da diferença usando pd.concat ao invés de append
nova_linha = pd.DataFrame({'valor': [diff]})
dif_depr_bens_mov = pd.concat([dif_depr_bens_mov, nova_linha], ignore_index=True)
# Pega então somente esta linha final (da diferença)
d2_00018_t = dif_depr_bens_mov.iloc[[len(dif_depr_bens_mov)-1]]
d2_00018_t['dimensao'] = 'D2_00018_DIF Depreciação e Bens Móveis'
#d2_00018_t = dif_depr_bens_mov.reset_index()
d2_00018_t = d2_00018_t.reset_index()


#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
condicao = d2_00018_t['valor'] > 0  

if condicao.any():
    resposta_d2_00018 = 'OK'
else:
    resposta_d2_00018 = 'ERRO'

d2_00018 = pd.DataFrame([resposta_d2_00018], columns=['Resposta'])
d2_00018.insert(0, 'Dimensão', 'D2_00018')
d2_00018.insert(2, 'Descrição da Dimensão', 'Avalia se o valor dos bens móveis é maior que sua depreciação acumulada ')
d2_00018.insert(3, 'OBS', 'Anexo I-AB da DCA')
d2_00018


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00018,OK,Avalia se o valor dos bens móveis é maior que sua depreciação acumulada,Anexo I-AB da DCA


In [95]:
d2_00018_t

,index,valor,dimensao
0,2,"12,754,150,766.04",D2_00018_DIF Depreciação e Bens Móveis


## D2_00019

In [96]:
#D2_00019_Bens Imóveis
bens_imoveis = df_dca_ab.query('cod_conta == "P1.2.3.2.1.00.00"')
bens_imoveis['dimensao'] = 'D2_00019_Bens Imóveis'
bens_imoveis = bens_imoveis.groupby('dimensao').agg({'valor': 'sum'})
d2_00019_t = bens_imoveis.reset_index()

#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
condicao = d2_00019_t['valor'] > 0  

if condicao.any():
    resposta_d2_00019 = 'OK'
else:
    resposta_d2_00019 = 'ERRO'

d2_00019 = pd.DataFrame([resposta_d2_00019], columns=['Resposta'])
d2_00019.insert(0, 'Dimensão', 'D2_00019')
d2_00019.insert(2, 'Descrição da Dimensão', 'Verifica a Informação do valor patrimonial de bens imóveis')
d2_00019.insert(3, 'OBS', 'Anexo I-AB da DCA')
d2_00019


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00019,OK,Verifica a Informação do valor patrimonial de bens imóveis,Anexo I-AB da DCA


In [97]:
d2_00019_t

,dimensao,valor
0,D2_00019_Bens Imóveis,"44,265,663,527.85"


## D2_00020

In [98]:
#D2_00020_Depreciação de Bens Imóveis
depr_bens_imoveis = df_dca_ab.query('cod_conta == "P1.2.3.8.1.02.00"')
depr_bens_imoveis['dimensao'] = 'D2_00020_Depreciação de Bens Imóveis'
depr_bens_imoveis = depr_bens_imoveis.groupby('dimensao').agg({'valor': 'sum'})
d2_00020_t = depr_bens_imoveis.reset_index()

#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
condicao = d2_00020_t['valor'] < 0  #como o sinal esta invertido da depreciação, para verificar precisei colocar menor que zero (para verificar que existe saldo)

if condicao.any():
    resposta_d2_00020 = 'OK'
else:
    resposta_d2_00020 = 'ERRO'

d2_00020 = pd.DataFrame([resposta_d2_00020], columns=['Resposta'])
d2_00020.insert(0, 'Dimensão', 'D2_00020')
d2_00020.insert(2, 'Descrição da Dimensão', 'Verifica a Informação de depreciação acumulada de bens imóveis')
d2_00020.insert(3, 'OBS', 'Anexo I-AB da DCA')
d2_00020


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00020,OK,Verifica a Informação de depreciação acumulada de bens imóveis,Anexo I-AB da DCA


In [99]:
d2_00020_t

,dimensao,valor
0,D2_00020_Depreciação de Bens Imóveis,"-1,939,933,293.47"


## D2_00021

In [100]:
# D2_00018_DIF_Entre Depreciação e Bens Móveis
dif_depr_bens_imov = pd.concat([bens_imoveis, depr_bens_imoveis])
# Calcula a diferença
diff = dif_depr_bens_imov['valor'].iloc[-2] - dif_depr_bens_imov['valor'].iloc[-1]
# Cria uma linha no final com este cálculo da diferença usando pd.concat ao invés de append
nova_linha = pd.DataFrame({'valor': [diff]})
dif_depr_bens_imov = pd.concat([dif_depr_bens_imov, nova_linha], ignore_index=True)
# Pega então somente esta linha final (da diferença)
dif_depr_bens_imov = dif_depr_bens_imov.iloc[[len(dif_depr_bens_imov)-1]]
dif_depr_bens_imov['dimensao'] = 'D2_00021_ DIF entre Depreciação e Bens Imóveis'
d2_00021_t = dif_depr_bens_imov.reset_index()

#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
condicao = d2_00021_t['valor'] > 0  

if condicao.any():
    resposta_d2_00021 = 'OK'
else:
    resposta_d2_00021 = 'ERRO'

d2_00021 = pd.DataFrame([resposta_d2_00021], columns=['Resposta'])
d2_00021.insert(0, 'Dimensão', 'D2_00021')
d2_00021.insert(2, 'Descrição da Dimensão', 'Avalia se o valor dos bens imóveis é maior que sua depreciação acumulada')
d2_00021.insert(3, 'OBS', 'Anexo I-AB da DCA')
d2_00021

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00021,OK,Avalia se o valor dos bens imóveis é maior que sua depreciação acumulada,Anexo I-AB da DCA


In [101]:
d2_00021_t = d2_00021_t[['dimensao', 'valor']]
d2_00021_t

,dimensao,valor
0,D2_00021_ DIF entre Depreciação e Bens Imóveis,"46,205,596,821.32"


## D2_00023

In [102]:
# D2_00023_Inscrição de RPNP com a Diferença entre Liquidados e Empenhados
dif_rpnp = df_dca_d.query('cod_conta == "TotalDespesas" and (coluna == "Despesas Empenhadas" or coluna == "Despesas Liquidadas" or coluna == "Inscrição de Restos a Pagar Não Processados")')
# Calcula a primeira diferença
diff1 = dif_rpnp['valor'].iloc[-2] - dif_rpnp['valor'].iloc[-3]
# Adiciona a primeira diferença ao DataFrame
nova_linha1 = pd.DataFrame({'valor': [diff1]})
dif_rpnp = pd.concat([dif_rpnp, nova_linha1], ignore_index=True)
# Calcula a soma dos dois últimos valores (incluindo a primeira diferença adicionada)
diff2 = dif_rpnp['valor'].iloc[-1] + dif_rpnp['valor'].iloc[-2]
# Adiciona a soma ao DataFrame como uma nova linha
nova_linha2 = pd.DataFrame({'valor': [diff2]})
dif_rpnp = pd.concat([dif_rpnp, nova_linha2], ignore_index=True)
# Seleciona a última linha adicionada, que contém a soma
dif_rpnp = dif_rpnp.iloc[[len(dif_rpnp)-1]]
dif_rpnp['dimensao'] = 'D2_00023_DIF_Inscr. de RPNP com a Dif. entre Liquid. e Emp.'
dif_rpnp = dif_rpnp.assign(DIF=dif_rpnp['valor'])
dif_rpnp = dif_rpnp.filter(items=['dimensao', 'DIF'])
d2_00023_t = dif_rpnp.reset_index()

#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
# Verifica se o valor inscrito em RPNP no exercício é menor ou igual à diferença entre os valores registrados de despesa empenhada e liquidada no Anexo I-D da DCA.
condicao = d2_00023_t['DIF'] != 0  

if condicao.any():
    resposta_d2_00023 = 'OK'
else:
    resposta_d2_00023 = 'ERRO'

d2_00023 = pd.DataFrame([resposta_d2_00023], columns=['Resposta'])
d2_00023.insert(0, 'Dimensão', 'D2_00023')
d2_00023.insert(2, 'Descrição da Dimensão', 'Avalia se o valor de inscrição de RPNP é menor ou igual à diferença entre despesas empenhadas e liquidadas')
d2_00023.insert(3, 'OBS', 'Anexo I-D da DCA')
d2_00023


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00023,OK,Avalia se o valor de inscrição de RPNP é menor ou igual à diferença entre despesas empenhadas e liquidadas,Anexo I-D da DCA


In [103]:
d2_00023_t = d2_00023_t[['dimensao', 'DIF']]
d2_00023_t

,dimensao,DIF
0,D2_00023_DIF_Inscr. de RPNP com a Dif. entre Liquid. e Emp.,-0.00


## D2_00024

In [104]:
#D2_00023_Inscrição de RPP com a Diferença entre Pagos e Liquidados
dif_rpp = df_dca_d.query('cod_conta == "TotalDespesas" and (coluna == "Despesas Liquidadas" or coluna == "Despesas Pagas" or coluna == "Inscrição de Restos a Pagar Processados")')
# Calcula a primeira diferença
diff1 = dif_rpp['valor'].iloc[-2] - dif_rpp['valor'].iloc[-3]
# Adiciona a primeira diferença ao DataFrame
nova_linha1 = pd.DataFrame({'valor': [diff1]})
dif_rpp = pd.concat([dif_rpp, nova_linha1], ignore_index=True)
# Calcula a soma dos dois últimos valores (incluindo a primeira diferença adicionada)
diff2 = dif_rpp['valor'].iloc[-1] + dif_rpp['valor'].iloc[-2]
# Adiciona a soma ao DataFrame como uma nova linha
nova_linha2 = pd.DataFrame({'valor': [diff2]})
dif_rpp = pd.concat([dif_rpp, nova_linha2], ignore_index=True)
# Seleciona a última linha adicionada, que contém a soma
dif_rpp = dif_rpp.iloc[[len(dif_rpp)-1]]
dif_rpp['dimensao'] = 'D2_00024_DIF_Inscr. de RPP com a Dif. entre pagos e Liquid.'
dif_rpp = dif_rpp.assign(DIF=dif_rpp['valor'])
dif_rpp = dif_rpp.filter(items=['dimensao', 'DIF'])
d2_00024_t = dif_rpp.reset_index()

#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
# Verifica se o valor inscrito em RPP no exercício é menor ou igual à diferença entre os valores registrados de despesa liquidada e paga no Anexo I-D da DCA
condicao = d2_00024_t['DIF'] != 0 

if condicao.any():
    resposta_d2_00024 = 'OK'
else:
    resposta_d2_00024 = 'ERRO'

d2_00024 = pd.DataFrame([resposta_d2_00024], columns=['Resposta'])
d2_00024.insert(0, 'Dimensão', 'D2_00024')
d2_00024.insert(2, 'Descrição da Dimensão', 'Avalia se o valor de inscrição de RPP é menor ou igual à diferença entre despesas liquidadas e pagas')
d2_00024.insert(3, 'OBS', 'Anexo I-D da DCA')
d2_00024


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00024,OK,Avalia se o valor de inscrição de RPP é menor ou igual à diferença entre despesas liquidadas e pagas,Anexo I-D da DCA


In [105]:
d2_00024_t = d2_00024_t[['dimensao', 'DIF']]
d2_00024_t

,dimensao,DIF
0,D2_00024_DIF_Inscr. de RPP com a Dif. entre pagos e Liquid.,-0.00


## D2_00028

In [106]:
#D2_00028_Passivo Circulante Financeiro é menor ou igual ao valor de Passivo Circulante
pass_circ = df_dca_ab.query('cod_conta == "P2.1.0.0.0.00.00"')
pass_circ_financ = df_dca_ab.query('cod_conta == "P2.1.0.0.0.00.00F"')

d2_00028_t = pd.concat([pass_circ, pass_circ_financ])
d2_00028_t['dimensao'] = 'D2_00028_Passivo Circulante Financeiro <= ao Passivo Circulante'

# trocar o sinal do Passivo F
d2_00028_t['valor'] = d2_00028_t.apply(lambda x: x['valor'] * -1
if (str(x['cod_conta']) == 'P2.1.0.0.0.00.00F')
else x['valor'], axis=1)

d2_00028_t = d2_00028_t.groupby('dimensao').agg({'valor': 'sum'})
d2_00028_t = d2_00028_t.reset_index()

condicao = d2_00028_t['valor'] >= 0 

if condicao.any():
    resposta_d2_00028 = 'OK'
else:
    resposta_d2_00028 = 'ERRO'

d2_00028 = pd.DataFrame([resposta_d2_00028], columns=['Resposta'])
d2_00028.insert(0, 'Dimensão', 'D2_00028')
d2_00028.insert(2, 'Descrição da Dimensão', 'Avalia se o valor de Passivo Circulante Financeiro é menor ou igual ao Passivo Circulante')
d2_00028.insert(3, 'OBS', 'Anexo I-AB da DCA')
#d2_00013.insert(4, 'Nota', f'{nota_d2_13}')
d2_00028



,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00028,OK,Avalia se o valor de Passivo Circulante Financeiro é menor ou igual ao Passivo Circulante,Anexo I-AB da DCA


In [107]:
d2_00028_t

,dimensao,valor
0,D2_00028_Passivo Circulante Financeiro <= ao Passivo Circulante,"21,584,266,070.68"


## D2_00029

In [108]:
vpd_juros = df_dca_hi.query('cod_conta == "P3.4.1.0.0.00.00"')
emprest = df_dca_ab.query('cod_conta == "P2.1.2.0.0.00.00" or cod_conta == "P2.2.2.0.0.00.00"')
d2_00029_t = pd.concat([vpd_juros, emprest])
d2_00029_t = d2_00029_t.groupby('conta').agg({'valor': 'sum'})

# Verifica se 'emprest_cp' não está vazio
if not emprest.empty:
    # Verifica se há algum saldo maior que zero em 'vpd_juros'
    # Substitua 'saldo' pela coluna que representa o saldo em 'vpd_juros'
    if (vpd_juros['valor'] > 0).any():
        resposta_d2_00029 = 'OK'
    else:
        resposta_d2_00029 = 'ERRO'
else:
    resposta_d2_00029 = 'OK'

d2_00029 = pd.DataFrame([resposta_d2_00029], columns=['Resposta'])
d2_00029.insert(0, 'Dimensão', 'D2_00029')
d2_00029.insert(2, 'Descrição da Dimensão', 'Verifica a existência de VPD de Juros e Encargos de Empréstimos e Financiamentos quando foram informados valores de Obrigações decorrentes de empréstimos e financiamentos')
d2_00029.insert(3, 'OBS', 'Anexo I-AB e Anexo I-HI da DCA')
d2_00029

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00029,OK,Verifica a existência de VPD de Juros e Encargos de Empréstimos e Financiamentos quando foram informados valores de Obrigações decorrentes de empréstimos e financiamentos,Anexo I-AB e Anexo I-HI da DCA


In [109]:
d2_00029_t

,valor
conta,
2.1.2.0.0.00.00 - Empréstimos e Financiamentos a Curto Prazo,"10,991,522,205.83"
2.2.2.0.0.00.00 - Empréstimos e Financiamentos a Longo Prazo,"196,062,931,809.39"
3.4.1.0.0.00.00 - Juros e Encargos de Empréstimos e Financiamentos Obtidos,"9,097,900,982.13"


## D2_00030

In [110]:
# Dividir a coluna 'cod_conta' em varias colunas
df_dca_ab['q1'] = df_dca_ab['cod_conta'].astype(str).str[:5]
df_dca_ab['q2'] = df_dca_ab['cod_conta'].astype(str).str[5:6]
df_dca_ab['q3'] = df_dca_ab['cod_conta'].astype(str).str[6:7]
df_dca_ab['q4'] = df_dca_ab['cod_conta'].astype(str).str[7:8]
# Pegando só o Terceiro Nível de contas
d2_00030_t = df_dca_ab[(df_dca_ab['q2'] != '0') & (df_dca_ab['q4'] == '0')]
# Filtrar para excluir linhas onde a coluna 'cod_conta' começa com '2.3'
d2_00030_t = d2_00030_t[~d2_00030_t['cod_conta'].str.startswith('P2.3.')]

condicao = d2_00030_t['valor'] >= 0 

if condicao.all():
    resposta_d2_00030 = 'OK'
else:
    resposta_d2_00030 = 'ERRO'

d2_00030 = pd.DataFrame([resposta_d2_00030], columns=['Resposta'])
d2_00030.insert(0, 'Dimensão', 'D2_00030')
d2_00030.insert(2, 'Descrição da Dimensão', 'Verifica a existência de contas de terceiro nível do PCASP com saldos negativos')
d2_00030.insert(3, 'OBS', 'Anexo I-AB da DCA')
d2_00030


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00030,OK,Verifica a existência de contas de terceiro nível do PCASP com saldos negativos,Anexo I-AB da DCA


In [111]:
d2_00030_t.tail(3)



,exercicio,instituicao,cod_ibge,uf,anexo,rotulo,coluna,cod_conta,conta,valor,populacao,q1,q2,q3,q4
259,2024,Governo do Estado do Rio de Janeiro,33,RJ,DCA-Anexo I-AB,Padrão,31/12/2024,P2.2.7.0.0.00.00,2.2.7.0.0.00.00 - Provisões a Longo Prazo,"13,755,689,292.69",16615526,P2.2.,7,.,0
275,2024,Governo do Estado do Rio de Janeiro,33,RJ,DCA-Anexo I-AB,Padrão,31/12/2024,P2.2.8.0.0.00.00,2.2.8.0.0.00.00 - Demais Obrigações a Longo Prazo,"16,347,785,930.36",16615526,P2.2.,8,.,0
282,2024,Governo do Estado do Rio de Janeiro,33,RJ,DCA-Anexo I-AB,Padrão,31/12/2024,P2.2.9.0.0.00.00,2.2.9.0.0.00.00 - Resultado Diferido,"25,326,627,936.53",16615526,P2.2.,9,.,0


## D2_00031

In [112]:
# Dividir a coluna 'cod_conta' em varias colunas
df_dca_hi['q1'] = df_dca_hi['cod_conta'].astype(str).str[:5]
df_dca_hi['q2'] = df_dca_hi['cod_conta'].astype(str).str[5:6]
df_dca_hi['q3'] = df_dca_hi['cod_conta'].astype(str).str[6:7]
df_dca_hi['q4'] = df_dca_hi['cod_conta'].astype(str).str[7:8]
# Pegando só o Terceiro Nível de contas
d2_00031_t = df_dca_hi[(df_dca_hi['q2'] != '0') & (df_dca_hi['q4'] == '0')]

condicao = d2_00031_t['valor'] >= 0 

if condicao.all():
    resposta_d2_00031 = 'OK'
else:
    resposta_d2_00031 = 'ERRO'

d2_00031 = pd.DataFrame([resposta_d2_00031], columns=['Resposta'])
d2_00031.insert(0, 'Dimensão', 'D2_00031')
d2_00031.insert(2, 'Descrição da Dimensão', 'Existência de contas de terceiro nível do PCASP com saldos negativos')
d2_00031.insert(3, 'OBS', 'Anexo I-HI da DCA')
d2_00031


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00031,OK,Existência de contas de terceiro nível do PCASP com saldos negativos,Anexo I-HI da DCA


In [113]:
d2_00031_t.head(3)

,exercicio,instituicao,cod_ibge,uf,anexo,rotulo,coluna,cod_conta,conta,valor,populacao,q1,q2,q3,q4
2,2024,Governo do Estado do Rio de Janeiro,33,RJ,DCA-Anexo I-HI,Padrão,31/12/2024,P3.1.1.0.0.00.00,3.1.1.0.0.00.00 - Remuneração a Pessoal,"28,214,930,773.60",16615526,P3.1.,1,.,0
9,2024,Governo do Estado do Rio de Janeiro,33,RJ,DCA-Anexo I-HI,Padrão,31/12/2024,P3.1.2.0.0.00.00,3.1.2.0.0.00.00 - Encargos Patronais,"492,389,851.14",16615526,P3.1.,2,.,0
24,2024,Governo do Estado do Rio de Janeiro,33,RJ,DCA-Anexo I-HI,Padrão,31/12/2024,P3.1.3.0.0.00.00,3.1.3.0.0.00.00 - Benefícios a Pessoal,"1,565,312,767.71",16615526,P3.1.,3,.,0


## D2_00032

In [114]:
#D2_00032_Reconhecimento de Ajuste de Dívida Ativa
divida_ativa_cp = df_dca_ab.query('cod_conta == "P1.1.2.5.0.00.00" or cod_conta == "P1.1.2.6.0.00.00"')
divida_ativa_cp['dimensao'] = 'D2_00032_Dívida Ativa CP'
divida_ativa_cp = divida_ativa_cp.groupby('dimensao').agg({'valor': 'sum'})

divida_ativa_lp = df_dca_ab.query('cod_conta == "P1.2.1.1.1.04.00" or cod_conta == "P1.2.1.1.1.05.00"')
divida_ativa_lp['dimensao'] = 'D2_00032_Dívida Ativa LP'
divida_ativa_lp = divida_ativa_lp.groupby('dimensao').agg({'valor': 'sum'})

ajuste_divida_ativa_cp = df_dca_ab.query('cod_conta == "P1.1.2.9.0.00.00"')
ajuste_divida_ativa_cp['dimensao'] = 'D2_00032_Ajuste da Dívida Ativa CP'
ajuste_divida_ativa_cp = ajuste_divida_ativa_cp.groupby('dimensao').agg({'valor': 'sum'})

ajuste_divida_ativa_lp = df_dca_ab.query('cod_conta == "P1.2.1.1.1.04.00" or cod_conta == "P1.2.1.1.1.05.00"')
ajuste_divida_ativa_lp['dimensao'] = 'D2_00032_Ajuste da Dívida Ativa LP'
ajuste_divida_ativa_lp = ajuste_divida_ativa_lp.groupby('dimensao').agg({'valor': 'sum'})

d2_00032_t = pd.concat([divida_ativa_cp, divida_ativa_lp, ajuste_divida_ativa_cp, ajuste_divida_ativa_lp])
d2_00032_t = d2_00032_t.reset_index()

condicao1 = d2_00032_t['valor'] != 0
condicao2 = not ajuste_divida_ativa_cp.empty and (divida_ativa_cp['valor'] != 0).all()
condicao3 = not ajuste_divida_ativa_lp.empty and (divida_ativa_lp['valor'] != 0).all()

if condicao1.any() and condicao2 and condicao3:
    resposta_d2_00032 = 'OK'
else:
    resposta_d2_00032 = 'ERRO'

d2_00032 = pd.DataFrame([resposta_d2_00032], columns=['Resposta'])
d2_00032.insert(0, 'Dimensão', 'D2_00032')
d2_00032.insert(2, 'Descrição da Dimensão', 'Verifica a informação de Ajuste de Dívida Ativa (Tributária + Não Tributária)')
d2_00032.insert(3, 'OBS', 'Anexo I-AB da DCA')
#d2_00013.insert(4, 'Nota', f'{nota_d2_13}')
d2_00032


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00032,OK,Verifica a informação de Ajuste de Dívida Ativa (Tributária + Não Tributária),Anexo I-AB da DCA


In [115]:
d2_00032_t

,dimensao,valor
0,D2_00032_Dívida Ativa CP,"1,046,328,814.82"
1,D2_00032_Dívida Ativa LP,"172,400,394,251.32"
2,D2_00032_Ajuste da Dívida Ativa CP,"-806,314,581.45"
3,D2_00032_Ajuste da Dívida Ativa LP,"172,400,394,251.32"


# D2_00033

In [116]:
#D2_00033_receita ou dedução de receita no Anexo I-C da DCA que não é de competência do ente (NÃO RECEITA)
d2_00033_t = df_dca_c.query('cod_conta == "RO1.1.1.1.00.0.0" or cod_conta == "RO1.1.1.2.50.0.0" or cod_conta == "RO1.1.1.3.01.0.0" or \
                             cod_conta == "RO1.1.1.3.02.0.0" or cod_conta == "RO1.1.1.4.01.0.0" or cod_conta == "RO1.1.1.4.51.0.0" or \
                             cod_conta == "RO1.1.1.5.00.0.0"')


condicao = d2_00033_t.empty

if condicao:
    resposta_d2_00033 = 'OK'
else:
    resposta_d2_00033 = 'ERRO'


d2_00033 = pd.DataFrame([resposta_d2_00033], columns=['Resposta'])
d2_00033.insert(0, 'Dimensão', 'D2_00033')
d2_00033.insert(2, 'Descrição da Dimensão', 'Verifica a Informação de receitas que não são de competência do ente')
d2_00033.insert(3, 'OBS', 'Anexo I-C da DCA')
#d2_00013.insert(4, 'Nota', f'{nota_d2_13}')
d2_00033



,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00033,OK,Verifica a Informação de receitas que não são de competência do ente,Anexo I-C da DCA


In [117]:
d2_00033_t

,exercicio,instituicao,cod_ibge,uf,anexo,rotulo,coluna,cod_conta,conta,valor,populacao


## D2_00034

In [118]:
# Dividir a coluna 'cod_conta' em varias colunas
df_dca_hi['q1'] = df_dca_hi['cod_conta'].astype(str).str[:9]
df_dca_hi['q2'] = df_dca_hi['cod_conta'].astype(str).str[9:10]
df_dca_hi['q3'] = df_dca_hi['cod_conta'].astype(str).str[10:11]
df_dca_hi['q4'] = df_dca_hi['cod_conta'].astype(str).str[11:12]
# Pegando só o Quinto Nível de contas
d2_00034_t = df_dca_hi[(df_dca_hi['q2'] != '0') & (df_dca_hi['q4'] == '0')]

condicao = d2_00034_t['valor'] >= 0 

if condicao.all():
    resposta_d2_00034 = 'OK'
else:
    resposta_d2_00034 = 'ERRO'

d2_00034 = pd.DataFrame([resposta_d2_00034], columns=['Resposta'])
d2_00034.insert(0, 'Dimensão', 'D2_00034')
d2_00034.insert(2, 'Descrição da Dimensão', 'Verifica a existência de contas de quinto nível do PCASP com saldos negativos')
d2_00034.insert(3, 'OBS', 'Anexo I-HI da DCA')
d2_00034


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00034,OK,Verifica a existência de contas de quinto nível do PCASP com saldos negativos,Anexo I-HI da DCA


In [119]:
# Filtrando as linhas onde os valores na coluna 'valor' são menores que zero
d2_00034_ta = d2_00034_t[d2_00034_t['valor'] < 0]
d2_00034_ta


,exercicio,instituicao,cod_ibge,uf,anexo,rotulo,coluna,cod_conta,conta,valor,populacao,q1,q2,q3,q4


In [120]:
# OBS: NA DCA HI não temos q ter valores negativos (este anexo não fazemos a máscara para invertr saldos de redutoras ou refitifacoreas como fazemos na DCA C e DCA AB)
#d2_00034_t.to_excel('d2_00034_t.xlsx')
d2_00034_t

,exercicio,instituicao,cod_ibge,uf,anexo,rotulo,coluna,cod_conta,conta,valor,populacao,q1,q2,q3,q4
4,2024,Governo do Estado do Rio de Janeiro,33,RJ,DCA-Anexo I-HI,Padrão,31/12/2024,P3.1.1.1.1.00.00,3.1.1.1.1.00.00 - Remuneração a Pessoal Ativo Civil ¿ Abrangidos pelo RPPS - Consolidação,"18,004,080,246.66",16615526,P3.1.1.1.,1,.,0
6,2024,Governo do Estado do Rio de Janeiro,33,RJ,DCA-Anexo I-HI,Padrão,31/12/2024,P3.1.1.2.1.00.00,3.1.1.2.1.00.00 - Remuneração a Pessoal Ativo Civil - Abrangidos pelo RGPS - Consolidação,"1,256,884,162.50",16615526,P3.1.1.2.,1,.,0
8,2024,Governo do Estado do Rio de Janeiro,33,RJ,DCA-Anexo I-HI,Padrão,31/12/2024,P3.1.1.3.1.00.00,3.1.1.3.1.00.00 - Remuneração a Pessoal Ativo Militar - Consolidação,"8,953,966,364.44",16615526,P3.1.1.3.,1,.,0
11,2024,Governo do Estado do Rio de Janeiro,33,RJ,DCA-Anexo I-HI,Padrão,31/12/2024,P3.1.2.1.3.00.00,3.1.2.1.3.00.00 - Encargos Patronais - RPPS - Inter OFSS - União,"248,618.57",16615526,P3.1.2.1.,3,.,0
12,2024,Governo do Estado do Rio de Janeiro,33,RJ,DCA-Anexo I-HI,Padrão,31/12/2024,P3.1.2.1.4.00.00,3.1.2.1.4.00.00 - Encargos Patronais - RPPS - Inter OFSS - Estado,"2,667.86",16615526,P3.1.2.1.,4,.,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,2024,Governo do Estado do Rio de Janeiro,33,RJ,DCA-Anexo I-HI,Padrão,31/12/2024,P4.9.9.6.2.00.00,"4.9.9.6.2.00.00 - Indenizações, Restituições e Ressarcimentos - Intra OFSS","384,220,597.51",16615526,P4.9.9.6.,2,.,0
367,2024,Governo do Estado do Rio de Janeiro,33,RJ,DCA-Anexo I-HI,Padrão,31/12/2024,P4.9.9.6.3.00.00,"4.9.9.6.3.00.00 - Indenizações, Restituições e Ressarcimentos - Inter OFSS - União","699,972.59",16615526,P4.9.9.6.,3,.,0
368,2024,Governo do Estado do Rio de Janeiro,33,RJ,DCA-Anexo I-HI,Padrão,31/12/2024,P4.9.9.6.4.00.00,"4.9.9.6.4.00.00 - Indenizações, Restituições e Ressarcimentos - Inter OFSS - Estado","133,560.93",16615526,P4.9.9.6.,4,.,0
369,2024,Governo do Estado do Rio de Janeiro,33,RJ,DCA-Anexo I-HI,Padrão,31/12/2024,P4.9.9.6.5.00.00,"4.9.9.6.5.00.00 - Indenizações, Restituições e Ressarcimentos - Inter OFSS - Município","576,102.15",16615526,P4.9.9.6.,5,.,0


## D2_00035

In [121]:
# Criar uma cópia para volar com os valores negativos
df_dca_c_n = df_dca_c.copy()
# Voltando com os valores negativos
df_dca_c_n['valor'] = df_dca_c_n.apply(lambda row: -row['valor'] if 'Deduções' in row['coluna'] else row['valor'], axis=1)

d2_00035_t = df_dca_c_n.query('coluna == "Deduções - Transferências Constitucionais" or coluna == "Deduções - FUNDEB" or coluna == "Outras Deduções da Receita"')

condicao = d2_00035_t['valor'] > 0 

if condicao.any():
    resposta_d2_00035 = 'OK'
else:
    resposta_d2_00035 = 'ERRO'

d2_00035 = pd.DataFrame([resposta_d2_00035], columns=['Resposta'])
d2_00035.insert(0, 'Dimensão', 'D2_00035')
d2_00035.insert(2, 'Descrição da Dimensão', 'Verifica a Informação de deduções de Receitas com sinal negativo')
d2_00035.insert(3, 'OBS', 'Anexo I-C da DCA')
d2_00035


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00035,OK,Verifica a Informação de deduções de Receitas com sinal negativo,Anexo I-C da DCA


In [122]:
d2_00035_t.head(3)

,exercicio,instituicao,cod_ibge,uf,anexo,rotulo,coluna,cod_conta,conta,valor,populacao
1,2024,Governo do Estado do Rio de Janeiro,33,RJ,DCA-Anexo I-C,Padrão,Deduções - Transferências Constitucionais,ReceitasExcetoIntraOrcamentarias,RECEITAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I),"17,321,797,787.81",16615526
2,2024,Governo do Estado do Rio de Janeiro,33,RJ,DCA-Anexo I-C,Padrão,Deduções - FUNDEB,ReceitasExcetoIntraOrcamentarias,RECEITAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I),"10,659,629,456.79",16615526
3,2024,Governo do Estado do Rio de Janeiro,33,RJ,DCA-Anexo I-C,Padrão,Outras Deduções da Receita,ReceitasExcetoIntraOrcamentarias,RECEITAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I),"2,498,003,422.76",16615526


## D2_00036

In [123]:
#D2_00036_Créditos Tributários X VPA
cred_trib = df_dca_ab.query('cod_conta == "P1.1.2.1.0.00.00" or cod_conta == "P1.2.1.1.1.01.00" or cod_conta == "P1.2.1.1.2.01.00" or cod_conta == "P1.2.1.1.3.01.00" or \
                            cod_conta == "P1.2.1.1.4.01.00" or cod_conta == "P1.2.1.1.5.01.00"')
cred_trib['dimensao'] = 'D2_00036_Crédito Tributário'
cred_trib = cred_trib.groupby('dimensao').agg({'valor': 'sum'})

vpa_cred_trib = df_dca_hi.query('cod_conta == "P4.1.0.0.0.00.00"')
vpa_cred_trib['dimensao'] = 'D2_00036_VPA de Crédito Tributário'
vpa_cred_trib = vpa_cred_trib.groupby('dimensao').agg({'valor': 'sum'})

d2_00036_t = pd.concat([cred_trib, vpa_cred_trib])
d2_00036_t = d2_00036_t.reset_index()

# Verifica se 'vpa_cred_trib' não está vazio
if not vpa_cred_trib.empty:
    # Verifica se há algum saldo maior que zero em 'cred_trib'
    # Substitua 'saldo' pela coluna que representa o saldo em 'cred_trib'
    if (cred_trib['valor'] > 0).any():
        resposta_d2_00036 = 'OK'
    else:
        resposta_d2_00036 = 'ERRO'
else:
    resposta_d2_00036 = 'OK'

d2_00036 = pd.DataFrame([resposta_d2_00036], columns=['Resposta'])
d2_00036.insert(0, 'Dimensão', 'D2_00036')
d2_00036.insert(2, 'Descrição da Dimensão', 'Verifica a evidenciação de créditos tributários na DCA')
d2_00036.insert(3, 'OBS', 'Anexo I-AB e Anexo I-HI da DCA')
d2_00036

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00036,OK,Verifica a evidenciação de créditos tributários na DCA,Anexo I-AB e Anexo I-HI da DCA


In [124]:
d2_00036_t

,dimensao,valor
0,D2_00036_Crédito Tributário,"2,003,174,912.97"
1,D2_00036_VPA de Crédito Tributário,"73,533,028,431.78"


# D2_00037 (DÚVIDA)

In [125]:
#P4.1.0.0.0.00.00
d2_00037_t = df_dca_hi.query('cod_conta == "P4.1.0.0.0.00.00"')
d2_00037_t = d2_00037_t.groupby('conta').agg({'valor': 'sum'})

condicao = d2_00037_t['valor'] >= 0 

if condicao.all():
    resposta_d2_00037 = 'OK'
else:
    resposta_d2_00037 = 'ERRO'

d2_00037 = pd.DataFrame([resposta_d2_00037], columns=['Resposta'])
d2_00037.insert(0, 'Dimensão', 'D2_00037')
d2_00037.insert(2, 'Descrição da Dimensão', 'Verifica se houve registro de créditos tributários no período')
d2_00037.insert(3, 'OBS', 'Anexo I-HI da DCA')
d2_00037


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00037,OK,Verifica se houve registro de créditos tributários no período,Anexo I-HI da DCA


In [126]:
d2_00037_t

,valor
conta,
"4.1.0.0.0.00.00 - Impostos, Taxas e Contribuições de Melhoria","73,533,028,431.78"


## D2_00038

In [127]:
#D2_00038_Créditos Previdenciários a Receber
cred_trib_rcb = df_dca_ab.query('cod_conta == "P1.1.3.6.0.00.00"') # Créditos Previdenciários a Receber a CURTO PRAZO
cred_trib_rcb['dimensao'] = 'D2_00038_Créditos Previdenciários a Receber'
cred_trib_rcb = cred_trib_rcb.groupby('dimensao').agg({'valor': 'sum'})
d2_00038_t = cred_trib_rcb.reset_index()

#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
condicao = d2_00038_t['valor'] > 0  

if condicao.any():
    resposta_d2_00038 = 'OK'
else:
    resposta_d2_00038 = 'ERRO'

d2_00038 = pd.DataFrame([resposta_d2_00038], columns=['Resposta'])
d2_00038.insert(0, 'Dimensão', 'D2_00038')
d2_00038.insert(2, 'Descrição da Dimensão', 'Verifica a informação de créditos previdenciários')
d2_00038.insert(3, 'OBS', 'Anexo I-AB da DCA')
d2_00038


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00038,OK,Verifica a informação de créditos previdenciários,Anexo I-AB da DCA


In [128]:
d2_00038_t

,dimensao,valor
0,D2_00038_Créditos Previdenciários a Receber,"893,479,222.98"


## D2_00039

In [129]:
prov = df_dca_ab.query('cod_conta == "P2.1.7.1.0.00.00" or cod_conta == "P2.1.7.2.0.00.00" or cod_conta == "P2.1.7.3.0.00.00" or \
                               cod_conta == "P2.1.7.4.0.00.00" or cod_conta == "P2.1.7.9.0.00.00" or \
                       cod_conta == "P2.2.7.1.0.00.00" or cod_conta == "P2.2.7.2.0.00.00" or cod_conta == "P2.2.7.3.0.00.00" or \
                               cod_conta == "P2.2.7.4.0.00.00" or cod_conta == "P2.2.7.9.0.00.00"')

vpd_prov = df_dca_hi.query('cod_conta == "P3.9.7.1.0.00.00" or cod_conta == "P3.9.7.2.0.00.00" or cod_conta == "P3.9.7.3.0.00.00" or \
                           cod_conta == "P3.9.7.4.0.00.00" or cod_conta == "P3.9.7.9.0.00.00"')

d2_00039_t = pd.concat([prov, vpd_prov])
d2_00039_t['tipo_prov'] = d2_00039_t['cod_conta'].str[7]
d2_00039_t = d2_00039_t.groupby(['tipo_prov', 'conta']).agg({'valor': 'sum'}).reset_index()

# Condições individuais
condicao_1 = d2_00039_t['conta'].str.startswith('3.9.7.1').any() # Verifica se existe alguma conta começando com '3.9.7.1'
condicao_2 = (~d2_00039_t['conta'].str.startswith('2.2.7.1')).all() # Verifica se nenhuma conta começa com '2.2.7.1'
condicao_3 = d2_00039_t['conta'].str.startswith('3.9.7.2').any()
condicao_4 = (~d2_00039_t['conta'].str.startswith('2.2.7.2')).all()
condicao_5 = d2_00039_t['conta'].str.startswith('3.9.7.3').any()
condicao_6 = (~d2_00039_t['conta'].str.startswith('2.2.7.3')).all()
condicao_7 = d2_00039_t['conta'].str.startswith('3.9.7.4').any()
condicao_8 = (~d2_00039_t['conta'].str.startswith('2.2.7.4')).all()
condicao_9 = d2_00039_t['conta'].str.startswith('3.9.7.9').any()
condicao_10 = (~d2_00039_t['conta'].str.startswith('2.2.7.9')).all()

# Combinando as condições em pares (por grupos do >> tipo_prov)
par_1_e_2 = condicao_1 and condicao_2
par_3_e_4 = condicao_3 and condicao_4
par_5_e_6 = condicao_5 and condicao_6
par_7_e_8 = condicao_7 and condicao_8
par_9_e_10 = condicao_9 and condicao_10


# Verificando se ambos os pares de condições são verdadeiros (se existir apenas um verdadeiro, ou seja >> Existe VPD sem PASSIVO = ERRO)
resultado_final = par_1_e_2 and par_3_e_4 and par_5_e_6 and par_7_e_8 and par_9_e_10
 
# Imprimindo o resultado
if resultado_final.any():
    resposta_d2_00039 = 'ERRO'
else:
    resposta_d2_00039 = 'OK'

d2_00039 = pd.DataFrame([resposta_d2_00039], columns=['Resposta'])
d2_00039.insert(0, 'Dimensão', 'D2_00039')
d2_00039.insert(2, 'Descrição da Dimensão', 'Verifica se há valor de VPD de provisão, então deve haver o registro no passivo de curto ou de longo prazo')
d2_00039.insert(3, 'OBS', 'Anexo I-AB e Anexo I-HI da DCA')
d2_00039


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00039,OK,"Verifica se há valor de VPD de provisão, então deve haver o registro no passivo de curto ou de longo prazo",Anexo I-AB e Anexo I-HI da DCA


In [130]:
d2_00039_t

,tipo_prov,conta,valor
0,1,2.1.7.1.0.00.00 - Provisão para Riscos Trabalhistas a Curto Prazo,"141,191,177.61"
1,1,2.2.7.1.0.00.00 - Provisão para Riscos Trabalhistas a Longo Prazo,"461,545,273.88"
2,1,3.9.7.1.0.00.00 - VPD de Provisão para Riscos Trabalhistas,"29,342,413.46"
3,2,2.2.7.2.0.00.00 - Provisões Matemáticas Previdenciárias a Longo Prazo,"12,916,618,333.06"
4,2,3.9.7.2.0.00.00 - VPD de Provisões Matemáticas Previdenciárias a Longo Prazo,"3,920,149,462.99"
5,3,2.1.7.3.0.00.00 - Provisões para Riscos Fiscais a Curto Prazo,"652,598.26"
6,3,2.2.7.3.0.00.00 - Provisão para Riscos Fiscais a Longo Prazo,"33,755,064.08"
7,4,2.1.7.4.0.00.00 - Provisão para Riscos Cíveis a Curto Prazo,"71,388,179.97"
8,4,2.2.7.4.0.00.00 - Provisão para Riscos Cíveis a Longo Prazo,"318,134,690.14"
9,4,3.9.7.4.0.00.00 - VPD de Provisão para Riscos Cíveis,"150,450,057.98"


## D2_00040

In [131]:
# Dividir a coluna 'cod_conta' em varias colunas
df_dca_ab['q1'] = df_dca_ab['cod_conta'].astype(str).str[:9]
df_dca_ab['q2'] = df_dca_ab['cod_conta'].astype(str).str[9:10]
df_dca_ab['q3'] = df_dca_ab['cod_conta'].astype(str).str[10:11]
df_dca_ab['q4'] = df_dca_ab['cod_conta'].astype(str).str[11:12]
# Pegando só o Quinto Nível de contas
d2_00040_t = df_dca_ab[(df_dca_ab['q2'] != '0') & (df_dca_ab['q4'] == '0')]

condicao = d2_00040_t['valor'] >= 0 

if condicao.any():
    resposta_d2_00040 = 'OK'
else:
    resposta_d2_00040 = 'ERRO'

d2_00040 = pd.DataFrame([resposta_d2_00040], columns=['Resposta'])
d2_00040.insert(0, 'Dimensão', 'D2_00040')
d2_00040.insert(2, 'Descrição da Dimensão', 'Verifica a existência de contas de quinto nível do PCASP com saldos negativos')
d2_00040.insert(3, 'OBS', 'Anexo I-AB da DCA')
d2_00040


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00040,OK,Verifica a existência de contas de quinto nível do PCASP com saldos negativos,Anexo I-AB da DCA


In [132]:
d2_00040_t.head(3)

,exercicio,instituicao,cod_ibge,uf,anexo,rotulo,coluna,cod_conta,conta,valor,populacao,q1,q2,q3,q4
4,2024,Governo do Estado do Rio de Janeiro,33,RJ,DCA-Anexo I-AB,Padrão,31/12/2024,P1.1.1.1.1.00.00,1.1.1.1.1.00.00 - Caixa e Equivalentes de Caixa em Moeda Nacional - Consolidação,"33,966,339,653.71",16615526,P1.1.1.1.,1,.,0
5,2024,Governo do Estado do Rio de Janeiro,33,RJ,DCA-Anexo I-AB,Padrão,31/12/2024,P1.1.1.1.2.00.00,1.1.1.1.2.00.00 - Caixa e Equivalentes de Caixa em Moeda Nacional - Intra OFSS,"227,118,657.16",16615526,P1.1.1.1.,2,.,0
7,2024,Governo do Estado do Rio de Janeiro,33,RJ,DCA-Anexo I-AB,Padrão,31/12/2024,P1.1.1.3.1.00.00,1.1.1.3.1.00.00 - Depósitos Restituíveis e Valores Vinculados - Consolidação,"2,655,279,132.84",16615526,P1.1.1.3.,1,.,0


# D2_00044

In [133]:
###################  >> D2_00044_Receita Realizada MSC Encerramento
rec_total = msc_encerr.query('tipo_valor == "beginning_balance" and (conta_contabil == "621200000" or conta_contabil == "621310100" or conta_contabil == "621310200" or conta_contabil == "621390000")')
rec_total['dimensao'] = 'D2_00044_Rec.Realizada'
rec_msc = rec_total.groupby('dimensao').agg({'valor': 'sum'})

###################  >> D2_00044_Receita Realizada DCA
rec_dca = df_dca_c.query('cod_conta == "TotalReceitas"')
rec_dca['dimensao'] = 'D2_00044_Rec.Realizada'
rec_dca = rec_dca.filter(items=['dimensao', 'valor'])
rec_dca = rec_dca.set_index("dimensao")
rec_dca = rec_dca.groupby('dimensao').agg({'valor': 'sum'})

d2_00044_t = rec_msc.merge(rec_dca, on='dimensao')
d2_00044_t['DIF'] = d2_00044_t['valor_x'] - d2_00044_t['valor_y']
d2_00044_t.columns = ['MSC ENCERR', 'DCA C e D', 'DIF']

tolerancia = 0.01  # Exemplo de tolerância
condicao = ~np.isclose(d2_00044_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d2_00044 = 'Diferença'
else:
    resposta_d2_00044 = 'OK'

d2_00044 = pd.DataFrame([resposta_d2_00044], columns=['Resposta'])
d2_00044.insert(0, 'Dimensão', 'D2_00044')
d2_00044.insert(2, 'Descrição da Dimensão', 'Avalia a igualdade das receitas arrecadadas')
d2_00044.insert(3, 'OBS', 'MSC de encerramento e no Anexo I-C da DCA')
d2_00044

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00044,OK,Avalia a igualdade das receitas arrecadadas,MSC de encerramento e no Anexo I-C da DCA


In [134]:
d2_00044_t

,MSC ENCERR,DCA C e D,DIF
dimensao,,,
D2_00044_Rec.Realizada,"106,112,454,653.40","106,112,454,653.40",0.00


## D2_00045


In [135]:
###################  >> D2_00045_Receita de Impostos MSC (pelo Bruto...sem Deduções)
rec_total = msc_encerr.query('tipo_valor == "beginning_balance" and (conta_contabil == "621200000" or conta_contabil == "621390000")')
imposto_msc_1 = rec_total[rec_total['natureza_receita'].str.contains("111251", case=False, regex=True)] # IPVA
imposto_msc_2 = rec_total[rec_total['natureza_receita'].str.contains("111252", case=False, regex=True)] # ITBI
imposto_msc_3 = rec_total[rec_total['natureza_receita'].str.contains("111303", case=False, regex=True)] # IRRF
imposto_msc_4 = rec_total[rec_total['natureza_receita'].str.contains("111450", case=False, regex=True)] # ICMS
imposto_msc = pd.concat([imposto_msc_1, imposto_msc_2, imposto_msc_3, imposto_msc_4], axis=0)
imposto_msc['dimensao'] = 'D2_00045_Rec.Impostos'
imposto_msc = imposto_msc.groupby('dimensao').agg({'valor': 'sum'})

###################  >> D2_00045_Receita de Impostos DCA
imposto_dca = df_dca_c.query('(cod_conta == "RO1.1.1.0.00.0.0" and coluna == "Receitas Brutas Realizadas") or (cod_conta == "RO1.1.1.0.00.0.0" and coluna == "Outras Deduções da Receita")')
imposto_dca['dimensao'] = 'D2_00045_Rec.Impostos'
imposto_dca = imposto_dca.filter(items=['dimensao', 'valor'])
imposto_dca = imposto_dca.set_index("dimensao")
imposto_dca = imposto_dca.groupby('dimensao').agg({'valor': 'sum'})

###################  >> D2_00045_Receita de Impostos
d2_00045_t = imposto_msc.merge(imposto_dca, on='dimensao')
d2_00045_t['DIF'] = d2_00045_t['valor_x'] - d2_00045_t['valor_y']
d2_00045_t.columns = ['MSC ENCERR', 'DCA C e D', 'DIF']

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d2_00045_t['DIF'], 0, atol=tolerancia)


if condicao.any():
    resposta_d2_00045 = 'Diferença'
else:
    resposta_d2_00045 = 'OK'

d2_00045 = pd.DataFrame([resposta_d2_00045], columns=['Resposta'])
d2_00045.insert(0, 'Dimensão', 'D2_00045')
d2_00045.insert(2, 'Descrição da Dimensão', 'Avalia a igualdade das receitas com tributos estaduais')
d2_00045.insert(3, 'OBS', 'MSC de encerramento e no Anexo I-C da DCA')
d2_00045


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00045,OK,Avalia a igualdade das receitas com tributos estaduais,MSC de encerramento e no Anexo I-C da DCA


In [136]:
d2_00045_t = d2_00045_t.reset_index()
d2_00045_t


,dimensao,MSC ENCERR,DCA C e D,DIF
0,D2_00045_Rec.Impostos,"71,276,657,196.75","71,276,657,196.75",0.00


## D2_00047

In [137]:
###################  >> D2_00047_Transferências Constitucionais MSC (FPE e FUNDEB)
rec_total = msc_encerr.query('tipo_valor == "beginning_balance" and conta_contabil == "621200000"')
#fpe_msc = rec_total[rec_total["natureza_receita"].str.contains("1718011")] # Em 2021
fpe_msc = rec_total[rec_total["natureza_receita"].str.contains("17115001")] # Em 2022
#fundeb_msc = rec_total[rec_total["natureza_receita"].str.contains("17580111")] # Em 2021
fundeb_msc = rec_total[rec_total["natureza_receita"].str.contains("17515001")] # Em 2022
complem_fundeb_msc = rec_total[rec_total["natureza_receita"].str.contains("1715")] # Em 2022
transf = pd.concat([fpe_msc, fundeb_msc, complem_fundeb_msc], axis=0)
transf['dimensao'] = 'D2_00047_Transf.Const'
transf = transf.groupby('dimensao').agg({'valor': 'sum'})

###################  >> D4_00011_Transferências Constitucionais DCA (#@ MUDAR quando for 2021 as NRs)
# Cota-Parte do FPE          >> NR: 1.7.1.1.50.0.0 + NR: 7.7.1.1.50.0.0
# Transferências do FUNDEB   >> NR: 1.7.1.5.00.0.0 + NR: 1.7.5.1.00.0.0 + NR: 7.7.1.5.00.0.0 + NR: 7.7.5.1.00.0.0

###################  >> D2_00047_Transferências Constitucionais DCA
transf_dca = df_dca_c.query('(cod_conta == "RO1.7.1.1.50.0.0" or cod_conta == "RO1.7.5.1.00.0.0" or cod_conta == "RO1.7.1.5.00.0.0") \
                            and (coluna == "Receitas Brutas Realizadas")')
#transf_fpe_dca = df_dca_c.query('coluna == "Receitas Brutas Realizadas" & conta == "1.7.1.8.01.1.0 - Cota-Parte do Fundo de Participação dos Estados e do Distrito Federal"')
#transf_fundeb_dca = df_dca_c.query('conta == "1.7.5.8.01.0.0 - Transferências de Recursos do Fundo de Manutenção e Desenvolvimento da Educação Básica e de Valorização dos Profissionais da Educação ¿ FUNDEB"')
#transf_dca = pd.concat([transf_fpe_dca, transf_fundeb_dca])
transf_dca['dimensao'] = 'D2_00047_Transf.Const'
transf_dca = transf_dca.filter(items=['dimensao', 'valor'])
transf_dca = transf_dca.set_index("dimensao")
transf_dca = transf_dca.groupby('dimensao').agg({'valor': 'sum'})

###################  >> D2_00047_Transferências Constitucionais
d2_00047_t = transf.merge(transf_dca, on='dimensao')
d2_00047_t['DIF'] = d2_00047_t['valor_x'] - d2_00047_t['valor_y']
d2_00047_t.columns = ['MSC ENCERR', 'DCA C e D', 'DIF']

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d2_00047_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d2_00047 = 'ERRO'
else:
    resposta_d2_00047 = 'OK'

d2_00047 = pd.DataFrame([resposta_d2_00047], columns=['Resposta'])
d2_00047.insert(0, 'Dimensão', 'D2_00047')
d2_00047.insert(2, 'Descrição da Dimensão', 'Avalia a igualdade das receitas estaduais com transferências constitucionais')
d2_00047.insert(3, 'OBS', 'MSC de encerramento e no Anexo I-C da DCA')
d2_00047


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00047,OK,Avalia a igualdade das receitas estaduais com transferências constitucionais,MSC de encerramento e no Anexo I-C da DCA


In [138]:
d2_00047_t = d2_00047_t.reset_index()
d2_00047_t

,dimensao,MSC ENCERR,DCA C e D,DIF
0,D2_00047_Transf.Const,"7,678,621,514.36","7,678,621,514.36",0.00


## D2_00049

In [139]:
###################  >> D2_00049_DPS Empenhada, Liquidada e Paga MSC
emp_msc = msc_encerr.query('tipo_valor == "beginning_balance" and (conta_contabil == "622130500" or conta_contabil == "622130600" or conta_contabil == "622130700" or conta_contabil == "622130400")').copy()
emp_msc['dimensao'] = 'D2_00049_Empenhado'
emp_msc = emp_msc.groupby('dimensao').agg({'valor': 'sum'})

liq_msc = msc_encerr.query('tipo_valor == "beginning_balance" and (conta_contabil == "622130700" or conta_contabil == "622130400")').copy()
liq_msc['dimensao'] = 'D2_00049_Liquidado'
liq_msc = liq_msc.groupby('dimensao').agg({'valor': 'sum'})

pago_msc = msc_encerr.query('tipo_valor == "beginning_balance" and conta_contabil == "622130400"').copy()
pago_msc['dimensao'] = 'D2_00049_Pago'
pago_msc = pago_msc.groupby('dimensao').agg({'valor': 'sum'})

###################  >> D2_00049_Empenhado, Liquidado e Pago DCA
emp_dca = df_dca_d.query('coluna == "Despesas Empenhadas" & cod_conta == "TotalDespesas"').copy()
emp_dca['dimensao'] = 'D2_00049_Empenhado'
emp_dca = emp_dca.filter(items=['dimensao', 'valor'])
emp_dca = emp_dca.set_index("dimensao")

liq_dca = df_dca_d.query('coluna == "Despesas Liquidadas" & cod_conta == "TotalDespesas"').copy()
liq_dca['dimensao'] = 'D2_00049_Liquidado'
liq_dca = liq_dca.filter(items=['dimensao', 'valor'])
liq_dca = liq_dca.set_index("dimensao")

pago_dca = df_dca_d.query('coluna == "Despesas Pagas" & cod_conta == "TotalDespesas"').copy()
pago_dca['dimensao'] = 'D2_00049_Pago'
pago_dca = pago_dca.filter(items=['dimensao', 'valor'])
pago_dca = pago_dca.set_index("dimensao")

###################  >> MERGE >> D2_00049_Empenhado, Liquidado e Pago
d2_00049_emp = emp_msc.merge(emp_dca, on='dimensao')
d2_00049_emp['DIF'] = d2_00049_emp['valor_x'] - d2_00049_emp['valor_y']
d2_00049_emp.columns = ['MSC ENCERR', 'DCA C e D', 'DIF']

d2_00049_liq = liq_msc.merge(liq_dca, on='dimensao')
d2_00049_liq['DIF'] = d2_00049_liq['valor_x'] - d2_00049_liq['valor_y']
d2_00049_liq.columns = ['MSC ENCERR', 'DCA C e D', 'DIF']

d2_00049_pago = pago_msc.merge(pago_dca, on='dimensao')
d2_00049_pago['DIF'] = d2_00049_pago['valor_x'] - d2_00049_pago['valor_y']
d2_00049_pago.columns = ['MSC ENCERR', 'DCA C e D', 'DIF']

d2_00049_t = pd.concat([d2_00049_emp, d2_00049_liq, d2_00049_pago])

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d2_00049_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d2_00049 = 'Diferença'
else:
    resposta_d2_00049 = 'OK'

d2_00049 = pd.DataFrame([resposta_d2_00049], columns=['Resposta'])
d2_00049.insert(0, 'Dimensão', 'D2_00049')
d2_00049.insert(2, 'Descrição da Dimensão', 'Verifica a igualdade das Despesas Orçamentárias empenhadas, liquidadas e pagas')
d2_00049.insert(3, 'OBS', 'MSC de encerramento e no Anexo I-C da DCA')
d2_00049


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00049,OK,"Verifica a igualdade das Despesas Orçamentárias empenhadas, liquidadas e pagas",MSC de encerramento e no Anexo I-C da DCA


In [140]:
d2_00049_t = d2_00049_t.reset_index()
d2_00049_t

,dimensao,MSC ENCERR,DCA C e D,DIF
0,D2_00049_Empenhado,"108,526,792,476.10","108,526,792,476.10",0.00
1,D2_00049_Liquidado,"107,262,617,813.00","107,262,617,813.00",0.00
2,D2_00049_Pago,"104,823,674,009.28","104,823,674,009.28",0.00


## D2_00050

In [141]:
###################  >> D2_00050_RPP e RPNP na MSC
rpp_msc = msc_encerr.query('tipo_valor == "beginning_balance" and conta_contabil == "622130700"')
rpp_msc['dimensao'] = 'D2_00050_Inscrição RPP'
rpp_msc = rpp_msc.groupby('dimensao').agg({'valor': 'sum'})

rpnp_msc = msc_encerr.query('tipo_valor == "beginning_balance" and (conta_contabil == "622130500" or conta_contabil == "622130600")')
rpnp_msc['dimensao'] = 'D2_00050_Inscrição RPNP'
rpnp_msc = rpnp_msc.groupby('dimensao').agg({'valor': 'sum'})

###################  >> D2_00050_RPP e RPNP na DCA
rpp_dca = df_dca_d.query('coluna == "Inscrição de Restos a Pagar Processados" & cod_conta == "TotalDespesas"')
rpp_dca['dimensao'] = 'D2_00050_Inscrição RPP'
rpp_dca = rpp_dca.filter(items=['dimensao', 'valor'])
rpp_dca = rpp_dca.set_index("dimensao")

rpnp_dca = df_dca_d.query('coluna == "Inscrição de Restos a Pagar Não Processados" & cod_conta == "TotalDespesas"')
rpnp_dca['dimensao'] = 'D2_00050_Inscrição RPNP'
rpnp_dca = rpnp_dca.filter(items=['dimensao', 'valor'])
rpnp_dca = rpnp_dca.set_index("dimensao")

###################  >> D2_00050_RPP e RPNP
d2_00050_rpp = rpp_msc.merge(rpp_dca, on='dimensao')
d2_00050_rpp['DIF'] = d2_00050_rpp['valor_x'] - d2_00050_rpp['valor_y']
d2_00050_rpp.columns = ['MSC ENCERR', 'DCA C e D', 'DIF']

d2_00050_rpnp = rpnp_msc.merge(rpnp_dca, on='dimensao')
d2_00050_rpnp['DIF'] = d2_00050_rpnp['valor_x'] - d2_00050_rpnp['valor_y']
d2_00050_rpnp.columns = ['MSC ENCERR', 'DCA C e D', 'DIF']

d2_00050_t = pd.concat([d2_00050_rpp, d2_00050_rpnp])

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d2_00050_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d2_00050 = 'Diferença'
else:
    resposta_d2_00050 = 'OK'

d2_00050 = pd.DataFrame([resposta_d2_00050], columns=['Resposta'])
d2_00050.insert(0, 'Dimensão', 'D2_00050')
d2_00050.insert(2, 'Descrição da Dimensão', 'Verifica a igualdade dos Restos a Pagar processados e não processados')
d2_00050.insert(3, 'OBS', 'MSC de encerramento e no Anexo I-D da DCA')
d2_00050


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00050,OK,Verifica a igualdade dos Restos a Pagar processados e não processados,MSC de encerramento e no Anexo I-D da DCA


In [142]:
d2_00050_t = d2_00050_t.reset_index()
d2_00050_t

,dimensao,MSC ENCERR,DCA C e D,DIF
0,D2_00050_Inscrição RPP,"2,438,943,803.72","2,438,943,803.72",0.00
1,D2_00050_Inscrição RPNP,"1,264,174,663.10","1,264,174,663.10",0.00


# D2_00051

In [143]:
#Verificar se o do total do Ajuste para perdas em Estoques é inferior ao total do saldo dos Estoques no Anexo I-AB da DCA.
d2_00051_t = df_dca_ab.query('cod_conta == "P1.1.5.0.0.00.00"')
d2_00051_t = d2_00051_t.groupby('conta').agg({'valor': 'sum'})

condicao = d2_00051_t['valor'] >= 0 

if condicao.all():
    resposta_d2_00051 = 'OK'
else:
    resposta_d2_00051 = 'ERRO'

d2_00051 = pd.DataFrame([resposta_d2_00051], columns=['Resposta'])
d2_00051.insert(0, 'Dimensão', 'D2_00051')
d2_00051.insert(2, 'Descrição da Dimensão', 'Verificar se o do total do Ajuste para perdas em Estoques é inferior ao total do saldo dos Estoques')
d2_00051.insert(3, 'OBS', ' Anexo I-AB da DCA')
d2_00051


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00051,OK,Verificar se o do total do Ajuste para perdas em Estoques é inferior ao total do saldo dos Estoques,Anexo I-AB da DCA


In [144]:
d2_00051_t

,valor
conta,
1.1.5.0.0.00.00 - Estoques,"1,804,508,912.86"


# D2_00052

In [145]:
eq_patrim = df_dca_ab.query('cod_conta == "P1.2.2.1.1.01.00" or cod_conta == "P1.2.2.1.2.01.00" or cod_conta == "P1.2.2.1.3.01.00" or cod_conta == "P1.2.2.1.4.01.00" or cod_conta == "P1.2.2.1.5.01.00"')
res_eq_patrim = df_dca_hi.query('cod_conta == "P3.9.2.0.0.00.00" or cod_conta == "P4.9.2.0.0.00.00"')

d2_00052_t = pd.concat([eq_patrim, res_eq_patrim])
d2_00052_t = d2_00052_t.groupby('conta').agg({'valor': 'sum'})


# Verifica se 'res_eq_patrim' não está vazio
if not res_eq_patrim.empty:
    # Verifica se há algum saldo maior que zero em 'eq_patrim'
    if (eq_patrim['valor'] > 0).any():
        resposta_d2_00052 = 'OK'
    else:
        resposta_d2_00052 = 'ERRO'
else:
    resposta_d2_00052 = 'OK'

d2_00052 = pd.DataFrame([resposta_d2_00052], columns=['Resposta'])
d2_00052.insert(0, 'Dimensão', 'D2_00052')
d2_00052.insert(2, 'Descrição da Dimensão', 'Verificar se o do total do Ajuste para perdas em Estoques é inferior ao total do saldo dos Estoques')
d2_00052.insert(3, 'OBS', ' Anexo I-AB da DCA')
d2_00052

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00052,OK,Verificar se o do total do Ajuste para perdas em Estoques é inferior ao total do saldo dos Estoques,Anexo I-AB da DCA


In [146]:
d2_00052_t

,valor
conta,
1.2.2.1.1.01.00 - Participações Avaliadas pelo Método de Equivalência Patrimonial,"6,734,844,872.40"
1.2.2.1.2.01.00 - Participações Avaliadas pelo Método de Equivalência Patrimonial,"228,066,288.04"
1.2.2.1.3.01.00 - Participações Avaliadas pelo Método de Equivalência Patrimonial,"321,300.76"
3.9.2.0.0.00.00 - Resultado Negativo de Participações,"2,025,286,759.51"
4.9.2.0.0.00.00 - Resultado Positivo de Participações,"149,198,386.00"


# D2_00053

In [147]:
#filtrado = msc_patrimonial_encerr.query('tipo_valor == "ending_balance"')
filtrado = msc_encerr.query('tipo_valor == "ending_balance"')

d2_00053_t = filtrado[filtrado['conta_contabil'].str.startswith('115')]
d2_00053_t = d2_00053_t.groupby(['poder_orgao'])['valor'].sum().reset_index()

condicao = d2_00053_t['valor'] >= 0 

if condicao.all():
    resposta_d2_00053 = 'OK'
else:
    resposta_d2_00053 = 'ERRO'

d2_00053 = pd.DataFrame([resposta_d2_00053], columns=['Resposta'])
d2_00053.insert(0, 'Dimensão', 'D2_00053')
d2_00053.insert(2, 'Descrição da Dimensão', 'Verificar se o total do Ajuste para perdas em Estoques é inferior ao total do saldo dos Estoque')
d2_00053.insert(3, 'OBS', 'MSC de Encerramento')
d2_00053


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00053,OK,Verificar se o total do Ajuste para perdas em Estoques é inferior ao total do saldo dos Estoque,MSC de Encerramento


In [148]:
d2_00053_t

,poder_orgao,valor
0,10111,"1,779,278,189.29"
1,10112,"249,494.06"
2,20211,"485,537.68"
3,20212,"2,366,758.58"
4,30390,"8,755,474.73"
5,50511,"5,525,903.18"
6,60611,"7,847,555.34"


# D2_00054

In [149]:
#filtrado2 = msc_patrimonial_encerr.query('tipo_valor == "beginning_balance"')
filtrado2 = msc_encerr.query('tipo_valor == "beginning_balance"')

eq_patrim_msc_ = filtrado2[filtrado2['conta_contabil'].str.startswith('1221')]
res_eq_patrim_msc = filtrado2[(filtrado2['conta_contabil'].str.startswith('392')) | (filtrado2['conta_contabil'].str.startswith('492'))]

d2_00054_t = pd.concat([eq_patrim_msc_, res_eq_patrim_msc])
d2_00054_t = d2_00054_t.groupby('conta_contabil').agg({'valor': 'sum'})


# Verifica se 'res_eq_patrim' não está vazio
if not res_eq_patrim_msc.empty:
    # Verifica se há algum saldo maior que zero em 'eq_patrim'
    if (eq_patrim_msc_['valor'] > 0).any():
        resposta_d2_00054 = 'OK'
    else:
        resposta_d2_00054 = 'ERRO'
else:
    resposta_d2_00054 = 'OK'


d2_00054 = pd.DataFrame([resposta_d2_00054], columns=['Resposta'])
d2_00054.insert(0, 'Dimensão', 'D2_00054')
d2_00054.insert(2, 'Descrição da Dimensão', 'Verificar se ente está registrando os investimentos permanentes, em conformidade com o PIPCP')
d2_00054.insert(3, 'OBS', 'MSC de Encerramento')
d2_00054

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00054,OK,"Verificar se ente está registrando os investimentos permanentes, em conformidade com o PIPCP",MSC de Encerramento


In [150]:
d2_00054_t

,valor
conta_contabil,
122110101,"17,260,613,811.35"
122110103,"2,139,914.25"
122110195,"1,910.40"
122110201,"3,839,771.05"
122120101,"228,066,288.04"
122130101,"321,300.76"
392110000,"2,025,286,759.51"
492110000,"49,061,685.31"
492210000,"100,136,700.69"


# D2_00055

In [151]:
software = filtrado[(filtrado['conta_contabil'].str.startswith('1241')) | (filtrado['conta_contabil'].str.startswith('1248101'))]
software = software.groupby(['poder_orgao'])['valor'].sum().reset_index()

marca = filtrado[(filtrado['conta_contabil'].str.startswith('1242')) | (filtrado['conta_contabil'].str.startswith('1248102'))]
marca = marca.groupby(['poder_orgao'])['valor'].sum().reset_index()

d2_00055_t = pd.concat([software, marca])

condicao = d2_00055_t['valor'] >= 0 

if condicao.all():
    resposta_d2_00055 = 'OK'
else:
    resposta_d2_00055 = 'ERRO'

d2_00055 = pd.DataFrame([resposta_d2_00055], columns=['Resposta'])
d2_00055.insert(0, 'Dimensão', 'D2_00055')
d2_00055.insert(2, 'Descrição da Dimensão', 'Verificar (por grupo de ativos) se a amortização acumulada de ativos intangíveis é maior que o do ativo intangível')
d2_00055.insert(3, 'OBS', 'MSC de Encerramento')
d2_00055


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00055,OK,Verificar (por grupo de ativos) se a amortização acumulada de ativos intangíveis é maior que o do ativo intangível,MSC de Encerramento


In [152]:
d2_00055_t

,poder_orgao,valor
0,10111,"226,077,083.43"
1,10112,"51,825,403.06"
2,20211,"7,161,846.21"
3,20212,"415,435.77"
4,30390,"19,935,057.29"
5,50511,"14,930,769.09"
6,60611,"47,516,464.66"
0,10111,"5,839,345.92"
1,20212,"10,812.60"
2,60611,"603,225.52"



# <span style="color: #FF5733">D2_00058 >> PERDEMOS PONTOS EM 2022</span>

In [153]:
#Erro está na DCA-HI, na conta 4.5.2.2.3.00.00 - Transferências do FUNDEB - Inter OFSS - União (valores do FNDE)

# CUIDADO na DCA (o ideal era reclassificar o saldo e mudar o nosso Plano de Contas que FNDE está dentro do FUNDEB >> grupo 4522)

#D2_00058 - Verificar a igualdade entre os valores informados de VPA do FUNDEB (União e Estados) na DCA e MSC de Encerramento
# DCA - 4.5.2.2.0.00.00 - Transferências do FUNDEB

#vpa_fundeb_msc = msc_patrimonial_encerr.query('tipo_valor == "beginning_balance" and conta_contabil == "452240000"')
vpa_fundeb_msc = msc_encerr.query('tipo_valor == "beginning_balance" and conta_contabil == "452240000"')
vpa_fundeb_msc['dimensao'] = 'D2_00058_MSC_Transferências do FUNDEB - Inter OFSS - Estado'
vpa_fundeb_msc = vpa_fundeb_msc.groupby('dimensao').agg({'valor': 'sum'})

vpa_fundeb_dcahi = df_dca_hi.query('cod_conta == "P4.5.2.2.4.00.00"')
vpa_fundeb_dcahi['dimensao'] = 'D2_00058_DCA_Transferências do FUNDEB - Inter OFSS - Estado'
vpa_fundeb_dcahi = vpa_fundeb_dcahi.groupby('dimensao').agg({'valor': 'sum'})

#vpa_fundeb_msc_u = msc_patrimonial_encerr.query('tipo_valor == "beginning_balance" and conta_contabil == "452230000"')
vpa_fundeb_msc_u = msc_encerr.query('tipo_valor == "beginning_balance" and conta_contabil == "452230000"')
vpa_fundeb_msc_u['dimensao'] = 'D2_00058_MSC_Transferências do FUNDEB - Inter OFSS - União'
vpa_fundeb_msc_u = vpa_fundeb_msc_u.groupby('dimensao').agg({'valor': 'sum'})

vpa_fundeb_dcahi_u = df_dca_hi.query('cod_conta == "P4.5.2.2.3.00.00"')
vpa_fundeb_dcahi_u['dimensao'] = 'D2_00058_DCA_Transferências do FUNDEB - Inter OFSS - União'
vpa_fundeb_dcahi_u = vpa_fundeb_dcahi_u.groupby('dimensao').agg({'valor': 'sum'})


d2_00058_t = pd.concat([vpa_fundeb_msc, vpa_fundeb_dcahi, vpa_fundeb_msc_u, vpa_fundeb_dcahi_u])

# Calcular a diferença entre os valores
d2_00058_t['diff_valor'] = d2_00058_t['valor'].diff()

d2_00058_ta = d2_00058_t.reset_index()

# Determinar os índices desejados dinamicamente
# Por exemplo, você pode querer modificar a cada dois índices
# Aqui, você pode ajustar a lógica para determinar quais índices deseja baseado em suas necessidades
indices_desejados = range(0, len(d2_00058_ta), 2)  # Isso irá gerar índices 0, 2, 4, ...
# Filtrar apenas os índices que existem no DataFrame
indices_existentes = [indice for indice in indices_desejados if indice in d2_00058_ta.index]
# Atribuição condicional
# Não é necessário verificar se a lista está vazia, pois o .loc[] lidará com uma lista vazia sem erros
d2_00058_ta.loc[indices_existentes, 'diff_valor'] = 0.0


# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d2_00058_ta['diff_valor'], 0, atol=tolerancia)

#condicao = d2_00058_ta['valor'] == 0

if condicao.any():
    resposta_d2_00058 = 'Diferença'
else:
    resposta_d2_00058 = 'OK'


# Criar DataFrame final
d2_00058 = pd.DataFrame([resposta_d2_00058], columns=['Resposta'])
d2_00058.insert(0, 'Dimensão', 'D2_00058')
d2_00058.insert(2, 'Descrição da Dimensão', 'Avalia se os valores de VPA e VPD com Fundeb estão iguais')
d2_00058.insert(3, 'OBS', 'MSC de encerramento e no Anexo I-HI da DCA')

d2_00058

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00058,OK,Avalia se os valores de VPA e VPD com Fundeb estão iguais,MSC de encerramento e no Anexo I-HI da DCA


In [154]:
# Temos problemas no Plano de CONTA com as COntas do FNDE que ficam dentro do Grupo de FUNDEB
d2_00058_ta

,dimensao,valor,diff_valor
0,D2_00058_MSC_Transferências do FUNDEB - Inter OFSS - Estado,"4,080,033,051.89",0.00
1,D2_00058_DCA_Transferências do FUNDEB - Inter OFSS - Estado,"4,080,033,051.89",0.00
2,D2_00058_MSC_Transferências do FUNDEB - Inter OFSS - União,"299,472,533.75",0.00
3,D2_00058_DCA_Transferências do FUNDEB - Inter OFSS - União,"299,472,533.75",0.00


# D2_00059

In [155]:
cred_cp = df_dca_ab.query('cod_conta == "P1.1.2.0.0.00.00"')
cred_lp = df_dca_ab.query('cod_conta == "P1.2.1.1.1.00.00" or cod_conta == "P1.2.1.1.2.00.00" or cod_conta == "P1.2.1.1.3.00.00" or cod_conta == "P1.2.1.1.4.00.00" or cod_conta == "P1.2.1.1.5.00.00"')



In [156]:
cred_cp_msc = filtrado[(filtrado['conta_contabil'].str.startswith('112'))]
cred_cp_msc = cred_cp_msc.groupby(['poder_orgao'])['valor'].sum().reset_index()

cred_lp_msc = filtrado[(filtrado['conta_contabil'].str.startswith('1211'))]
cred_lp_msc = cred_lp_msc.groupby(['poder_orgao'])['valor'].sum().reset_index()

d2_00059_t = pd.concat([cred_cp_msc, cred_lp_msc])

condicao = d2_00059_t['valor'] >= 0 

if condicao.all():
    resposta_d2_00059 = 'OK'
else:
    resposta_d2_00059 = 'ERRO'

d2_00059 = pd.DataFrame([resposta_d2_00059], columns=['Resposta'])
d2_00059.insert(0, 'Dimensão', 'D2_00059')
d2_00059.insert(2, 'Descrição da Dimensão', 'Verificar a relação entre o valor de ajuste para perdas dos "Créditos a curto e longo prazo"')
d2_00059.insert(3, 'OBS', 'MSC de Encerramento')
d2_00059



,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00059,OK,"Verificar a relação entre o valor de ajuste para perdas dos ""Créditos a curto e longo prazo""",MSC de Encerramento


In [157]:
d2_00059_t

,poder_orgao,valor
0,10111,"2,280,118,556.90"
1,10112,"368,292.01"
2,20212,"56,939,250.14"
3,30390,"203,835.55"
0,10111,"24,251,384,393.69"
1,10112,"1,130,584,749.29"
2,20211,"3,137,261.26"
3,20212,"57,002,434.65"
4,30390,"5,428,538.10"
5,50511,"2,415,763.57"


# D2_00060

In [158]:
outros_cred_cp_msc = filtrado[(filtrado['conta_contabil'].str.startswith('113'))]
outros_cred_cp_msc = outros_cred_cp_msc.groupby(['poder_orgao'])['valor'].sum().reset_index()

outros_cred_lp_msc = filtrado[(filtrado['conta_contabil'].str.startswith('1212'))]
outros_cred_lp_msc = outros_cred_lp_msc.groupby(['poder_orgao'])['valor'].sum().reset_index()

d2_00060_t = pd.concat([outros_cred_cp_msc, outros_cred_lp_msc])

condicao = d2_00060_t['valor'] >= 0 

if condicao.all():
    resposta_d2_00060 = 'OK'
else:
    resposta_d2_00060 = 'ERRO'

d2_00060 = pd.DataFrame([resposta_d2_00060], columns=['Resposta'])
d2_00060.insert(0, 'Dimensão', 'D2_00060')
d2_00060.insert(2, 'Descrição da Dimensão', 'Verificar a relação entre o valor de ajuste para perdas dos "Demais créditos e valores a curto e longo prazo"')
d2_00060.insert(3, 'OBS', 'MSC de Encerramento')
d2_00060



,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00060,OK,"Verificar a relação entre o valor de ajuste para perdas dos ""Demais créditos e valores a curto e longo prazo""",MSC de Encerramento


In [159]:
d2_00060_t

,poder_orgao,valor
0,10111,"14,598,871,967.91"
1,10112,"2,475,699,056.71"
2,20211,"81,256,356.02"
3,20212,"5,283,292.29"
4,30390,"67,913,742.50"
5,50511,"415,950.27"
6,60611,"1,521,678.08"
0,10111,"5,920,874,084.08"
1,10112,"342,510,106.85"
2,50511,"2,288,970.00"


# D2_00061

In [160]:
d2_00061_t = df_dca_hi.query('cod_conta == "P4.5.2.2.0.00.00"')
d2_00061_t = d2_00061_t.groupby(['conta'])['valor'].sum().reset_index()


condicao = d2_00061_t['valor'] >= 0 

if condicao.all():
    resposta_d2_00061 = 'OK'
else:
    resposta_d2_00061 = 'ERRO'

d2_00061 = pd.DataFrame([resposta_d2_00061], columns=['Resposta'])
d2_00061.insert(0, 'Dimensão', 'D2_00061')
d2_00061.insert(2, 'Descrição da Dimensão', 'Verificar se foi informada Variação Patrimonial Aumentativa com o FUNDEB')
d2_00061.insert(3, 'OBS', 'DCA - Anexo I-HI')
d2_00061


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00061,OK,Verificar se foi informada Variação Patrimonial Aumentativa com o FUNDEB,DCA - Anexo I-HI


In [161]:
d2_00061_t

,conta,valor
0,4.5.2.2.0.00.00 - Transferências do FUNDEB,"4,379,505,585.64"


# D2_00066

In [162]:
software = df_dca_ab.query('cod_conta == "P1.2.4.1.0.00.00" or cod_conta == "P1.2.4.8.1.01.00"')
software['tipo'] = 'software'
software = software.groupby(['tipo'])['valor'].sum().reset_index()

marca = df_dca_ab.query('cod_conta == "P1.2.4.2.0.00.00" or cod_conta == "P1.2.4.8.1.02.00"')
marca['tipo'] = 'marca'
marca = marca.groupby(['tipo'])['valor'].sum().reset_index()

d2_00066_t = pd.concat([software, marca])

condicao = d2_00066_t['valor'] >= 0 

if condicao.all():
    resposta_d2_00066 = 'OK'
else:
    resposta_d2_00066 = 'ERRO'

d2_00066 = pd.DataFrame([resposta_d2_00066], columns=['Resposta'])
d2_00066.insert(0, 'Dimensão', 'D2_00066')
d2_00066.insert(2, 'Descrição da Dimensão', 'Verificar (por grupo de ativos) se a amortização acumulada de ativos intangíveis é maior que o do ativo intangível')
d2_00066.insert(3, 'OBS', 'DCA - Anexo I-AB')
d2_00066

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00066,OK,Verificar (por grupo de ativos) se a amortização acumulada de ativos intangíveis é maior que o do ativo intangível,DCA - Anexo I-AB


In [163]:
d2_00066_t

,tipo,valor
0,software,"367,862,059.51"
0,marca,"6,453,384.04"


# D2_00067

In [164]:
bm = filtrado[(filtrado['conta_contabil'].str.startswith('1231'))]

d_bm = filtrado[(filtrado['conta_contabil'].str.startswith('1238101'))]

d2_00067_t = pd.concat([bm, d_bm])
d2_00067_t = d2_00067_t.groupby(['poder_orgao'])['valor'].sum().reset_index()

condicao = d2_00067_t['valor'] >= 0 

if condicao.all():
    resposta_d2_00067 = 'OK'
else:
    resposta_d2_00067 = 'ERRO'

d2_00067 = pd.DataFrame([resposta_d2_00067], columns=['Resposta'])
d2_00067.insert(0, 'Dimensão', 'D2_00067')
d2_00067.insert(2, 'Descrição da Dimensão', 'Verificar se os valores de depreciação de bens móveis é inferior ao valor  total de bens móveis')
d2_00067.insert(3, 'OBS', 'MSC de Encerramento')
d2_00067

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00067,OK,Verificar se os valores de depreciação de bens móveis é inferior ao valor total de bens móveis,MSC de Encerramento


In [165]:
d2_00067_t

,poder_orgao,valor
0,10111,"7,640,871,728.66"
1,10112,"33,508,188.35"
2,20211,"31,145,329.89"
3,20212,"28,600,815.40"
4,30390,"313,691,683.16"
5,50511,"76,902,055.29"
6,60611,"37,946,618.49"


# D2_00068

In [166]:
bi = filtrado[(filtrado['conta_contabil'].str.startswith('1232'))]

d_bi = filtrado[(filtrado['conta_contabil'].str.startswith('1238102'))]

d2_00068_t = pd.concat([bi, d_bi])
d2_00068_t = d2_00068_t.groupby(['poder_orgao'])['valor'].sum().reset_index()

condicao = d2_00068_t['valor'] >= 0 

if condicao.all():
    resposta_d2_00068 = 'OK'
else:
    resposta_d2_00068 = 'ERRO'

d2_00068 = pd.DataFrame([resposta_d2_00068], columns=['Resposta'])
d2_00068.insert(0, 'Dimensão', 'D2_00068')
d2_00068.insert(2, 'Descrição da Dimensão', 'Verificar se os valores de depreciação de bens imóveis é inferior ao valor  total de bens imóveis')
d2_00068.insert(3, 'OBS', 'MSC de Encerramento')
d2_00068

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00068,OK,Verificar se os valores de depreciação de bens imóveis é inferior ao valor total de bens imóveis,MSC de Encerramento


In [167]:
d2_00068_t

,poder_orgao,valor
0,10111,"38,358,363,198.74"
1,10112,"31,553,073.09"
2,20211,"346,024,607.26"
3,20212,"148,217,814.11"
4,30390,"2,960,446,654.02"
5,50511,"403,648,594.38"
6,60611,"77,476,292.78"


# D2_00069

In [168]:
previ_msc = emp_msc_encerr.query('funcao == "09" & DIGITO_INTRA != "91"')
previ_msc['dimensao'] = 'D2_00069_Previdência Social'
previ_msc = previ_msc.groupby('dimensao').agg({'valor': 'sum'})

previ_dca = df_dca_e.query('coluna == "Despesas Empenhadas" and conta == "09 - Previdência Social"')
previ_dca['dimensao'] = 'D2_00069_Previdência Social'
previ_dca = previ_dca.groupby('dimensao').agg({'valor': 'sum'})

d2_00069_t = pd.concat([previ_msc, previ_dca])
d2_00069_ta = d2_00069_t.diff().reset_index()
d2_00069_ta.loc[0, 'valor'] = 0

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d2_00069_ta['valor'], 0, atol=tolerancia)

if condicao.any():
    resposta_d2_00069 = 'Diferença'
else:
    resposta_d2_00069 = 'OK'

d2_00069 = pd.DataFrame([resposta_d2_00069], columns=['Resposta'])
# Insere novas colunas
d2_00069.insert(0, 'Dimensão', 'D2_00069')
d2_00069.insert(2, 'Descrição da Dimensão', 'Avalia se o valor de despesas exceto-intra na função 09 (Prev. Social)')
d2_00069.insert(3, 'OBS', 'MSC de Encerramento e o Anexo E da DCA')
d2_00069

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00069,OK,Avalia se o valor de despesas exceto-intra na função 09 (Prev. Social),MSC de Encerramento e o Anexo E da DCA


In [169]:
d2_00069_t

,valor
dimensao,
D2_00069_Previdência Social,"27,561,141,336.96"
D2_00069_Previdência Social,"27,561,141,336.96"


# D2_00070

In [170]:
saude_msc = emp_msc_encerr.query('funcao == "10" & DIGITO_INTRA != "91"')
saude_msc['dimensao'] = 'D2_00070_Saúde'
saude_msc = saude_msc.groupby('dimensao').agg({'valor': 'sum'})

saude_dca = df_dca_e.query('coluna == "Despesas Empenhadas" and conta == "10 - Saúde"')
saude_dca['dimensao'] = 'D2_00070_Saúde'
saude_dca = saude_dca.groupby('dimensao').agg({'valor': 'sum'})

d2_00070_t = pd.concat([saude_msc, saude_dca])
d2_00070_ta = d2_00070_t.diff().reset_index()
d2_00070_ta.loc[0, 'valor'] = 0

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d2_00070_ta['valor'], 0, atol=tolerancia)

if condicao.any():
    resposta_d2_00070 = 'Diferença'
else:
    resposta_d2_00070 = 'OK'

d2_00070 = pd.DataFrame([resposta_d2_00070], columns=['Resposta'])
# Insere novas colunas
d2_00070.insert(0, 'Dimensão', 'D2_00070')
d2_00070.insert(2, 'Descrição da Dimensão', 'Avalia se o valor de despesas exceto-intra na função 10 (Saúde)')
d2_00070.insert(3, 'OBS', 'MSC de Encerramento e o Anexo E da DCA')
d2_00070

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00070,OK,Avalia se o valor de despesas exceto-intra na função 10 (Saúde),MSC de Encerramento e o Anexo E da DCA


In [171]:
d2_00070_t

,valor
dimensao,
D2_00070_Saúde,"10,439,322,135.46"
D2_00070_Saúde,"10,439,322,135.46"


# D2_00071

In [172]:
edu_msc = emp_msc_encerr.query('funcao == "12" & DIGITO_INTRA != "91"')
edu_msc['dimensao'] = 'D2_00071_Educação'
edu_msc = edu_msc.groupby('dimensao').agg({'valor': 'sum'})

edu_dca = df_dca_e.query('coluna == "Despesas Empenhadas" and conta == "12 - Educação"')
edu_dca['dimensao'] = 'D2_00071_Educação'
edu_dca = edu_dca.groupby('dimensao').agg({'valor': 'sum'})

d2_00071_t = pd.concat([edu_msc, edu_dca])
d2_00071_ta = d2_00071_t.diff().reset_index()
d2_00071_ta.loc[0, 'valor'] = 0

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d2_00071_ta['valor'], 0, atol=tolerancia)

if condicao.any():
    resposta_d2_00071 = 'Diferença'
else:
    resposta_d2_00071 = 'OK'

d2_00071 = pd.DataFrame([resposta_d2_00071], columns=['Resposta'])
# Insere novas colunas
d2_00071.insert(0, 'Dimensão', 'D2_00071')
d2_00071.insert(2, 'Descrição da Dimensão', 'Avalia se o valor de despesas exceto-intra na função 12 (Educação)')
d2_00071.insert(3, 'OBS', 'MSC de Encerramento e o Anexo E da DCA')
d2_00071

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00071,OK,Avalia se o valor de despesas exceto-intra na função 12 (Educação),MSC de Encerramento e o Anexo E da DCA


In [173]:
d2_00071_t

,valor
dimensao,
D2_00071_Educação,"9,969,994,244.61"
D2_00071_Educação,"9,969,994,244.61"


# D2_00072

In [174]:
demais_msc = emp_msc_encerr.query('(funcao != "09" & funcao != "10" & funcao != "12") & DIGITO_INTRA != "91"')
demais_msc['dimensao'] = 'D2_00072_Demais Funções'
demais_msc = demais_msc.groupby('dimensao').agg({'valor': 'sum'})


demais_dca = df_dca_e.query('coluna == "Despesas Empenhadas" & \
                            conta not in ["09 - Previdência Social", "10 - Saúde", "12 - Educação", "Despesas Exceto Intraorçamentárias", "Despesas Intraorçamentárias"]')
# A expressão regular "^\\d{2} - " busca por strings que começam (^) com exatamente dois dígitos (\\d{2}),
# seguidos de um espaço e um traço.
demais_dca = demais_dca[demais_dca['conta'].str.match(r"^\d{2} - ")]
demais_dca['dimensao'] = 'D2_00072_Demais Funções'
demais_dca = demais_dca.groupby('dimensao').agg({'valor': 'sum'})

d2_00072_t = pd.concat([demais_msc, demais_dca])
d2_00072_ta = d2_00072_t.diff().reset_index()
d2_00072_ta.loc[0, 'valor'] = 0

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d2_00072_ta['valor'], 0, atol=tolerancia)

if condicao.any():
    resposta_d2_00072 = 'Diferença'
else:
    resposta_d2_00072 = 'OK'

d2_00072 = pd.DataFrame([resposta_d2_00072], columns=['Resposta'])
# Insere novas colunas
d2_00072.insert(0, 'Dimensão', 'D2_00072')
d2_00072.insert(2, 'Descrição da Dimensão', 'Avalia se o valor de despesas exceto-intra na Demais Funções')
d2_00072.insert(3, 'OBS', 'MSC de Encerramento e o Anexo E da DCA')
d2_00072



,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00072,OK,Avalia se o valor de despesas exceto-intra na Demais Funções,MSC de Encerramento e o Anexo E da DCA


In [175]:
d2_00072_t

,valor
dimensao,
D2_00072_Demais Funções,"52,537,793,079.49"
D2_00072_Demais Funções,"52,537,793,079.49"


# D2_00073

In [176]:
intra_msc = emp_msc_encerr.query('DIGITO_INTRA == "91"')
intra_msc['dimensao'] = 'D2_00073_Funções Intraorçamentárias'
intra_msc = intra_msc.groupby('dimensao').agg({'valor': 'sum'})

intra_dca = df_dca_e.query('coluna == "Despesas Empenhadas" & conta == "Despesas Intraorçamentárias"')
intra_dca['dimensao'] = 'D2_00073_Funções Intraorçamentárias'
intra_dca = intra_dca.groupby('dimensao').agg({'valor': 'sum'})

d2_00073_t = pd.concat([intra_msc, intra_dca])
d2_00073_ta = d2_00073_t.diff().reset_index()
d2_00073_ta.loc[0, 'valor'] = 0

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d2_00073_ta['valor'], 0, atol=tolerancia)

if condicao.any():
    resposta_d2_00073 = 'Diferença'
else:
    resposta_d2_00073 = 'OK'

d2_00073 = pd.DataFrame([resposta_d2_00073], columns=['Resposta'])
# Insere novas colunas
d2_00073.insert(0, 'Dimensão', 'D2_00073')
d2_00073.insert(2, 'Descrição da Dimensão', 'Avalia se o valor de despesas com Funções Intraorçamentárias')
d2_00073.insert(3, 'OBS', 'MSC de Encerramento e o Anexo E da DCA')
d2_00073



,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00073,OK,Avalia se o valor de despesas com Funções Intraorçamentárias,MSC de Encerramento e o Anexo E da DCA


In [177]:
d2_00073_t

,valor
dimensao,
D2_00073_Funções Intraorçamentárias,"8,018,541,679.58"
D2_00073_Funções Intraorçamentárias,"8,018,541,679.58"


# D2_00074

In [178]:
rpp_pago_dca = df_dca_f.query('cod_conta == "TotalDespesas" and (coluna == "Restos a Pagar Processados Pagos" or coluna == "Restos a Pagar Não Processados Pagos")')
rpp_pago_dca = rpp_pago_dca.groupby('coluna').agg({'valor': 'sum'})

rpnp_pago_msc = msc_encerr.query('tipo_valor == "beginning_balance" and conta_contabil == "631400000"')
rpnp_pago_msc['cod_conta'] = 'Restos a Pagar Não Processados Pagos'
rpnp_pago_msc = rpnp_pago_msc.groupby('cod_conta').agg({'valor': 'sum'})

rpp_pago_msc = msc_encerr.query('tipo_valor == "beginning_balance" and conta_contabil == "632200000"')
rpp_pago_msc['cod_conta'] = 'Restos a Pagar Processados Pagos'
rpp_pago_msc = rpp_pago_msc.groupby('cod_conta').agg({'valor': 'sum'})

rps_msc_pagos = pd.concat([rpnp_pago_msc, rpp_pago_msc])

d2_00074_t = pd.concat([rpp_pago_dca, rps_msc_pagos]).reset_index(drop=True)
d2_00074_t.loc[0, 'cod_conta'] = 'DCA F - Restos a Pagar Não Processados Pagos'  
d2_00074_t.loc[1, 'cod_conta'] = 'DCA F - Restos a Pagar Processados Pagos'
d2_00074_t.loc[2, 'cod_conta'] = 'MSC ENCERR - estos a Pagar Não Processados Pagos'
d2_00074_t.loc[3, 'cod_conta'] = 'MSC ENCERR - Restos a Pagar Processados Pagos'

# Extrair os valores
dca_rpnp_pago = d2_00074_t.iloc[0, 0]  # Primeira linha, Primeira coluna
dca_rpp_pago = d2_00074_t.iloc[1, 0]  # Segunda linha, Primeira coluna
msc_rpnp_pago = d2_00074_t.iloc[2, 0]  # Terceira linha, Primeira coluna
msc_rpp_pago = d2_00074_t.iloc[3, 0]  # Quarta linha, Primeira coluna

# Calcular a diferença
diferenca_rpp_pago = dca_rpp_pago - msc_rpp_pago
diferenca_rpnp_pago = dca_rpnp_pago - msc_rpnp_pago

# Adicionar a linha "DIF" ao final do DataFrame
d2_00074_t.loc['DIF_RPNP'] = [diferenca_rpnp_pago, 'DIF_RPNP']
d2_00074_t.loc['DIF_RPP'] = [diferenca_rpp_pago, 'DIF_RPP']

d2_00074_t['valor'] = d2_00074_t['valor'].astype(float)

# Selecionando a linha de diferença
linha_diferenca = d2_00074_t[d2_00074_t['cod_conta'] == 'DIF_RPNP'].reset_index(drop=True)
linha_diferenca2 = d2_00074_t[d2_00074_t['cod_conta'] == 'DIF_RPP'].reset_index(drop=True)

d2_00074_ta = pd.concat([linha_diferenca, linha_diferenca2])


# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d2_00074_ta['valor'], 0, atol=tolerancia)

if condicao.any():
    resposta_d2_00074 = 'ERRO'
else:
    resposta_d2_00074 = 'OK'

d2_00074 = pd.DataFrame([resposta_d2_00074], columns=['Resposta'])
# Insere novas colunas
d2_00074.insert(0, 'Dimensão', 'D2_00074')
d2_00074.insert(2, 'Descrição da Dimensão', 'Compara o saldo final de RPPP e RPNPP Pagos')
d2_00074.insert(3, 'OBS', 'MSC de Encerramento e o Anexo F da DCA')
d2_00074

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00074,OK,Compara o saldo final de RPPP e RPNPP Pagos,MSC de Encerramento e o Anexo F da DCA


In [179]:
d2_00074_t

,valor,cod_conta
0,"660,505,752.08",DCA F - Restos a Pagar Não Processados Pagos
1,"1,492,300,289.28",DCA F - Restos a Pagar Processados Pagos
2,"660,505,752.08",MSC ENCERR - estos a Pagar Não Processados Pagos
3,"1,492,300,289.28",MSC ENCERR - Restos a Pagar Processados Pagos
DIF_RPNP,0.00,DIF_RPNP
DIF_RPP,0.00,DIF_RPP


<hr style="border: none; border-top: 4px solid gray;">
<p style="color: lightgreen; font-size: 26px;"><strong>NOVAS 2023</strong></p>
<hr style="border: none; border-top: 4px solid gray;">


# D2_00076

# D2_00077

### Cria uma verificação para as D2_00077 a D2_00079

In [180]:
# Verifica se existem registros para os meses de referência 1 e 12
condicao_meses = (
    (msc_consolidada['mes_referencia'] == 1) | 
    (msc_consolidada['mes_referencia'] == 12)
)

In [181]:
# Filtra o DataFrame com base na condição e verifica se o resultado é vazio
if not msc_consolidada[condicao_meses].empty:
    # Seu código continua aqui para quando existem registros para ambos os meses
    filtro_1 = msc_consolidada[
        (msc_consolidada['tipo_valor'] == "beginning_balance") & 
        (msc_consolidada['mes_referencia'] == 1) &
        (msc_consolidada['conta_contabil'].str.startswith("227") | 
         msc_consolidada['conta_contabil'].str.startswith("228"))
    ]
    filtro_1 = filtro_1.groupby(['conta_contabil'])['valor'].sum().reset_index()

    filtro_2 = msc_consolidada[
        (msc_consolidada['tipo_valor'] == "ending_balance") & 
        (msc_consolidada['mes_referencia'] == 12) &
        (msc_consolidada['conta_contabil'].str.startswith("227") | 
         msc_consolidada['conta_contabil'].str.startswith("228"))
    ]
    filtro_2 = filtro_2.groupby(['conta_contabil'])['valor'].sum().reset_index()

    # Fazendo o merge dos DataFrames baseado em 'conta_contabil'
    d2_00077_t = pd.merge(filtro_1, filtro_2, on='conta_contabil', suffixes=('_inicio', '_fim'))

    # Calculando a diferença de 'valor' para cada 'conta_contabil'
    d2_00077_t['DIF'] = d2_00077_t['valor_fim'] - d2_00077_t['valor_inicio']

    # Condição para dar a resposta
    condicao = d2_00077_t['DIF'] == 0 

    if condicao.any():
        resposta_d2_00077 = 'ERRO'
    else:
        resposta_d2_00077 = 'OK'

    d2_00077 = pd.DataFrame([resposta_d2_00077], columns=['Resposta'])
    d2_00077.insert(0, 'Dimensão', 'D2_00077')
    d2_00077.insert(2, 'Descrição da Dimensão', 'Comparativo do saldo das contas começadas por 227 e 228')
    d2_00077.insert(3, 'OBS', 'msc de janeiro (saldo inicial) com a msc de dezembro (saldo final)')

else:
    # Cria um DataFrame com a informação de vazio se não existirem registros para ambos os meses
    d2_00077 = pd.DataFrame({
        'Dimensão': ['D2_00077'],
        'Resposta': ['VAZIO - Nenhum valor informado para a Matriz nos meses de referência 1 e 12'],
        'Descrição da Dimensão': ['Comparativo do saldo das contas começadas por 227 e 228'],
        'OBS': ['msc de janeiro (saldo inicial) com a msc de dezembro (saldo final)'],
    })

d2_00077

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00077,OK,Comparativo do saldo das contas começadas por 227 e 228,msc de janeiro (saldo inicial) com a msc de dezembro (saldo final)


In [182]:
d2_00077_t

,conta_contabil,valor_inicio,valor_fim,DIF
0,227110100,"446,671,325.54","923,090,547.76","476,419,222.22"
1,227210101,"168,949,237,379.00","329,749,142,431.90","160,799,905,052.90"
2,227210103,"-5,924,401,048.52","-11,330,042,973.70","-5,405,641,925.18"
3,227210104,"-2,723,399,608.40","-5,616,431,567.48","-2,893,031,959.08"
4,227210105,"-941,131,587.16","-2,015,029,895.16","-1,073,898,308.00"
5,227210201,"114,000,652,729.20","229,645,571,596.22","115,644,918,867.02"
6,227210202,"-13,587,185,193.17","-26,714,251,540.72","-13,127,066,347.55"
7,227210203,"-12,866,019,957.40","-25,928,704,710.70","-13,062,684,753.30"
8,227210204,"-4,074,947,543.74","-9,652,353,235.58","-5,577,405,691.84"
9,227210301,"148,287,262.89","338,269,911.86","189,982,648.97"


# D2_00079

In [183]:
# Filtra o DataFrame com base na condição e verifica se o resultado é vazio
if not msc_consolidada[condicao_meses].empty:
    # Seu código continua aqui para quando existem registros para ambos os meses
    filtro_1 = msc_consolidada[
        (msc_consolidada['tipo_valor'] == "beginning_balance") & 
        (msc_consolidada['mes_referencia'] == 1) &
        (msc_consolidada['conta_contabil'].str.startswith("119"))
    ]
    filtro_1 = filtro_1.groupby(['conta_contabil'])['valor'].sum().reset_index()

    filtro_2 = msc_consolidada[
        (msc_consolidada['tipo_valor'] == "ending_balance") & 
        (msc_consolidada['mes_referencia'] == 12) &
        (msc_consolidada['conta_contabil'].str.startswith("119"))
    ]
    filtro_2 = filtro_2.groupby(['conta_contabil'])['valor'].sum().reset_index()

    # Fazendo o merge dos DataFrames baseado em 'conta_contabil'
    d2_00079_t = pd.merge(filtro_1, filtro_2, on='conta_contabil', suffixes=('_inicio', '_fim'))

    # Calculando a diferença de 'valor' para cada 'conta_contabil'
    d2_00079_t['DIF'] = d2_00079_t['valor_fim'] - d2_00079_t['valor_inicio']

    # Condição para dar a resposta
    condicao = d2_00079_t['DIF'] == 0 

    if condicao.any():
        resposta_d2_00079 = 'ERRO'
    else:
        resposta_d2_00079 = 'OK'

    d2_00079 = pd.DataFrame([resposta_d2_00079], columns=['Resposta'])
    d2_00079.insert(0, 'Dimensão', 'D2_00079')
    d2_00079.insert(2, 'Descrição da Dimensão', 'Verifica o somatório dos saldos das contas começam com 119, na msc de janeiro serão diferentes do somatório do saldo final na msc de dezembro.')
    d2_00079.insert(3, 'OBS', 'msc de janeiro (saldo inicial) com a msc de dezembro (saldo final)')

else:
    # Cria um DataFrame com a informação de vazio se não existirem registros para ambos os meses
    d2_00079 = pd.DataFrame({
        'Dimensão': ['D2_00079'],
        'Resposta': ['VAZIO - Nenhum valor informado para a Matriz nos meses de referência 1 e 12'],
        'Descrição da Dimensão': ['Verifica o somatório dos saldos das contas começam com 119, na msc de janeiro serão diferentes do somatório do saldo final na msc de dezembro.'],
        'OBS': ['msc de janeiro (saldo inicial) com a msc de dezembro (saldo final)'],
    })

d2_00079

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00079,OK,"Verifica o somatório dos saldos das contas começam com 119, na msc de janeiro serão diferentes do somatório do saldo final na msc de dezembro.",msc de janeiro (saldo inicial) com a msc de dezembro (saldo final)


In [184]:
d2_00079_t

,conta_contabil,valor_inicio,valor_fim,DIF
0,119110100,"172,267.29","276,570.66","104,303.37"
1,119310000,"909,797.36","1,386,771.46","476,974.10"
2,119410000,"73,767.62","147,535.24","73,767.62"
3,119710000,"7,437,277.44","697,137.92","-6,740,139.52"
4,119810000,"10,087,769.84","39,983,956.78","29,896,186.94"


# D2_00080

### Cria uma verificação para as D2_00080 a D2_00083

In [185]:
# Lista para verificar a presença de todos os meses
meses_presentes = [msc_consolidada['mes_referencia'].eq(mes).any() for mes in range(1, 13)]

In [186]:
# Verifica se todos os meses estão presentes
if all(meses_presentes):
    filtro_1 = msc_consolidada[
    (msc_consolidada['tipo_valor'] == "ending_balance") & 
    (msc_consolidada['conta_contabil'].str.startswith("1156"))
]
    d2_00080_t = filtro_1.groupby(['conta_contabil', 'mes_referencia'])['valor'].sum().reset_index()

    # Primeiro, agrupamos o DataFrame filtrado por 'conta_contabil' e contamos os meses únicos para cada conta
    filtro_1 = d2_00080_t.groupby('conta_contabil')['mes_referencia'].nunique()

    # Verificamos se todas as contas têm 12 meses registrados
    condicao = filtro_1 == 12
    ## Aqui Tem que ser o ANY (e não o ALL), porque precisa apenas que tenha uma conta do grupo 1156 com saldo nos 12 meses
    if condicao.any():
        resposta_d2_00080 = 'OK'
    else:
        resposta_d2_00080 = 'ERRO'

    d2_00080 = pd.DataFrame([resposta_d2_00080], columns=['Resposta'])
    d2_00080.insert(0, 'Dimensão', 'D2_00080')
    d2_00080.insert(2, 'Descrição da Dimensão', 'Avaliação do saldo das contas contábeis começadas por 1156')
    d2_00080.insert(3, 'OBS', 'Em todas as MSCs')
else:
    # Cria um DataFrame com a informação de vazio se qualquer mês estiver faltando
    d2_00080 = pd.DataFrame({
        'Dimensão': ['D2_00080'],
        'Resposta': ['VAZIO - Informações ausentes para um ou mais meses do ano'],
        'Descrição da Dimensão': ['Avaliação do saldo das contas contábeis começadas por 1156'],
        'OBS': ['Em todas as MSCs'],
    })

d2_00080

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00080,OK,Avaliação do saldo das contas contábeis começadas por 1156,Em todas as MSCs


In [187]:
d2_00080_t

,conta_contabil,mes_referencia,valor
0,115610100,1,"1,316,657,834.06"
1,115610100,2,"1,338,396,926.18"
2,115610100,3,"1,366,890,588.98"
3,115610100,4,"1,413,070,442.23"
4,115610100,5,"1,447,775,563.13"
5,115610100,6,"1,535,724,613.29"
6,115610100,7,"1,545,731,273.96"
7,115610100,8,"1,574,910,546.96"
8,115610100,9,"1,440,343,565.21"
9,115610100,10,"1,465,497,890.41"


In [188]:
filtro_1

conta_contabil
115610100    12
115610500    12
115610900    12
Name: mes_referencia, dtype: int64

# D2_00081

In [189]:
# Verifica se todos os meses estão presentes
if all(meses_presentes):
    filtro_1 = msc_consolidada[
    (msc_consolidada['tipo_valor'] == "period_change") & 
    (msc_consolidada['natureza_conta'] == "C") &
    (msc_consolidada['conta_contabil'].str.startswith("211110102") | 
        msc_consolidada['conta_contabil'].str.startswith("211110103"))
]
    d2_00081_t = filtro_1.groupby(['conta_contabil', 'mes_referencia'])['valor'].sum().reset_index()

    # Primeiro, agrupamos o DataFrame filtrado por 'conta_contabil' e contamos os meses únicos para cada conta
    filtro_1 = d2_00081_t.groupby('conta_contabil')['mes_referencia'].nunique()

    # Verificamos se todas as contas têm 12 meses registrados
    condicao = filtro_1 == 12

    if condicao.all():
        resposta_d2_00081 = 'OK'
    else:
        resposta_d2_00081 = 'ERRO'

    d2_00081 = pd.DataFrame([resposta_d2_00081], columns=['Resposta'])
    d2_00081.insert(0, 'Dimensão', 'D2_00081')
    d2_00081.insert(2, 'Descrição da Dimensão', 'Avalia a existência de movimento credor nas contas 2.1.1.1.1.01.02 e 2.1.1.1.1.01.03')
    d2_00081.insert(3, 'OBS', 'Em todas as MSCs, utliza-se a movimentação credora')
else:
    # Cria um DataFrame com a informação de vazio se qualquer mês estiver faltando
    d2_00081 = pd.DataFrame({
        'Dimensão': ['D2_00081'],
        'Resposta': ['VAZIO - Informações ausentes para um ou mais meses do ano'],
        'Descrição da Dimensão': ['Avalia a existência de movimento credor nas contas 2.1.1.1.1.01.02 e 2.1.1.1.1.01.03'],
        'OBS': ['Em todas as MSCs, utliza-se a movimentação credora'],
    })

d2_00081

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00081,OK,Avalia a existência de movimento credor nas contas 2.1.1.1.1.01.02 e 2.1.1.1.1.01.03,"Em todas as MSCs, utliza-se a movimentação credora"


In [190]:
d2_00081_t

,conta_contabil,mes_referencia,valor
0,211110102,1,"117,287,941.51"
1,211110102,2,"284,388,050.03"
2,211110102,3,"243,727,206.96"
3,211110102,4,"179,675,555.57"
4,211110102,5,"192,336,264.39"
5,211110102,6,"1,834,470,402.43"
6,211110102,7,"172,001,673.26"
7,211110102,8,"177,030,214.26"
8,211110102,9,"86,519,658.79"
9,211110102,10,"271,125,659.67"


In [191]:
filtro_1

conta_contabil
211110102    12
211110103    12
Name: mes_referencia, dtype: int64

# D2_00082

In [192]:
# Verifica se todos os meses estão presentes
if all(meses_presentes):
    filtro_1 = msc_consolidada[
    (msc_consolidada['tipo_valor'] == "period_change") & 
    (msc_consolidada['natureza_conta'] == "C") &
    (msc_consolidada['conta_contabil'].str.startswith("1238101") | 
        msc_consolidada['conta_contabil'].str.startswith("1238103") | 
        msc_consolidada['conta_contabil'].str.startswith("1238105"))
]
    d2_00082_t = filtro_1.groupby(['conta_contabil', 'mes_referencia'])['valor'].sum().reset_index()

    # Primeiro, agrupamos o DataFrame filtrado por 'conta_contabil' e contamos os meses únicos para cada conta
    filtro_1 = d2_00082_t.groupby('conta_contabil')['mes_referencia'].nunique()

    # Verificamos se todas as contas têm 12 meses registrados
    condicao = filtro_1 == 12

    if condicao.all():
        resposta_d2_00082 = 'OK'
    else:
        resposta_d2_00082 = 'ERRO'

    d2_00082 = pd.DataFrame([resposta_d2_00082], columns=['Resposta'])
    d2_00082.insert(0, 'Dimensão', 'D2_00082')
    d2_00082.insert(2, 'Descrição da Dimensão', 'Avalia a existência de movimento credor nas contas 1.2.3.8.1.01.XX, 1.2.3.8.1.03.XX e 1.2.3.8.1.05.XX')
    d2_00082.insert(3, 'OBS', 'Em todas as MSCs, utliza-se a movimentação credora')

else:
    # Cria um DataFrame com a informação de vazio se qualquer mês estiver faltando
    d2_00082 = pd.DataFrame({
        'Dimensão': ['D2_00082'],
        'Resposta': ['VAZIO - Informações ausentes para um ou mais meses do ano'],
        'Descrição da Dimensão': ['Avalia a existência de movimento credor nas contas 1.2.3.8.1.01.XX, 1.2.3.8.1.03.XX e 1.2.3.8.1.05.XX'],
        'OBS': ['Em todas as MSCs, utliza-se a movimentação credora'],
    })

d2_00082

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00082,OK,"Avalia a existência de movimento credor nas contas 1.2.3.8.1.01.XX, 1.2.3.8.1.03.XX e 1.2.3.8.1.05.XX","Em todas as MSCs, utliza-se a movimentação credora"


In [193]:
d2_00082_t

,conta_contabil,mes_referencia,valor
0,123810199,1,"-24,830,217.33"
1,123810199,2,"-19,929,848.95"
2,123810199,3,"-307,971,763.37"
3,123810199,4,"-41,451,440.89"
4,123810199,5,"-26,192,718.68"
5,123810199,6,"-17,733,746.00"
6,123810199,7,"-42,388,724.96"
7,123810199,8,"-35,760,878.94"
8,123810199,9,"-26,571,794.58"
9,123810199,10,"-18,402,841.21"


In [194]:
filtro_1

conta_contabil
123810199    12
Name: mes_referencia, dtype: int64

In [195]:
#d2_00082_t.query('conta_contabil == "123810110"')

# D2_00083

In [196]:
# Verifica se todos os meses estão presentes
if all(meses_presentes):
    # Seu código original para filtrar e agrupar os dados
    filtro_1 = msc_consolidada[
        (msc_consolidada['tipo_valor'] == "ending_balance") & 
        (msc_consolidada['conta_contabil'].str.startswith("721"))
    ]
    filtro_1 = filtro_1.groupby(['mes_referencia'])['valor'].sum().reset_index()

    filtro_2 = msc_consolidada[
        (msc_consolidada['tipo_valor'] == "ending_balance") & 
        (msc_consolidada['conta_contabil'].str.startswith("821"))
    ]
    filtro_2 = filtro_2.groupby(['mes_referencia'])['valor'].sum().reset_index()

    # Fazendo o merge dos DataFrames baseado em 'conta_contabil'
    d2_00083_t = pd.merge(filtro_1, filtro_2, on='mes_referencia', suffixes=('_7', '_8'))

    # Calculando a diferença de 'valor' para cada 'conta_contabil'
    d2_00083_t['DIF'] = d2_00083_t['valor_7'] - d2_00083_t['valor_8']


    # Defina uma tolerância absoluta. Por exemplo, 0.00001 para considerar diferenças menores que isso como zero.
    tolerancia = 0.001

    # Use numpy.isclose para comparar cada valor com 0, considerando a tolerância definida
    condicao = np.isclose(d2_00083_t['DIF'], 0, atol=tolerancia)

    if condicao.all():
        resposta_d2_00083 = 'OK'
    else:
        resposta_d2_00083 = 'ERRO'

    d2_00083 = pd.DataFrame([resposta_d2_00083], columns=['Resposta'])
    d2_00083.insert(0, 'Dimensão', 'D2_00083')
    d2_00083.insert(2, 'Descrição da Dimensão', 'Avalia o saldo final das contas contábeis começadas por 721 e 821 é igual')
    d2_00083.insert(3, 'OBS', 'Em todas as MSCs')

else:
    # Cria um DataFrame com a informação de vazio se qualquer mês estiver faltando
    d2_00083 = pd.DataFrame({
        'Dimensão': ['D2_00083'],
        'Resposta': ['VAZIO - Informações ausentes para um ou mais meses do ano'],
        'Descrição da Dimensão': ['Avalia o saldo final das contas contábeis começadas por 721 e 821 é igual'],
        'OBS': ['Em todas as MSCs'],
    })

d2_00083

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00083,OK,Avalia o saldo final das contas contábeis começadas por 721 e 821 é igual,Em todas as MSCs


In [197]:
d2_00083_t

,mes_referencia,valor_7,valor_8,DIF
0,1,"470,446,827,137.59","470,446,827,137.59",0.00
1,2,"492,858,667,653.54","492,858,667,653.54",-0.00
2,3,"510,463,146,571.89","510,463,146,571.89",0.00
3,4,"532,617,684,092.76","532,617,684,092.76",0.00
4,5,"554,736,953,946.67","554,736,953,946.67",0.00
5,6,"573,601,807,022.81","573,601,807,022.81",0.00
6,7,"591,717,438,488.42","591,717,438,488.42",0.00
7,8,"615,615,860,140.93","615,615,860,140.93",-0.00
8,9,"639,032,099,568.24","639,032,099,568.24",0.00
9,10,"659,858,441,464.87","659,858,441,464.87",0.00


# D2_00084

### Condição para verificar se Existe MSC de DEZ

In [198]:
# Verifica se existem registros para os meses de referência 1 e 12
condicao_msc_dez = (
    (msc['mes_referencia'] == 12)
)

In [199]:

# Verifica se a MSC de DEZ não foi enviada
if msc[condicao_msc_dez].empty:
    d2_00084 = pd.DataFrame({
        'Dimensão': ['D2_00084'],
        'Resposta': ['VAZIO - Nenhum valor informado para a MSC de Dezembro'],
        'Descrição da Dimensão': ['Avalia se o total de passivo financeiro + permanente na DCA é maior ou igual ao total de ativo circulante + passivo circulante na MSC de dezembro'],
        'OBS': ['Anexo I-AB da DCA x MSC DEZ'],
        'Nota': ['0']
    })
else:
    fin_perm_dca = df_dca_ab.query('cod_conta == "PassivoFinanceiro" or cod_conta == "PassivoPermanente"')
    filtro_fin_perm_dca = fin_perm_dca['valor'].sum()

    circ_nao_circ_msc = msc[
    (msc['tipo_valor'] == "ending_balance") & 
    (msc['mes_referencia'] == 12) &
    (msc['conta_contabil'].str.startswith("21") | 
        msc['conta_contabil'].str.startswith("22"))
    ]
    filtro_circ_nao_circ_msc = circ_nao_circ_msc['valor'].sum()


    # Criando a Tabela
    fin_perm_dca['cod'] = 'passivo financeiro e passivo permanente da DCA'
    filtro_1 = fin_perm_dca.groupby(['cod'])['valor'].sum().reset_index()

    circ_nao_circ_msc['cod'] = 'passivo circulante e não circulante do saldo final da MSC de dezembro'
    filtro_2 = circ_nao_circ_msc.groupby(['cod'])['valor'].sum().reset_index()
    

    d2_00084_t = pd.concat([filtro_1, filtro_2])


    if filtro_fin_perm_dca >= filtro_circ_nao_circ_msc:
        resposta_d2_00084 = 'OK'
    else:
        resposta_d2_00084 = 'ERRO'

    d2_00084 = pd.DataFrame([resposta_d2_00084], columns=['Resposta'])
    d2_00084.insert(0, 'Dimensão', 'D2_00084')
    d2_00084.insert(2, 'Descrição da Dimensão', 'Avalia se o total de passivo financeiro + permanente na DCA é maior ou igual ao total de ativo circulante + passivo circulante na MSC de dezembro')
    d2_00084.insert(3, 'OBS', 'Anexo I-AB da DCA x MSC DEZ')

d2_00084

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00084,ERRO,Avalia se o total de passivo financeiro + permanente na DCA é maior ou igual ao total de ativo circulante + passivo circulante na MSC de dezembro,Anexo I-AB da DCA x MSC DEZ


In [200]:
d2_00084_t

,cod,valor
0,passivo financeiro e passivo permanente da DCA,"300,660,387,391.91"
0,passivo circulante e não circulante do saldo final da MSC de dezembro,"305,237,746,341.49"


In [201]:
teste = msc[msc['conta_contabil'].str.startswith("22")]
teste.query('natureza_conta == "D" and tipo_valor == "ending_balance" and mes_referencia == 12')


,tipo_matriz,cod_ibge,classe_conta,conta_contabil,poder_orgao,financeiro_permanente,ano_fonte_recursos,fonte_recursos,exercicio,mes_referencia,...,valor,natureza_conta,tipo_valor,complemento_fonte,funcao,subfuncao,educacao_saude,natureza_despesa,ano_inscricao,natureza_receita
51502,MSCC,33,2,222810100,10111,2.00,NaN,None,2024,12,...,"-7,364,593,183.63",D,ending_balance,None,NaN,NaN,NaN,NaN,NaN,NaN
51503,MSCC,33,2,222830100,10111,2.00,NaN,None,2024,12,...,"-112,409,772,559.53",D,ending_balance,None,NaN,NaN,NaN,NaN,NaN,NaN
51504,MSCC,33,2,222910100,10111,2.00,NaN,None,2024,12,...,"-5,782,723,738.00",D,ending_balance,None,NaN,NaN,NaN,NaN,NaN,NaN
51520,MSCC,33,2,227210103,10112,2.00,NaN,None,2024,12,...,"-5,665,021,486.85",D,ending_balance,None,NaN,NaN,NaN,NaN,NaN,NaN
51521,MSCC,33,2,227210104,10112,2.00,NaN,None,2024,12,...,"-2,808,215,783.74",D,ending_balance,None,NaN,NaN,NaN,NaN,NaN,NaN
51522,MSCC,33,2,227210105,10112,2.00,NaN,None,2024,12,...,"-1,007,514,947.58",D,ending_balance,None,NaN,NaN,NaN,NaN,NaN,NaN
51524,MSCC,33,2,227210202,10112,2.00,NaN,None,2024,12,...,"-13,357,125,770.36",D,ending_balance,None,NaN,NaN,NaN,NaN,NaN,NaN
51525,MSCC,33,2,227210203,10112,2.00,NaN,None,2024,12,...,"-12,964,352,355.35",D,ending_balance,None,NaN,NaN,NaN,NaN,NaN,NaN
51526,MSCC,33,2,227210204,10112,2.00,NaN,None,2024,12,...,"-4,826,176,617.79",D,ending_balance,None,NaN,NaN,NaN,NaN,NaN,NaN
51528,MSCC,33,2,227210305,10112,2.00,NaN,None,2024,12,...,"-498,049.21",D,ending_balance,None,NaN,NaN,NaN,NaN,NaN,NaN


# D2_00085

In [202]:
# Verifica se a DCA esta vazia de fato
if df_dca_ab.empty:
    d2_00085 = pd.DataFrame({
        'Dimensão': ['D2_00085'],
        'Resposta': ['VAZIO - Nenhum valor informado para a DCA'],
        'Descrição da Dimensão': ['Avalia se o total de passivo financeiro + permanente na DCA é maior ou igual ao total de ativo circulante + passivo circulante na DCA'],
        'OBS': ['Anexo I-AB da DCA'],
        'Nota': ['0']
    })
else:
    fin_perm_dca = df_dca_ab.query('cod_conta == "PassivoFinanceiro" or cod_conta == "PassivoPermanente"')
    filtro_fin_perm_dca = fin_perm_dca['valor'].sum()

    circ_nao_circ_dca = df_dca_ab.query('cod_conta == "P2.1.0.0.0.00.00" or cod_conta == "P2.2.0.0.0.00.00"')
    filtro_circ_nao_circ_dca = circ_nao_circ_dca['valor'].sum()


    fin_perm_dca['cod'] = 'passivo financeiro e passivo permanente da DCA'
    filtro_1 = fin_perm_dca.groupby(['cod'])['valor'].sum().reset_index()

    circ_nao_circ_dca['cod'] = 'passivo circulante e não circulante da DCA'
    filtro_2 = circ_nao_circ_dca.groupby(['cod'])['valor'].sum().reset_index()
    

    d2_00085_t = pd.concat([filtro_1, filtro_2])


    if filtro_fin_perm_dca >= filtro_circ_nao_circ_dca:
        resposta_d2_00085 = 'OK'
    else:
        resposta_d2_00085 = 'ERRO'

    d2_00085 = pd.DataFrame([resposta_d2_00085], columns=['Resposta'])
    d2_00085.insert(0, 'Dimensão', 'D2_00085')
    d2_00085.insert(2, 'Descrição da Dimensão', 'Avalia se o total de passivo financeiro + permanente na DCA é maior ou igual ao total de ativo circulante + passivo circulante na DCA')
    d2_00085.insert(3, 'OBS', 'Anexo I-AB da DCA')
 
d2_00085

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00085,OK,Avalia se o total de passivo financeiro + permanente na DCA é maior ou igual ao total de ativo circulante + passivo circulante na DCA,Anexo I-AB da DCA


In [203]:
d2_00085_t

,cod,valor
0,passivo financeiro e passivo permanente da DCA,"300,660,387,391.91"
0,passivo circulante e não circulante da DCA,"299,392,641,728.06"


<hr style="border: none; border-top: 4px solid gray;">
<p style="color: orange; font-size: 26px;"><strong>NOVAS 2024</strong></p>
<hr style="border: none; border-top: 4px solid gray;">


# D2_00090 >> Verifica a relação entre o valor de ajuste para perdas dos "Demais créditos e valores a curto e longo prazo" e suas contas principais. Se há valores para créditos a receber, deve haver valor de ajustes de perdas respectivos, então, estas rubricas não podem estar zeradas.

In [204]:
# A princípio a análise é Só do Nível 1 OFSS
demais_cred_cp_1 = df_dca_ab_orig.query('cod_conta == "P1.1.3.1.1.00.00" or cod_conta == "P1.1.3.2.1.00.00" or cod_conta == "P1.1.3.3.1.00.00" or cod_conta == "P1.1.3.4.1.00.00" \
                                         or cod_conta == "P1.1.3.5.1.00.00" or cod_conta == "P1.1.3.6.1.00.00" or cod_conta == "P1.1.3.8.1.00.00"')
demais_cred_cp_1['dimensao'] = 'D2_00090_Demais Créditos CP - OFSS 1'
demais_cred_cp_1 = demais_cred_cp_1.groupby('dimensao').agg({'valor': 'sum'})

demais_cred_lp_1 = df_dca_ab_orig.query('cod_conta == "P1.2.1.2.1.01.00" or cod_conta == "P1.2.1.2.1.02.00" or cod_conta == "P1.2.1.2.1.03.00" or cod_conta == "P1.2.1.2.1.04.00" \
                                         or cod_conta == "P1.2.1.2.1.05.00" or cod_conta == "P1.2.1.2.1.06.00" or cod_conta == "P1.2.1.2.1.07.00" \
                                        or cod_conta == "P1.2.1.2.1.09.00" or cod_conta == "P1.2.1.2.1.98.00"')
demais_cred_lp_1['dimensao'] = 'D2_00090_Demais Créditos LP - OFSS 1'
demais_cred_lp_1 = demais_cred_lp_1.groupby('dimensao').agg({'valor': 'sum'})

ajuste_demais_cred_cp_1 = df_dca_ab_orig.query('cod_conta == "P1.1.3.9.1.00.00"')
ajuste_demais_cred_cp_1['dimensao'] = 'D2_00090_Ajuste Demais Créditos CP - OFSS 1'
ajuste_demais_cred_cp_1 = ajuste_demais_cred_cp_1.groupby('dimensao').agg({'valor': 'sum'})

ajuste_demais_cred_lp_1 = df_dca_ab_orig.query('cod_conta == "P1.2.1.2.1.99.00"')
ajuste_demais_cred_lp_1['dimensao'] = 'D2_00090_Ajuste Demais Créditos LP - OFSS 1'
ajuste_demais_cred_lp_1 = ajuste_demais_cred_lp_1.groupby('dimensao').agg({'valor': 'sum'})

d2_00090_t = pd.concat([demais_cred_cp_1, demais_cred_lp_1, ajuste_demais_cred_cp_1, ajuste_demais_cred_lp_1])
d2_00090_t = d2_00090_t.reset_index()

condicao1 = d2_00090_t['valor'] != 0
condicao2 = not ajuste_demais_cred_cp_1.empty and (demais_cred_cp_1['valor'] != 0).all()
condicao3 = not ajuste_demais_cred_lp_1.empty and (demais_cred_lp_1['valor'] != 0).all()

if condicao1.any() and condicao2 and condicao3:
    resposta_d2_00090 = 'OK'
else:
    resposta_d2_00090 = 'ERRO'

d2_00090 = pd.DataFrame([resposta_d2_00090], columns=['Resposta'])
d2_00090.insert(0, 'Dimensão', 'D2_00090')
d2_00090.insert(2, 'Descrição da Dimensão', 'Verifica a relação entre o valor de ajuste para perdas dos "Demais créditos e valores a curto e longo prazo" e suas contas principais.')
d2_00090.insert(3, 'OBS', 'Anexo I-AB da DCA')

d2_00090

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00090,ERRO,"Verifica a relação entre o valor de ajuste para perdas dos ""Demais créditos e valores a curto e longo prazo"" e suas contas principais.",Anexo I-AB da DCA


In [205]:
d2_00090_t

,dimensao,valor
0,D2_00090_Demais Créditos CP - OFSS 1,"14,352,285,908.33"
1,D2_00090_Demais Créditos LP - OFSS 1,"3,555,394,001.92"
2,D2_00090_Ajuste Demais Créditos CP - OFSS 1,"2,159,948,016.64"


### Importar a Tabela e Criar as Variáveis para as Emendas

In [206]:
emendas = pd.read_csv('Emendas-Parlamentares.csv', sep=';', encoding='latin1')
emendas_rj = emendas.query('`Código IBGE` == 33 and Ano == 2024')
emendas_rj_individual = emendas_rj.query('`Nome Emenda` == "Emenda Individual"')
emendas_rj_bancada = emendas_rj.query('`Nome Emenda` == "Emenda de Bancada"')

# Remove pontos (separadores de milhar) e substitui vírgulas (decimais) por ponto
emendas_rj_individual['Valor'] = (
    emendas_rj_individual['Valor']
    .astype(str)
    .str.replace('.', '', regex=False)  # remove pontos
    .str.replace(',', '.', regex=False)  # troca vírgula por ponto
    .astype(float)  # converte para float
)

# Remove pontos (separadores de milhar) e substitui vírgulas (decimais) por ponto
emendas_rj_bancada['Valor'] = (
    emendas_rj_bancada['Valor']
    .astype(str)
    .str.replace('.', '', regex=False)  # remove pontos
    .str.replace(',', '.', regex=False)  # troca vírgula por ponto
    .astype(float)  # converte para float
)


FileNotFoundError: [Errno 2] No such file or directory: 'Emendas-Parlamentares.csv'

# D2_00096 >> Verificar se o valor total transferido pela União por meio de emendas parlamentares individuais é menor ou igual ao valor total de receitas decorrentes dessas emendas registrados pelo ente.

In [ ]:
emendas_rj

,Nome Ente,UF,Código Siafi,Código IBGE,Data,Ano,Mês,Tipo Ente,OB,CNPJ do Favorecido,Nome Favorecido,Nome Emenda,Transferência Especial,Categoria Econômica Despesa,Valor
165136,Rio de Janeiro,RJ,RJ,33,45352,2024,março,Estado,200330000012024OB000036,32690668000102,SECRETARIA DE ESTADO DE POLICIA MILITAR - SEPM,Emenda Individual,Não,DESPESAS DE CAPITAL,"1320132,8"
165158,Rio de Janeiro,RJ,RJ,33,45352,2024,março,Estado,200330000012024OB000070,32690668000102,SECRETARIA DE ESTADO DE POLICIA MILITAR - SEPM,Emenda de Bancada,Não,DESPESAS DE CAPITAL,"1173355,29"
165159,Rio de Janeiro,RJ,RJ,33,45352,2024,março,Estado,200330000012024OB000071,32690668000102,SECRETARIA DE ESTADO DE POLICIA MILITAR - SEPM,Emenda de Bancada,Não,DESPESAS DE CAPITAL,1255530
165946,Rio de Janeiro,RJ,RJ,33,45352,2024,março,Estado,257001000012024OB005508,35949791000185,FUNDO ESTADUAL DE SAUDE FES,Emenda Individual,Não,DESPESAS DE CAPITAL,717093
165947,Rio de Janeiro,RJ,RJ,33,45352,2024,março,Estado,257001000012024OB005508,35949791000185,FUNDO ESTADUAL DE SAUDE FES,Emenda Individual,Não,DESPESAS DE CAPITAL,822483
165948,Rio de Janeiro,RJ,RJ,33,45352,2024,março,Estado,257001000012024OB005508,35949791000185,FUNDO ESTADUAL DE SAUDE FES,Emenda Individual,Não,DESPESAS DE CAPITAL,999453
165949,Rio de Janeiro,RJ,RJ,33,45352,2024,março,Estado,257001000012024OB005508,35949791000185,FUNDO ESTADUAL DE SAUDE FES,Emenda Individual,Não,DESPESAS DE CAPITAL,1001451
165950,Rio de Janeiro,RJ,RJ,33,45352,2024,março,Estado,257001000012024OB005508,35949791000185,FUNDO ESTADUAL DE SAUDE FES,Emenda Individual,Não,DESPESAS DE CAPITAL,2956242
167145,Rio de Janeiro,RJ,RJ,33,45413,2024,maio,Estado,200330000012024OB000189,28176998000107,SECRETARIA DE ESTADO DE DEFESA CIVIL,Emenda Individual,Não,DESPESAS DE CAPITAL,"390150,51"
167146,Rio de Janeiro,RJ,RJ,33,45413,2024,maio,Estado,200330000012024OB000190,28176998000107,SECRETARIA DE ESTADO DE DEFESA CIVIL,Emenda Individual,Não,DESPESAS DE CAPITAL,584000


In [ ]:
msc.query('complemento_fonte == "3110"')

,tipo_matriz,cod_ibge,classe_conta,conta_contabil,poder_orgao,financeiro_permanente,ano_fonte_recursos,fonte_recursos,exercicio,mes_referencia,...,valor,natureza_conta,tipo_valor,complemento_fonte,funcao,subfuncao,educacao_saude,natureza_despesa,ano_inscricao,natureza_receita
2277,MSCC,33,1,111111900,10111,1.0,1.0,1706,2024,4,...,36.50,D,ending_balance,3110,NaN,NaN,NaN,NaN,NaN,NaN
2355,MSCC,33,1,111115000,10111,1.0,1.0,1706,2024,4,...,"3,757,502.64",D,ending_balance,3110,NaN,NaN,NaN,NaN,NaN,NaN
3079,MSCC,33,1,111111900,10111,1.0,1.0,1706,2024,5,...,36.50,D,ending_balance,3110,NaN,NaN,NaN,NaN,NaN,NaN
3158,MSCC,33,1,111115000,10111,1.0,1.0,1706,2024,5,...,"3,757,502.64",D,ending_balance,3110,NaN,NaN,NaN,NaN,NaN,NaN
3890,MSCC,33,1,111111900,10111,1.0,1.0,1706,2024,6,...,36.50,D,ending_balance,3110,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1301291,MSCC,33,8,821110100,10111,NaN,1.0,1706,2024,12,...,"3,846,002.12",D,period_change,3110,NaN,NaN,NaN,NaN,NaN,NaN
1301684,MSCC,33,8,821130100,10111,NaN,1.0,1706,2024,12,...,"2,343,001.06",C,period_change,3110,NaN,NaN,NaN,NaN,NaN,NaN
1301685,MSCC,33,8,821130100,10111,NaN,1.0,1706,2024,12,...,"840,000.00",D,period_change,3110,NaN,NaN,NaN,NaN,NaN,NaN
1302020,MSCC,33,8,821140100,10111,NaN,1.0,1706,2024,12,...,0.00,C,period_change,3110,NaN,NaN,NaN,NaN,NaN,NaN


# D2_00097 >> Verificar se o valor total transferido pela União por meio de transferências especiais de emendas individuais é menor ou igual ao valor total de receitas decorrentes dessas transferências registrados pelo ente.

In [ ]:
emendas_rj_individual_97 = emendas_rj_individual.copy()
emendas_rj_individual_97['dimensao'] = 'D2_00097_Emendas Individual Portal'
emendas_rj_individual_97 = emendas_rj_individual_97[['Código IBGE', 'dimensao', 'Valor']]
emendas_rj_individual_97 = emendas_rj_individual_97.groupby(['Código IBGE', 'dimensao'])['Valor'].sum().reset_index()

emendas_rj_individual_97 = emendas_rj_individual_97.rename(columns={
    'Código IBGE': 'cod_ibge',
    'Valor': 'valor'
})

msc_97 = msc_dez.copy()
msc_97 = msc_97.query('conta_contabil == "621200000" and complemento_fonte == "3110"')
msc_97['dimensao'] = 'D2_00097_Emendas Individual MSC'
msc_97 = msc_97[['cod_ibge', 'dimensao', 'valor']]
msc_97 = msc_97.groupby(['cod_ibge', 'dimensao'])['valor'].sum().reset_index()

#d2_00097_t = pd.concat([emendas_rj_individual_97, msc_97])
d2_00097_t = pd.concat([msc_97, emendas_rj_individual_97]).reset_index(drop=True)

d2_00097_t['DIF'] = d2_00097_t['valor'] - d2_00097_t['valor'].shift(1)
d2_00097_t.loc[0, 'DIF'] = 0  


if len(d2_00097_t) >= 2 and (d2_00097_t['DIF'] >= 0).all():
    resposta_d2_00097 = 'OK'
else:
    resposta_d2_00097 = 'ERRO'

d2_00097 = pd.DataFrame([resposta_d2_00097], columns=['Resposta'])
d2_00097.insert(0, 'Dimensão', 'D2_00097')
d2_00097.insert(2, 'Descrição da Dimensão', 'Verificar se o valor total transferido pela União por meio de transferências especiais de emendas individuais é menor ou igual ao valor total de receitas decorrentes dessas transferências registrados pelo ente.')
d2_00097.insert(3, 'OBS', 'MSC DEZ X Relatório de Transferências da União')
d2_00097

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00097,OK,Verificar se o valor total transferido pela União por meio de transferências especiais de emendas individuais é menor ou igual ao valor total de receitas decorrentes dessas transferências registrados pelo ente.,MSC DEZ X Relatório de Transferências da União


In [ ]:
d2_00097_t

,cod_ibge,dimensao,valor,DIF
0,33,D2_00097_Emendas Individual MSC,"16,420,184.26",0.00
1,33,D2_00097_Emendas Individual Portal,"41,757,105.42","25,336,921.16"


# D2_00098 >> Verificar se o valor total transferido pela União por meio de emendas parlamentares de bancada é maior ou igual ao valor total de receitas decorrentes dessas emendas registrados pelo ente.

In [207]:
emendas_rj_bancada_98 = emendas_rj_bancada.copy()
emendas_rj_bancada_98['dimensao'] = 'D2_00098_Emendas Bancada Portal'
emendas_rj_bancada_98 = emendas_rj_bancada_98[['Código IBGE', 'dimensao', 'Valor']]
emendas_rj_bancada_98 = emendas_rj_bancada_98.groupby(['Código IBGE', 'dimensao'])['Valor'].sum().reset_index()

emendas_rj_bancada_98 = emendas_rj_bancada_98.rename(columns={
    'Código IBGE': 'cod_ibge',
    'Valor': 'valor'
})

msc_98 = msc_dez.copy()
msc_98 = msc_98.query('conta_contabil == "621200000" and complemento_fonte == "3120"')
msc_98['dimensao'] = 'D2_00098_Emendas Bancada MSC'
msc_98 = msc_98[['cod_ibge', 'dimensao', 'valor']]
msc_98 = msc_98.groupby(['cod_ibge', 'dimensao'])['valor'].sum().reset_index()

d2_00098_t = pd.concat([msc_98, emendas_rj_bancada_98]).reset_index(drop=True)

d2_00098_t['DIF'] = d2_00098_t['valor'] - d2_00098_t['valor'].shift(1)
d2_00098_t.loc[0, 'DIF'] = 0  

if len(d2_00098_t) >= 2 and (d2_00098_t['DIF'] >= 0).all():
    resposta_d2_00098 = 'OK'
else:
    resposta_d2_00098 = 'ERRO'

d2_00098 = pd.DataFrame([resposta_d2_00098], columns=['Resposta'])
d2_00098.insert(0, 'Dimensão', 'D2_00098')
d2_00098.insert(2, 'Descrição da Dimensão', 'Verificar se o valor total transferido pela União por meio de emendas parlamentares de bancada é maior ou igual ao valor total de receitas decorrentes dessas emendas registrados pelo ente.')
d2_00098.insert(3, 'OBS', 'MSC DEZ X Relatório de Transferências da União')
d2_00098

NameError: name 'emendas_rj_bancada' is not defined

In [208]:
d2_00098_t

NameError: name 'd2_00098_t' is not defined

In [ ]:
d2 = pd.concat([d2_00002, d2_00003, d2_00004, d2_00005, d2_00006, d2_00007, d2_00008, d2_00010, d2_00011, d2_00012, d2_00013, 
                d2_00014, d2_00015, d2_00016, d2_00017, d2_00018, d2_00019, d2_00020, d2_00021, d2_00023, d2_00024, 
                d2_00028, d2_00029, d2_00030, d2_00031,
                d2_00032, d2_00033, d2_00034, d2_00035, d2_00036, d2_00037, d2_00038, d2_00039, d2_00040, d2_00044, d2_00045, 
                d2_00047, d2_00049, d2_00050, d2_00051, d2_00052, d2_00053, d2_00054, d2_00055, d2_00058, d2_00059, d2_00060, 
                d2_00061, d2_00066, d2_00067, d2_00068,
                d2_00069, d2_00070, d2_00071, d2_00072, d2_00073, d2_00074, d2_00077, d2_00079, d2_00080, d2_00081, 
                d2_00082, d2_00083, d2_00084, d2_00085, d2_00090, d2_00097, d2_00098
               ]).reset_index(drop=True)

d2

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D2_00002,OK,Verifica se foi informado o valor de VPD do FUNDEB,Anexo I-HI da DCA
1,D2_00003,OK,Verifica se foi informado o valor de Deduções de Receitas para formação do FUNDEB,Anexo I-C da DCA
2,D2_00004,OK,Verifica se foi informado o valor de Receitas Orçamentárias do FUNDEB,Anexo I-C da DCA
3,D2_00005,OK,Verifica se foi informado o valor de Despesas Orçamentárias com Encargos Patronais,Anexo I-D da DCA
4,D2_00006,OK,Verifica se foi informado o valor de Despesas Orçamentárias com Pessoal,Anexo I-D da DCA
...,...,...,...,...
63,D2_00084,ERRO,Avalia se o total de passivo financeiro + permanente na DCA é maior ou igual ao total de ativo circulante + passivo circulante na MSC de dezembro,Anexo I-AB da DCA x MSC DEZ
64,D2_00085,OK,Avalia se o total de passivo financeiro + permanente na DCA é maior ou igual ao total de ativo circulante + passivo circulante na DCA,Anexo I-AB da DCA
65,D2_00090,ERRO,"Verifica a relação entre o valor de ajuste para perdas dos ""Demais créditos e valores a curto e longo prazo"" e suas contas principais.",Anexo I-AB da DCA
66,D2_00097,OK,Verificar se o valor total transferido pela União por meio de transferências especiais de emendas individuais é menor ou igual ao valor total de receitas decorrentes dessas transferências registrados pelo ente.,MSC DEZ X Relatório de Transferências da União


In [ ]:
contagem_total_dim = d2.Dimensão.unique()
contagem_total_dim = len(contagem_total_dim)
contagem_total_dim

68

In [ ]:
total_dim = 66
dim_falta = total_dim - contagem_total_dim
dim_falta

-2

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

## D3_00001

In [ ]:
###@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
##@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ Criando As Dimensões do RREO Anexo 1 para Cálculo do Superavit ou Defcit

# Superavit ou Defcit EMPENHO
rec_rreo_1 = df_rreo_1.query('coluna == "Até o Bimestre (c)" & cod_conta == "TotalReceitas"')
rec_rreo_1['dimensao'] = 'D3_00001_Superavit ou Defcit_ Empenhado'
rec_rreo_1 = rec_rreo_1.groupby('dimensao').agg({'valor': 'sum'})

dps_rreo_1 = df_rreo_1.query('coluna == "DESPESAS EMPENHADAS ATÉ O BIMESTRE (f)" & cod_conta == "TotalDespesas"')
dps_rreo_1['dimensao'] = 'D3_00001_Superavit ou Defcit_ Empenhado'
dps_rreo_1 = dps_rreo_1.groupby('dimensao').agg({'valor': 'sum'})

sup_ou_def_rreo_1 = df_rreo_1.query('coluna == "DESPESAS EMPENHADAS ATÉ O BIMESTRE (f)" & cod_conta == "Superavit"')
sup_ou_def_rreo_1['dimensao'] = 'D3_00001_Superavit ou Defcit_ Empenhado'
sup_ou_def_rreo_1 = sup_ou_def_rreo_1.groupby('dimensao')['valor'].sum().to_frame()

d3_00001_e = rec_rreo_1.merge(dps_rreo_1, on='dimensao')
d3_00001_e['DIF'] = d3_00001_e['valor_x'] - d3_00001_e['valor_y']
d3_00001_e.columns = ['REC', 'DPS EMP', 'DIF']

d3_00001_final1 = d3_00001_e.merge(sup_ou_def_rreo_1, on='dimensao')
d3_00001_final1['DIF Final'] = d3_00001_final1['DIF'] - d3_00001_final1['valor']
d3_00001_final1.columns = ['REC', 'DPS EMP', 'DIF', 'Superávit ou Défcit', 'DIF Final']


# Superavit ou Defcit LIQUIDAÇÃO
rec_rreo_1 = df_rreo_1.query('coluna == "Até o Bimestre (c)" & cod_conta == "TotalReceitas"')
rec_rreo_1['dimensao'] = 'D3_00001_Superavit ou Defcit_ Liquidado'
rec_rreo_1 = rec_rreo_1.groupby('dimensao').agg({'valor': 'sum'})

dps_rreo_1 = df_rreo_1.query('coluna == "DESPESAS LIQUIDADAS ATÉ O BIMESTRE (h)" & cod_conta == "TotalDespesas"')
dps_rreo_1['dimensao'] = 'D3_00001_Superavit ou Defcit_ Liquidado'
dps_rreo_1 = dps_rreo_1.groupby('dimensao').agg({'valor': 'sum'})

sup_ou_def_rreo_1 = df_rreo_1.query('coluna == "DESPESAS LIQUIDADAS ATÉ O BIMESTRE (h)" & cod_conta == "Superavit"')
sup_ou_def_rreo_1['dimensao'] = 'D3_00001_Superavit ou Defcit_ Liquidado'
sup_ou_def_rreo_1 = sup_ou_def_rreo_1.groupby('dimensao')['valor'].sum().to_frame()

d3_00001_l = rec_rreo_1.merge(dps_rreo_1, on='dimensao')
d3_00001_l['DIF'] = d3_00001_l['valor_x'] - d3_00001_l['valor_y']
d3_00001_l.columns = ['REC', 'DPS EMP', 'DIF']

d3_00001_final2 = d3_00001_l.merge(sup_ou_def_rreo_1, on='dimensao')
d3_00001_final2['DIF Final'] = d3_00001_final2['DIF'] - d3_00001_final2['valor']
d3_00001_final2.columns = ['REC', 'DPS EMP', 'DIF', 'Superávit ou Défcit', 'DIF Final']


# Superavit ou Defcit PAGAS
rec_rreo_1 = df_rreo_1.query('coluna == "Até o Bimestre (c)" & cod_conta == "TotalReceitas"')
rec_rreo_1['dimensao'] = 'D3_00001_Superavit ou Defcit_ Pago'
rec_rreo_1 = rec_rreo_1.groupby('dimensao').agg({'valor': 'sum'})

dps_rreo_1 = df_rreo_1.query('coluna == "DESPESAS PAGAS ATÉ O BIMESTRE (j)" & cod_conta == "TotalDespesas"')
dps_rreo_1['dimensao'] = 'D3_00001_Superavit ou Defcit_ Pago'
dps_rreo_1 = dps_rreo_1.groupby('dimensao').agg({'valor': 'sum'})

sup_ou_def_rreo_1 = df_rreo_1.query('coluna == "DESPESAS PAGAS ATÉ O BIMESTRE (j)" & cod_conta == "Superavit"')
sup_ou_def_rreo_1['dimensao'] = 'D3_00001_Superavit ou Defcit_ Pago'
sup_ou_def_rreo_1 = sup_ou_def_rreo_1.groupby('dimensao')['valor'].sum().to_frame()

d3_00001_p = rec_rreo_1.merge(dps_rreo_1, on='dimensao')
d3_00001_p['DIF'] = d3_00001_p['valor_x'] - d3_00001_p['valor_y']
d3_00001_p.columns = ['REC', 'DPS EMP', 'DIF']

d3_00001_final3 = d3_00001_p.merge(sup_ou_def_rreo_1, on='dimensao')
d3_00001_final3['DIF Final'] = d3_00001_final3['DIF'] - d3_00001_final3['valor']
d3_00001_final3.columns = ['REC', 'DPS EMP', 'DIF', 'Superávit ou Défcit', 'DIF Final']

d3_00001_t = pd.concat([d3_00001_final1, d3_00001_final2, d3_00001_final3])

limiar = 1e-2  # Definir um LIMITE (número decimal 0.01 em notação científica)
# Substituir valores abaixo do limiar por zero (Isso significa que os valores que estão dentro do limiar tanto acima quanto abaixo de zero serão substituídos por zero)
d3_00001_t['DIF Final'] = d3_00001_t['DIF Final'].apply(lambda x: 0 if abs(x) < limiar else x)

#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
condicao = d3_00001_t['DIF Final'] == 0 

if condicao.all():
    resposta_d3_00001 = 'OK'
else:
    resposta_d3_00001 = 'ERRO'

d3_00001 = pd.DataFrame([resposta_d3_00001], columns=['Resposta'])
# Insere novas colunas
d3_00001.insert(0, 'Dimensão', 'D3_00001')
d3_00001.insert(2, 'Descrição da Dimensão', 'Verifica se o resultado orçamentário foi calculado corretamente no Balanço Orçamentário')
d3_00001.insert(3, 'OBS', f'Anexo 01 do RREO 6ºB')
d3_00001

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D3_00001,OK,Verifica se o resultado orçamentário foi calculado corretamente no Balanço Orçamentário,Anexo 01 do RREO 6ºB


In [ ]:
d3_00001_t

,REC,DPS EMP,DIF,Superávit ou Défcit,DIF Final
dimensao,,,,,
D3_00001_Superavit ou Defcit_ Pago,"106,112,454,653.40","104,823,674,009.28","1,288,780,644.12","1,288,780,644.12",0


## D3_00002

In [ ]:
############ >> D3_00003_Despesa do BO (Anexo 01) X Demonstrativo da Execução da Despesa por Função/Subfunção (Anexo 02)

##@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ Criando As Dimensões do RREO Anexo 2 

# Dotação Inicial
dotinic_rreo_2 = df_rreo_2.query('coluna == "DOTAÇÃO INICIAL" & cod_conta == "RREO2TotalDespesas" & conta == "DESPESAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I)"')
dotinic_rreo_2['dimensao'] = 'D3_00002_Dotação_Inicial'
dotinic_rreo_2 = dotinic_rreo_2.filter(items=['dimensao', 'valor'])
dotinic_rreo_2 = dotinic_rreo_2.set_index("dimensao")

# Dotação Inicial INTRA
dotinic_intra_rreo_2 = df_rreo_2.query('coluna == "DOTAÇÃO INICIAL" & cod_conta == "RREO2TotalDespesas" & conta == "DESPESAS (INTRA-ORÇAMENTÁRIAS) (II)"')
dotinic_intra_rreo_2['dimensao'] = 'D3_00002_Dotação_Inicial_INTRA'
dotinic_intra_rreo_2 = dotinic_intra_rreo_2.filter(items=['dimensao', 'valor'])
dotinic_intra_rreo_2 = dotinic_intra_rreo_2.set_index("dimensao")

# Dotação Atualizada
dotatualiz_rreo_2 = df_rreo_2.query('coluna == "DOTAÇÃO ATUALIZADA (a)" & cod_conta == "RREO2TotalDespesas" & conta == "DESPESAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I)"')
dotatualiz_rreo_2['dimensao'] = 'D3_00002_Dotação_Atualizada'
dotatualiz_rreo_2 = dotatualiz_rreo_2.filter(items=['dimensao', 'valor'])
dotatualiz_rreo_2 = dotatualiz_rreo_2.set_index("dimensao")

# Dotação Atualizada INTRA
dotatualiz_intra_rreo_2 = df_rreo_2.query('coluna == "DOTAÇÃO ATUALIZADA (a)" & cod_conta == "RREO2TotalDespesas" & conta == "DESPESAS (INTRA-ORÇAMENTÁRIAS) (II)"')
dotatualiz_intra_rreo_2['dimensao'] = 'D3_00002_Dotação_Atualizada_INTRA'
dotatualiz_intra_rreo_2 = dotatualiz_intra_rreo_2.filter(items=['dimensao', 'valor'])
dotatualiz_intra_rreo_2 = dotatualiz_intra_rreo_2.set_index("dimensao")

# Empenhados
emp_rreo_2 = df_rreo_2.query('coluna == "DESPESAS EMPENHADAS ATÉ O BIMESTRE (b)" & cod_conta == "RREO2TotalDespesas" & conta == "DESPESAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I)"')
emp_rreo_2['dimensao'] = 'D3_00002_Empenhado'
emp_rreo_2 = emp_rreo_2.filter(items=['dimensao', 'valor'])
emp_rreo_2 = emp_rreo_2.set_index("dimensao")

# Empenhados INTRA
emp_intra_rreo_2 = df_rreo_2.query('coluna == "DESPESAS EMPENHADAS ATÉ O BIMESTRE (b)" & cod_conta == "RREO2TotalDespesas" & conta == "DESPESAS (INTRA-ORÇAMENTÁRIAS) (II)"')
emp_intra_rreo_2['dimensao'] = 'D3_00002_Empenhado_INTRA'
emp_intra_rreo_2 = emp_intra_rreo_2.filter(items=['dimensao', 'valor'])
emp_intra_rreo_2 = emp_intra_rreo_2.set_index("dimensao")

# Liquidados
liq_rreo_2 = df_rreo_2.query('coluna == "DESPESAS LIQUIDADAS ATÉ O BIMESTRE (d)" & cod_conta == "RREO2TotalDespesas" & conta == "DESPESAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I)"')
liq_rreo_2['dimensao'] = 'D3_00002_Liquidado'
liq_rreo_2 = liq_rreo_2.filter(items=['dimensao', 'valor'])
liq_rreo_2 = liq_rreo_2.set_index("dimensao")

# Liquidados INTRA
liq_intra_rreo_2 = df_rreo_2.query('coluna == "DESPESAS LIQUIDADAS ATÉ O BIMESTRE (d)" & cod_conta == "RREO2TotalDespesas" & conta == "DESPESAS (INTRA-ORÇAMENTÁRIAS) (II)"')
liq_intra_rreo_2['dimensao'] = 'D3_00002_Liquidado_INTRA'
liq_intra_rreo_2 = liq_intra_rreo_2.filter(items=['dimensao', 'valor'])
liq_intra_rreo_2 = liq_intra_rreo_2.set_index("dimensao")

# Inscritas em RPNP
rpnp_rreo_2 = df_rreo_2.query('coluna == "INSCRITAS EM RESTOS A PAGAR NÃO PROCESSADOS (f)" & cod_conta == "RREO2TotalDespesas" & conta == "DESPESAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I)"')
rpnp_rreo_2['dimensao'] = 'D3_00002_Inscrição RPNP'
rpnp_rreo_2 = rpnp_rreo_2.filter(items=['dimensao', 'valor'])
rpnp_rreo_2 = rpnp_rreo_2.set_index("dimensao")

# Inscritas em RPNP INTRA
rpnp_intra_rreo_2 = df_rreo_2.query('coluna == "INSCRITAS EM RESTOS A PAGAR NÃO PROCESSADOS (f)" & cod_conta == "RREO2TotalDespesas" & conta == "DESPESAS (INTRA-ORÇAMENTÁRIAS) (II)"')
rpnp_intra_rreo_2['dimensao'] = 'D3_00002_Inscrição RPNP_INTRA'
rpnp_intra_rreo_2 = rpnp_intra_rreo_2.filter(items=['dimensao', 'valor'])
rpnp_intra_rreo_2 = rpnp_intra_rreo_2.set_index("dimensao")

###@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
##@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ Criando As Dimensões do RREO Anexo 1

# Dotação Inicial
dotinic_rreo_1a = df_rreo_1.query('coluna == "DOTAÇÃO INICIAL (d)" & cod_conta == "DespesasExcetoIntraOrcamentarias"')
dotinic_rreo_1b = df_rreo_1.query('coluna == "DOTAÇÃO INICIAL (d)" & cod_conta == "AmortizacaoRefinanciamentoDaDivida"')
dotinic_rreo_1 = pd.concat([dotinic_rreo_1a, dotinic_rreo_1b])
dotinic_rreo_1['dimensao'] = 'D3_00002_Dotação_Inicial'
dotinic_rreo_1 = dotinic_rreo_1.groupby('dimensao').agg({'valor': 'sum'})

# Dotação Inicial INTRA
dotinic_intra_rreo_1 = df_rreo_1.query('coluna == "DOTAÇÃO INICIAL (d)" & cod_conta == "DespesasIntraOrcamentariasTotal"')
dotinic_intra_rreo_1['dimensao'] = 'D3_00002_Dotação_Inicial_INTRA'
dotinic_intra_rreo_1 = dotinic_intra_rreo_1.filter(items=['dimensao', 'valor'])
dotinic_intra_rreo_1 = dotinic_intra_rreo_1.set_index("dimensao")

# Dotação Atualizada
dotatualiz_rreo_1a = df_rreo_1.query('coluna == "DOTAÇÃO ATUALIZADA (e)" & cod_conta == "DespesasExcetoIntraOrcamentarias"')
dotatualiz_rreo_1b = df_rreo_1.query('coluna == "DOTAÇÃO ATUALIZADA (e)" & cod_conta == "AmortizacaoRefinanciamentoDaDivida"')
dotatualiz_rreo_1 = pd.concat([dotatualiz_rreo_1a, dotatualiz_rreo_1b])
dotatualiz_rreo_1['dimensao'] = 'D3_00002_Dotação_Atualizada'
dotatualiz_rreo_1 = dotatualiz_rreo_1.groupby('dimensao').agg({'valor': 'sum'})

# Dotação Atualizada INTRA
dotatualiz_intra_rreo_1 = df_rreo_1.query('coluna == "DOTAÇÃO ATUALIZADA (e)" & cod_conta == "DespesasIntraOrcamentariasTotal"')
dotatualiz_intra_rreo_1['dimensao'] = 'D3_00002_Dotação_Atualizada_INTRA'
dotatualiz_intra_rreo_1 = dotatualiz_intra_rreo_1.filter(items=['dimensao', 'valor'])
dotatualiz_intra_rreo_1 = dotatualiz_intra_rreo_1.set_index("dimensao")

# Empenhados
emp_rreo_1a = df_rreo_1.query('coluna == "DESPESAS EMPENHADAS ATÉ O BIMESTRE (f)" & cod_conta == "DespesasExcetoIntraOrcamentarias"')
emp_rreo_1b = df_rreo_1.query('coluna == "DESPESAS EMPENHADAS ATÉ O BIMESTRE (f)" & cod_conta == "AmortizacaoRefinanciamentoDaDivida"')
emp_rreo_1 = pd.concat([emp_rreo_1a, emp_rreo_1b])
emp_rreo_1['dimensao'] = 'D3_00002_Empenhado'
emp_rreo_1 = emp_rreo_1.groupby('dimensao').agg({'valor': 'sum'})

# Empenhados INTRA
emp_intra_rreo_1 = df_rreo_1.query('coluna == "DESPESAS EMPENHADAS ATÉ O BIMESTRE (f)" & cod_conta == "DespesasIntraOrcamentariasTotal"')
emp_intra_rreo_1['dimensao'] = 'D3_00002_Empenhado_INTRA'
emp_intra_rreo_1 = emp_intra_rreo_1.filter(items=['dimensao', 'valor'])
emp_intra_rreo_1 = emp_intra_rreo_1.set_index("dimensao")

# Liquidados
liq_rreo_1a = df_rreo_1.query('coluna == "DESPESAS LIQUIDADAS ATÉ O BIMESTRE (h)" & cod_conta == "DespesasExcetoIntraOrcamentarias"')
liq_rreo_1b = df_rreo_1.query('coluna == "DESPESAS LIQUIDADAS ATÉ O BIMESTRE (h)" & cod_conta == "AmortizacaoRefinanciamentoDaDivida"')
liq_rreo_1 = pd.concat([liq_rreo_1a, liq_rreo_1b])
liq_rreo_1['dimensao'] = 'D3_00002_Liquidado'
liq_rreo_1 = liq_rreo_1.groupby('dimensao').agg({'valor': 'sum'})

# Liquidados INTRA
liq_intra_rreo_1 = df_rreo_1.query('coluna == "DESPESAS LIQUIDADAS ATÉ O BIMESTRE (h)" & cod_conta == "DespesasIntraOrcamentariasTotal"')
liq_intra_rreo_1['dimensao'] = 'D3_00002_Liquidado_INTRA'
liq_intra_rreo_1 = liq_intra_rreo_1.filter(items=['dimensao', 'valor'])
liq_intra_rreo_1 = liq_intra_rreo_1.set_index("dimensao")

# Inscritas em RPNP
rpnp_rreo_1a = df_rreo_1.query('coluna == "INSCRITAS EM RESTOS A PAGAR NÃO PROCESSADOS (k)" & cod_conta == "DespesasExcetoIntraOrcamentarias"')
rpnp_rreo_1b = df_rreo_1.query('coluna == "INSCRITAS EM RESTOS A PAGAR NÃO PROCESSADOS (k)" & cod_conta == "AmortizacaoRefinanciamentoDaDivida"')
rpnp_rreo_1 = pd.concat([rpnp_rreo_1a, rpnp_rreo_1b])
rpnp_rreo_1['dimensao'] = 'D3_00002_Inscrição RPNP'
rpnp_rreo_1 = rpnp_rreo_1.groupby('dimensao').agg({'valor': 'sum'})

# Inscritas em RPNP INTRA
rpnp_intra_rreo_1 = df_rreo_1.query('coluna == "INSCRITAS EM RESTOS A PAGAR NÃO PROCESSADOS (k)" & cod_conta == "DespesasIntraOrcamentariasTotal"')
rpnp_intra_rreo_1['dimensao'] = 'D3_00002_Inscrição RPNP_INTRA'
rpnp_intra_rreo_1 = rpnp_intra_rreo_1.filter(items=['dimensao', 'valor'])
rpnp_intra_rreo_1 = rpnp_intra_rreo_1.set_index("dimensao")

###@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#############################Criando o Merge entre os DF >> RREO 2 X RREO 1

###################  >> D3_00002_Dotação Inicial
d3_00002_dot_inicial = dotinic_rreo_2.merge(dotinic_rreo_1, on='dimensao')
d3_00002_dot_inicial['DIF'] = d3_00002_dot_inicial['valor_x'] - d3_00002_dot_inicial['valor_y']
d3_00002_dot_inicial.columns = ['RREO 2', 'RREO 1', 'DIF']

###################  >> D3_00002_Dotação Inicial INTRA
d3_00002_dot_inicial_intra = dotinic_intra_rreo_2.merge(dotinic_intra_rreo_1, on='dimensao')
d3_00002_dot_inicial_intra['DIF'] = d3_00002_dot_inicial_intra['valor_x'] - d3_00002_dot_inicial_intra['valor_y']
d3_00002_dot_inicial_intra.columns = ['RREO 2', 'RREO 1', 'DIF']

###################  >> D3_00002_Dotação Atualizada
d3_00002_dot_atualiz = dotatualiz_rreo_2.merge(dotatualiz_rreo_1, on='dimensao')
d3_00002_dot_atualiz['DIF'] = d3_00002_dot_atualiz['valor_x'] - d3_00002_dot_atualiz['valor_y']
d3_00002_dot_atualiz.columns = ['RREO 2', 'RREO 1', 'DIF']

###################  >> D3_00002_Dotação Atualizada INTRA
d3_00002_dot_atualiz_intra = dotatualiz_intra_rreo_2.merge(dotatualiz_intra_rreo_1, on='dimensao')
d3_00002_dot_atualiz_intra['DIF'] = d3_00002_dot_atualiz_intra['valor_x'] - d3_00002_dot_atualiz_intra['valor_y']
d3_00002_dot_atualiz_intra.columns = ['RREO 2', 'RREO 1', 'DIF']

###################  >> D3_00002_Empenhado
d3_00002_emp = emp_rreo_2.merge(emp_rreo_1, on='dimensao')
d3_00002_emp['DIF'] = d3_00002_emp['valor_x'] - d3_00002_emp['valor_y']
d3_00002_emp.columns = ['RREO 2', 'RREO 1', 'DIF']

###################  >> D3_00002_Empenhado INTRA
d3_00002_emp_intra = emp_intra_rreo_2.merge(emp_intra_rreo_1, on='dimensao')
d3_00002_emp_intra['DIF'] = d3_00002_emp_intra['valor_x'] - d3_00002_emp_intra['valor_y']
d3_00002_emp_intra.columns = ['RREO 2', 'RREO 1', 'DIF']

###################  >> D3_00002_Liquidado
d3_00002_liq = liq_rreo_2.merge(liq_rreo_1, on='dimensao')
d3_00002_liq['DIF'] = d3_00002_liq['valor_x'] - d3_00002_liq['valor_y']
d3_00002_liq.columns = ['RREO 2', 'RREO 1', 'DIF']

###################  >> D3_00002_Liquidado INTRA
d3_00002_liq_intra = liq_intra_rreo_2.merge(liq_intra_rreo_1, on='dimensao')
d3_00002_liq_intra['DIF'] = d3_00002_liq_intra['valor_x'] - d3_00002_liq_intra['valor_y']
d3_00002_liq_intra.columns = ['RREO 2', 'RREO 1', 'DIF']

###################  >> D3_00002_Inscrição de RPNP
d3_00002_rpnp = rpnp_rreo_2.merge(rpnp_rreo_1, on='dimensao')
d3_00002_rpnp['DIF'] = d3_00002_rpnp['valor_x'] - d3_00002_rpnp['valor_y']
d3_00002_rpnp.columns = ['RREO 2', 'RREO 1', 'DIF']

###################  >> D3_00002_Inscrição de RPNP INTRA
d3_00002_rpnp_intra = rpnp_intra_rreo_2.merge(rpnp_intra_rreo_1, on='dimensao')
d3_00002_rpnp_intra['DIF'] = d3_00002_rpnp_intra['valor_x'] - d3_00002_rpnp_intra['valor_y']
d3_00002_rpnp_intra.columns = ['RREO 2', 'RREO 1', 'DIF']

final_a = pd.concat([d3_00002_dot_inicial, d3_00002_dot_inicial_intra, d3_00002_dot_atualiz, d3_00002_dot_atualiz_intra, d3_00002_emp, d3_00002_emp_intra])
final_b = pd.concat([d3_00002_liq, d3_00002_liq_intra, d3_00002_rpnp, d3_00002_rpnp_intra])

d3_00002_t = pd.concat([final_a, final_b])

tolerancia = 1e-2
#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
condicao = d3_00002_t['DIF'].abs() <= tolerancia

if condicao.all():
    resposta_d3_00002 = 'OK'
else:
    resposta_d3_00002 = 'ERRO'

d3_00002 = pd.DataFrame([resposta_d3_00002], columns=['Resposta'])
# Insere novas colunas
d3_00002.insert(0, 'Dimensão', 'D3_00002')
d3_00002.insert(2, 'Descrição da Dimensão', 'Verifica a igualdade dos valores de despesa entre o Balanço Orçamentário e o Demonstrativo da Execução da Despesa por Função/Subfunção')
d3_00002.insert(3, 'OBS', f'Anexo 01 e Anexo 02 do RREO')
d3_00002

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D3_00002,OK,Verifica a igualdade dos valores de despesa entre o Balanço Orçamentário e o Demonstrativo da Execução da Despesa por Função/Subfunção,Anexo 01 e Anexo 02 do RREO


In [ ]:
d3_00002_t

,RREO 2,RREO 1,DIF
dimensao,,,
D3_00002_Dotação_Inicial,"105,606,489,534.00","105,606,489,534.00",0.00
D3_00002_Dotação_Inicial_INTRA,"7,534,120,647.00","7,534,120,647.00",0.00
D3_00002_Dotação_Atualizada,"120,818,321,152.63","120,818,321,152.63",0.00
D3_00002_Dotação_Atualizada_INTRA,"8,364,049,766.16","8,364,049,766.16",0.00
D3_00002_Empenhado,"100,508,250,796.52","100,508,250,796.52",0.00
D3_00002_Empenhado_INTRA,"8,018,541,679.58","8,018,541,679.58",0.00
D3_00002_Liquidado,"99,267,937,429.71","99,267,937,429.71",0.00
D3_00002_Liquidado_INTRA,"7,994,680,383.29","7,994,680,383.29",0.00
D3_00002_Inscrição RPNP,"1,240,313,366.81","1,240,313,366.81",0.00


# D3_00005 >> Igualdade da Receita Corrente Líquida entre o Anexo 03 do RREO e os Anexos 01, 02, 03 e 04 do RGF do poder executivo.

In [ ]:
rcl_rreo3_df = df_rreo_3.query('coluna == "TOTAL (ÚLTIMOS 12 MESES)"').copy()
rcl_rreo_3 = rcl_rreo3_df.query('cod_conta == "RREO3ReceitaCorrenteLiquida"')
rcl_rreo_3_divida = rcl_rreo3_df.query('cod_conta == "RREO3ReceitaCorrenteLiquidaAjustadaParaCalculoDosLimitesDeEndividamento"')
rcl_rreo_3_pessoal = rcl_rreo3_df.query('cod_conta == "RREO3ReceitaCorrenteLiquidaAjustadaParaCalculoDosLimitesDaDespesaComPessoal"')

rcl_rgf1 = df_rgf_1e.query('cod_conta == "ReceitaCorrenteLiquidaLimiteLegal"')
rcl_rgf1_pessoal = df_rgf_1e.query('cod_conta == "ReceitaCorrenteLiquidaAjustada"')

rcl_rgf2_df = df_rgf_2e.query('coluna == "Até o 3º Quadrimestre"')
rcl_rgf2 = rcl_rgf2_df.query('cod_conta == "RGF2ReceitaCorrenteLiquida"')
rcl_rgf2_divida = rcl_rgf2_df.query('cod_conta == "ReceitaCorrenteLiquidaAjustadaParaCalculoDosLimitesDeEndividamento"')

rcl_rgf3_df = df_rgf_3e.query('coluna == "Até o 3º Quadrimestre"')
rcl_rgf3 = rcl_rgf3_df.query('cod_conta == "RGF3ReceitaCorrenteLiquida"')
rcl_rgf3_divida = rcl_rgf3_df.query('cod_conta == "ReceitaCorrenteLiquidaAjustadaParaCalculoDosLimitesDeEndividamento"')

rcl_rgf4 = df_rgf_4e.query('cod_conta == "RGF4ReceitaCorrenteLiquida"')
rcl_rgf4_divida = df_rgf_4e.query('cod_conta == "ReceitaCorrenteLiquidaAjustadaParaCalculoDosLimitesDeEndividamento"')

d3_00005_t1 = pd.concat([rcl_rreo_3, rcl_rgf1, rcl_rgf2, rcl_rgf3, rcl_rgf4]).reset_index()
d3_00005_t1['DIF'] = d3_00005_t1['valor'].diff()
d3_00005_t1 = d3_00005_t1[['instituicao', 'anexo', 'cod_conta', 'valor', 'DIF']]
d3_00005_t1.loc[0,'DIF'] = 0

d3_00005_t2 = pd.concat([rcl_rreo_3_pessoal, rcl_rgf1_pessoal]).reset_index()
d3_00005_t2['DIF'] = d3_00005_t2['valor'].diff()
d3_00005_t2 = d3_00005_t2[['instituicao', 'anexo', 'cod_conta', 'valor', 'DIF']]
d3_00005_t2.loc[0,'DIF'] = 0

d3_00005_t3 = pd.concat([rcl_rreo_3_divida, rcl_rgf2_divida, rcl_rgf3_divida, rcl_rgf4_divida]).reset_index()
d3_00005_t3['DIF'] = d3_00005_t3['valor'].diff()
d3_00005_t3 = d3_00005_t3[['instituicao', 'anexo', 'cod_conta', 'valor', 'DIF']]
d3_00005_t3.loc[0,'DIF'] = 0

d3_00005_t = pd.concat([d3_00005_t1, d3_00005_t2, d3_00005_t3]).reset_index(drop=True)

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d3_00005_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d3_00005 = 'ERRO'
else:
    resposta_d3_00005 = 'OK'

d3_00005 = pd.DataFrame([resposta_d3_00005], columns=['Resposta'])
# Insere novas colunas
d3_00005.insert(0, 'Dimensão', 'D3_00005')
d3_00005.insert(2, 'Descrição da Dimensão', 'Verifica a Igualdade da Receita Corrente Líquida (RCL)')
d3_00005.insert(3, 'OBS', f' Anexo 03 do RREO e os Anexos 01, 02, 03 e 04 do RGF do poder executivo')
d3_00005

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D3_00005,OK,Verifica a Igualdade da Receita Corrente Líquida (RCL),"Anexo 03 do RREO e os Anexos 01, 02, 03 e 04 do RGF do poder executivo"


In [ ]:
d3_00005_t

,instituicao,anexo,cod_conta,valor,DIF
0,Governo do Estado do Rio de Janeiro,RREO-Anexo 03,RREO3ReceitaCorrenteLiquida,"92,764,348,281.93",0.00
1,Governo do Estado do Rio de Janeiro,RGF-Anexo 01,ReceitaCorrenteLiquidaLimiteLegal,"92,764,348,281.93",0.00
2,Governo do Estado do Rio de Janeiro,RGF-Anexo 02,RGF2ReceitaCorrenteLiquida,"92,764,348,281.93",0.00
3,Governo do Estado do Rio de Janeiro,RGF-Anexo 03,RGF3ReceitaCorrenteLiquida,"92,764,348,281.93",0.00
4,Governo do Estado do Rio de Janeiro,RGF-Anexo 04,RGF4ReceitaCorrenteLiquida,"92,764,348,281.93",0.00
5,Governo do Estado do Rio de Janeiro,RREO-Anexo 03,RREO3ReceitaCorrenteLiquidaAjustadaParaCalculoDosLimitesDaDespesaComPessoal,"92,709,068,228.22",0.00
6,Governo do Estado do Rio de Janeiro,RGF-Anexo 01,ReceitaCorrenteLiquidaAjustada,"92,709,068,228.22",0.00
7,Governo do Estado do Rio de Janeiro,RREO-Anexo 03,RREO3ReceitaCorrenteLiquidaAjustadaParaCalculoDosLimitesDeEndividamento,"92,722,591,176.51",0.00
8,Governo do Estado do Rio de Janeiro,RGF-Anexo 02,ReceitaCorrenteLiquidaAjustadaParaCalculoDosLimitesDeEndividamento,"92,722,591,176.51",0.00
9,Governo do Estado do Rio de Janeiro,RGF-Anexo 03,ReceitaCorrenteLiquidaAjustadaParaCalculoDosLimitesDeEndividamento,"92,722,591,176.51",0.00


# D3_00006 >> Igualdade da Dívida Consolidada Líquida (DCL) entre o Anexo 06 do RREO e o Anexo 02 do RGF do poder executivo

In [ ]:
# Anexo 2 RGF 
dcl_rgf2 = df_rgf_2e.query('cod_conta == "DividaConsolidadaLiquida" and coluna == "Até o 3º Quadrimestre"')
# Anexo 6 RREO
dcl_rreo6 = df_rreo_6.query(f'cod_conta == "DividaConsolidadaLiquida" and coluna == "Até o Bimestre {ano} (b)"')


d3_00006_t = pd.concat([dcl_rgf2, dcl_rreo6]).reset_index()
d3_00006_t['DIF'] = d3_00006_t['valor'].diff()
d3_00006_t = d3_00006_t[['instituicao', 'anexo', 'cod_conta', 'valor', 'DIF']]
d3_00006_t.loc[0, 'DIF'] = 0

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d3_00006_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d3_00006 = 'ERRO'
else:
    resposta_d3_00006 = 'OK'

d3_00006 = pd.DataFrame([resposta_d3_00006], columns=['Resposta'])
# Insere novas colunas
d3_00006.insert(0, 'Dimensão', 'D3_00006')
d3_00006.insert(2, 'Descrição da Dimensão', 'Verifica a Igualdade da Dívida Consolidada Líquida (DCL)')
d3_00006.insert(3, 'OBS', f' Anexo 06 do RREO e o Anexo 02 do RGF do poder executivo')
d3_00006


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D3_00006,OK,Verifica a Igualdade da Dívida Consolidada Líquida (DCL),Anexo 06 do RREO e o Anexo 02 do RGF do poder executivo


In [ ]:
d3_00006_t

,instituicao,anexo,cod_conta,valor,DIF
0,Governo do Estado do Rio de Janeiro,RGF-Anexo 02,DividaConsolidadaLiquida,"195,944,554,023.44",0.00
1,Governo do Estado do Rio de Janeiro,RREO-Anexo 06,DividaConsolidadaLiquida,"195,944,554,023.44",0.00


## D3_00008 >> Verifica a igualdade dos valores dos restos a pagar não processados entre o Anexo 01 do RREO e a soma dos valores do Anexo 05 do RGF de todos os poderes/órgãos.

In [ ]:
# EXECUTIVO mudou >> ´´conta == "TOTAL (IV) = (I + II + III)"``
# Inscrição de RPNP Executivo
rpnp_rgf_5e = df_rgf_5e.query('cod_conta == "RestosAPagarEmpenhadosENaoLiquidadosDoExercicio" & conta == "TOTAL (IV) = (I + II + III)"')  
rpnp_rgf_5e = rpnp_rgf_5e.groupby(['anexo'])['valor'].sum().reset_index()

#DEMAIS PODRES
#RPNP a Pagar >> RGF 5 dos Demais Poderes
rpnp_rgf_5_o = rgf_o.query('cod_conta == "RestosAPagarEmpenhadosENaoLiquidadosDoExercicio" & conta == "TOTAL (III) = (I + II)"')
rpnp_rgf_5_o = rpnp_rgf_5_o.groupby(['anexo'])['valor'].sum().reset_index()

#CONCAT
rpnp_rgf = pd.concat([rpnp_rgf_5e, rpnp_rgf_5_o])
rpnp_rgf['cod'] = "RPNP_Inscrito"

#RREO 7
#RPNP a Pagar
rpnp_rreo_1 = df_rreo_1.query('coluna == "INSCRITAS EM RESTOS A PAGAR NÃO PROCESSADOS (k)" & cod_conta == "TotalDespesas"')
rpnp_rreo_1 = rpnp_rreo_1.groupby(['anexo'])['valor'].sum().reset_index()
rpnp_rreo_1['cod'] = "RPNP_Inscrito"


d3_00008_t = pd.concat([rpnp_rgf, rpnp_rreo_1])
d3_00008_t = d3_00008_t.groupby(['cod', 'anexo'])['valor'].sum().reset_index()
d3_00008_t = d3_00008_t.pivot(index='cod', columns='anexo', values='valor')
d3_00008_t['DIF'] = d3_00008_t['RGF-Anexo 05'] - d3_00008_t['RREO-Anexo 01']
d3_00008_t.reset_index(inplace=True)
d3_00008_t.fillna(0, inplace=True)

# # Supondo que tolerancia seja um valor definido anteriormente no seu código
# tolerancia = 0.99  # Exemplo de tolerância
# # Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
# condicao = ~np.isclose(d3_00008_t['DIF'], 0, atol=tolerancia)

# if condicao.any():
#     resposta_d3_00008 = 'ERRO'
# else:
#     resposta_d3_00008 = 'OK'

# Tolerância para considerar diferença de centavos (menos de 1 real em valor absoluto)
tolerancia_centavos = 0.99999
# Tolerância para considerar o "-0,00" como zero
tolerancia_zero = 1e-3  # Um valor muito pequeno (0,001)

diferenca_encontrada = d3_00008_t['DIF'].abs().max()

if np.isclose(diferenca_encontrada, 0, atol=tolerancia_zero):
    resposta_d3_00008 = 'OK'
elif diferenca_encontrada <= tolerancia_centavos and not np.isclose(diferenca_encontrada, 0, atol=tolerancia_zero):
    resposta_d3_00008 = 'OK (com dif centavos)'
else:
    resposta_d3_00008 = 'ERRO' 

d3_00008 = pd.DataFrame([resposta_d3_00008], columns=['Resposta'])
# Insere novas colunas
d3_00008.insert(0, 'Dimensão', 'D3_00008')
d3_00008.insert(2, 'Descrição da Dimensão', 'Verifica a igualdade dos valores dos restos a pagar não processados')
d3_00008.insert(3, 'OBS', 'Anexo 01 do RREO e a soma dos valores do Anexo 05 do RGF de todos os poderes/órgãos')

d3_00008

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D3_00008,OK (com dif centavos),Verifica a igualdade dos valores dos restos a pagar não processados,Anexo 01 do RREO e a soma dos valores do Anexo 05 do RGF de todos os poderes/órgãos


In [ ]:
d3_00008_t

anexo,cod,RGF-Anexo 05,RREO-Anexo 01,DIF
0,RPNP_Inscrito,"1,264,174,663.07","1,264,174,663.10",-0.03


## D3_00009 >> Verifica a igualdade dos valores dos restos a pagar processados e não processados entre Anexo 07 do RREO e a soma dos valores dos Anexos 05 do RGF de todos os poderes/órgãos (pega a SOMA dos RPs)

In [ ]:
# EXECUTIVO mudou >> ´´conta == "TOTAL (IV) = (I + II + III)"``
# RPNP a Pagar >> RGF 5 do Executivo 
rpnp_a_pagar_rgf_e = df_rgf_5e.query('cod_conta == "RestosAPagarEmpenhadosENaoLiquidadosDeExerciciosAnteriores" & conta == "TOTAL (IV) = (I + II + III)"') 
rpnp_a_pagar_rgf_e = rpnp_a_pagar_rgf_e.groupby(['anexo'])['valor'].sum().reset_index()

# RPP a Pagar  >> RGF 5
rpp_a_pagar_rgf_e = df_rgf_5e.query('cod_conta == "RestosAPagarLiquidadosENaoPagosDeExerciciosAnteriores" & conta == "TOTAL (IV) = (I + II + III)"')
rpp_a_pagar_rgf_e = rpp_a_pagar_rgf_e.groupby(['anexo'])['valor'].sum().reset_index()

#DEMAIS PODRES
#RPNP a Pagar >> RGF 5 dos Demais Poderes
rpnp_a_pagar_rgf_o = rgf_o.query('cod_conta == "RestosAPagarEmpenhadosENaoLiquidadosDeExerciciosAnteriores" & conta == "TOTAL (III) = (I + II)"')
rpnp_a_pagar_rgf_o = rpnp_a_pagar_rgf_o.groupby(['anexo'])['valor'].sum().reset_index()
#RPPP a Pagar >> RGF 5 dos Demais Poderes
rpp_a_pagar_rgf_o = rgf_o.query('cod_conta == "RestosAPagarLiquidadosENaoPagosDeExerciciosAnteriores" & conta == "TOTAL (III) = (I + II)"')
rpp_a_pagar_rgf_o = rpp_a_pagar_rgf_o.groupby(['anexo'])['valor'].sum().reset_index()

#CONCAT
rpnp_a_pagar_rgf = pd.concat([rpnp_a_pagar_rgf_e, rpnp_a_pagar_rgf_o])
rpnp_a_pagar_rgf['cod'] = "RPNP"
rpp_a_pagar_rgf = pd.concat([rpp_a_pagar_rgf_e, rpp_a_pagar_rgf_o])
rpp_a_pagar_rgf['cod'] = "RPP"

#RREO 7
#RPNP a Pagar
rpnp_a_pagar_rreo_7 = df_rreo_7.query('cod_conta == "RestosAPagarNaoProcessadosAPagar" & conta == "TOTAL (III) = (I + II)"')
rpnp_a_pagar_rreo_7 = rpnp_a_pagar_rreo_7.groupby(['anexo'])['valor'].sum().reset_index()
rpnp_a_pagar_rreo_7['cod'] = "RPNP"
# RPP a Pagar
rpp_a_pagar_rreo_7 = df_rreo_7.query('cod_conta == "RestosAPagarProcessadosENaoProcessadosLiquidadosAPagar" & conta == "TOTAL (III) = (I + II)"')
rpp_a_pagar_rreo_7 = rpp_a_pagar_rreo_7.groupby(['anexo'])['valor'].sum().reset_index()
rpp_a_pagar_rreo_7['cod'] = "RPP"


# Juntando
d3_00009_t = pd.concat([rpnp_a_pagar_rreo_7, rpnp_a_pagar_rgf, rpp_a_pagar_rreo_7, rpp_a_pagar_rgf])
d3_00009_t = d3_00009_t.groupby(['cod', 'anexo'])['valor'].sum().reset_index()
d3_00009_t = d3_00009_t.pivot(index='cod', columns='anexo', values='valor')
d3_00009_t['DIF'] = d3_00009_t['RGF-Anexo 05'] - d3_00009_t['RREO-Anexo 07']
d3_00009_t = d3_00009_t.reset_index(drop=True)

# 1. Calcular os totais das colunas numéricas
totais = d3_00009_t[['RGF-Anexo 05', 'RREO-Anexo 07', 'DIF']].sum()
# 2. Criar uma nova linha (Series) com os totais e um rótulo
linha_total = pd.Series([totais['RGF-Anexo 05'], totais['RREO-Anexo 07'], totais['DIF']], index=d3_00009_t.columns)
# 3. Adicionar a nova linha ao DataFrame usando .loc
d3_00009_t.loc['Total'] = linha_total
# Deixando só aúltima linha para fazer a condição
d3_00009_t = d3_00009_t.drop(d3_00009_t.index[:-1])

# Tolerância para considerar diferença de centavos (menos de 1 real em valor absoluto)
tolerancia_centavos = 0.99999
# Tolerância para considerar o "-0,00" como zero
tolerancia_zero = 1e-3  # Um valor muito pequeno (0,001)

diferenca_encontrada = d3_00009_t['DIF'].abs().max()

if np.isclose(diferenca_encontrada, 0, atol=tolerancia_zero):
    resposta_d3_00009 = 'OK'
elif diferenca_encontrada <= tolerancia_centavos and not np.isclose(diferenca_encontrada, 0, atol=tolerancia_zero):
    resposta_d3_00009 = 'OK (com dif centavos)'
else:
    resposta_d3_00009 = 'ERRO' 


d3_00009 = pd.DataFrame([resposta_d3_00009], columns=['Resposta'])
# Insere novas colunas
d3_00009.insert(0, 'Dimensão', 'D3_00009')
d3_00009.insert(2, 'Descrição da Dimensão', 'Verifica a igualdade dos valores dos restos a pagar processados e não processados')
d3_00009.insert(3, 'OBS', f'Anexo 07 do RREO e os Anexos 05 do RGF de todos os poderes/órgãos')
d3_00009


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D3_00009,OK,Verifica a igualdade dos valores dos restos a pagar processados e não processados,Anexo 07 do RREO e os Anexos 05 do RGF de todos os poderes/órgãos


In [ ]:
d3_00009_t

anexo,RGF-Anexo 05,RREO-Anexo 07,DIF
Total,"650,769,373.06","650,769,373.06",0.00


# D3_00010 >> Igualdade da Receita Corrente Líquida (RCL) no Anexo 01 do RGF de todos os poderes/órgãos

In [ ]:
df_rgf_1m.head(2)

,exercicio,periodo,periodicidade,instituicao,cod_ibge,uf,co_poder,populacao,anexo,esfera,rotulo,coluna,cod_conta,conta,valor
0,2024,3,Q,Ministério Público do Estado do Rio de Janeiro,33,RJ,M,16615526,RGF-Anexo 01,E,Padrão,<MR-11>,DespesaComPessoalBruta,DESPESA BRUTA COM PESSOAL (I),"212,695,025.85"
1,2024,3,Q,Ministério Público do Estado do Rio de Janeiro,33,RJ,M,16615526,RGF-Anexo 01,E,Padrão,<MR-10>,DespesaComPessoalBruta,DESPESA BRUTA COM PESSOAL (I),"233,890,707.10"


In [ ]:
## Anexo 1 >> EXECUTIVO
rcl_rgf1e = df_rgf_1e.query('cod_conta == "ReceitaCorrenteLiquidaLimiteLegal"')
## Anexo 1 >> LEGISLATIVO
rcl_rgf1l = df_rgf_1l.query('cod_conta == "ReceitaCorrenteLiquidaLimiteLegal"')
## Anexo 1 >> JUDICIÁRIO
rcl_rgf1j = df_rgf_1j.query('cod_conta == "ReceitaCorrenteLiquidaLimiteLegal"')
## Anexo 1 >> MP
rcl_rgf1m = df_rgf_1m.query('cod_conta == "ReceitaCorrenteLiquidaLimiteLegal"')
## Anexo 1 >> DP
rcl_rgf1d = df_rgf_1d.query('cod_conta == "ReceitaCorrenteLiquidaLimiteLegal"')

d3_00010_t = pd.concat([rcl_rgf1e, rcl_rgf1l, rcl_rgf1j, rcl_rgf1m, rcl_rgf1d]).reset_index()
d3_00010_t['DIF'] = d3_00010_t['valor'].diff()
d3_00010_t = d3_00010_t[['instituicao', 'anexo', 'cod_conta', 'valor', 'DIF']]
d3_00010_t.loc[0, "DIF"] = 0

#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
condicao = d3_00010_t['DIF'] == 0 

if condicao.all():
    resposta_d3_00010 = 'OK'
else:
    resposta_d3_00010 = 'ERRO'

d3_00010 = pd.DataFrame([resposta_d3_00010], columns=['Resposta'])
# Insere novas colunas
d3_00010.insert(0, 'Dimensão', 'D3_00010')
d3_00010.insert(2, 'Descrição da Dimensão', 'Verifica a Igualdade da Receita Corrente Líquida (RCL) no Anexo 01 do RGF de todos os poderes/órgãos')
d3_00010.insert(3, 'OBS', f' Demonstrativo da Despesa com Pessoal do RGF (Anexo 1) de Todos os poderes/órgãos')
d3_00010

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D3_00010,OK,Verifica a Igualdade da Receita Corrente Líquida (RCL) no Anexo 01 do RGF de todos os poderes/órgãos,Demonstrativo da Despesa com Pessoal do RGF (Anexo 1) de Todos os poderes/órgãos


In [ ]:
d3_00010_t

,instituicao,anexo,cod_conta,valor,DIF
0,Governo do Estado do Rio de Janeiro,RGF-Anexo 01,ReceitaCorrenteLiquidaLimiteLegal,"92,764,348,281.93",0.00
1,Assembleia Legislativa do Estado do Rio de Janeiro,RGF-Anexo 01,ReceitaCorrenteLiquidaLimiteLegal,"92,764,348,281.93",0.00
2,Tribunal de Contas do Estado do Rio de Janeiro,RGF-Anexo 01,ReceitaCorrenteLiquidaLimiteLegal,"92,764,348,281.93",0.00
3,Tribunal de Justiça do Estado do Rio de Janeiro,RGF-Anexo 01,ReceitaCorrenteLiquidaLimiteLegal,"92,764,348,281.93",0.00
4,Ministério Público do Estado do Rio de Janeiro,RGF-Anexo 01,ReceitaCorrenteLiquidaLimiteLegal,"92,764,348,281.93",0.00


# D3_00011 >> Dedução de inativos e pensionistas com recursos vinculados menor ou igual ao valor com inativos e pensionistas no Anexo 01 do RGF de todos os poderes/órgãos

In [ ]:
## Anexo 1 >> EXECUTIVO
inativo_rgf1e = df_rgf_1e.query('(cod_conta == "DespesaComPessoalInativoEPensionistasBruta" or cod_conta == "DespesaComPessoalNaoComputadaInativosEPensionistasComRecursosVinculados")\
                                and coluna == "TOTAL (ÚLTIMOS 12 MESES) (a)"')

# ## Anexo 1 >> LEGISLATIVO
inativo_rgf1l = df_rgf_1l.query('(cod_conta == "DespesaComPessoalInativoEPensionistasBruta" or cod_conta == "DespesaComPessoalNaoComputadaInativosEPensionistasComRecursosVinculados")\
                                and coluna == "TOTAL (ÚLTIMOS 12 MESES) (a)"')

# ## Anexo 1 >> JUDICIÁRIO
inativo_rgf1j = df_rgf_1j.query('(cod_conta == "DespesaComPessoalInativoEPensionistasBruta" or cod_conta == "DespesaComPessoalNaoComputadaInativosEPensionistasComRecursosVinculados")\
                                and coluna == "TOTAL (ÚLTIMOS 12 MESES) (a)"')

# ## Anexo 1 >> MP
inativo_rgf1m = df_rgf_1m.query('(cod_conta == "DespesaComPessoalInativoEPensionistasBruta" or cod_conta == "DespesaComPessoalNaoComputadaInativosEPensionistasComRecursosVinculados")\
                                and coluna == "TOTAL (ÚLTIMOS 12 MESES) (a)"')

# ## Anexo 1 >> DP
inativo_rgf1d = df_rgf_1d.query('(cod_conta == "DespesaComPessoalInativoEPensionistasBruta" or cod_conta == "DespesaComPessoalNaoComputadaInativosEPensionistasComRecursosVinculados")\
                                and coluna == "TOTAL (ÚLTIMOS 12 MESES) (a)"')

d3_00011_t = pd.concat([inativo_rgf1e, inativo_rgf1l, inativo_rgf1j, inativo_rgf1m, inativo_rgf1d]).reset_index()
d3_00011_t['DIF'] = d3_00011_t['valor'].diff()

d3_00011_t = d3_00011_t[['instituicao', 'anexo', 'cod_conta', 'valor', 'DIF']]
d3_00011_t = d3_00011_t.query('cod_conta == "DespesaComPessoalNaoComputadaInativosEPensionistasComRecursosVinculados"')

#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)

condicao = d3_00011_t['DIF'] > 0

if condicao.any():
    resposta_d3_00011 = 'ERRO'
else:
    resposta_d3_00011 = 'OK'

d3_00011 = pd.DataFrame([resposta_d3_00011], columns=['Resposta'])
# Insere novas colunas
d3_00011.insert(0, 'Dimensão', 'D3_00011')
d3_00011.insert(2, 'Descrição da Dimensão', 'Verifica se a Dedução de inativos e pensionistas com recursos vinculados é menor ou igual ao valor com inativos e pensionistas no Anexo 01 do RGF de todos os poderes/órgãos')
d3_00011.insert(3, 'OBS', f' Demonstrativo da Despesa com Pessoal do RGF (Anexo 1) de Todos os poderes/órgãos')
d3_00011


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D3_00011,OK,Verifica se a Dedução de inativos e pensionistas com recursos vinculados é menor ou igual ao valor com inativos e pensionistas no Anexo 01 do RGF de todos os poderes/órgãos,Demonstrativo da Despesa com Pessoal do RGF (Anexo 1) de Todos os poderes/órgãos


In [ ]:
d3_00011_t

,instituicao,anexo,cod_conta,valor,DIF
1,Governo do Estado do Rio de Janeiro,RGF-Anexo 01,DespesaComPessoalNaoComputadaInativosEPensionistasComRecursosVinculados,"6,450,168,505.62","-16,523,035,050.35"
3,Assembleia Legislativa do Estado do Rio de Janeiro,RGF-Anexo 01,DespesaComPessoalNaoComputadaInativosEPensionistasComRecursosVinculados,"379,412,174.76","-200,596,814.24"
5,Tribunal de Contas do Estado do Rio de Janeiro,RGF-Anexo 01,DespesaComPessoalNaoComputadaInativosEPensionistasComRecursosVinculados,"644,492,867.07","-89,567,225.59"
7,Tribunal de Justiça do Estado do Rio de Janeiro,RGF-Anexo 01,DespesaComPessoalNaoComputadaInativosEPensionistasComRecursosVinculados,"2,992,738,847.44",0.00
9,Ministério Público do Estado do Rio de Janeiro,RGF-Anexo 01,DespesaComPessoalNaoComputadaInativosEPensionistasComRecursosVinculados,"395,220,250.94",0.00
11,Defensoria Pública do Estado do Rio de Janeiro,RGF-Anexo 01,DespesaComPessoalNaoComputadaInativosEPensionistasComRecursosVinculados,"176,553,634.72","-50,987,698.66"


# D3_00012 >> Informação de valores negativos no RREO

In [ ]:
rreo_consol = pd.concat ([df_rreo_1, df_rreo_2, df_rreo_3, df_rreo_4, df_rreo_6, df_rreo_7, df_rreo_9, df_rreo_11, df_rreo_14])
d3_00012_t = rreo_consol.query('valor < 0')
d3_00012_t.coluna.unique()

array(['SALDO (a-c)', 'No Bimestre (b)', '% (b/a)',
       'DESPESAS EMPENHADAS NO BIMESTRE', 'DOTAÇÃO ATUALIZADA (c)',
       'DESPESAS EMPENHADAS ATÉ O BIMESTRE (d)',
       'DESPESAS LIQUIDADAS ATÉ O BIMESTRE (e)',
       'DESPESAS PAGAS ATÉ O BIMESTRE (f)', 'VALOR', 'VALOR CORRENTE',
       'SALDO (c) = (a - b)', 'Até o Bimestre',
       'Meta Fixada no Anexo de Metas Fiscais da LDO (a)',
       'Resultado Apurado até o Bimestre (b)', 'Exercício',
       '10º Exercício', '20º Exercício', '35º Exercício',
       'Saldo a Realizar'], dtype=object)

# D3_00013 >> Verifica se foram informados valores com sinal negativo no Relatório de Gestão Fiscal (RGF) de todos os poderes/órgãos em campos que não deveriam apresentar valores negativos.

# D3_00014 >> Verifica a igualdade do valor da Transferências Obrigatórias da União Relativas às Emendas Individuais entre os anexos 1, 2, 3 e 4 do RGF do poder executivo de final de período.

In [ ]:

filtro1 = df_rgf_1e['cod_conta'].str.contains('EmendasIndividuais', case=False, na=False)
emenda_indiv_rgf1e = df_rgf_1e[filtro1]

df_rgf_2e = df_rgf_2e.query('coluna == "Até o 3º Quadrimestre"')
filtro2 = df_rgf_2e['cod_conta'].str.contains('EmendasIndividuais', case=False, na=False)
emenda_indiv_rgf2e = df_rgf_2e[filtro2]

df_rgf_3e = df_rgf_3e.query('coluna == "Até o 3º Quadrimestre"')
filtro3 = df_rgf_3e['cod_conta'].str.contains('EmendasIndividuais', case=False, na=False)
emenda_indiv_rgf3e = df_rgf_3e[filtro3]

filtro4 = df_rgf_4e['cod_conta'].str.contains('EmendasIndividuais', case=False, na=False)
emenda_indiv_rgf4e = df_rgf_4e[filtro4]

d3_00014_t = pd.concat([emenda_indiv_rgf1e, emenda_indiv_rgf2e, emenda_indiv_rgf3e, emenda_indiv_rgf4e]).reset_index(drop=True)

d3_00014_t['DIF'] = d3_00014_t['valor'].diff()
d3_00014_t.loc[0, 'DIF'] = 0

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d3_00014_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d3_00014 = 'ERRO'
else:
    resposta_d3_00014 = 'OK'


d3_00014_t = d3_00014_t[['anexo', 'cod_conta', 'valor', 'DIF']]

d3_00014 = pd.DataFrame([resposta_d3_00014], columns=['Resposta'])
# Insere novas colunas
d3_00014.insert(0, 'Dimensão', 'D3_00014')
d3_00014.insert(2, 'Descrição da Dimensão', 'Verifica a igualdade do valor da Transferências Obrigatórias da União Relativas às Emendas Individuais')
d3_00014.insert(3, 'OBS', 'Anexos 1, 2, 3 e 4 do RGF do poder executivo')
d3_00014

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D3_00014,OK,Verifica a igualdade do valor da Transferências Obrigatórias da União Relativas às Emendas Individuais,"Anexos 1, 2, 3 e 4 do RGF do poder executivo"


In [ ]:
d3_00014_t

,anexo,cod_conta,valor,DIF
0,RGF-Anexo 01,TransferenciasObrigatoriasDaUniaoRelativasAsEmendasIndividuais,"41,757,105.42",0.00
1,RGF-Anexo 02,TransferenciasObrigatoriasDaUniaoRelativasAsEmendasIndividuais,"41,757,105.42",0.00
2,RGF-Anexo 03,TransferenciasObrigatoriasDaUniaoRelativasAsEmendasIndividuais,"41,757,105.42",0.00
3,RGF-Anexo 04,TransferenciasObrigatoriasDaUniaoRelativasAsEmendasIndividuais,"41,757,105.42",0.00


# D3_00015

In [ ]:
df_rreo_3 = df_rreo_3.query('coluna == "TOTAL (ÚLTIMOS 12 MESES)"')
filtro5 = df_rreo_3['cod_conta'].str.contains('EmendasIndividuais', case=False, na=False)
emenda_indiv_rreo3 = df_rreo_3[filtro5]

d3_00015_t = pd.concat([emenda_indiv_rgf1e, emenda_indiv_rreo3]).reset_index(drop=True)

d3_00015_t['DIF'] = d3_00015_t['valor'].diff()
d3_00015_t.loc[0, 'DIF'] = 0

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d3_00015_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d3_00015 = 'ERRO'
else:
    resposta_d3_00015 = 'OK'

d3_00015_t = d3_00015_t[['anexo', 'cod_conta', 'valor', 'DIF']]

d3_00015 = pd.DataFrame([resposta_d3_00015], columns=['Resposta'])
# Insere novas colunas
d3_00015.insert(0, 'Dimensão', 'D3_00015')
d3_00015.insert(2, 'Descrição da Dimensão', 'Verifica a igualdade do valor da Transferências Obrigatórias da União Relativas às Emendas Individuais')
d3_00015.insert(3, 'OBS', 'Anexo 03 do RREO e o Anexo 01 do RGF do poder executivo')
d3_00015

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D3_00015,OK,Verifica a igualdade do valor da Transferências Obrigatórias da União Relativas às Emendas Individuais,Anexo 03 do RREO e o Anexo 01 do RGF do poder executivo


In [ ]:
d3_00015_t

,anexo,cod_conta,valor,DIF
0,RGF-Anexo 01,TransferenciasObrigatoriasDaUniaoRelativasAsEmendasIndividuais,"41,757,105.42",0.00
1,RREO-Anexo 03,RREO3TransferenciasObrigatoriasDaUniaoRelativasAsEmendasIndividuais,"41,757,105.42",0.00


# D3_00016

In [ ]:

filtro6 = df_rgf_1e['cod_conta'].str.contains('Bancada', case=False, na=False)
emenda_bancada_rgf1e = df_rgf_1e[filtro6]

filtro7 = df_rreo_3['cod_conta'].str.contains('Bancada', case=False, na=False)
emenda_bancada_rreo3 = df_rreo_3[filtro7]

d3_00016_t = pd.concat([emenda_bancada_rgf1e, emenda_bancada_rreo3]).reset_index(drop=True)

d3_00016_t['DIF'] = d3_00016_t['valor'].diff()
d3_00016_t.loc[0, 'DIF'] = 0

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d3_00016_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d3_00016 = 'ERRO'
else:
    resposta_d3_00016 = 'OK'

d3_00016_t = d3_00016_t[['anexo', 'cod_conta', 'valor', 'DIF']]

d3_00016 = pd.DataFrame([resposta_d3_00016], columns=['Resposta'])
# Insere novas colunas
d3_00016.insert(0, 'Dimensão', 'D3_00016')
d3_00016.insert(2, 'Descrição da Dimensão', 'Verifica a igualdade do valor da Transferências Obrigatórias da União Relativas às Emendas de Bancada')
d3_00016.insert(3, 'OBS', 'Anexo 03 do RREO e o Anexo 01 do RGF do poder executivo')
d3_00016

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D3_00016,OK,Verifica a igualdade do valor da Transferências Obrigatórias da União Relativas às Emendas de Bancada,Anexo 03 do RREO e o Anexo 01 do RGF do poder executivo


In [ ]:
d3_00016_t

,anexo,cod_conta,valor,DIF
0,RGF-Anexo 01,TransferenciasObrigatoriasDaUniaoRelativasAsEmendasDeBancada,"13,522,948.29",0.00
1,RREO-Anexo 03,RREO3TransferenciasObrigatoriasDaUniaoRelativasAsEmendasDeBancada,"13,522,948.29",0.00


<hr style="border: none; border-top: 4px solid gray;">
<p style="color: lightgreen; font-size: 26px;"><strong>NOVAS 2023</strong></p>
<hr style="border: none; border-top: 4px solid gray;">


# D3_00017 >> Igualdade entre os Restos a Pagar (Processados e Não-Processados) Pagos no Exercício do Anexo 6 do RREO com os Restos a Pagar  (Processados e Não-Processados) Pagos do Anexo 7 do RREO 

In [ ]:
###################  >> D3_00017_RPP Pagos
rpp_pago_rreo_7 = df_rreo_7.query('cod_conta == "RestosAPagarProcessadosENaoProcessadosLiquidadosPagos" & conta == "TOTAL (III) = (I + II)"')
rpp_pago_rreo_7['dimensao'] = 'D3_00017_RPP'
rpp_pago_rreo_7 = rpp_pago_rreo_7.groupby('dimensao').agg({'valor': 'sum'})

rpp_pago_rreo_6 = df_rreo_6.query('coluna == "RESTOS A PAGAR PROCESSADOS PAGOS (b)" & (cod_conta == "DespesasCorrentesExcetoFontesRPPS" | cod_conta == "DespesasPrimariasCorrentesComFontesRPPS"\
                                | cod_conta == "DespesasDeCapitalExcetoFontesRPPS" | cod_conta == "DespesasPrimariasDeCapitalComFontesRPPS" | cod_conta == "RREO6ReservaDeContingencia")')
rpp_pago_rreo_6['dimensao'] = 'D3_00017_RPP'
rpp_pago_rreo_6 = rpp_pago_rreo_6.groupby('dimensao').agg({'valor': 'sum'})

###################  >> D3_00017_RPNP Pagos
rpnp_pago_rreo_7 = df_rreo_7.query('cod_conta == "RestosAPagarNaoProcessadosPagos" & conta == "TOTAL (III) = (I + II)"')
rpnp_pago_rreo_7['dimensao'] = 'D3_00017_RPNP'
rpnp_pago_rreo_7 = rpnp_pago_rreo_7.groupby('dimensao').agg({'valor': 'sum'})

rpnp_pago_rreo_6 = df_rreo_6.query('coluna == "PAGOS (c)" & (cod_conta == "DespesasCorrentesExcetoFontesRPPS" | cod_conta == "DespesasPrimariasCorrentesComFontesRPPS"\
                                | cod_conta == "DespesasDeCapitalExcetoFontesRPPS" | cod_conta == "DespesasPrimariasDeCapitalComFontesRPPS" | cod_conta == "RREO6ReservaDeContingencia")')
rpnp_pago_rreo_6['dimensao'] = 'D3_00017_RPNP'
rpnp_pago_rreo_6 = rpnp_pago_rreo_6.groupby('dimensao').agg({'valor': 'sum'})

rreo_7 = pd.concat([rpp_pago_rreo_7, rpnp_pago_rreo_7]).reset_index()
rreo_6 = pd.concat([rpp_pago_rreo_6, rpnp_pago_rreo_6]).reset_index()

d3_00017_t = rreo_7.merge(rreo_6, on="dimensao", how="inner")

d3_00017_t['DIF'] = d3_00017_t['valor_x'] - d3_00017_t['valor_y']

d3_00017_t = d3_00017_t.groupby(['dimensao'])['DIF'].sum().reset_index()

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d3_00017_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d3_00017 = 'ERRO'
else:
    resposta_d3_00017 = 'OK'

d3_00017 = pd.DataFrame([resposta_d3_00017], columns=['Resposta'])
# Insere novas colunas
d3_00017.insert(0, 'Dimensão', 'D3_00017')
d3_00017.insert(2, 'Descrição da Dimensão', 'Igualdade entre os Restos a Pagar (Processados e Não-Processados) Pagos no Exercício')
d3_00017.insert(3, 'OBS', f'Anexo 6 do RREO e Anexo 7 do RREO')
d3_00017


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D3_00017,ERRO,Igualdade entre os Restos a Pagar (Processados e Não-Processados) Pagos no Exercício,Anexo 6 do RREO e Anexo 7 do RREO


In [ ]:
d3_00017_t

,dimensao,DIF
0,D3_00017_RPNP,"19,628,012.87"
1,D3_00017_RPP,"123,687,803.27"


# D3_00018 >> Igualdade entre o valor da Disponibilidade de Caixa Bruta de cada FR dos Recursos Vinculados ao RPPS do Anexo 5 do RGF (Poder Executivo) e o valor da respectiva linha de Caixa e Equivalentes de Caixa do Anexo 4 do RREO dos Fundos do RPPS

In [ ]:
disp_bruta_rgf_5e = df_rgf_5e.query('cod_conta == "DisponibilidadeDeCaixaBruta"')
disp_bruta_rgf_5e

,exercicio,periodo,periodicidade,instituicao,cod_ibge,uf,co_poder,populacao,anexo,esfera,rotulo,coluna,cod_conta,conta,valor
0,2024,3,Q,Governo do Estado do Rio de Janeiro,33,RJ,E,16615526,RGF-Anexo 05,E,Padrão,DISPONIBILIDADE DE CAIXA BRUTA (a),DisponibilidadeDeCaixaBruta,TOTAL DOS RECURSOS NÃO VINCULADOS (I),"14,222,375,488.39"
1,2024,3,Q,Governo do Estado do Rio de Janeiro,33,RJ,E,16615526,RGF-Anexo 05,E,Padrão,DISPONIBILIDADE DE CAIXA BRUTA (a),DisponibilidadeDeCaixaBruta,Outros Recursos não Vinculados,"14,222,375,488.39"
2,2024,3,Q,Governo do Estado do Rio de Janeiro,33,RJ,E,16615526,RGF-Anexo 05,E,Padrão,DISPONIBILIDADE DE CAIXA BRUTA (a),DisponibilidadeDeCaixaBruta,TOTAL DOS RECURSOS VINCULADOS (EXCETO AO RPPS) (II),"14,353,029,351.62"
3,2024,3,Q,Governo do Estado do Rio de Janeiro,33,RJ,E,16615526,RGF-Anexo 05,E,Padrão,DISPONIBILIDADE DE CAIXA BRUTA (a),DisponibilidadeDeCaixaBruta,Recursos Vinculados à Educação,"518,352,796.88"
4,2024,3,Q,Governo do Estado do Rio de Janeiro,33,RJ,E,16615526,RGF-Anexo 05,E,Padrão,DISPONIBILIDADE DE CAIXA BRUTA (a),DisponibilidadeDeCaixaBruta,Transferências do FUNDEB,"73,120,113.03"
5,2024,3,Q,Governo do Estado do Rio de Janeiro,33,RJ,E,16615526,RGF-Anexo 05,E,Padrão,DISPONIBILIDADE DE CAIXA BRUTA (a),DisponibilidadeDeCaixaBruta,Outros Recursos Vinculados à Educação,"445,232,683.85"
6,2024,3,Q,Governo do Estado do Rio de Janeiro,33,RJ,E,16615526,RGF-Anexo 05,E,Padrão,DISPONIBILIDADE DE CAIXA BRUTA (a),DisponibilidadeDeCaixaBruta,Recursos Vinculados à Saúde,"1,250,597,249.88"
7,2024,3,Q,Governo do Estado do Rio de Janeiro,33,RJ,E,16615526,RGF-Anexo 05,E,Padrão,DISPONIBILIDADE DE CAIXA BRUTA (a),DisponibilidadeDeCaixaBruta,Transferências Fundo a Fundo de Recursos do SUS,"201,365,877.59"
8,2024,3,Q,Governo do Estado do Rio de Janeiro,33,RJ,E,16615526,RGF-Anexo 05,E,Padrão,DISPONIBILIDADE DE CAIXA BRUTA (a),DisponibilidadeDeCaixaBruta,Outros Recursos Vinculados à Saúde,"1,049,231,372.29"
9,2024,3,Q,Governo do Estado do Rio de Janeiro,33,RJ,E,16615526,RGF-Anexo 05,E,Padrão,DISPONIBILIDADE DE CAIXA BRUTA (a),DisponibilidadeDeCaixaBruta,Recursos Vinculados à Previdência Social (Exceto ao RPPS),"390,040,062.12"


# D3_00021 >> Verificar se o somatório do passivo circulante e passivo não circulante (apenas financeiros) é maior ou igual aos Restos a Pagar (liquidados ou em liquidação) da MSC dezembro - PO - Poder Executivo + PO RPPS (MSC dezembro)

In [ ]:
############################# IMPORTANDO A MSC Orçamentária GRUPO 6 de DEZEMBRO
msc_orcam6 = pd.DataFrame()

pular = 0
while True:
    link = f'https://apidatalake.tesouro.gov.br/ords/siconfi/tt/msc_orcamentaria?id_ente={ente}&an_referencia={ano}&me_referencia=12&co_tipo_matriz=MSCC&classe_conta=6&id_tv=ending_balance&offset={pular}&limit=5000'
    requisicao = requests.get(link)
        
    # Verifique se a resposta não está vazia antes de tentar decodificar o JSON
    if requisicao.content:
        informacoes = requisicao.json()
        df_msc_parte = pd.DataFrame(informacoes["items"])
        if len(informacoes["items"]) < 1:
                break
        msc_orcam6 = pd.concat([msc_orcam6, df_msc_parte])
        pular += 5000
    else:
        break


C:\Users\mjandussi\AppData\Local\Temp\ipykernel_12956\3900988929.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  msc_orcam6 = pd.concat([msc_orcam6, df_msc_parte])


In [ ]:
# Cria a máscara para identificar as linhas que atendem à condição da retificadora
mascara_retificadora2 = (
    ((msc_orcam6['conta_contabil'].astype(str).str[0] == '6') & (msc_orcam6['natureza_conta'] == 'D') & (~msc_orcam6['tipo_valor'].eq('period_change')))
)

# Verifica se não existem valores negativos na coluna 'valor' antes de aplicar a troca de sinal
if not (msc_orcam6.loc[mascara_retificadora2, 'valor'] < 0).any():
    # Aplica a troca de sinal apenas nos valores selecionados pela máscara
    msc_orcam6.loc[mascara_retificadora2, 'valor'] = msc_orcam6.loc[mascara_retificadora2, 'valor'] * -1

In [ ]:
############################# IMPORTANDO A MSC Orçamentária GRUPO 2 de DEZEMBRO
msc_passivo = pd.DataFrame()

pular = 0
while True:
    link = f'https://apidatalake.tesouro.gov.br/ords/siconfi/tt/msc_patrimonial?id_ente={ente}&an_referencia={ano}&me_referencia=12&co_tipo_matriz=MSCC&classe_conta=2&id_tv=ending_balance&offset={pular}&limit=5000'
    requisicao = requests.get(link)
        
    # Verifique se a resposta não está vazia antes de tentar decodificar o JSON
    if requisicao.content:
        informacoes = requisicao.json()
        df_msc_parte = pd.DataFrame(informacoes["items"])
        if len(informacoes["items"]) < 1:
                break
        msc_passivo = pd.concat([msc_passivo, df_msc_parte])
        pular += 5000
    else:
        break


In [ ]:
msc_rp = msc_orcam6.query('conta_contabil == "632100000" and (poder_orgao == "10111" or poder_orgao == "10112")')
msc_rp['descricao'] = "msc_rp"
msc_rp = msc_rp.groupby(['descricao'])['valor'].sum().reset_index()

msc_passivo = msc_passivo.query('financeiro_permanente == 1 and (poder_orgao == "10111" or poder_orgao == "10112")')
msc_passivo['descricao'] = "msc_passivo_f"
msc_passivo = msc_passivo.groupby(['descricao'])['valor'].sum().reset_index()

d3_00021_t = pd.concat([msc_rp, msc_passivo])
d3_00021_t["DIF"] = d3_00021_t['valor'].diff()
d3_00021_t.loc[0, 'DIF'] = 0

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d3_00021_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d3_00021 = 'ERRO'
else:
    resposta_d3_00021 = 'OK'

d3_00021 = pd.DataFrame([resposta_d3_00021], columns=['Resposta'])
# Insere novas colunas
d3_00021.insert(0, 'Dimensão', 'D3_00021')
d3_00021.insert(2, 'Descrição da Dimensão', 'Verificar se o somatório do passivo circulante e passivo não circulante (apenas financeiros) é maior ou igual aos Restos a Pagar (liquidados ou em liquidação)')
d3_00021.insert(3, 'OBS', f'MSC dezembro - PO - Poder Executivo + PO RPPS')
d3_00021

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D3_00021,OK,Verificar se o somatório do passivo circulante e passivo não circulante (apenas financeiros) é maior ou igual aos Restos a Pagar (liquidados ou em liquidação),MSC dezembro - PO - Poder Executivo + PO RPPS


In [ ]:
d3_00021_t

,descricao,valor,DIF
0,msc_rp,"638,845,961.38",0.00
0,msc_passivo_f,"13,607,992,490.79",0.00


## Criando Receita Corrente e de Capital 

In [ ]:
receita = msc_orcam6.query('conta_contabil == "621200000" or conta_contabil == "621310100" or conta_contabil == "621310200" or conta_contabil == "621390000"')
receita['cat_receita'] = receita['natureza_receita'].astype(str).str[0]

receita_corr = receita.query('cat_receita == "1"')
receita_capi = receita.query('cat_receita == "2"')


# D3_00022 >> Igualdade das "Receitas  Correntes Orçamentárias" e "Receitas Correntes Intraorçamentárias" no Anexo 1 do RREO e dos valores da MSC de dezembro. 

In [ ]:
receita_corr['conta'] = "RECEITAS CORRENTES"
receita_corr_msc = receita_corr.groupby(['conta'])['valor'].sum().reset_index()

receita_corr_rreo = df_rreo_1.query('cod_conta == "ReceitasCorrentes" and coluna == "Até o Bimestre (c)"')
receita_corr_rreo = receita_corr_rreo.groupby(['conta'])['valor'].sum().reset_index()

d3_00022_t = pd.concat([receita_corr_msc, receita_corr_rreo])
d3_00022_t["DIF"] = d3_00022_t['valor'].diff()
d3_00022_t.loc[0, 'DIF'] = 0

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d3_00022_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d3_00022 = 'ERRO'
else:
    resposta_d3_00022 = 'OK'

d3_00022 = pd.DataFrame([resposta_d3_00022], columns=['Resposta'])
# Insere novas colunas
d3_00022.insert(0, 'Dimensão', 'D3_00022')
d3_00022.insert(2, 'Descrição da Dimensão', 'Verificar a Igualdade das "Receitas Correntes Orçamentárias" e "Receitas Correntes Intraorçamentárias"')
d3_00022.insert(3, 'OBS', f'Anexo 1 do RREO e dos valores da MSC de dezembro')
d3_00022

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D3_00022,OK,"Verificar a Igualdade das ""Receitas Correntes Orçamentárias"" e ""Receitas Correntes Intraorçamentárias""",Anexo 1 do RREO e dos valores da MSC de dezembro


In [ ]:
d3_00022_t

,conta,valor,DIF
0,RECEITAS CORRENTES,"97,995,152,741.34",0.00
0,RECEITAS CORRENTES,"97,995,152,741.34",0.00


# D3_00023 >>Igualdade das "Receitas de Capital Orçamentárias" e "Receitas de Capital Intraorçamentárias" no Anexo 1 do RREO e da MSC de dezembro. 

In [ ]:
receita_capi['conta'] = "RECEITAS DE CAPITAL"
receita_capi_msc = receita_capi.groupby(['conta'])['valor'].sum().reset_index()

receita_capi_rreo = df_rreo_1.query('cod_conta == "ReceitasDeCapital" and coluna == "Até o Bimestre (c)"')
receita_capi_rreo = receita_capi_rreo.groupby(['conta'])['valor'].sum().reset_index()

d3_00023_t = pd.concat([receita_capi_msc, receita_capi_rreo])
d3_00023_t["DIF"] = d3_00023_t['valor'].diff()
d3_00023_t.loc[0, 'DIF'] = 0

limiar = 1e-2  # Definir um LIMITE (número decimal 0.01 em notação científica)
# Substituir valores abaixo do limiar por zero (Isso significa que os valores que estão dentro do limiar tanto acima quanto abaixo de zero serão substituídos por zero)
d3_00023_t['DIF'] = d3_00023_t['DIF'].apply(lambda x: 0 if abs(x) < limiar else x)

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d3_00023_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d3_00023 = 'ERRO'
else:
    resposta_d3_00023 = 'OK'

d3_00023 = pd.DataFrame([resposta_d3_00023], columns=['Resposta'])
# Insere novas colunas
d3_00023.insert(0, 'Dimensão', 'D3_00023')
d3_00023.insert(2, 'Descrição da Dimensão', 'Verificar a Igualdade das "Receitas de Capital Orçamentárias" e "Receitas de Capital Intraorçamentárias"')
d3_00023.insert(3, 'OBS', f'Anexo 1 do RREO e dos valores da MSC de dezembro')
d3_00023

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D3_00023,OK,"Verificar a Igualdade das ""Receitas de Capital Orçamentárias"" e ""Receitas de Capital Intraorçamentárias""",Anexo 1 do RREO e dos valores da MSC de dezembro


In [ ]:
d3_00023_t

,conta,valor,DIF
0,RECEITAS DE CAPITAL,"301,179,199.31",0
0,RECEITAS DE CAPITAL,"301,179,199.31",0


## Criando Despesa Corrente e de Capital 

In [ ]:
msc_orcam6['cat_despesa'] = msc_orcam6['natureza_despesa'].astype(str).str[0]

despesa_corr = msc_orcam6.query('cat_despesa == "3"')
despesa_capi = msc_orcam6.query('cat_despesa == "4"')


# D3_00024 >> Igualdade das "Despesas  Correntes Orçamentárias" e "Despesas Correntes Intraorçamentárias" no Anexo 1 do RREO e dos valores da MSC de dezembro. 

In [ ]:
# MSC
despesa_corr_emp_msc = despesa_corr.query('conta_contabil == "622130400" or conta_contabil == "622130500" or conta_contabil == "622130600" or conta_contabil == "622130700"')
despesa_corr_emp_msc['conta'] = 'DESPESAS EMP CORRENTES'
despesa_corr_emp_msc = despesa_corr_emp_msc.groupby(['conta'])['valor'].sum().reset_index()

despesa_corr_liq_msc = despesa_corr.query('conta_contabil == "622130400" or conta_contabil == "622130700"')
despesa_corr_liq_msc['conta'] = 'DESPESAS LIQ CORRENTES'
despesa_corr_liq_msc = despesa_corr_liq_msc.groupby(['conta'])['valor'].sum().reset_index()

despesa_corr_paga_msc = despesa_corr.query('conta_contabil == "622130400"')
despesa_corr_paga_msc['conta'] = 'DESPESAS PAGAS CORRENTES'
despesa_corr_paga_msc = despesa_corr_paga_msc.groupby(['conta'])['valor'].sum().reset_index()

#RREO
despesa_corr_emp_rreo = df_rreo_1.query('(cod_conta == "DespesasCorrentes" or cod_conta == "DespesasCorrentesIntra") and coluna == "DESPESAS EMPENHADAS ATÉ O BIMESTRE (f)"')
despesa_corr_emp_rreo['conta'] = 'DESPESAS EMP CORRENTES'
despesa_corr_emp_rreo = despesa_corr_emp_rreo.groupby(['conta'])['valor'].sum().reset_index()

despesa_corr_liq_rreo = df_rreo_1.query('(cod_conta == "DespesasCorrentes" or cod_conta == "DespesasCorrentesIntra") and coluna == "DESPESAS LIQUIDADAS ATÉ O BIMESTRE (h)"')
despesa_corr_liq_rreo['conta'] = 'DESPESAS LIQ CORRENTES'
despesa_corr_liq_rreo = despesa_corr_liq_rreo.groupby(['conta'])['valor'].sum().reset_index()

despesa_corr_paga_rreo = df_rreo_1.query('(cod_conta == "DespesasCorrentes" or cod_conta == "DespesasCorrentesIntra") and coluna == "DESPESAS PAGAS ATÉ O BIMESTRE (j)"')
despesa_corr_paga_rreo['conta'] = 'DESPESAS PAGAS CORRENTES'
despesa_corr_paga_rreo = despesa_corr_paga_rreo.groupby(['conta'])['valor'].sum().reset_index()

d3_00024_t = pd.concat([despesa_corr_emp_msc, despesa_corr_emp_rreo, despesa_corr_liq_msc, despesa_corr_liq_rreo, despesa_corr_paga_msc, despesa_corr_paga_rreo]).reset_index()
d3_00024_t["DIF"] = d3_00024_t['valor'].diff()
d3_00024_t.loc[0, 'DIF'] = 0
d3_00024_t.loc[2, 'DIF'] = 0
d3_00024_t.loc[4, 'DIF'] = 0
d3_00024_t = d3_00024_t.drop(columns=['index'])

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d3_00024_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d3_00024 = 'ERRO'
else:
    resposta_d3_00024 = 'OK'

d3_00024 = pd.DataFrame([resposta_d3_00024], columns=['Resposta'])
# Insere novas colunas
d3_00024.insert(0, 'Dimensão', 'D3_00024')
d3_00024.insert(2, 'Descrição da Dimensão', 'Verificar a Igualdade das "Despesas  Correntes Orçamentárias" e "Despesas Correntes Intraorçamentárias"')
d3_00024.insert(3, 'OBS', f'Anexo 1 do RREO e dos valores da MSC de dezembro')
d3_00024

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D3_00024,OK,"Verificar a Igualdade das ""Despesas Correntes Orçamentárias"" e ""Despesas Correntes Intraorçamentárias""",Anexo 1 do RREO e dos valores da MSC de dezembro


In [ ]:
d3_00024_t

,conta,valor,DIF
0,DESPESAS EMP CORRENTES,"101,192,875,074.38",0.00
1,DESPESAS EMP CORRENTES,"101,192,875,074.38",0.00
2,DESPESAS LIQ CORRENTES,"100,235,292,386.58",0.00
3,DESPESAS LIQ CORRENTES,"100,235,292,386.58",-0.00
4,DESPESAS PAGAS CORRENTES,"98,018,255,635.54",0.00
5,DESPESAS PAGAS CORRENTES,"98,018,255,635.54",0.00


# D3_00025 >> Igualdade das "Despesas de Capital Orçamentárias" e "Despesas de Capital Intraorçamentárias" no Anexo 1 do RREO e da MSC de dezembro. 

In [ ]:
# MSC
despesa_capi_emp_msc = despesa_capi.query('conta_contabil == "622130400" or conta_contabil == "622130500" or conta_contabil == "622130600" or conta_contabil == "622130700"')
despesa_capi_emp_msc['conta'] = 'DESPESAS EMP CAPITAL'
despesa_capi_emp_msc = despesa_capi_emp_msc.groupby(['conta'])['valor'].sum().reset_index()

despesa_capi_liq_msc = despesa_capi.query('conta_contabil == "622130400" or conta_contabil == "622130700"')
despesa_capi_liq_msc['conta'] = 'DESPESAS LIQ CAPITAL'
despesa_capi_liq_msc = despesa_capi_liq_msc.groupby(['conta'])['valor'].sum().reset_index()

despesa_capi_paga_msc = despesa_capi.query('conta_contabil == "622130400"')
despesa_capi_paga_msc['conta'] = 'DESPESAS PAGAS CAPITAL'
despesa_capi_paga_msc = despesa_capi_paga_msc.groupby(['conta'])['valor'].sum().reset_index()

#RREO
despesa_capi_emp_rreo = df_rreo_1.query('(cod_conta == "DespesasDeCapital" or cod_conta == "DespesasDeCapitalIntra" or cod_conta == "AmortizacaoRefinanciamentoDaDivida") and coluna == "DESPESAS EMPENHADAS ATÉ O BIMESTRE (f)"')
despesa_capi_emp_rreo['conta'] = 'DESPESAS EMP CAPITAL'
despesa_capi_emp_rreo = despesa_capi_emp_rreo.groupby(['conta'])['valor'].sum().reset_index()

despesa_capi_liq_rreo = df_rreo_1.query('(cod_conta == "DespesasDeCapital" or cod_conta == "DespesasDeCapitalIntra" or cod_conta == "AmortizacaoRefinanciamentoDaDivida") and coluna == "DESPESAS LIQUIDADAS ATÉ O BIMESTRE (h)"')
despesa_capi_liq_rreo['conta'] = 'DESPESAS LIQ CAPITAL'
despesa_capi_liq_rreo = despesa_capi_liq_rreo.groupby(['conta'])['valor'].sum().reset_index()

despesa_capi_paga_rreo = df_rreo_1.query('(cod_conta == "DespesasDeCapital" or cod_conta == "DespesasDeCapitalIntra" or cod_conta == "AmortizacaoRefinanciamentoDaDivida") and coluna == "DESPESAS PAGAS ATÉ O BIMESTRE (j)"')
despesa_capi_paga_rreo['conta'] = 'DESPESAS PAGAS CAPITAL'
despesa_capi_paga_rreo = despesa_capi_paga_rreo.groupby(['conta'])['valor'].sum().reset_index()

d3_00025_t = pd.concat([despesa_capi_emp_msc, despesa_capi_emp_rreo, despesa_capi_liq_msc, despesa_capi_liq_rreo, despesa_capi_paga_msc, despesa_capi_paga_rreo]).reset_index()
d3_00025_t["DIF"] = d3_00025_t['valor'].diff()
d3_00025_t.loc[0, 'DIF'] = 0
d3_00025_t.loc[2, 'DIF'] = 0
d3_00025_t.loc[4, 'DIF'] = 0
d3_00025_t = d3_00025_t.drop(columns=['index'])

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d3_00025_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d3_00025 = 'ERRO'
else:
    resposta_d3_00025 = 'OK'

d3_00025 = pd.DataFrame([resposta_d3_00025], columns=['Resposta'])
# Insere novas colunas
d3_00025.insert(0, 'Dimensão', 'D3_00025')
d3_00025.insert(2, 'Descrição da Dimensão', 'Verificar a Igualdade das "Despesas de Capital Orçamentárias" e "Despesas de Capital Intraorçamentárias"')
d3_00025.insert(3, 'OBS', f'Anexo 1 do RREO e dos valores da MSC de dezembro')
d3_00025

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D3_00025,OK,"Verificar a Igualdade das ""Despesas de Capital Orçamentárias"" e ""Despesas de Capital Intraorçamentárias""",Anexo 1 do RREO e dos valores da MSC de dezembro


In [ ]:
d3_00025_t

,conta,valor,DIF
0,DESPESAS EMP CAPITAL,"7,333,917,401.72",0.00
1,DESPESAS EMP CAPITAL,"7,333,917,401.72",0.00
2,DESPESAS LIQ CAPITAL,"7,027,325,426.42",0.00
3,DESPESAS LIQ CAPITAL,"7,027,325,426.42",0.00
4,DESPESAS PAGAS CAPITAL,"6,805,418,373.74",0.00
5,DESPESAS PAGAS CAPITAL,"6,805,418,373.74",0.00


# D3_00027 >> Verifica a igualdade entre Dotação atualizada, Despesas empenhadas até o bimestre, Despesas Liquidadas até o bimestre nos Anexos 1 e 6 do RREO (substituirá a D3_00003)

In [ ]:
# Dotação Atualizada >> RREO 1
dotatualiz_rreo1 = df_rreo_1.query('coluna == "DOTAÇÃO ATUALIZADA (e)" & cod_conta == "TotalDespesas"')
dotatualiz_rreo1['conta'] = 'DESPESAS DOTAÇÃO ATUALIZADA'
dotatualiz_rreo1 = dotatualiz_rreo1.groupby(['conta'])['valor'].sum().reset_index()

# Dotação Atualizada >> RREO 6
dotatualiz_rreo6 = df_rreo_6.query('coluna == "DOTAÇÃO ATUALIZADA" & (cod_conta == "DespesasCorrentesExcetoFontesRPPS" | cod_conta == "DespesasPrimariasCorrentesComFontesRPPS"\
                                | cod_conta == "DespesasDeCapitalExcetoFontesRPPS" | cod_conta == "DespesasPrimariasDeCapitalComFontesRPPS" | cod_conta == "RREO6ReservaDeContingencia")')
dotatualiz_rreo6['conta'] = 'DESPESAS DOTAÇÃO ATUALIZADA'
dotatualiz_rreo6 = dotatualiz_rreo6.groupby(['conta'])['valor'].sum().reset_index()

# Dotação Atualizada >> RREO 1
emp_rreo1 = df_rreo_1.query('coluna == "DESPESAS EMPENHADAS ATÉ O BIMESTRE (f)" & cod_conta == "TotalDespesas"')
emp_rreo1['conta'] = 'DESPESAS EMPENHADAS ATÉ O BIMESTRE'
emp_rreo1 = emp_rreo1.groupby(['conta'])['valor'].sum().reset_index()

# Dotação Atualizada >> RREO 6
emp_rreo6 = df_rreo_6.query('coluna == "DESPESAS EMPENHADAS" & (cod_conta == "DespesasCorrentesExcetoFontesRPPS" | cod_conta == "DespesasPrimariasCorrentesComFontesRPPS"\
                                | cod_conta == "DespesasDeCapitalExcetoFontesRPPS" | cod_conta == "DespesasPrimariasDeCapitalComFontesRPPS" | cod_conta == "RREO6ReservaDeContingencia")')
emp_rreo6['conta'] = 'DESPESAS EMPENHADAS ATÉ O BIMESTRE'
emp_rreo6 = emp_rreo6.groupby(['conta'])['valor'].sum().reset_index()

# Dotação Atualizada >> RREO 1
liq_rreo1 = df_rreo_1.query('coluna == "DESPESAS LIQUIDADAS ATÉ O BIMESTRE (h)" & cod_conta == "TotalDespesas"')
liq_rreo1['conta'] = 'DESPESAS LIQUIDADAS ATÉ O BIMESTRE'
liq_rreo1 = liq_rreo1.groupby(['conta'])['valor'].sum().reset_index()

# Dotação Atualizada >> RREO 6
liq_rreo6 = df_rreo_6.query('coluna == "DESPESAS LIQUIDADAS" & (cod_conta == "DespesasCorrentesExcetoFontesRPPS" | cod_conta == "DespesasPrimariasCorrentesComFontesRPPS"\
                                | cod_conta == "DespesasDeCapitalExcetoFontesRPPS" | cod_conta == "DespesasPrimariasDeCapitalComFontesRPPS" | cod_conta == "RREO6ReservaDeContingencia")')
liq_rreo6['conta'] = 'DESPESAS LIQUIDADAS ATÉ O BIMESTRE'
liq_rreo6 = liq_rreo6.groupby(['conta'])['valor'].sum().reset_index()

d3_00027_t = pd.concat([dotatualiz_rreo1, dotatualiz_rreo6, emp_rreo1, emp_rreo6, liq_rreo1, liq_rreo6]).reset_index()
d3_00027_t["DIF"] = d3_00027_t['valor'].diff()
d3_00027_t.loc[0, 'DIF'] = 0
d3_00027_t.loc[2, 'DIF'] = 0
d3_00027_t.loc[4, 'DIF'] = 0
d3_00027_t = d3_00027_t.drop(columns=['index'])


# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d3_00027_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d3_00027 = 'ERRO'
else:
    resposta_d3_00027 = 'OK'

d3_00027 = pd.DataFrame([resposta_d3_00027], columns=['Resposta'])
# Insere novas colunas
d3_00027.insert(0, 'Dimensão', 'D3_00027')
d3_00027.insert(2, 'Descrição da Dimensão', 'Verifica a igualdade entre Dotação atualizada, Despesas Empenhadas e Despesas Liquidadas')
d3_00027.insert(3, 'OBS', f'Anexos 1 e 6 do RREO')
d3_00027

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D3_00027,ERRO,"Verifica a igualdade entre Dotação atualizada, Despesas Empenhadas e Despesas Liquidadas",Anexos 1 e 6 do RREO


In [ ]:
d3_00027_t

,conta,valor,DIF
0,DESPESAS DOTAÇÃO ATUALIZADA,"129,182,370,918.79",0.00
1,DESPESAS DOTAÇÃO ATUALIZADA,"120,818,311,152.63","-8,364,059,766.16"
2,DESPESAS EMPENHADAS ATÉ O BIMESTRE,"108,526,792,476.10",0.00
3,DESPESAS EMPENHADAS ATÉ O BIMESTRE,"100,508,250,796.52","-8,018,541,679.58"
4,DESPESAS LIQUIDADAS ATÉ O BIMESTRE,"107,262,617,813.00",0.00
5,DESPESAS LIQUIDADAS ATÉ O BIMESTRE,"99,267,937,429.71","-7,994,680,383.29"


# D3_00028 >> Verifica a igualdade entre Previsão atualizada e Receitas Realizadas até o bimestre nos Anexos 1 e 6 do RREO (substituirá a D3_00007)

In [ ]:
# Receita Realizada >> RREO 1
recrealiz_rreo1 = df_rreo_1.query('coluna == "Até o Bimestre (c)" & cod_conta == "TotalReceitas"')
recrealiz_rreo1['conta'] = 'REC Realizada até o Bimestre'
recrealiz_rreo1 = recrealiz_rreo1.groupby(['conta'])['valor'].sum().reset_index()

# Receita Realizada >> RREO 6
recrealiz_rreo6 = df_rreo_6.query('coluna == "RECEITAS REALIZADAS (a)" & (cod_conta == "ReceitasCorrentesExcetoFontesRPPS" or cod_conta == "ReceitasPrimariasCorrentesComFontesRPPS" or\
                                            cod_conta == "ReceitasNaoPrimariasCorrentesComFontesRPPS" or cod_conta == "ReceitasDeCapitalExcetoFontesRPPS")')
recrealiz_rreo6['conta'] = 'REC Realizada até o Bimestre'
recrealiz_rreo6 = recrealiz_rreo6.groupby(['conta'])['valor'].sum().reset_index()

# Previsão Atualizada >> RREO 1
prevatualiz_rreo1 = df_rreo_1.query('coluna == "PREVISÃO ATUALIZADA (a)" & cod_conta == "TotalReceitas"')
prevatualiz_rreo1['conta'] = 'Previsão Atualizada'
prevatualiz_rreo1 = prevatualiz_rreo1.groupby(['conta'])['valor'].sum().reset_index()

# Previsão Atualizada >> RREO 6
prevatualiz_rreo6 = df_rreo_6.query('coluna == "PREVISÃO ATUALIZADA" & (cod_conta == "ReceitasCorrentesExcetoFontesRPPS" or cod_conta == "ReceitasPrimariasCorrentesComFontesRPPS" or\
                                            cod_conta == "ReceitasNaoPrimariasCorrentesComFontesRPPS" or cod_conta == "ReceitasDeCapitalExcetoFontesRPPS")')
prevatualiz_rreo6['conta'] = 'Previsão Atualizada'
prevatualiz_rreo6 = prevatualiz_rreo6.groupby(['conta'])['valor'].sum().reset_index()

d3_00028_t = pd.concat([recrealiz_rreo1, recrealiz_rreo6, prevatualiz_rreo1, prevatualiz_rreo6]).reset_index()
d3_00028_t["DIF"] = d3_00028_t['valor'].diff()
d3_00028_t.loc[0, 'DIF'] = 0
d3_00028_t.loc[2, 'DIF'] = 0
d3_00028_t = d3_00028_t.drop(columns=['index'])

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d3_00028_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d3_00028 = 'ERRO'
else:
    resposta_d3_00028 = 'OK'

d3_00028 = pd.DataFrame([resposta_d3_00028], columns=['Resposta'])
# Insere novas colunas
d3_00028.insert(0, 'Dimensão', 'D3_00028')
d3_00028.insert(2, 'Descrição da Dimensão', 'Verifica a igualdade entre Previsão atualizada e Receitas Realizadas')
d3_00028.insert(3, 'OBS', f'Anexos 1 e 6 do RREO')
d3_00028

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D3_00028,ERRO,Verifica a igualdade entre Previsão atualizada e Receitas Realizadas,Anexos 1 e 6 do RREO


In [ ]:
d3_00028_t

,conta,valor,DIF
0,REC Realizada até o Bimestre,"106,112,454,653.40",0.00
1,REC Realizada até o Bimestre,"98,296,331,940.65","-7,816,122,712.75"
2,Previsão Atualizada,"105,155,633,240.49",0.00
3,Previsão Atualizada,"97,493,025,875.27","-7,662,607,365.22"


<hr style="border: none; border-top: 4px solid gray;">
<p style="color: orange; font-size: 26px;"><strong>NOVAS 2024</strong></p>
<hr style="border: none; border-top: 4px solid gray;">


# D3_00029

In [ ]:
condicao = df_rgf_1e['cod_conta'].str.startswith("Parcela")
parcela_dedut_rgf_1 = df_rgf_1e[condicao]
parcela_dedut_rgf_1 = parcela_dedut_rgf_1.query('coluna == "TOTAL (ÚLTIMOS 12 MESES) (a)"')
parcela_dedut_rgf_1['conta'] = 'Parcela Dedutível RGF 1'
parcela_dedut_rgf_1 = parcela_dedut_rgf_1.groupby(['conta'])['valor'].sum().reset_index()

# Definindo as listas para as condições 'isin'
contas_contabeis = ["622130300", "622130400", "622130500", "622130600", "622130700"]
poderes_orgaos = ["10111", "10112", "60611"]
fontes_recursos = ["1605", "2605", "9605"]

# Expressão regular para o padrão 33XX34XX
regex_pattern_natureza = r'^33\d{2}34\d{2}$'

# Combinando todas as condições
condicao_final = (
    msc['conta_contabil'].isin(contas_contabeis) &
    msc['poder_orgao'].isin(poderes_orgaos) &
    (msc['tipo_valor'] == "ending_balance") &
    msc['fonte_recursos'].isin(fontes_recursos) &
    (
        msc['natureza_despesa'].str.startswith("31") |
        msc['natureza_despesa'].str.startswith("33909134") |
        msc['natureza_despesa'].str.startswith("33909234") |
        msc['natureza_despesa'].str.contains(regex_pattern_natureza, regex=True)
    )
)

# Aplicar o filtro ao DataFrame
msc_dez_29_t = msc_dez[condicao_final]

msc_dez_29_t['conta'] = 'Parcela Dedutível MSC'
msc_dez_29 = msc_dez_29_t.groupby(['conta'])['valor'].sum().reset_index()

####################################################

d3_00029_t = pd.concat([parcela_dedut_rgf_1, msc_dez_29]).reset_index(drop=True)
d3_00029_t["DIF"] = d3_00029_t['valor'].diff()
d3_00029_t.loc[0, 'DIF'] = 0

##################################################

# Certificar que a coluna 'valor' é numérica
d3_00029_t['valor'] = pd.to_numeric(d3_00029_t['valor'], errors='coerce')

# Acessar os valores das linhas específicas
# Assumimos que 'Parcela Dedutível RGF 1' está na linha de índice 0
valor_rgf_1 = d3_00029_t.loc[d3_00029_t['conta'] == 'Parcela Dedutível RGF 1', 'valor'].iloc[0]

# Assumimos que 'Parcela Dedutível MSC' está na linha de índice 1
valor_msc = d3_00029_t.loc[d3_00029_t['conta'] == 'Parcela Dedutível MSC', 'valor'].iloc[0]

# Calcular a métrica: RGF 1 deve ser menor ou igual a 90% do valor da MSC
# Multiplicamos valor_msc por 0.90 (ou 90/100)
metrica_condicao = valor_rgf_1 <= (valor_msc * 0.90)

d3_00029_t['Metrica_RGF_MSC'] = metrica_condicao

# Se você quiser uma saída 'OK' ou 'ERRO' baseada na métrica:
if metrica_condicao:
    resposta_d3_00029 = 'OK'
else:
    resposta_d3_00029 = 'ERRO'

d3_00029 = pd.DataFrame([resposta_d3_00029], columns=['Resposta'])
# Insere novas colunas
d3_00029.insert(0, 'Dimensão', 'D3_00029')
d3_00029.insert(2, 'Descrição da Dimensão', 'A Parcela dedutível referente ao piso salarial do Enfermeiro, Técnico de Enfermagem, Auxiliar de Enfermage deve ser menor ou igual a 90% do somatório dos valores informados na MSC DEZ')
d3_00029.insert(3, 'OBS', f'RGF 01 e MSC DEZ')
d3_00029

C:\Users\mjandussi\AppData\Local\Temp\ipykernel_12956\3883169188.py:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  msc_dez_29_t = msc_dez[condicao_final]


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D3_00029,ERRO,"A Parcela dedutível referente ao piso salarial do Enfermeiro, Técnico de Enfermagem, Auxiliar de Enfermage deve ser menor ou igual a 90% do somatório dos valores informados na MSC DEZ",RGF 01 e MSC DEZ


In [ ]:
d3_00029_t

,conta,valor,DIF,Metrica_RGF_MSC
0,Parcela Dedutível RGF 1,"11,930,109.10",0.00,False
1,Parcela Dedutível MSC,"3,936,496.34","-7,993,612.76",False


# D3_00030

# CONSOLIDANDO D3

In [ ]:
d3 = pd.concat([d3_00001, d3_00002, d3_00005, d3_00006, d3_00008, d3_00009, d3_00010, d3_00011, d3_00014, d3_00015, d3_00016, d3_00017, d3_00021, 
                d3_00022, d3_00023, d3_00024, d3_00025, d3_00027, d3_00028, d3_00029]).reset_index(drop=True)
    

d3


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D3_00001,OK,Verifica se o resultado orçamentário foi calculado corretamente no Balanço Orçamentário,Anexo 01 do RREO 6ºB
1,D3_00002,OK,Verifica a igualdade dos valores de despesa entre o Balanço Orçamentário e o Demonstrativo da Execução da Despesa por Função/Subfunção,Anexo 01 e Anexo 02 do RREO
2,D3_00005,OK,Verifica a Igualdade da Receita Corrente Líquida (RCL),"Anexo 03 do RREO e os Anexos 01, 02, 03 e 04 do RGF do poder executivo"
3,D3_00006,OK,Verifica a Igualdade da Dívida Consolidada Líquida (DCL),Anexo 06 do RREO e o Anexo 02 do RGF do poder executivo
4,D3_00008,OK (com dif centavos),Verifica a igualdade dos valores dos restos a pagar não processados,Anexo 01 do RREO e a soma dos valores do Anexo 05 do RGF de todos os poderes/órgãos
5,D3_00009,OK,Verifica a igualdade dos valores dos restos a pagar processados e não processados,Anexo 07 do RREO e os Anexos 05 do RGF de todos os poderes/órgãos
6,D3_00010,OK,Verifica a Igualdade da Receita Corrente Líquida (RCL) no Anexo 01 do RGF de todos os poderes/órgãos,Demonstrativo da Despesa com Pessoal do RGF (Anexo 1) de Todos os poderes/órgãos
7,D3_00011,OK,Verifica se a Dedução de inativos e pensionistas com recursos vinculados é menor ou igual ao valor com inativos e pensionistas no Anexo 01 do RGF de todos os poderes/órgãos,Demonstrativo da Despesa com Pessoal do RGF (Anexo 1) de Todos os poderes/órgãos
8,D3_00014,OK,Verifica a igualdade do valor da Transferências Obrigatórias da União Relativas às Emendas Individuais,"Anexos 1, 2, 3 e 4 do RGF do poder executivo"
9,D3_00015,OK,Verifica a igualdade do valor da Transferências Obrigatórias da União Relativas às Emendas Individuais,Anexo 03 do RREO e o Anexo 01 do RGF do poder executivo


In [ ]:
contagem_total_dim = d3.Dimensão.unique()
contagem_total_dim = len(contagem_total_dim)
contagem_total_dim

19

In [ ]:
total_dim = 22
dim_falta = total_dim - contagem_total_dim
dim_falta

3

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

<hr style="border: none; border-top: 4px dashed red;">

# D4_00001

In [ ]:
## D4_00001
###################  >> D4_00001_Receita Realizada RREO 2
rec_rreo_2 = df_rreo_1.query('coluna == "Até o Bimestre (c)" & cod_conta == "TotalReceitas"')
rec_rreo_2['dimensao'] = 'D4_00001_Rec.Realizada'
rec_rreo_2 = rec_rreo_2.filter(items=['dimensao', 'valor'])

###################  >> D4_00001_Receita Realizada DCA
rec_dca = df_dca_c.query('cod_conta == "TotalReceitas"')
rec_dca['dimensao'] = 'D4_00001_Rec.Realizada'
rec_dca = rec_dca.groupby('dimensao').agg({'valor': 'sum'})
rec_dca = rec_dca.reset_index()

d4_00001 = rec_rreo_2.merge(rec_dca, on='dimensao')
d4_00001['DIF'] = d4_00001['valor_x'] - d4_00001['valor_y']
d4_00001_t = d4_00001.filter(items=['dimensao', 'valor_x', 'valor_y', 'DIF'])
d4_00001_t.columns = ['Dimensão', 'RREO', 'DCA', 'DIF']

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d4_00001_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d4_00001 = 'ERRO'
else:
    resposta_d4_00001 = 'OK'

d4_00001 = pd.DataFrame([resposta_d4_00001], columns=['Resposta'])
# Insere novas colunas
d4_00001.insert(0, 'Dimensão', 'D4_00001')
d4_00001.insert(2, 'Descrição da Dimensão', 'Igualdade da receita realizada')
d4_00001.insert(3, 'OBS', 'Anexo I-C da DCA e o Anexo 01 do RREO 6ºB')
d4_00001

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00001,OK,Igualdade da receita realizada,Anexo I-C da DCA e o Anexo 01 do RREO 6ºB


In [ ]:
d4_00001_t

,Dimensão,RREO,DCA,DIF
0,D4_00001_Rec.Realizada,"106,112,454,653.40","106,112,454,653.40",0.00


# D4_00002

In [ ]:
## D4_00002
###################  >> D4_00002_DPS Emp., Liq. e Paga DCA X RREO
## RREO 2
emp_rreo_2 = df_rreo_1.query('coluna == "DESPESAS EMPENHADAS ATÉ O BIMESTRE (f)" & cod_conta == "TotalDespesas"')
emp_rreo_2['dimensao'] = 'D4_00002_Empenhado'
emp_rreo_2 = emp_rreo_2.filter(items=['dimensao', 'valor'])
emp_rreo_2 = emp_rreo_2.set_index("dimensao")
emp_rreo_2 = emp_rreo_2.reset_index()

liq_rreo_2 = df_rreo_1.query('coluna == "DESPESAS LIQUIDADAS ATÉ O BIMESTRE (h)" & cod_conta == "TotalDespesas"')
liq_rreo_2['dimensao'] = 'D4_00002_Liquidado'
liq_rreo_2 = liq_rreo_2.filter(items=['dimensao', 'valor'])
liq_rreo_2 = liq_rreo_2.set_index("dimensao")
liq_rreo_2 = liq_rreo_2.reset_index()

pago_rreo_2 = df_rreo_1.query('coluna == "DESPESAS PAGAS ATÉ O BIMESTRE (j)" & cod_conta == "TotalDespesas"')
pago_rreo_2['dimensao'] = 'D4_00002_Pago'
pago_rreo_2 = pago_rreo_2.filter(items=['dimensao', 'valor'])
pago_rreo_2 = pago_rreo_2.set_index("dimensao")
pago_rreo_2 = pago_rreo_2.reset_index()

rpnp_rreo_2 = df_rreo_1.query('coluna == "INSCRITAS EM RESTOS A PAGAR NÃO PROCESSADOS (k)" & cod_conta == "TotalDespesas"')
rpnp_rreo_2['dimensao'] = 'D4_00002_Inscrição RPNP'
rpnp_rreo_2 = rpnp_rreo_2.filter(items=['dimensao', 'valor'])
rpnp_rreo_2 = rpnp_rreo_2.set_index("dimensao")
rpnp_rreo_2 = rpnp_rreo_2.reset_index()


## DCA
emp_dca = df_dca_d.query('cod_conta == "TotalDespesas" & coluna == "Despesas Empenhadas"')
emp_dca['dimensao'] = 'D4_00002_Empenhado'
emp_dca = emp_dca.filter(items=['dimensao', 'valor'])
emp_dca = emp_dca.set_index("dimensao")
emp_dca = emp_dca.reset_index()

liq_dca = df_dca_d.query('cod_conta == "TotalDespesas" & coluna == "Despesas Liquidadas"')
liq_dca['dimensao'] = 'D4_00002_Liquidado'
liq_dca = liq_dca.filter(items=['dimensao', 'valor'])
liq_dca = liq_dca.set_index("dimensao")
liq_dca = liq_dca.reset_index()

pago_dca = df_dca_d.query('cod_conta == "TotalDespesas" & coluna == "Despesas Pagas"')
pago_dca['dimensao'] = 'D4_00002_Pago'
pago_dca = pago_dca.filter(items=['dimensao', 'valor'])
pago_dca = pago_dca.set_index("dimensao")
pago_dca = pago_dca.reset_index()

rpnp_dca = df_dca_d.query('cod_conta == "TotalDespesas" & coluna == "Inscrição de Restos a Pagar Não Processados"')
rpnp_dca['dimensao'] = 'D4_00002_Inscrição RPNP'
rpnp_dca = rpnp_dca.filter(items=['dimensao', 'valor'])
rpnp_dca = rpnp_dca.set_index("dimensao")
rpnp_dca = rpnp_dca.reset_index()

## MERGE Empenhos
d4_00002_emp = emp_rreo_2.merge(emp_dca, on='dimensao')
d4_00002_emp['DIF'] = d4_00002_emp['valor_x'] - d4_00002_emp['valor_y']
d4_00002_emp = d4_00002_emp.filter(items=['dimensao', 'valor_x', 'valor_y', 'DIF'])
d4_00002_emp.columns = ['Dimensão', 'RREO', 'DCA', 'DIF']

## MERGE Liquidados
d4_00002_liq = liq_rreo_2.merge(liq_dca, on='dimensao')
d4_00002_liq['DIF'] = d4_00002_liq['valor_x'] - d4_00002_liq['valor_y']
d4_00002_liq = d4_00002_liq.filter(items=['dimensao', 'valor_x', 'valor_y', 'DIF'])
d4_00002_liq.columns = ['Dimensão', 'RREO', 'DCA', 'DIF']

## MERGE Pagos
d4_00002_pago = pago_rreo_2.merge(pago_dca, on='dimensao')
d4_00002_pago['DIF'] = d4_00002_pago['valor_x'] - d4_00002_pago['valor_y']
d4_00002_pago = d4_00002_pago.filter(items=['dimensao', 'valor_x', 'valor_y', 'DIF'])
d4_00002_pago.columns = ['Dimensão', 'RREO', 'DCA', 'DIF']

## MERGE RPNP
d4_00002_rpnp = rpnp_rreo_2.merge(rpnp_dca, on='dimensao')
d4_00002_rpnp['DIF'] = d4_00002_rpnp['valor_x'] - d4_00002_rpnp['valor_y']
d4_00002_rpnp = d4_00002_rpnp.filter(items=['dimensao', 'valor_x', 'valor_y', 'DIF'])
d4_00002_rpnp.columns = ['Dimensão', 'RREO', 'DCA', 'DIF']

d4_00002_t = pd.concat([d4_00002_emp, d4_00002_liq, d4_00002_pago, d4_00002_rpnp])

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d4_00002_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d4_00002 = 'ERRO'
else:
    resposta_d4_00002 = 'OK'

d4_00002 = pd.DataFrame([resposta_d4_00002], columns=['Resposta'])
# Insere novas colunas
d4_00002.insert(0, 'Dimensão', 'D4_00002')
d4_00002.insert(2, 'Descrição da Dimensão', 'Igualdade da execução da despesa')
d4_00002.insert(3, 'OBS', 'Anexo I-D da DCA e o Anexo 01 do RREO 6ºB')
d4_00002

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00002,OK,Igualdade da execução da despesa,Anexo I-D da DCA e o Anexo 01 do RREO 6ºB


In [ ]:
d4_00002_t

,Dimensão,RREO,DCA,DIF
0,D4_00002_Empenhado,"108,526,792,476.10","108,526,792,476.10",0.00
0,D4_00002_Liquidado,"107,262,617,813.00","107,262,617,813.00",0.00
0,D4_00002_Pago,"104,823,674,009.28","104,823,674,009.28",0.00
0,D4_00002_Inscrição RPNP,"1,264,174,663.10","1,264,174,663.10",0.00


In [ ]:
## D4_00003

############################## RREO 2
# Empenhados
emp_rreo_2_2 = df_rreo_2.query('coluna == "DESPESAS EMPENHADAS ATÉ O BIMESTRE (b)" & cod_conta == "RREO2TotalDespesas" & conta == "DESPESAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I)"')
emp_rreo_2_2['dimensao'] = 'D4_00003_Empenhado'
emp_rreo_2_2 = emp_rreo_2_2.filter(items=['dimensao', 'valor'])

# Liquidados
liq_rreo_2_2 = df_rreo_2.query('coluna == "DESPESAS LIQUIDADAS ATÉ O BIMESTRE (d)" & cod_conta == "RREO2TotalDespesas" & conta == "DESPESAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I)"')
liq_rreo_2_2['dimensao'] = 'D4_00003_Liquidado'
liq_rreo_2_2 = liq_rreo_2_2.filter(items=['dimensao', 'valor'])

# Inscritas em RPNP
rpnp_rreo_2_2 = df_rreo_2.query('coluna == "INSCRITAS EM RESTOS A PAGAR NÃO PROCESSADOS (f)" & cod_conta == "RREO2TotalDespesas" & conta == "DESPESAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I)"')
rpnp_rreo_2_2['dimensao'] = 'D4_00003_Inscrição RPNP'
rpnp_rreo_2_2 = rpnp_rreo_2_2.filter(items=['dimensao', 'valor'])


################################# DCA E
#Empenhados
emp_dca_2 = df_dca_e.query('conta == "Despesas Exceto Intraorçamentárias" and coluna == "Despesas Empenhadas"')
emp_dca_2['dimensao'] = 'D4_00003_Empenhado'
emp_dca_2 = emp_dca_2.groupby('dimensao').agg({'valor': 'sum'})

#Liquidados
liq_dca_2 = df_dca_e.query('conta == "Despesas Exceto Intraorçamentárias" and coluna == "Despesas Liquidadas"')
liq_dca_2['dimensao'] = 'D4_00003_Liquidado'
liq_dca_2 = liq_dca_2.groupby('dimensao').agg({'valor': 'sum'})

# Inscritas em RPNP
rpnp_dca_2 = df_dca_e.query('conta == "Despesas Exceto Intraorçamentárias" and coluna == "Inscrição de Restos a Pagar Não Processados"')
rpnp_dca_2['dimensao'] = 'D4_00003_Inscrição RPNP'
rpnp_dca_2 = rpnp_dca_2.groupby('dimensao').agg({'valor': 'sum'})

############################################ MERGE
###################  >> D4_00003_Empenhado
d4_00003_emp = emp_rreo_2_2.merge(emp_dca_2, on='dimensao')
d4_00003_emp['DIF'] = d4_00003_emp['valor_x'] - d4_00003_emp['valor_y']
d4_00003_emp.columns = ['Dimensão', 'RREO 2', 'DCA E', 'DIF']

###################  >> D4_00003_Liquidado
d4_00003_liq = liq_rreo_2_2.merge(liq_dca_2, on='dimensao')
d4_00003_liq['DIF'] = d4_00003_liq['valor_x'] - d4_00003_liq['valor_y']
d4_00003_liq.columns = ['Dimensão', 'RREO 2', 'DCA E', 'DIF']

###################  >> D4_00003_Inscrição de RPNP
d4_00003_rpnp = rpnp_rreo_2_2.merge(rpnp_dca_2, on='dimensao')
d4_00003_rpnp['DIF'] = d4_00003_rpnp['valor_x'] - d4_00003_rpnp['valor_y']
d4_00003_rpnp.columns = ['Dimensão', 'RREO 2', 'DCA E', 'DIF']


# Juntando os DF
d4_00003_t = pd.concat([d4_00003_emp, d4_00003_liq, d4_00003_rpnp])

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d4_00003_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d4_00003 = 'ERRO'
else:
    resposta_d4_00003 = 'OK'

d4_00003 = pd.DataFrame([resposta_d4_00003], columns=['Resposta'])
# Insere novas colunas
d4_00003.insert(0, 'Dimensão', 'D4_00003')
d4_00003.insert(2, 'Descrição da Dimensão', 'Igualdade da execução da despesa por função')
d4_00003.insert(3, 'OBS', 'Anexo I-E da DCA e o Anexo 02 do RREO 6ºB (exceto intraorçamentária)')
d4_00003

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00003,OK,Igualdade da execução da despesa por função,Anexo I-E da DCA e o Anexo 02 do RREO 6ºB (exceto intraorçamentária)


In [ ]:
d4_00003_t

,Dimensão,RREO 2,DCA E,DIF
0,D4_00003_Empenhado,"100,508,250,796.52","100,508,250,796.52",0.00
0,D4_00003_Liquidado,"99,267,937,429.71","99,267,937,429.71",0.00
0,D4_00003_Inscrição RPNP,"1,240,313,366.81","1,240,313,366.81",0.00


In [ ]:
## D4_00004
############################## RREO 2
# Empenhados INTRA
emp_intra_rreo_2_2 = df_rreo_2.query('coluna == "DESPESAS EMPENHADAS ATÉ O BIMESTRE (b)" & cod_conta == "RREO2TotalDespesas" & conta == "DESPESAS (INTRA-ORÇAMENTÁRIAS) (II)"')
emp_intra_rreo_2_2['dimensao'] = 'D4_00004_Empenhado_INTRA'
emp_intra_rreo_2_2 = emp_intra_rreo_2_2.filter(items=['dimensao', 'valor'])
emp_intra_rreo_2_2 = emp_intra_rreo_2_2.set_index("dimensao")

# Liquidados INTRA
liq_intra_rreo_2_2 = df_rreo_2.query('coluna == "DESPESAS LIQUIDADAS ATÉ O BIMESTRE (d)" & cod_conta == "RREO2TotalDespesas" & conta == "DESPESAS (INTRA-ORÇAMENTÁRIAS) (II)"')
liq_intra_rreo_2_2['dimensao'] = 'D4_00004_Liquidado_INTRA'
liq_intra_rreo_2_2 = liq_intra_rreo_2_2.filter(items=['dimensao', 'valor'])
liq_intra_rreo_2_2 = liq_intra_rreo_2_2.set_index("dimensao")

# Inscritas em RPNP INTRA
rpnp_intra_rreo_2_2 = df_rreo_2.query('coluna == "INSCRITAS EM RESTOS A PAGAR NÃO PROCESSADOS (f)" & cod_conta == "RREO2TotalDespesas" & conta == "DESPESAS (INTRA-ORÇAMENTÁRIAS) (II)"')
rpnp_intra_rreo_2_2['dimensao'] = 'D4_00004_Inscrição RPNP_INTRA'
rpnp_intra_rreo_2_2 = rpnp_intra_rreo_2_2.filter(items=['dimensao', 'valor'])
rpnp_intra_rreo_2_2 = rpnp_intra_rreo_2_2.set_index("dimensao")


################################# DCA E
#Empenhados INTRA
emp_intra_dca_2 = df_dca_e.query('conta == "Despesas Intraorçamentárias" and coluna == "Despesas Empenhadas"')
emp_intra_dca_2['dimensao'] = 'D4_00004_Empenhado_INTRA'
emp_intra_dca_2 = emp_intra_dca_2.groupby('dimensao').agg({'valor': 'sum'})

#Liquidados INTRA
liq_intra_dca_2 = df_dca_e.query('conta == "Despesas Intraorçamentárias" and coluna == "Despesas Liquidadas"')
liq_intra_dca_2['dimensao'] = 'D4_00004_Liquidado_INTRA'
liq_intra_dca_2 = liq_intra_dca_2.groupby('dimensao').agg({'valor': 'sum'})

# Inscritas em RPNP INTRA
rpnp_intra_dca_2 = df_dca_e.query('conta == "Despesas Intraorçamentárias" and coluna == "Inscrição de Restos a Pagar Não Processados"')
rpnp_intra_dca_2['dimensao'] = 'D4_00004_Inscrição RPNP_INTRA'
rpnp_intra_dca_2 = rpnp_intra_dca_2.groupby('dimensao').agg({'valor': 'sum'})

############################################ MERGE
###################  >> D4_00004_Empenhado INTRA
d4_00004_emp_intra = emp_intra_rreo_2_2.merge(emp_intra_dca_2, on='dimensao')
d4_00004_emp_intra['DIF'] = d4_00004_emp_intra['valor_x'] - d4_00004_emp_intra['valor_y']
d4_00004_emp_intra.columns = ['RREO 2', 'DCA E', 'DIF']

###################  >> D4_00004_Liquidado INTRA
d4_00004_liq_intra = liq_intra_rreo_2_2.merge(liq_intra_dca_2, on='dimensao')
d4_00004_liq_intra['DIF'] = d4_00004_liq_intra['valor_x'] - d4_00004_liq_intra['valor_y']
d4_00004_liq_intra.columns = ['RREO 2', 'DCA E', 'DIF']

###################  >> D4_00004_Inscrição de RPNP INTRA
d4_00004_rpnp_intra = rpnp_intra_rreo_2_2.merge(rpnp_intra_dca_2, on='dimensao')
d4_00004_rpnp_intra['DIF'] = d4_00004_rpnp_intra['valor_x'] - d4_00004_rpnp_intra['valor_y']
d4_00004_rpnp_intra.columns = ['RREO 2', 'DCA E', 'DIF']


# Juntando os DF
d4_00004_t = pd.concat([d4_00004_emp_intra, d4_00004_liq_intra, d4_00004_rpnp_intra])

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d4_00004_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d4_00004 = 'ERRO'
else:
    resposta_d4_00004 = 'OK'

d4_00004 = pd.DataFrame([resposta_d4_00004], columns=['Resposta'])
# Insere novas colunas
d4_00004.insert(0, 'Dimensão', 'D4_00004')
d4_00004.insert(2, 'Descrição da Dimensão', 'Igualdade da execução da despesa por função')
d4_00004.insert(3, 'OBS', 'Anexo I-E da DCA e o Anexo 02 do RREO 6ºB (intraorçamentária)')
d4_00004

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00004,OK,Igualdade da execução da despesa por função,Anexo I-E da DCA e o Anexo 02 do RREO 6ºB (intraorçamentária)


In [ ]:
d4_00004_t

,RREO 2,DCA E,DIF
dimensao,,,
D4_00004_Empenhado_INTRA,"8,018,541,679.58","8,018,541,679.58",0.00
D4_00004_Liquidado_INTRA,"7,994,680,383.29","7,994,680,383.29",0.00
D4_00004_Inscrição RPNP_INTRA,"23,861,296.29","23,861,296.29",0.00


In [ ]:
## D4_00005
########## RREO 7
# RPP em Exercícios Anteriores
rpp_exerc_ant_rreo_7 = df_rreo_7.query('conta == "TOTAL (III) = (I + II)" & cod_conta == "RestosAPagarProcessadosENaoProcessadosLiquidadosInscritosEmExerciciosAnteriores"')
rpp_exerc_ant_rreo_7['dimensao'] = 'D4_00005_RPP em Exercícios Anteriores'
rpp_exerc_ant_rreo_7 = rpp_exerc_ant_rreo_7.groupby('dimensao').agg({'valor': 'sum'})

# RPP em Dez do Ano Anterior
rpp_ano_ant_rreo_7 = df_rreo_7.query('conta == "TOTAL (III) = (I + II)" & cod_conta == "RestosAPagarProcessadosENaoProcessadosLiquidadosInscritosEmExercicioAnterior"')
rpp_ano_ant_rreo_7['dimensao'] = 'D4_00005_RPP em Dez do Ano Anterior'
rpp_ano_ant_rreo_7 = rpp_ano_ant_rreo_7.groupby('dimensao').agg({'valor': 'sum'})

# RPP Pagos
rpp_pagos_7 = df_rreo_7.query('conta == "TOTAL (III) = (I + II)" & cod_conta == "RestosAPagarProcessadosENaoProcessadosLiquidadosPagos"')
rpp_pagos_7['dimensao'] = 'D4_00005_RPP Pagos'
rpp_pagos_7 = rpp_pagos_7.groupby('dimensao').agg({'valor': 'sum'})

# RPP Cancelados
rpp_cancelados_7 = df_rreo_7.query('conta == "TOTAL (III) = (I + II)" & cod_conta == "RestosAPagarProcessadosENaoProcessadosLiquidadosCancelados"')
rpp_cancelados_7['dimensao'] = 'D4_00005_RPP Cancelados'
rpp_cancelados_7 = rpp_cancelados_7.groupby('dimensao').agg({'valor': 'sum'})

# RPNP em Exercícios Anteriores
rpnp_exerc_ant_rreo_7 = df_rreo_7.query('conta == "TOTAL (III) = (I + II)" & cod_conta == "RestosAPagarNaoProcessadosInscritosEmExerciciosAnteriores"')
rpnp_exerc_ant_rreo_7['dimensao'] = 'D4_00005_RPNP em Exercícios Anteriores'
rpnp_exerc_ant_rreo_7 = rpnp_exerc_ant_rreo_7.groupby('dimensao').agg({'valor': 'sum'})

# RPNP em Dez do Ano Anterior
rpnp_ano_ant_rreo_7 = df_rreo_7.query('conta == "TOTAL (III) = (I + II)" & cod_conta == "RestosAPagarNaoProcessadosInscritosEmExercicioAnterior"')
rpnp_ano_ant_rreo_7['dimensao'] = 'D4_00005_RPNP em Dez do Ano Anterior'
rpnp_ano_ant_rreo_7 = rpnp_ano_ant_rreo_7.groupby('dimensao').agg({'valor': 'sum'})

# RPNP Liquidados
rpnp_liquidados_7 = df_rreo_7.query('conta == "TOTAL (III) = (I + II)" & cod_conta == "RestosAPagarNaoProcessadosLiquidados"')
rpnp_liquidados_7['dimensao'] = 'D4_00005_RPNP Liquidados'
rpnp_liquidados_7 = rpnp_liquidados_7.groupby('dimensao').agg({'valor': 'sum'})

# RPNP Pagos
rpnp_pagos_7 = df_rreo_7.query('conta == "TOTAL (III) = (I + II)" & cod_conta == "RestosAPagarNaoProcessadosPagos"')
rpnp_pagos_7['dimensao'] = 'D4_00005_RPNP Pagos'
rpnp_pagos_7 = rpnp_pagos_7.groupby('dimensao').agg({'valor': 'sum'})

# RPNP Cancelados
rpnp_cancelados_7 = df_rreo_7.query('conta == "TOTAL (III) = (I + II)" & cod_conta == "RestosAPagarNaoProcessadosCancelados"')
rpnp_cancelados_7['dimensao'] = 'D4_00005_RPNP Cancelados'
rpnp_cancelados_7 = rpnp_cancelados_7.groupby('dimensao').agg({'valor': 'sum'})


########## DCA F
# RPP em Exercícios Anteriores
rpp_exerc_ant_dca = df_dca_f.query('conta == "Total Despesas" and coluna == "Restos a Pagar Processados Inscritos em Exercícios Anteriores"')
rpp_exerc_ant_dca['dimensao'] = 'D4_00005_RPP em Exercícios Anteriores'
rpp_exerc_ant_dca = rpp_exerc_ant_dca.groupby('dimensao').agg({'valor': 'sum'})

# RPP em Dez do Ano Anterior
rpp_ano_ant_dca = df_dca_f.query('conta == "Total Despesas" and coluna == "Restos a Pagar Processados Inscritos em 31 de Dezembro do Exercício Anterior"')
rpp_ano_ant_dca['dimensao'] = 'D4_00005_RPP em Dez do Ano Anterior'
rpp_ano_ant_dca = rpp_ano_ant_dca.groupby('dimensao').agg({'valor': 'sum'})

# RPP Pagos
rpp_pagos_dca = df_dca_f.query('conta == "Total Despesas" and coluna == "Restos a Pagar Processados Pagos"')
rpp_pagos_dca['dimensao'] = 'D4_00005_RPP Pagos'
rpp_pagos_dca = rpp_pagos_dca.groupby('dimensao').agg({'valor': 'sum'})

# RPP Cancelados
rpp_cancelados_dca = df_dca_f.query('conta == "Total Despesas" and coluna == "Restos a Pagar Processados Cancelados"')
rpp_cancelados_dca['dimensao'] = 'D4_00005_RPP Cancelados'
rpp_cancelados_dca = rpp_cancelados_dca.groupby('dimensao').agg({'valor': 'sum'})

# RPNP em Exercícios Anteriores
rpnp_exerc_ant_dca = df_dca_f.query('conta == "Total Despesas" and coluna == "Restos a Pagar Não Processados Inscritos em Exercícios Anteriores"')
rpnp_exerc_ant_dca['dimensao'] = 'D4_00005_RPNP em Exercícios Anteriores'
rpnp_exerc_ant_dca = rpnp_exerc_ant_dca.groupby('dimensao').agg({'valor': 'sum'})

# RPNP em Dez do Ano Anterior
rpnp_ano_ant_dca = df_dca_f.query('conta == "Total Despesas" and coluna == "Restos a Pagar Não Processados Inscritos em 31 de Dezembro do Exercício Anterior"')
rpnp_ano_ant_dca['dimensao'] = 'D4_00005_RPNP em Dez do Ano Anterior'
rpnp_ano_ant_dca = rpnp_ano_ant_dca.groupby('dimensao').agg({'valor': 'sum'})

# RPNP Liquidados
rpnp_liquidados_dca = df_dca_f.query('conta == "Total Despesas" and coluna == "Restos a Pagar Não Processados Liquidados"')
rpnp_liquidados_dca['dimensao'] = 'D4_00005_RPNP Liquidados'
rpnp_liquidados_dca = rpnp_liquidados_dca.groupby('dimensao').agg({'valor': 'sum'})

# RPNP Pagos
rpnp_pagos_dca = df_dca_f.query('conta == "Total Despesas" and coluna == "Restos a Pagar Não Processados Pagos"')
rpnp_pagos_dca['dimensao'] = 'D4_00005_RPNP Pagos'
rpnp_pagos_dca = rpnp_pagos_dca.groupby('dimensao').agg({'valor': 'sum'})

# RPNP Cancelados
rpnp_cancelados_dca = df_dca_f.query('conta == "Total Despesas" and coluna == "Restos a Pagar Não Processados Cancelados"')
rpnp_cancelados_dca['dimensao'] = 'D4_00005_RPNP Cancelados'
rpnp_cancelados_dca = rpnp_cancelados_dca.groupby('dimensao').agg({'valor': 'sum'})


######################################## Juntando 

###################  >> D4_00005_RPP em Exercícios Anteriores
d4_00005_rpp_exerc_ant = rpp_exerc_ant_rreo_7.merge(rpp_exerc_ant_dca, on='dimensao')
d4_00005_rpp_exerc_ant['DIF'] = d4_00005_rpp_exerc_ant['valor_x'] - d4_00005_rpp_exerc_ant['valor_y']
d4_00005_rpp_exerc_ant.columns = ['RREO 7', 'DCA F', 'DIF']

###################  >> D4_00005_RPP em Dez do Ano Anterior
d4_00005_rpp_ano_ant = rpp_ano_ant_rreo_7.merge(rpp_ano_ant_dca, on='dimensao')
d4_00005_rpp_ano_ant['DIF'] = d4_00005_rpp_ano_ant['valor_x'] - d4_00005_rpp_ano_ant['valor_y']
d4_00005_rpp_ano_ant.columns = ['RREO 7', 'DCA F', 'DIF']

###################  >> D4_00005_RPP Pagos
d4_00005_rpp_pagos = rpp_pagos_7.merge(rpp_pagos_dca, on='dimensao')
d4_00005_rpp_pagos['DIF'] = d4_00005_rpp_pagos['valor_x'] - d4_00005_rpp_pagos['valor_y']
d4_00005_rpp_pagos.columns = ['RREO 7', 'DCA F', 'DIF']

###################  >> D4_00005_RPP Cancelados
d4_00005_rpp_cancelados = rpp_cancelados_7.merge(rpp_cancelados_dca, on='dimensao')
d4_00005_rpp_cancelados['DIF'] = d4_00005_rpp_cancelados['valor_x'] - d4_00005_rpp_cancelados['valor_y']
d4_00005_rpp_cancelados.columns = ['RREO 7', 'DCA F', 'DIF']

###################  >> D4_00005_RPNP em Exercícios Anteriores
d4_00005_rpnp_exerc_ant = rpnp_exerc_ant_rreo_7.merge(rpnp_exerc_ant_dca, on='dimensao')
d4_00005_rpnp_exerc_ant['DIF'] = d4_00005_rpnp_exerc_ant['valor_x'] - d4_00005_rpnp_exerc_ant['valor_y']
d4_00005_rpnp_exerc_ant.columns = ['RREO 7', 'DCA F', 'DIF']

###################  >> D4_00005_RPNP em Dez do Ano Anterior
d4_00005_rpnp_ano_ant = rpnp_ano_ant_rreo_7.merge(rpnp_ano_ant_dca, on='dimensao')
d4_00005_rpnp_ano_ant['DIF'] = d4_00005_rpnp_ano_ant['valor_x'] - d4_00005_rpnp_ano_ant['valor_y']
d4_00005_rpnp_ano_ant.columns = ['RREO 7', 'DCA F', 'DIF']

###################  >> D4_00005_RPNP Liquidados
d4_00005_rpnp_liquidados = rpnp_liquidados_7.merge(rpnp_liquidados_dca, on='dimensao')
d4_00005_rpnp_liquidados['DIF'] = d4_00005_rpnp_liquidados['valor_x'] - d4_00005_rpnp_liquidados['valor_y']
d4_00005_rpnp_liquidados.columns = ['RREO 7', 'DCA F', 'DIF']

###################  >> D4_00005_RPNP Pagos
d4_00005_rpnp_pagos = rpnp_pagos_7.merge(rpnp_pagos_dca, on='dimensao')
d4_00005_rpnp_pagos['DIF'] = d4_00005_rpnp_pagos['valor_x'] - d4_00005_rpnp_pagos['valor_y']
d4_00005_rpnp_pagos.columns = ['RREO 7', 'DCA F', 'DIF']

###################  >> D4_00005_RPNP Cancelados
d4_00005_rpnp_cancelados = rpnp_cancelados_7.merge(rpnp_cancelados_dca, on='dimensao')
d4_00005_rpnp_cancelados['DIF'] = d4_00005_rpnp_cancelados['valor_x'] - d4_00005_rpnp_cancelados['valor_y']
d4_00005_rpnp_cancelados.columns = ['RREO 7', 'DCA F', 'DIF']


# Juntando os DF
d4_00005_t = pd.concat([d4_00005_rpp_exerc_ant, d4_00005_rpp_ano_ant, d4_00005_rpp_pagos, d4_00005_rpp_cancelados, d4_00005_rpnp_exerc_ant, d4_00005_rpnp_ano_ant, d4_00005_rpnp_liquidados, d4_00005_rpnp_pagos, d4_00005_rpnp_cancelados], axis=0)
d4_00005_t = d4_00005_t.reset_index()

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d4_00005_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d4_00005 = 'ERRO'
else:
    resposta_d4_00005 = 'OK'

d4_00005 = pd.DataFrame([resposta_d4_00005], columns=['Resposta'])
# Insere novas colunas
d4_00005.insert(0, 'Dimensão', 'D4_00005')
d4_00005.insert(2, 'Descrição da Dimensão', 'Igualdade dos restos a pagar não processados e processados')
d4_00005.insert(3, 'OBS', 'Anexo I-F da DCA e o Anexo 07 do RREO')
d4_00005

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00005,OK,Igualdade dos restos a pagar não processados e processados,Anexo I-F da DCA e o Anexo 07 do RREO


In [ ]:
d4_00005_t

,dimensao,RREO 7,DCA F,DIF
0,D4_00005_RPP em Exercícios Anteriores,"855,747,698.59","855,747,698.59",0.00
1,D4_00005_RPP em Dez do Ano Anterior,"1,453,140,543.32","1,453,140,543.32",0.00
2,D4_00005_RPP Pagos,"1,492,300,289.28","1,492,300,289.28",0.00
3,D4_00005_RPP Cancelados,"173,056,595.34","173,056,595.34",0.00
4,D4_00005_RPNP em Exercícios Anteriores,"5,855,558.30","5,855,558.30",0.00
5,D4_00005_RPNP em Dez do Ano Anterior,"1,619,724,634.61","1,619,724,634.61",0.00
6,D4_00005_RPNP Liquidados,"660,991,987.74","660,991,987.74",0.00
7,D4_00005_RPNP Pagos,"660,505,752.08","660,505,752.08",0.00
8,D4_00005_RPNP Cancelados,"957,836,425.06","957,836,425.06",0.00


In [ ]:
## D4_00006
########## RREO 7
# RPNP em Exercícios Anteriores
rpnp_exerc_ant_rreo_7 = df_rreo_7.query('conta == "TOTAL (III) = (I + II)" & cod_conta == "RestosAPagarNaoProcessadosInscritosEmExerciciosAnteriores"')
rpnp_exerc_ant_rreo_7['dimensao'] = 'D4_00006_RPNP em Exercícios Anteriores'
rpnp_exerc_ant_rreo_7 = rpnp_exerc_ant_rreo_7.groupby('dimensao').agg({'valor': 'sum'})

# RPNP em Dez do Ano Anterior
rpnp_ano_ant_rreo_7 = df_rreo_7.query('conta == "TOTAL (III) = (I + II)" & cod_conta == "RestosAPagarNaoProcessadosInscritosEmExercicioAnterior"')
rpnp_ano_ant_rreo_7['dimensao'] = 'D4_00006_RPNP em Dez do Ano Anterior'
rpnp_ano_ant_rreo_7 = rpnp_ano_ant_rreo_7.groupby('dimensao').agg({'valor': 'sum'})

# RPNP Pagos
rpnp_pagos_7 = df_rreo_7.query('conta == "TOTAL (III) = (I + II)" & cod_conta == "RestosAPagarNaoProcessadosPagos"')
rpnp_pagos_7['dimensao'] = 'D4_00006_RPNP Pagos'
rpnp_pagos_7 = rpnp_pagos_7.groupby('dimensao').agg({'valor': 'sum'})

# RPNP Cancelados
rpnp_cancelados_7 = df_rreo_7.query('conta == "TOTAL (III) = (I + II)" & cod_conta == "RestosAPagarNaoProcessadosCancelados"')
rpnp_cancelados_7['dimensao'] = 'D4_00006_RPNP Cancelados'
rpnp_cancelados_7 = rpnp_cancelados_7.groupby('dimensao').agg({'valor': 'sum'})

########## DCA G
# RPNP em Exercícios Anteriores
rpnp_exerc_ant_dca = df_dca_g.query('(conta == "Despesas Exceto Intraorçamentárias" or conta == "Despesas Intraorçamentárias") and coluna == "Restos a Pagar Não Processados Inscritos em Exercícios Anteriores"')
rpnp_exerc_ant_dca['dimensao'] = 'D4_00006_RPNP em Exercícios Anteriores'
rpnp_exerc_ant_dca = rpnp_exerc_ant_dca.groupby('dimensao').agg({'valor': 'sum'})

# RPNP em Dez do Ano Anterior
rpnp_ano_ant_dca = df_dca_g.query('(conta == "Despesas Exceto Intraorçamentárias" or conta == "Despesas Intraorçamentárias") and coluna == "Restos a Pagar Não Processados Inscritos em 31 de Dezembro do Exercício Anterior"')
rpnp_ano_ant_dca['dimensao'] = 'D4_00006_RPNP em Dez do Ano Anterior'
rpnp_ano_ant_dca = rpnp_ano_ant_dca.groupby('dimensao').agg({'valor': 'sum'})

# RPNP Pagos
rpnp_pagos_dca = df_dca_g.query('(conta == "Despesas Exceto Intraorçamentárias" or conta == "Despesas Intraorçamentárias") and coluna == "Restos a Pagar Não Processados Pagos"')
rpnp_pagos_dca['dimensao'] = 'D4_00006_RPNP Pagos'
rpnp_pagos_dca = rpnp_pagos_dca.groupby('dimensao').agg({'valor': 'sum'})

# RPNP Cancelados
rpnp_cancelados_dca = df_dca_g.query('(conta == "Despesas Exceto Intraorçamentárias" or conta == "Despesas Intraorçamentárias") and coluna == "Restos a Pagar Não Processados Cancelados"')
rpnp_cancelados_dca['dimensao'] = 'D4_00006_RPNP Cancelados'
rpnp_cancelados_dca = rpnp_cancelados_dca.groupby('dimensao').agg({'valor': 'sum'})


######################################## Juntando 

###################  >> D4_00005_RPNP em Exercícios Anteriores
d4_00006_rpnp_exerc_ant = rpnp_exerc_ant_rreo_7.merge(rpnp_exerc_ant_dca, on='dimensao')
d4_00006_rpnp_exerc_ant['DIF'] = d4_00006_rpnp_exerc_ant['valor_x'] - d4_00006_rpnp_exerc_ant['valor_y']
d4_00006_rpnp_exerc_ant.columns = ['RREO 7', 'DCA G', 'DIF']

###################  >> D4_00005_RPNP em Dez do Ano Anterior
d4_00006_rpnp_ano_ant = rpnp_ano_ant_rreo_7.merge(rpnp_ano_ant_dca, on='dimensao')
d4_00006_rpnp_ano_ant['DIF'] = d4_00006_rpnp_ano_ant['valor_x'] - d4_00006_rpnp_ano_ant['valor_y']
d4_00006_rpnp_ano_ant.columns = ['RREO 7', 'DCA G', 'DIF']

###################  >> D4_00005_RPNP Pagos
d4_00006_rpnp_pagos = rpnp_pagos_7.merge(rpnp_pagos_dca, on='dimensao')
d4_00006_rpnp_pagos['DIF'] = d4_00006_rpnp_pagos['valor_x'] - d4_00006_rpnp_pagos['valor_y']
d4_00006_rpnp_pagos.columns = ['RREO 7', 'DCA G', 'DIF']

###################  >> D4_00005_RPNP Cancelados
d4_00006_rpnp_cancelados = rpnp_cancelados_7.merge(rpnp_cancelados_dca, on='dimensao')
d4_00006_rpnp_cancelados['DIF'] = d4_00006_rpnp_cancelados['valor_x'] - d4_00006_rpnp_cancelados['valor_y']
d4_00006_rpnp_cancelados.columns = ['RREO 7', 'DCA G', 'DIF']


# Juntando os DF
d4_00006_t = pd.concat([d4_00006_rpnp_exerc_ant, d4_00006_rpnp_ano_ant, d4_00006_rpnp_pagos, d4_00006_rpnp_cancelados], axis=0)
d4_00006_t = d4_00006_t.reset_index(drop=True)

tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d4_00006_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d4_00006 = 'ERRO'
else:
    resposta_d4_00006 = 'OK'

d4_00006 = pd.DataFrame([resposta_d4_00006], columns=['Resposta'])
# Insere novas colunas
d4_00006.insert(0, 'Dimensão', 'D4_00006')
d4_00006.insert(2, 'Descrição da Dimensão', 'Igualdade dos restos a pagar não processados')
d4_00006.insert(3, 'OBS', 'Anexo I-G da DCA e o Anexo 07 do RREO')
d4_00006

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00006,OK,Igualdade dos restos a pagar não processados,Anexo I-G da DCA e o Anexo 07 do RREO


In [ ]:
d4_00006_t

,RREO 7,DCA G,DIF
0,"5,855,558.30","5,855,558.30",0.00
1,"1,619,724,634.61","1,619,724,634.61",-0.00
2,"660,505,752.08","660,505,752.08",0.00
3,"957,836,425.06","957,836,425.06",-0.00


In [ ]:
## D4_00007
########## RREO 7
# RPP em Exercícios Anteriores
rpp_exerc_ant_rreo_7 = df_rreo_7.query('conta == "TOTAL (III) = (I + II)" & cod_conta == "RestosAPagarProcessadosENaoProcessadosLiquidadosInscritosEmExerciciosAnteriores"')
rpp_exerc_ant_rreo_7['dimensao'] = 'D4_00005_RPP em Exercícios Anteriores'
rpp_exerc_ant_rreo_7 = rpp_exerc_ant_rreo_7.groupby('dimensao').agg({'valor': 'sum'})

# RPP em Dez do Ano Anterior
rpp_ano_ant_rreo_7 = df_rreo_7.query('conta == "TOTAL (III) = (I + II)" & cod_conta == "RestosAPagarProcessadosENaoProcessadosLiquidadosInscritosEmExercicioAnterior"')
rpp_ano_ant_rreo_7['dimensao'] = 'D4_00005_RPP em Dez do Ano Anterior'
rpp_ano_ant_rreo_7 = rpp_ano_ant_rreo_7.groupby('dimensao').agg({'valor': 'sum'})

# RPP Pagos
rpp_pagos_7 = df_rreo_7.query('conta == "TOTAL (III) = (I + II)" & cod_conta == "RestosAPagarProcessadosENaoProcessadosLiquidadosPagos"')
rpp_pagos_7['dimensao'] = 'D4_00005_RPP Pagos'
rpp_pagos_7 = rpp_pagos_7.groupby('dimensao').agg({'valor': 'sum'})

# RPP Cancelados
rpp_cancelados_7 = df_rreo_7.query('conta == "TOTAL (III) = (I + II)" & cod_conta == "RestosAPagarProcessadosENaoProcessadosLiquidadosCancelados"')
rpp_cancelados_7['dimensao'] = 'D4_00005_RPP Cancelados'
rpp_cancelados_7 = rpp_cancelados_7.groupby('dimensao').agg({'valor': 'sum'})

########## DCA G
# RPP em Exercícios Anteriores
rpp_exerc_ant_dca = df_dca_g.query('(conta == "Despesas Exceto Intraorçamentárias" or conta == "Despesas Intraorçamentárias") and coluna == "Restos a Pagar Processados Inscritos em Exercícios Anteriores"')
rpp_exerc_ant_dca['dimensao'] = 'D4_00005_RPP em Exercícios Anteriores'
rpp_exerc_ant_dca = rpp_exerc_ant_dca.groupby('dimensao').agg({'valor': 'sum'})

# RPP em Dez do Ano Anterior
rpp_ano_ant_dca = df_dca_g.query('(conta == "Despesas Exceto Intraorçamentárias" or conta == "Despesas Intraorçamentárias") and coluna == "Restos a Pagar Processados Inscritos em 31 de Dezembro do Exercício Anterior"')
rpp_ano_ant_dca['dimensao'] = 'D4_00005_RPP em Dez do Ano Anterior'
rpp_ano_ant_dca = rpp_ano_ant_dca.groupby('dimensao').agg({'valor': 'sum'})

# RPP Pagos
rpp_pagos_dca = df_dca_g.query('(conta == "Despesas Exceto Intraorçamentárias" or conta == "Despesas Intraorçamentárias") and coluna == "Restos a Pagar Processados Pagos"')
rpp_pagos_dca['dimensao'] = 'D4_00005_RPP Pagos'
rpp_pagos_dca = rpp_pagos_dca.groupby('dimensao').agg({'valor': 'sum'})

# RPP Cancelados
rpp_cancelados_dca = df_dca_g.query('(conta == "Despesas Exceto Intraorçamentárias" or conta == "Despesas Intraorçamentárias") and coluna == "Restos a Pagar Processados Cancelados"')
rpp_cancelados_dca['dimensao'] = 'D4_00005_RPP Cancelados'
rpp_cancelados_dca = rpp_cancelados_dca.groupby('dimensao').agg({'valor': 'sum'})


######################################## Juntando 

###################  >> D4_00005_RPP em Exercícios Anteriores
d4_00007_rpp_exerc_ant = rpp_exerc_ant_rreo_7.merge(rpp_exerc_ant_dca, on='dimensao')
d4_00007_rpp_exerc_ant['DIF'] = d4_00007_rpp_exerc_ant['valor_x'] - d4_00007_rpp_exerc_ant['valor_y']
d4_00007_rpp_exerc_ant.columns = ['RREO 7', 'DCA F', 'DIF']

###################  >> D4_00005_RPP em Dez do Ano Anterior
d4_00007_rpp_ano_ant = rpp_ano_ant_rreo_7.merge(rpp_ano_ant_dca, on='dimensao')
d4_00007_rpp_ano_ant['DIF'] = d4_00007_rpp_ano_ant['valor_x'] - d4_00007_rpp_ano_ant['valor_y']
d4_00007_rpp_ano_ant.columns = ['RREO 7', 'DCA F', 'DIF']

###################  >> D4_00005_RPP Pagos
d4_00007_rpp_pagos = rpp_pagos_7.merge(rpp_pagos_dca, on='dimensao')
d4_00007_rpp_pagos['DIF'] = d4_00007_rpp_pagos['valor_x'] - d4_00007_rpp_pagos['valor_y']
d4_00007_rpp_pagos.columns = ['RREO 7', 'DCA F', 'DIF']

###################  >> D4_00005_RPP Cancelados
d4_00007_rpp_cancelados = rpp_cancelados_7.merge(rpp_cancelados_dca, on='dimensao')
d4_00007_rpp_cancelados['DIF'] = d4_00007_rpp_cancelados['valor_x'] - d4_00007_rpp_cancelados['valor_y']
d4_00007_rpp_cancelados.columns = ['RREO 7', 'DCA F', 'DIF']


# Juntando os DF
d4_00007_t = pd.concat([d4_00007_rpp_exerc_ant, d4_00007_rpp_ano_ant, d4_00007_rpp_pagos, d4_00007_rpp_cancelados], axis=0)
d4_00007_t = d4_00007_t.reset_index(drop=True)

tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d4_00007_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d4_00007 = 'ERRO'
else:
    resposta_d4_00007 = 'OK'

d4_00007 = pd.DataFrame([resposta_d4_00007], columns=['Resposta'])
# Insere novas colunas
d4_00007.insert(0, 'Dimensão', 'D4_00007')
d4_00007.insert(2, 'Descrição da Dimensão', 'Igualdade dos restos a pagar processados')
d4_00007.insert(3, 'OBS', 'Anexo I-G da DCA e o Anexo 07 do RREO')

d4_00007

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00007,OK,Igualdade dos restos a pagar processados,Anexo I-G da DCA e o Anexo 07 do RREO


In [ ]:
d4_00007_t

,RREO 7,DCA F,DIF
0,"855,747,698.59","855,747,698.59",0.00
1,"1,453,140,543.32","1,453,140,543.32",0.00
2,"1,492,300,289.28","1,492,300,289.28",0.00
3,"173,056,595.34","173,056,595.34",0.00


In [ ]:
## D4_00009
###################  >> D4_00009_Receita de Impostos RREO 2
# (apresenta uma diferença que está lá na DCA ref. a Outras Deduções da Receita)
icms_rreo_2 = df_rreo_3[df_rreo_3["conta"].str.contains("ICMS")]
ipva_rreo_2 = df_rreo_3[df_rreo_3["conta"].str.contains("IPVA")]
itcd_rreo_2 = df_rreo_3[df_rreo_3["conta"].str.contains("ITCD")]
irrf_rreo_2 = df_rreo_3[df_rreo_3["conta"].str.contains("IRRF")]
imposto_rreo_2 = pd.concat([icms_rreo_2, ipva_rreo_2, itcd_rreo_2, irrf_rreo_2])
imposto_rreo_2 = imposto_rreo_2.query('coluna == "TOTAL (ÚLTIMOS 12 MESES)"')
imposto_rreo_2['dimensao'] = 'D4_00009_Rec.Impostos'
imposto_rreo_2 = imposto_rreo_2.groupby('dimensao').agg({'valor': 'sum'})
imposto_rreo_2 = imposto_rreo_2.reset_index()

###################  >> D4_00009_Receita de Impostos DCA
# tem que colocar o valor de ´´Outras Deduções da Receita``)
imposto_dca = df_dca_c.query('(cod_conta == "RO1.1.1.0.00.0.0" and coluna == "Receitas Brutas Realizadas") or (cod_conta == "RO1.1.1.0.00.0.0" and coluna == "Outras Deduções da Receita")')
imposto_dca['dimensao'] = 'D4_00009_Rec.Impostos'
imposto_dca = imposto_dca.groupby('dimensao').agg({'valor': 'sum'})
imposto_dca = imposto_dca.reset_index()

## MERGE Receita de Impostos
d4_00009 = imposto_rreo_2.merge(imposto_dca, on='dimensao')
d4_00009['DIF'] = d4_00009['valor_x'] - d4_00009['valor_y']
d4_00009_t = d4_00009.filter(items=['dimensao', 'valor_x', 'valor_y', 'DIF'])
d4_00009_t.columns = ['Dimensão', 'RREO', 'DCA', 'DIF']

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d4_00009_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d4_00009 = 'ERRO'
else:
    resposta_d4_00009 = 'OK'

d4_00009 = pd.DataFrame([resposta_d4_00009], columns=['Resposta'])
# Insere novas colunas
d4_00009.insert(0, 'Dimensão', 'D4_00009')
d4_00009.insert(2, 'Descrição da Dimensão', 'Igualdade das receitas com tributos estaduais')
d4_00009.insert(3, 'OBS', 'Anexo I-C da DCA e o Anexo 11 do RREO')
d4_00009

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00009,OK,Igualdade das receitas com tributos estaduais,Anexo I-C da DCA e o Anexo 11 do RREO


In [ ]:
d4_00009_t

,Dimensão,RREO,DCA,DIF
0,D4_00009_Rec.Impostos,"71,276,657,196.75","71,276,657,196.75",0.00


In [ ]:
## D4_00011
###################  >> D4_00011_Transferências Constitucionais RREO 2
fpe_rreo_2 = df_rreo_3[df_rreo_3["conta"].str.contains("Cota-Parte do FPE")]
fundeb_rreo_2 = df_rreo_3[df_rreo_3["conta"].str.contains("Transferências do FUNDEB")]
transf_rreo_2 = pd.concat([fpe_rreo_2, fundeb_rreo_2])
transf_rreo_2 = transf_rreo_2.query('coluna == "TOTAL (ÚLTIMOS 12 MESES)"')
transf_rreo_2['dimensao'] = 'D4_00011_Transf.Const'
transf_rreo_2 = transf_rreo_2.groupby('dimensao').agg({'valor': 'sum'})
transf_rreo_2 = transf_rreo_2.reset_index()

###################  >> D4_00011_Transferências Constitucionais DCA (#@ MUDAR quando for 2021 as NRs)
# Cota-Parte do FPE          >> NR: 1.7.1.1.50.0.0 + NR: 7.7.1.1.50.0.0
# Transferências do FUNDEB   >> NR: 1.7.1.5.00.0.0 + NR: 1.7.5.1.00.0.0 + NR: 7.7.1.5.00.0.0 + NR: 7.7.5.1.00.0.0

transf_dca = df_dca_c.query('(cod_conta == "RO1.7.1.1.50.0.0" or cod_conta == "RO1.7.5.1.00.0.0" or cod_conta == "RO1.7.1.5.00.0.0") and (coluna == "Receitas Brutas Realizadas")')
transf_dca['dimensao'] = 'D4_00011_Transf.Const'
transf_dca = transf_dca.groupby('dimensao').agg({'valor': 'sum'})
transf_dca = transf_dca.reset_index()

## MERGE Transferências Constitucionais
d4_00011 = transf_rreo_2.merge(transf_dca, on='dimensao')
d4_00011['DIF'] = d4_00011['valor_x'] - d4_00011['valor_y']
d4_00011_t = d4_00011.filter(items=['dimensao', 'valor_x', 'valor_y', 'DIF'])
d4_00011_t.columns = ['Dimensão', 'RREO', 'DCA', 'DIF']

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d4_00011_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d4_00011 = 'ERRO'
else:
    resposta_d4_00011 = 'OK'

d4_00011 = pd.DataFrame([resposta_d4_00011], columns=['Resposta'])
# Insere novas colunas
d4_00011.insert(0, 'Dimensão', 'D4_00011')
d4_00011.insert(2, 'Descrição da Dimensão', 'Igualdade das transferências estaduais')
d4_00011.insert(3, 'OBS', 'Anexo I-C da DCA e o Anexo 03 do RREO')
d4_00011

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00011,OK,Igualdade das transferências estaduais,Anexo I-C da DCA e o Anexo 03 do RREO


In [ ]:
d4_00011_t

,Dimensão,RREO,DCA,DIF
0,D4_00011_Transf.Const,"7,678,621,514.36","7,678,621,514.36",0.00


In [ ]:
## D4_00017
############################## RREO 3
# Contribuições dos servidores
contrib_serv_rreo_3 = df_rreo_3.query('coluna == "TOTAL (ÚLTIMOS 12 MESES)" & cod_conta == "ContribuicaoDoServidorParaOPlanoDePrevidencia"')
contrib_serv_rreo_3['dimensao'] = 'D4_00017_Contribuições dos Servidores'
contrib_serv_rreo_3 = contrib_serv_rreo_3.filter(items=['dimensao', 'valor'])

# Compensações Financeiras
comp_fin_rreo_3 = df_rreo_3.query('coluna == "TOTAL (ÚLTIMOS 12 MESES)" & cod_conta == "CompensacaoFinanceiraEntreRegimesPrevidencia"')
comp_fin_rreo_3['dimensao'] = 'D4_00017_Compensações Financeiras'
comp_fin_rreo_3 = comp_fin_rreo_3.filter(items=['dimensao', 'valor'])

############################## DCA C
# Contribuições dos servidores
contrib_serv_dca_c = df_dca_c.query('cod_conta == "RO1.2.1.5.00.0.0"')
contrib_serv_dca_c['dimensao'] = 'D4_00017_Contribuições dos Servidores'
contrib_serv_dca_c = contrib_serv_dca_c.filter(items=['dimensao', 'valor'])
contrib_serv_dca_c = contrib_serv_dca_c.groupby('dimensao').agg({'valor': 'sum'})

# Compensações Financeiras
comp_fin_dca_c = df_dca_c.query('cod_conta == "RO1.9.9.9.03.0.0"')
comp_fin_dca_c['dimensao'] = 'D4_00017_Compensações Financeiras'
comp_fin_dca_c = comp_fin_dca_c.filter(items=['dimensao', 'valor'])

#############################Criando o Merge entre os DF >> RREO 6 X DCA 
# Contribuições dos servidores
d4_00017_contrib_serv = contrib_serv_rreo_3.merge(contrib_serv_dca_c, on='dimensao')
d4_00017_contrib_serv['DIF'] = d4_00017_contrib_serv['valor_x'] - d4_00017_contrib_serv['valor_y']
d4_00017_contrib_serv.columns = ['Dimensão', 'RREO', 'DCA', 'DIF']

# Compensações Financeiras
d4_00017_comp_fin = comp_fin_rreo_3.merge(comp_fin_dca_c, on='dimensao')
d4_00017_comp_fin['DIF'] = d4_00017_comp_fin['valor_x'] - d4_00017_comp_fin['valor_y']
d4_00017_comp_fin.columns = ['Dimensão', 'RREO', 'DCA', 'DIF']

d4_00017_t = pd.concat([d4_00017_contrib_serv, d4_00017_comp_fin])

#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
condicao = d4_00017_t['DIF'] == 0

if condicao.any():
    resposta_d4_00017 = 'OK'
else:
    resposta_d4_00017 = 'ERRO'

d4_00017 = pd.DataFrame([resposta_d4_00017], columns=['Resposta'])
# Insere novas colunas
d4_00017.insert(0, 'Dimensão', 'D4_00017')
d4_00017.insert(2, 'Descrição da Dimensão', 'Igualdade das contrib. dos servidores e compensações financeiras')
d4_00017.insert(3, 'OBS', 'Anexo I-C da DCA e o Anexo 03 do RREO')
d4_00017

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00017,OK,Igualdade das contrib. dos servidores e compensações financeiras,Anexo I-C da DCA e o Anexo 03 do RREO


In [ ]:
d4_00017_t

,Dimensão,RREO,DCA,DIF
0,D4_00017_Contribuições dos Servidores,"4,145,305,275.37","4,145,305,275.37",0.00
0,D4_00017_Compensações Financeiras,"227,675,279.81","227,675,279.81",0.00


In [ ]:
## D4_00019
############################## RREO 9
# valor bruto das despesas de capital
df_rreo_9_dps_kap_bruto = df_rreo_9.query('coluna == "DESPESAS EMPENHADAS (e)" & cod_conta == "RREO9DespesasDeCapital"')
df_rreo_9_dps_kap_bruto['dimensao'] = 'D4_00019_valor bruto das despesas de capital'
df_rreo_9_dps_kap_bruto = df_rreo_9_dps_kap_bruto.filter(items=['dimensao', 'valor'])

# valor bruto das despesas de capital
df_dca_d_dps_kap_bruto = df_dca_d.query('coluna == "Despesas Empenhadas" and cod_conta == "DO4.0.00.00.00.00"')
df_dca_d_dps_kap_bruto['dimensao'] = 'D4_00019_valor bruto das despesas de capital'
df_dca_d_dps_kap_bruto = df_dca_d_dps_kap_bruto.filter(items=['dimensao', 'valor'])
df_dca_d_dps_kap_bruto = df_dca_d_dps_kap_bruto.groupby('dimensao').agg({'valor': 'sum'})

#############################Criando o Merge entre os DF >> RREO 6 X DCA 
d4_00019_t = df_rreo_9_dps_kap_bruto.merge(df_dca_d_dps_kap_bruto, on='dimensao')
d4_00019_t['DIF'] = d4_00019_t['valor_x'] - d4_00019_t['valor_y']
d4_00019_t.columns = ['Dimensão', 'RREO', 'DCA', 'DIF']

#FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
condicao = d4_00019_t['DIF'] == 0

if condicao.any():
    resposta_d4_00019 = 'OK'
else:
    resposta_d4_00019 = 'ERRO'

# Insere novas colunas
d4_00019 = pd.DataFrame([resposta_d4_00019], columns=['Resposta'])
d4_00019.insert(0, 'Dimensão', 'D4_00019')
d4_00019.insert(2, 'Descrição da Dimensão', 'Igualdade do valor das despesas de capital')
d4_00019.insert(3, 'OBS', 'Anexo I-D da DCA e o Anexo 09 do RREO')
d4_00019

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00019,OK,Igualdade do valor das despesas de capital,Anexo I-D da DCA e o Anexo 09 do RREO


In [ ]:
d4_00019_t

,Dimensão,RREO,DCA,DIF
0,D4_00019_valor bruto das despesas de capital,"7,333,917,401.72","7,333,917,401.72",0.00


In [ ]:
## D4_00020
###################  >> D4_00020_Receita Realizada MSC
rec_total = msc_dez.query('tipo_valor == "ending_balance" and (conta_contabil == "621200000" or conta_contabil == "621310100" or conta_contabil == "621310200" or conta_contabil == "621390000")')
rec_total['dimensao'] = 'D4_00020_Rec.Realizada'
rec_msc = rec_total.groupby('dimensao').agg({'valor': 'sum'})
rec_msc = rec_msc.reset_index()

###################  >> D4_00020_Receita Realizada RREO
rec_rreo = df_rreo_1.query('coluna == "Até o Bimestre (c)" & cod_conta == "TotalReceitas"')
rec_rreo['dimensao'] = 'D4_00020_Rec.Realizada'
rec_rreo = rec_rreo.filter(items=['dimensao', 'valor'])

## MERGE Receita Realizada
d4_00020 = rec_msc.merge(rec_rreo, on='dimensao')
d4_00020['DIF'] = d4_00020['valor_x'] - d4_00020['valor_y']
d4_00020_t = d4_00020.filter(items=['dimensao', 'valor_x', 'valor_y', 'DIF'])
d4_00020_t.columns = ['Dimensão', 'MSC', 'RREO', 'DIF']

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d4_00020_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d4_00020 = 'ERRO'
else:
    resposta_d4_00020 = 'OK'


d4_00020 = pd.DataFrame([resposta_d4_00020], columns=['Resposta'])
# Insere novas colunas
d4_00020.insert(0, 'Dimensão', 'D4_00020')
d4_00020.insert(2, 'Descrição da Dimensão', 'Igualdade nas receitas arrecadadas')
d4_00020.insert(3, 'OBS', 'MSC de Dezembro e no Anexo 01 do RREO')
d4_00020

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00020,OK,Igualdade nas receitas arrecadadas,MSC de Dezembro e no Anexo 01 do RREO


In [ ]:
d4_00020_t

,Dimensão,MSC,RREO,DIF
0,D4_00020_Rec.Realizada,"106,112,454,653.40","106,112,454,653.40",0.00


In [ ]:
## D4_00021
###################  >> D4_00021_Receita de Impostos MSC (pelo Bruto...sem Deduções)
rec_total = msc_dez.query('tipo_valor == "ending_balance" and (conta_contabil == "621200000" or conta_contabil == "621390000")')
imposto_msc_1 = rec_total[rec_total['natureza_receita'].str.contains("11125", case=False, regex=True)]
imposto_msc_2 = rec_total[rec_total['natureza_receita'].str.contains("11130", case=False, regex=True)]
imposto_msc_3 = rec_total[rec_total['natureza_receita'].str.contains("11145", case=False, regex=True)]
imposto_msc = pd.concat([imposto_msc_1, imposto_msc_2, imposto_msc_3], axis=0)
imposto_msc['dimensao'] = 'D4_00021_Rec.Impostos'
imposto_msc = imposto_msc.groupby('dimensao').agg({'valor': 'sum'})
imposto_msc = imposto_msc.reset_index()

###################  >> D4_00021_Receita de Impostos RREO
# (apresenta uma diferença que está lá na DCA ref. a Outras Deduções da Receita)
icms_rreo = df_rreo_3[df_rreo_3["conta"].str.contains("ICMS")]
ipva_rreo = df_rreo_3[df_rreo_3["conta"].str.contains("IPVA")]
itcd_rreo = df_rreo_3[df_rreo_3["conta"].str.contains("ITCD")]
irrf_rreo = df_rreo_3[df_rreo_3["conta"].str.contains("IRRF")]
imposto_rreo = pd.concat([icms_rreo, ipva_rreo, itcd_rreo, irrf_rreo])
imposto_rreo = imposto_rreo.query('coluna == "TOTAL (ÚLTIMOS 12 MESES)"')
imposto_rreo['dimensao'] = 'D4_00021_Rec.Impostos'
imposto_rreo = imposto_rreo.groupby('dimensao').agg({'valor': 'sum'})
imposto_rreo = imposto_rreo.reset_index()

## MERGE Receita de Impostos
d4_00021 = imposto_msc.merge(imposto_rreo, on='dimensao')
d4_00021['DIF'] = d4_00021['valor_x'] - d4_00021['valor_y']
d4_00021_t = d4_00021.filter(items=['dimensao', 'valor_x', 'valor_y', 'DIF'])
d4_00021_t.columns = ['Dimensão', 'MSC', 'RREO', 'DIF']

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d4_00021_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d4_00021 = 'ERRO'
else:
    resposta_d4_00021 = 'OK'

d4_00021 = pd.DataFrame([resposta_d4_00021], columns=['Resposta'])
# Insere novas colunas
d4_00021.insert(0, 'Dimensão', 'D4_00021')
d4_00021.insert(2, 'Descrição da Dimensão', 'Igualdade nas receitas com tributos estaduais')
d4_00021.insert(3, 'OBS', 'MSC de dezembro e no Anexo 03 do RREO')
d4_00021

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00021,OK,Igualdade nas receitas com tributos estaduais,MSC de dezembro e no Anexo 03 do RREO


In [ ]:
d4_00021_t

,Dimensão,MSC,RREO,DIF
0,D4_00021_Rec.Impostos,"71,276,657,196.75","71,276,657,196.75",0.00


In [ ]:
## D4_00023
rec_total = msc_dez.query('tipo_valor == "ending_balance" and conta_contabil == "621200000"')
#fpe_msc = rec_total[rec_total["natureza_receita"].str.contains("1718011")] # Em 2021
fpe_msc = rec_total[rec_total["natureza_receita"].str.contains("17115001")] # Em 2022
#fundeb_msc = rec_total[rec_total["natureza_receita"].str.contains("17580111")] # Em 2021
fundeb_msc = rec_total[rec_total["natureza_receita"].str.contains("17515001")] # Em 2022
# + COMPLEMENTO FUNDEB 17155101
complemento_fundeb_msc = rec_total[rec_total["natureza_receita"].str.contains("1715")] # Em 2022

transf = pd.concat([fpe_msc, fundeb_msc, complemento_fundeb_msc])
transf['dimensao'] = 'D4_00023_Transf.Const'
transf = transf.groupby('dimensao').agg({'valor': 'sum'})
transf = transf.reset_index()

###################  >> D4_00023_Transferências Constitucionais RREO
fpe_rreo = df_rreo_3[df_rreo_3["conta"].str.contains("Cota-Parte do FPE")]
fundeb_rreo = df_rreo_3[df_rreo_3["conta"].str.contains("Transferências do FUNDEB")]
transf_rreo = pd.concat([fpe_rreo, fundeb_rreo])
transf_rreo = transf_rreo.query('coluna == "TOTAL (ÚLTIMOS 12 MESES)"')
transf_rreo['dimensao'] = 'D4_00023_Transf.Const'
transf_rreo = transf_rreo.groupby('dimensao').agg({'valor': 'sum'})
transf_rreo = transf_rreo.reset_index()

## MERGE Transferências Constitucionais
d4_00023 = transf.merge(transf_rreo, on='dimensao')
d4_00023['DIF'] = d4_00023['valor_x'] - d4_00023['valor_y']
d4_00023_t = d4_00023.filter(items=['dimensao', 'valor_x', 'valor_y', 'DIF'])
d4_00023_t.columns = ['Dimensão', 'MSC', 'RREO', 'DIF']

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d4_00023_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d4_00023 = 'ERRO'
else:
    resposta_d4_00023 = 'OK'

d4_00023 = pd.DataFrame([resposta_d4_00023], columns=['Resposta'])
# Insere novas colunas
d4_00023.insert(0, 'Dimensão', 'D4_00023')
d4_00023.insert(2, 'Descrição da Dimensão', 'Igualdade nas receitas estaduais com transferências constitucionais')
d4_00023.insert(3, 'OBS', 'MSC de dezembro e no Anexo 03 do RREO')
d4_00023

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00023,OK,Igualdade nas receitas estaduais com transferências constitucionais,MSC de dezembro e no Anexo 03 do RREO


In [ ]:
d4_00023_t

,Dimensão,MSC,RREO,DIF
0,D4_00023_Transf.Const,"7,678,621,514.36","7,678,621,514.36",-0.00


In [ ]:
## D4_00025
###################  >> D4_00025_DPS Empenhada, Liquidada e Paga MSC
emp_msc = msc_dez.query('tipo_valor == "ending_balance" and (conta_contabil == "622130500" or conta_contabil == "622130600" or conta_contabil == "622130700" or conta_contabil == "622130400")')
emp_msc['dimensao'] = 'D4_00025_Empenhado'
emp_msc = emp_msc.groupby('dimensao').agg({'valor': 'sum'})
emp_msc = emp_msc.reset_index()

liq_msc = msc_dez.query('tipo_valor == "ending_balance" and (conta_contabil == "622130700" or conta_contabil == "622130400")')
liq_msc['dimensao'] = 'D4_00025_Liquidado'
liq_msc = liq_msc.groupby('dimensao').agg({'valor': 'sum'})
liq_msc = liq_msc.reset_index()

pago_msc = msc_dez.query('tipo_valor == "ending_balance" and conta_contabil == "622130400"')
pago_msc['dimensao'] = 'D4_00025_Pago'
pago_msc = pago_msc.groupby('dimensao').agg({'valor': 'sum'})
pago_msc = pago_msc.reset_index()


###################  >> D4_00025_Empenhado, Liquidado e Pago RREO
emp_rreo = df_rreo_1.query('coluna == "DESPESAS EMPENHADAS ATÉ O BIMESTRE (f)" & cod_conta == "TotalDespesas"')
emp_rreo['dimensao'] = 'D4_00025_Empenhado'
emp_rreo = emp_rreo.filter(items=['dimensao', 'valor'])
emp_rreo = emp_rreo.set_index("dimensao")
emp_rreo = emp_rreo.reset_index()

liq_rreo = df_rreo_1.query('coluna == "DESPESAS LIQUIDADAS ATÉ O BIMESTRE (h)" & cod_conta == "TotalDespesas"')
liq_rreo['dimensao'] = 'D4_00025_Liquidado'
liq_rreo = liq_rreo.filter(items=['dimensao', 'valor'])
liq_rreo = liq_rreo.set_index("dimensao")
liq_rreo = liq_rreo.reset_index()

pago_rreo = df_rreo_1.query('coluna == "DESPESAS PAGAS ATÉ O BIMESTRE (j)" & cod_conta == "TotalDespesas"')
pago_rreo['dimensao'] = 'D4_00025_Pago'
pago_rreo = pago_rreo.filter(items=['dimensao', 'valor'])
pago_rreo = pago_rreo.set_index("dimensao")
pago_rreo = pago_rreo.reset_index()

## MERGE Empenhos
d4_00025_emp = emp_msc.merge(emp_rreo, on='dimensao')
d4_00025_emp['DIF'] = d4_00025_emp['valor_x'] - d4_00025_emp['valor_y']
d4_00025_emp = d4_00025_emp.filter(items=['dimensao', 'valor_x', 'valor_y', 'DIF'])
d4_00025_emp.columns = ['Dimensão', 'MSC', 'RREO', 'DIF']

## MERGE Liquidados
d4_00025_liq = liq_msc.merge(liq_rreo, on='dimensao')
d4_00025_liq['DIF'] = d4_00025_liq['valor_x'] - d4_00025_liq['valor_y']
d4_00025_liq = d4_00025_liq.filter(items=['dimensao', 'valor_x', 'valor_y', 'DIF'])
d4_00025_liq.columns = ['Dimensão', 'MSC', 'RREO', 'DIF']

## MERGE Pagos
d4_00025_pago = pago_msc.merge(pago_rreo, on='dimensao')
d4_00025_pago['DIF'] = d4_00025_pago['valor_x'] - d4_00025_pago['valor_y']
d4_00025_pago = d4_00025_pago.filter(items=['dimensao', 'valor_x', 'valor_y', 'DIF'])
d4_00025_pago.columns = ['Dimensão', 'MSC', 'RREO', 'DIF']

d4_00025_t = pd.concat([d4_00025_emp, d4_00025_liq, d4_00025_pago])

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d4_00025_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d4_00025 = 'ERRO'
else:
    resposta_d4_00025 = 'OK'

d4_00025 = pd.DataFrame([resposta_d4_00025], columns=['Resposta'])
# Insere novas colunas
d4_00025.insert(0, 'Dimensão', 'D4_00025')
d4_00025.insert(2, 'Descrição da Dimensão', 'Igualdade das Despesas Orçamentárias empenhadas, liquidadas e pagas')
d4_00025.insert(3, 'OBS', 'MSC de dezembro e no Anexo 01 do RREO')
d4_00025

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00025,OK,"Igualdade das Despesas Orçamentárias empenhadas, liquidadas e pagas",MSC de dezembro e no Anexo 01 do RREO


In [ ]:
d4_00025_t

,Dimensão,MSC,RREO,DIF
0,D4_00025_Empenhado,"108,526,792,476.10","108,526,792,476.10",0.00
0,D4_00025_Liquidado,"107,262,617,813.00","107,262,617,813.00",0.00
0,D4_00025_Pago,"104,823,674,009.28","104,823,674,009.28",0.00


# D4_00026

In [ ]:
## D4_00026
###################  >> D4_00026_RPNP MSC
rpnp_msc = msc_dez.query('tipo_valor == "ending_balance" and (conta_contabil == "622130500" or conta_contabil == "622130600")')
rpnp_msc['dimensao'] = 'D4_00026_Inscrição RPNP'
rpnp_msc = rpnp_msc.groupby('dimensao').agg({'valor': 'sum'})
rpnp_msc = rpnp_msc.reset_index()

###################  >> D4_00026_RPNP RREO
rpnp_rreo = df_rreo_1.query('coluna == "INSCRITAS EM RESTOS A PAGAR NÃO PROCESSADOS (k)" & cod_conta == "TotalDespesas"')
rpnp_rreo['dimensao'] = 'D4_00026_Inscrição RPNP'
rpnp_rreo = rpnp_rreo.filter(items=['dimensao', 'valor'])
rpnp_rreo = rpnp_rreo.set_index("dimensao")
rpnp_rreo = rpnp_rreo.reset_index()

## MERGE RPNP
d4_00026 = rpnp_msc.merge(rpnp_rreo, on='dimensao')
d4_00026['DIF'] = d4_00026['valor_x'] - d4_00026['valor_y']
d4_00026_t = d4_00026.filter(items=['dimensao', 'valor_x', 'valor_y', 'DIF'])
d4_00026_t.columns = ['Dimensão', 'MSC', 'RREO', 'DIF']

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d4_00026_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d4_00026 = 'Diferença'
else:
    resposta_d4_00026 = 'OK'

d4_00026 = pd.DataFrame([resposta_d4_00026], columns=['Resposta'])
# Insere novas colunas
d4_00026.insert(0, 'Dimensão', 'D4_00026')
d4_00026.insert(2, 'Descrição da Dimensão', 'Igualdade dos Restos a Pagar não processados')
d4_00026.insert(3, 'OBS', 'MSC de dezembro e no Anexo 01 do RREO')
d4_00026

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00026,OK,Igualdade dos Restos a Pagar não processados,MSC de dezembro e no Anexo 01 do RREO


In [ ]:
d4_00026_t

,Dimensão,MSC,RREO,DIF
0,D4_00026_Inscrição RPNP,"1,264,174,663.10","1,264,174,663.10",0.00


# D4_00027 >> Compara o saldo da conta CEC da DCA com a disponibilidade de caixa bruta total do Anexo 2 do RGF. Disponibilidade de Caixa Bruta do Anexo 2 do RGF deve ser menor ou igual a 1.1.1.0.0.00.00 - Caixa e Equivalentes, do Anexo I-AB, da DCA

In [ ]:
caixa_dca = df_dca_ab.query('cod_conta == "P1.1.1.0.0.00.00"')
caixa_dca = caixa_dca[['cod_conta', 'valor']]

filtro = df_rgf_2e['cod_conta'].str.contains('DisponibilidadeDeCaixaBruta', case=False, na=False)
caixa_rgf2 = df_rgf_2e[filtro]
caixa_rgf2 = caixa_rgf2.query('coluna == "Até o 3º Quadrimestre"')
caixa_rgf2 = caixa_rgf2[['cod_conta', 'valor']]

d4_00027_t = pd.concat([caixa_dca, caixa_rgf2]).reset_index(drop=True)
d4_00027_t["DIF"] = d4_00027_t['valor'].diff()
d4_00027_t.loc[0, 'DIF'] = 0

# Substituindo os valores da coluna 'cod_conta' nas linhas de índice 0 e 1
novo_valor_linha_0 = "CEC_DCA-AB"
novo_valor_linha_1 = "CEC_RGF-2"
d4_00027_t.loc[0, 'cod_conta'] = novo_valor_linha_0
d4_00027_t.loc[1, 'cod_conta'] = novo_valor_linha_1


if (d4_00027_t['DIF'] > 0).any():
    resposta_d4_00027 = 'ERRO'  # Caso o valor seja positivo (CEC do RGF-2 maior que o DCA-AB)
else:
    resposta_d4_00027 = 'OK'    # Caso o valor seja menor ou igual a zero

# Criando o DataFrame final com a resposta
d4_00027 = pd.DataFrame([resposta_d4_00027], columns=['Resposta'])

# Adicionando mais colunas conforme o solicitado
d4_00027.insert(0, 'Dimensão', 'D4_00027')
d4_00027.insert(2, 'Descrição da Dimensão', 'Disponibilidade de Caixa Bruta do Anexo 2 do RGF deve ser menor ou igual a Caixa e Equivalentes (111), do Anexo I-AB, da DCA')
d4_00027.insert(3, 'OBS', 'RGF 2 e DCA AB')
d4_00027

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00027,OK,"Disponibilidade de Caixa Bruta do Anexo 2 do RGF deve ser menor ou igual a Caixa e Equivalentes (111), do Anexo I-AB, da DCA",RGF 2 e DCA AB


In [ ]:
d4_00027_t

,cod_conta,valor,DIF
0,CEC_DCA-AB,"36,848,737,443.71",0.00
1,CEC_RGF-2,"28,980,093,542.07","-7,868,643,901.64"


# D4_00028 >> Compara o saldo da conta CEC da DCA com a disponibilidade de caixa bruta total do Anexo 5 do RGF. Disponibilidade de Caixa Bruta do Anexo 5 do RGF deve ser menor ou igual a 1.1.1.0.0.00.00 - Caixa e Equivalentes do Anexo I-AB, da DCA.

In [ ]:


filtro = rgf_total['cod_conta'].str.contains('DisponibilidadeDeCaixaBruta', case=False, na=False)
caixa_rgf5_t = rgf_total[filtro]
caixa_rgf5_o = caixa_rgf5_t.query('conta == "TOTAL (III) = (I + II)"')
caixa_rgf5_o = caixa_rgf5_o[['cod_conta', 'valor']]
caixa_rgf5_e = caixa_rgf5_t.query('conta == "TOTAL (IV) = (I + II + III)"')
caixa_rgf5_e = caixa_rgf5_e[['cod_conta', 'valor']]
caixa_rgf_5 = pd.concat([caixa_rgf5_e, caixa_rgf5_o]).reset_index(drop=True)

# Calcular o total da coluna 'valor'
total_valor = caixa_rgf_5['valor'].sum()
# Criar a nova linha com o total
nova_linha = pd.DataFrame([{'cod_conta': 'TOTAL', 'valor': total_valor}])
# Concatenar a nova linha ao DataFrame original
caixa_rgf_5 = pd.concat([caixa_rgf_5, nova_linha], ignore_index=True)

caixa_rgf_5 = caixa_rgf_5.drop(caixa_rgf_5.index[:-1]) # deixando só a última linha

d4_00028_t = pd.concat([caixa_dca, caixa_rgf_5]).reset_index(drop=True)
d4_00028_t["DIF"] = d4_00028_t['valor'].diff()
d4_00028_t.loc[0, 'DIF'] = 0

# Substituindo os valores da coluna 'cod_conta' nas linhas de índice 0 e 1
novo_valor_linha_0 = "CEC_DCA-AB"
novo_valor_linha_1 = "CEC_RGF-5"
d4_00028_t.loc[0, 'cod_conta'] = novo_valor_linha_0
d4_00028_t.loc[1, 'cod_conta'] = novo_valor_linha_1

if (d4_00028_t['DIF'] > 0).any():
    resposta_d4_00028 = 'ERRO'  # Caso o valor seja positivo (CEC do RGF-2 maior que o DCA-AB)
else:
    resposta_d4_00028 = 'OK'    # Caso o valor seja menor ou igual a zero

# Criando o DataFrame final com a resposta
d4_00028 = pd.DataFrame([resposta_d4_00028], columns=['Resposta'])

# Adicionando mais colunas conforme o solicitado
d4_00028.insert(0, 'Dimensão', 'D4_00028')
d4_00028.insert(2, 'Descrição da Dimensão', 'Disponibilidade de Caixa Bruta do Anexo 5 do RGF deve ser menor ou igual a Caixa e Equivalentes (111), do Anexo I-AB, da DCA')
d4_00028.insert(3, 'OBS', 'RGF 5 e DCA AB')
d4_00028


,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00028,ERRO,"Disponibilidade de Caixa Bruta do Anexo 5 do RGF deve ser menor ou igual a Caixa e Equivalentes (111), do Anexo I-AB, da DCA",RGF 5 e DCA AB


In [ ]:
d4_00028_t

,cod_conta,valor,DIF
0,CEC_DCA-AB,"36,848,737,443.71",0.00
1,CEC_RGF-5,"41,242,651,352.03","4,393,913,908.32"


# D4_00029

In [ ]:
## D4_00029

############################## RREO 2
# D4_00029_Previdência Social
previ_rreo_2 = df_rreo_2.query('coluna == "DESPESAS EMPENHADAS ATÉ O BIMESTRE (b)" & \
                                cod_conta == "RREO2TotalDespesas" & \
                                conta == "Previdência Social"')
previ_rreo_2['dimensao'] = 'D4_00029_Previdência Social'
previ_rreo_2 = previ_rreo_2.filter(items=['dimensao', 'valor'])
previ_rreo_2 = previ_rreo_2.set_index("dimensao")

## D4_00029_Previdência Social
previ_msc = emp_msc_dez.query('funcao == "09" & DIGITO_INTRA != "91"')
previ_msc['dimensao'] = 'D4_00029_Previdência Social'
previ_msc = previ_msc.groupby('dimensao').agg({'valor': 'sum'})
previ_msc = previ_msc.reset_index()

###################  >> D4_00029_Previdência Social
d4_00029 = pd.merge(previ_rreo_2, previ_msc, on='dimensao')
d4_00029['DIF'] = d4_00029['valor_x'] - d4_00029['valor_y']
d4_00029_t = d4_00029.filter(items=['dimensao', 'valor_x', 'valor_y', 'DIF'])
d4_00029_t.columns = ['Dimensão', 'RREO 2', 'MSC', 'DIF']

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d4_00029_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d4_00029 = 'ERRO'
else:
    resposta_d4_00029 = 'OK'

d4_00029 = pd.DataFrame([resposta_d4_00029], columns=['Resposta'])
# Insere novas colunas
d4_00029.insert(0, 'Dimensão', 'D4_00029')
d4_00029.insert(2, 'Descrição da Dimensão', 'Avalia se o valor de despesas exceto-intra na função 09 (Prev. Social)')
d4_00029.insert(3, 'OBS', 'MSC de Dezembro e o Anexo 02 do RREO')
d4_00029

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00029,OK,Avalia se o valor de despesas exceto-intra na função 09 (Prev. Social),MSC de Dezembro e o Anexo 02 do RREO


In [ ]:
d4_00029_t

,Dimensão,RREO 2,MSC,DIF
0,D4_00029_Previdência Social,"27,561,141,336.96","27,561,141,336.96",0.00


In [ ]:
## D4_00030
############################## RREO 2
# D4_00030_Saúde
saude_rreo_2 = df_rreo_2.query('coluna == "DESPESAS EMPENHADAS ATÉ O BIMESTRE (b)" & \
                                cod_conta == "RREO2TotalDespesas" & \
                                conta == "Saúde"')
saude_rreo_2['dimensao'] = 'D4_00030_Saúde'
saude_rreo_2 = saude_rreo_2.filter(items=['dimensao', 'valor'])
saude_rreo_2 = saude_rreo_2.set_index("dimensao")

## D4_00030_Saúde
saude_msc = emp_msc_dez.query('funcao == "10" & DIGITO_INTRA != "91"')
saude_msc['dimensao'] = 'D4_00030_Saúde'
saude_msc = saude_msc.groupby('dimensao').agg({'valor': 'sum'})
saude_msc = saude_msc.reset_index()

###################  >> D4_00030_Saúde
d4_00030 = pd.merge(saude_rreo_2, saude_msc, on='dimensao')
d4_00030['DIF'] = d4_00030['valor_x'] - d4_00030['valor_y']
d4_00030_t = d4_00030.filter(items=['dimensao', 'valor_x', 'valor_y', 'DIF'])
d4_00030_t.columns = ['Dimensão', 'RREO 2', 'MSC', 'DIF']

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d4_00030_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d4_00030 = 'ERRO'
else:
    resposta_d4_00030 = 'OK'

d4_00030 = pd.DataFrame([resposta_d4_00030], columns=['Resposta'])
# Insere novas colunas
d4_00030.insert(0, 'Dimensão', 'D4_00030')
d4_00030.insert(2, 'Descrição da Dimensão', 'Avalia se o valor de despesas exceto-intra na função 10 (Saúde)')
d4_00030.insert(3, 'OBS', 'MSC de Dezembro e o Anexo 02 do RREO')
d4_00030

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00030,OK,Avalia se o valor de despesas exceto-intra na função 10 (Saúde),MSC de Dezembro e o Anexo 02 do RREO


In [ ]:
d4_00030_t

,Dimensão,RREO 2,MSC,DIF
0,D4_00030_Saúde,"10,439,322,135.46","10,439,322,135.46",-0.00


In [ ]:
## D4_00031
############################## RREO 2
# D4_00031_Educação
educa_rreo_2 = df_rreo_2.query('coluna == "DESPESAS EMPENHADAS ATÉ O BIMESTRE (b)" & \
                                cod_conta == "RREO2TotalDespesas" & \
                                conta == "Educação"')
educa_rreo_2['dimensao'] = 'D4_00031_Educação'
educa_rreo_2 = educa_rreo_2.filter(items=['dimensao', 'valor'])
educa_rreo_2 = educa_rreo_2.set_index("dimensao")

## D4_00031_Educação
educa_msc = emp_msc_dez.query('funcao == "12" & DIGITO_INTRA != "91"')
educa_msc['dimensao'] = 'D4_00031_Educação'
educa_msc = educa_msc.groupby('dimensao').agg({'valor': 'sum'})
educa_msc = educa_msc.reset_index()

###################  >> D4_00031_Educação
d4_00031 = pd.merge(educa_rreo_2, educa_msc, on='dimensao')
d4_00031['DIF'] = d4_00031['valor_x'] - d4_00031['valor_y']
d4_00031_t = d4_00031.filter(items=['dimensao', 'valor_x', 'valor_y', 'DIF'])
d4_00031_t.columns = ['Dimensão', 'RREO 2', 'MSC', 'DIF']

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d4_00031_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d4_00031 = 'ERRO'
else:
    resposta_d4_00031 = 'OK'

d4_00031 = pd.DataFrame([resposta_d4_00031], columns=['Resposta'])
# Insere novas colunas
d4_00031.insert(0, 'Dimensão', 'D4_00031')
d4_00031.insert(2, 'Descrição da Dimensão', 'Avalia se o valor de despesas exceto-intra na função 12 (Educação)')
d4_00031.insert(3, 'OBS', 'MSC de Dezembro e o Anexo 02 do RREO')
d4_00031

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00031,OK,Avalia se o valor de despesas exceto-intra na função 12 (Educação),MSC de Dezembro e o Anexo 02 do RREO


In [ ]:
d4_00031_t

,Dimensão,RREO 2,MSC,DIF
0,D4_00031_Educação,"9,969,994,244.61","9,969,994,244.61",0.00


In [ ]:
## D4_00032
############################## RREO 2
# D4_00032_Demais Funções
demais_rreo_2 = df_rreo_2.query('coluna == "DESPESAS EMPENHADAS ATÉ O BIMESTRE (b)" & \
                                            cod_conta == "RREO2TotalDespesas" & \
                        (conta == "Legislativa" | conta == "Judiciária" | conta == "Essencial à Justiça" | conta == "Administração" | conta == "Segurança Pública" | \
                        conta == "Assistência Social" | conta == "Trabalho" | conta == "Cultura" | conta == "Direitos da Cidadania" | conta == "Urbanismo" | \
                        conta == "Habitação" | conta == "Saneamento" | conta == "Gestão Ambiental" | conta == "Ciência e Tecnologia" | conta == "Agricultura" | \
                        conta == "Organização Agrária" | conta == "Indústria" | conta == "Comércio e Serviços" | conta == "Comunicações" | conta == "Energia" | conta == "Transporte" | \
                        conta == "Desporto e Lazer" | conta == "Encargos Especiais")')
demais_rreo_2['dimensao'] = 'D4_00032_Demais Funções'
demais_rreo_2 = demais_rreo_2.filter(items=['dimensao', 'valor'])
demais_rreo_2 = demais_rreo_2.set_index("dimensao")
demais_rreo_2 = demais_rreo_2.groupby('dimensao').agg({'valor': 'sum'})

## D4_00032_Demais Funções
demais_msc = emp_msc_dez.query('(funcao != "09" & funcao != "10" & funcao != "12") & DIGITO_INTRA != "91"')
demais_msc['dimensao'] = 'D4_00032_Demais Funções'
demais_msc = demais_msc.groupby('dimensao').agg({'valor': 'sum'})
demais_msc = demais_msc.reset_index()

###################  >> D4_00032_Demais Funções
d4_00032 = pd.merge(demais_rreo_2, demais_msc, on='dimensao')
d4_00032['DIF'] = d4_00032['valor_x'] - d4_00032['valor_y']
d4_00032_t = d4_00032.filter(items=['dimensao', 'valor_x', 'valor_y', 'DIF'])
d4_00032_t.columns = ['Dimensão', 'RREO 2', 'MSC', 'DIF']

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d4_00032_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d4_00032 = 'ERRO'
else:
    resposta_d4_00032 = 'OK'

d4_00032 = pd.DataFrame([resposta_d4_00032], columns=['Resposta'])
# Insere novas colunas
d4_00032.insert(0, 'Dimensão', 'D4_00032')
d4_00032.insert(2, 'Descrição da Dimensão', 'Avalia se o valor de despesas exceto-intra nas funções diferentes de 09, 10 e 12')
d4_00032.insert(3, 'OBS', 'MSC de Dezembro e o Anexo 02 do RREO')
d4_00032

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00032,OK,"Avalia se o valor de despesas exceto-intra nas funções diferentes de 09, 10 e 12",MSC de Dezembro e o Anexo 02 do RREO


In [ ]:
d4_00032_t

,Dimensão,RREO 2,MSC,DIF
0,D4_00032_Demais Funções,"52,537,793,079.49","52,537,793,079.49",0.00


In [ ]:
## D4_00033
############################## RREO 2
#D4_00033_Empenhados INTRA
emp_intra_rreo_2_3 = df_rreo_2.query('coluna == "DESPESAS EMPENHADAS ATÉ O BIMESTRE (b)" & cod_conta == "RREO2TotalDespesas" & conta == "DESPESAS (INTRA-ORÇAMENTÁRIAS) (II)"')
emp_intra_rreo_2_3['dimensao'] = 'D4_00033_Empenhados INTRA'
emp_intra_rreo_2_3 = emp_intra_rreo_2_3.filter(items=['dimensao', 'valor'])
emp_intra_rreo_2_3 = emp_intra_rreo_2_3.set_index("dimensao")
emp_intra_rreo_2_3 = emp_intra_rreo_2_3.reset_index()

# D4_00033_Empenhados INTRA
emp_msc_dez_intra_3 = emp_msc_dez.query('DIGITO_INTRA == "91"')
emp_msc_dez_intra_3['dimensao'] = 'D4_00033_Empenhados INTRA'
emp_msc_dez_intra_3 = emp_msc_dez_intra_3.groupby('dimensao').agg({'valor': 'sum'})
emp_msc_dez_intra_3 = emp_msc_dez_intra_3.reset_index()

###################  >> D4_00033_INTRA
d4_00033 = pd.merge(emp_intra_rreo_2_3, emp_msc_dez_intra_3, on='dimensao')
d4_00033['DIF'] = d4_00033['valor_x'] - d4_00033['valor_y']
d4_00033_t = d4_00033.filter(items=['dimensao', 'valor_x', 'valor_y', 'DIF'])
d4_00033_t.columns = ['Dimensão', 'RREO 2', 'MSC', 'DIF']

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d4_00033_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d4_00033 = 'ERRO'
else:
    resposta_d4_00033 = 'OK'

d4_00033 = pd.DataFrame([resposta_d4_00033], columns=['Resposta'])
# Insere novas colunas
d4_00033.insert(0, 'Dimensão', 'D4_00033')
d4_00033.insert(2, 'Descrição da Dimensão', 'Avalia se o valor de despesas intraorçamentárias com detalhamento de função/subfunção')
d4_00033.insert(3, 'OBS', 'MSC de Dezembro e o Anexo 02 do RREO')
d4_00033

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00033,OK,Avalia se o valor de despesas intraorçamentárias com detalhamento de função/subfunção,MSC de Dezembro e o Anexo 02 do RREO


In [ ]:
d4_00033_t

,Dimensão,RREO 2,MSC,DIF
0,D4_00033_Empenhados INTRA,"8,018,541,679.58","8,018,541,679.58",0.00


In [ ]:
# D4_00034
###################  >> D4_00034_RPNP e RPP Pago
rpnp_pago_msc = msc_dez.query('tipo_valor == "ending_balance" and conta_contabil == "631400000"')
rpnp_pago_msc['dimensao'] = 'D4_00034_RPNP Pago'
rpnp_pago_msc = rpnp_pago_msc.groupby('dimensao').agg({'valor': 'sum'})
rpnp_pago_msc = rpnp_pago_msc.reset_index()

rpnp_pago_rreo = df_rreo_7.query('conta == "TOTAL (III) = (I + II)" & cod_conta == "RestosAPagarNaoProcessadosPagos"')
rpnp_pago_rreo['dimensao'] = 'D4_00034_RPNP Pago'
rpnp_pago_rreo = rpnp_pago_rreo.filter(items=['dimensao', 'valor'])
rpnp_pago_rreo = rpnp_pago_rreo.set_index("dimensao")
rpnp_pago_rreo = rpnp_pago_rreo.reset_index()

rpp_pago_msc = msc_dez.query('tipo_valor == "ending_balance" and conta_contabil == "632200000"')
rpp_pago_msc['dimensao'] = 'D4_00034_RPP Pago'
rpp_pago_msc = rpp_pago_msc.groupby('dimensao').agg({'valor': 'sum'})
rpp_pago_msc = rpp_pago_msc.reset_index()

rpp_pago_rreo = df_rreo_7.query('conta == "TOTAL (III) = (I + II)" & cod_conta == "RestosAPagarProcessadosENaoProcessadosLiquidadosPagos"')
rpp_pago_rreo['dimensao'] = 'D4_00034_RPNP Pago'
rpp_pago_rreo = rpp_pago_rreo.filter(items=['dimensao', 'valor'])
rpp_pago_rreo = rpp_pago_rreo.set_index("dimensao")
rpp_pago_rreo = rpp_pago_rreo.reset_index()

d4_00034_t = pd.concat([rpnp_pago_msc, rpnp_pago_rreo, rpp_pago_msc, rpp_pago_rreo])

# Calcular a diferença entre os valores
d4_00034_t['DIF'] = d4_00034_t['valor'].diff()

# Convertendo as colunas para tipo numérico e tratando NaNs
d4_00034_t['DIF'] = pd.to_numeric(d4_00034_t['DIF'], errors='coerce').fillna(0)

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d4_00034_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d4_00034 = 'OK'
else:
    resposta_d4_00034 = 'ERRO'


d4_00034_t = d4_00034_t.reset_index(drop=True)
d4_00034_t.loc[0, 'DIF'] = 0
d4_00034_t.loc[2, 'DIF'] = 0

# Criar DataFrame final
d4_00034 = pd.DataFrame([resposta_d4_00034], columns=['Resposta'])
# Insere novas colunas
d4_00034.insert(0, 'Dimensão', 'D4_00034')
d4_00034.insert(2, 'Descrição da Dimensão', 'Avalia a igualdade entre os saldos finais de RPP pagos e RPNP pagos')
d4_00034.insert(3, 'OBS', 'MSC de Dezembro e o Anexo 07 do RREO')

d4_00034

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00034,OK,Avalia a igualdade entre os saldos finais de RPP pagos e RPNP pagos,MSC de Dezembro e o Anexo 07 do RREO


In [ ]:
d4_00034_t

,dimensao,valor,DIF
0,D4_00034_RPNP Pago,"660,505,752.08",0.00
1,D4_00034_RPNP Pago,"660,505,752.08",0.00
2,D4_00034_RPP Pago,"1,492,300,289.28",0.00
3,D4_00034_RPNP Pago,"1,492,300,289.28",0.00


# D4_00035

In [ ]:
msc_encerr_b = msc_encerr.query('tipo_valor == "beginning_balance"')
caixa_msc_encerr = msc_encerr_b[msc_encerr_b['conta_contabil'].str.startswith('111')]
caixa_msc_encerr = caixa_msc_encerr[['conta_contabil', 'valor']] 
caixa_msc_encerr = caixa_msc_encerr.rename(columns={'conta_contabil': 'cod_conta'})

# Calcular o total da coluna 'valor'
total_valor = caixa_msc_encerr['valor'].sum()
# Criar a nova linha com o total
nova_linha = pd.DataFrame([{'cod_conta': 'TOTAL', 'valor': total_valor}])
# Concatenar a nova linha ao DataFrame original
caixa_msc_encerr = pd.concat([caixa_msc_encerr, nova_linha], ignore_index=True)

caixa_msc_encerr = caixa_msc_encerr.drop(caixa_msc_encerr.index[:-1]) # retirando as primeiras linhas

d4_00035_t = pd.concat([caixa_msc_encerr, caixa_rgf_5]).reset_index(drop=True)
d4_00035_t["DIF"] = d4_00035_t['valor'].diff()
d4_00035_t.loc[0, 'DIF'] = 0

# Substituindo os valores da coluna 'cod_conta' nas linhas de índice 0 e 1
novo_valor_linha_0 = "CEC_MSC-ENCERR"
novo_valor_linha_1 = "CEC_RGF-5"
d4_00035_t.loc[0, 'cod_conta'] = novo_valor_linha_0
d4_00035_t.loc[1, 'cod_conta'] = novo_valor_linha_1


if (d4_00035_t['DIF'] > 0).any():
    resposta_d4_00035 = 'ERRO'  # Caso o valor seja positivo (CEC do RGF-2 maior que o DCA-AB)
else:
    resposta_d4_00035 = 'OK'    # Caso o valor seja menor ou igual a zero

# Criando o DataFrame final com a resposta
d4_00035 = pd.DataFrame([resposta_d4_00035], columns=['Resposta'])

# Adicionando mais colunas conforme o solicitado
d4_00035.insert(0, 'Dimensão', 'D4_00035')
d4_00035.insert(2, 'Descrição da Dimensão', 'Disponibilidade de Caixa Bruta do Anexo 5 do RGF deve ser menor ou igual a Caixa e Equivalentes (111), da Matriz de Encerramrento')
d4_00035.insert(3, 'OBS', 'RGF 5 e MSC ENCERR')
d4_00035

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00035,ERRO,"Disponibilidade de Caixa Bruta do Anexo 5 do RGF deve ser menor ou igual a Caixa e Equivalentes (111), da Matriz de Encerramrento",RGF 5 e MSC ENCERR


In [ ]:
d4_00035_t

,cod_conta,valor,DIF
0,CEC_MSC-ENCERR,"37,838,417,102.57",0.00
1,CEC_RGF-5,"41,242,651,352.03","3,404,234,249.46"


# D4_00036

In [ ]:
d4_00036_t = pd.concat([caixa_msc_encerr, caixa_rgf2]).reset_index(drop=True)
d4_00036_t["DIF"] = d4_00036_t['valor'].diff()
d4_00036_t.loc[0, 'DIF'] = 0

# Substituindo os valores da coluna 'cod_conta' nas linhas de índice 0 e 1
novo_valor_linha_0 = "CEC_MSC-ENCERR"
novo_valor_linha_1 = "CEC_RGF-2"
d4_00036_t.loc[0, 'cod_conta'] = novo_valor_linha_0
d4_00036_t.loc[1, 'cod_conta'] = novo_valor_linha_1


if (d4_00036_t['DIF'] > 0).any():
    resposta_d4_00036 = 'ERRO'  # Caso o valor seja positivo (CEC do RGF-2 maior que o DCA-AB)
else:
    resposta_d4_00036 = 'OK'    # Caso o valor seja menor ou igual a zero

# Criando o DataFrame final com a resposta
d4_00036 = pd.DataFrame([resposta_d4_00036], columns=['Resposta'])

# Adicionando mais colunas conforme o solicitado
d4_00036.insert(0, 'Dimensão', 'D4_00036')
d4_00036.insert(2, 'Descrição da Dimensão', 'Disponibilidade de Caixa Bruta do Anexo 2 do RGF deve ser menor ou igual a Caixa e Equivalentes (111), da Matriz de Encerramrento')
d4_00036.insert(3, 'OBS', 'RGF 2 e MSC ENCERR')
d4_00036

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00036,OK,"Disponibilidade de Caixa Bruta do Anexo 2 do RGF deve ser menor ou igual a Caixa e Equivalentes (111), da Matriz de Encerramrento",RGF 2 e MSC ENCERR


In [ ]:
d4_00036_t

,cod_conta,valor,DIF
0,CEC_MSC-ENCERR,"37,838,417,102.57",0.00
1,CEC_RGF-2,"28,980,093,542.07","-8,858,323,560.50"


<hr style="border: none; border-top: 4px solid gray;">
<p style="color: lightgreen; font-size: 26px;"><strong>NOVAS 2023</strong></p>
<hr style="border: none; border-top: 4px solid gray;">


In [ ]:
## D4_00037
###################  >> D4_00009_Receita de Impostos RREO 6
icms_rreo_6 = df_rreo_6[df_rreo_6["conta"].str.contains("ICMS")]
ipva_rreo_6 = df_rreo_6[df_rreo_6["conta"].str.contains("IPVA")]
itcd_rreo_6 = df_rreo_6[df_rreo_6["conta"].str.contains("ITCD")]
irrf_rreo_6 = df_rreo_6[df_rreo_6["conta"].str.contains("IRRF")]
imposto_rreo_6 = pd.concat([icms_rreo_6, ipva_rreo_6, itcd_rreo_6, irrf_rreo_6])
imposto_rreo_6 = imposto_rreo_6.query('coluna == "RECEITAS REALIZADAS (a)"')
imposto_rreo_6['dimensao'] = 'D4_00037_Rec.Impostos'
imposto_rreo_6 = imposto_rreo_6.groupby('dimensao').agg({'valor': 'sum'})
imposto_rreo_6 = imposto_rreo_6.reset_index()

###################  >> D4_00009_Receita de Impostos MSC
rec_realizada_imposto_msc = receita[receita['natureza_receita'].str.match(r"^(111450|111251|111252|111303)")]
rec_realizada_imposto_msc['dimensao'] = 'D4_00037_Rec.Impostos'
rec_realizada_imposto_msc = rec_realizada_imposto_msc.groupby(['dimensao'])['valor'].sum().reset_index()

## MERGE Receita de Impostos
d4_00037_t = imposto_rreo_6.merge(rec_realizada_imposto_msc, on='dimensao')
d4_00037_t['DIF'] = d4_00037_t['valor_x'] - d4_00037_t['valor_y']
d4_00037_t = d4_00037_t.filter(items=['dimensao', 'valor_x', 'valor_y', 'DIF'])
d4_00037_t.columns = ['Dimensão', 'RREO', 'MSC', 'DIF']

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d4_00037_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d4_00037 = 'ERRO'
else:
    resposta_d4_00037 = 'OK'

d4_00037 = pd.DataFrame([resposta_d4_00037], columns=['Resposta'])
# Insere novas colunas
d4_00037.insert(0, 'Dimensão', 'D4_00037')
d4_00037.insert(2, 'Descrição da Dimensão', 'Igualdade das receitas com tributos estaduais')
d4_00037.insert(3, 'OBS', 'MSC Dezembro e o Anexo 6 do RREO')
d4_00037

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00037,OK,Igualdade das receitas com tributos estaduais,MSC Dezembro e o Anexo 6 do RREO


In [ ]:
d4_00037_t

,Dimensão,RREO,MSC,DIF
0,D4_00037_Rec.Impostos,"46,296,339,460.89","46,296,339,460.89",0.00


In [ ]:
## D4_00039
###################  >> D4_00011_Transferências Constitucionais RREO 2
fpe_rreo_6 = df_rreo_6[df_rreo_6["conta"].str.contains("Cota-Parte do FPE")]
fundeb_rreo_6 = df_rreo_6[df_rreo_6["conta"].str.contains("Transferências do FUNDEB")]
transf_rreo_6 = pd.concat([fpe_rreo_6, fundeb_rreo_6])
transf_rreo_6 = transf_rreo_6.query('coluna == "RECEITAS REALIZADAS (a)"')
transf_rreo_6['dimensao'] = 'D4_00039_Transf.Const'
transf_rreo_6 = transf_rreo_6.groupby('dimensao').agg({'valor': 'sum'})
transf_rreo_6 = transf_rreo_6.reset_index()

# MSC
# Cota-Parte do FPE          >> NR: 1.7.1.1.50.0.0 + NR: 7.7.1.1.50.0.0
# Transferências do FUNDEB   >> NR: 1.7.1.5.00.0.0 + NR: 1.7.5.1.00.0.0 + NR: 7.7.1.5.00.0.0 + NR: 7.7.5.1.00.0.0
transf_msc = receita[receita['natureza_receita'].str.match(r"^(171150|771150|1715|1751|7715|7751)")]
transf_msc['dimensao'] = 'D4_00039_Transf.Const'
transf_msc = transf_msc.groupby(['dimensao'])['valor'].sum().reset_index()

## MERGE Transferências Constitucionais
d4_00039_t = transf_rreo_6.merge(transf_msc, on='dimensao')
d4_00039_t['DIF'] = d4_00039_t['valor_x'] - d4_00039_t['valor_y']
d4_00039_t = d4_00039_t.filter(items=['dimensao', 'valor_x', 'valor_y', 'DIF'])
d4_00039_t.columns = ['Dimensão', 'RREO', 'DCA', 'DIF']

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao = ~np.isclose(d4_00039_t['DIF'], 0, atol=tolerancia)

if condicao.any():
    resposta_d4_00039 = 'ERRO'
else:
    resposta_d4_00039 = 'OK'

d4_00039 = pd.DataFrame([resposta_d4_00039], columns=['Resposta'])
# Insere novas colunas
d4_00039.insert(0, 'Dimensão', 'D4_00039')
d4_00039.insert(2, 'Descrição da Dimensão', 'Avalia a igualdade nas transferências constitucionais (Cota-Parte do FPE e Transferências do FUNDEB)')
d4_00039.insert(3, 'OBS', 'MSC de Dezembro e o Anexo 06 do RREO')
d4_00039

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00039,OK,Avalia a igualdade nas transferências constitucionais (Cota-Parte do FPE e Transferências do FUNDEB),MSC de Dezembro e o Anexo 06 do RREO


In [ ]:
d4_00039_t

,Dimensão,RREO,DCA,DIF
0,D4_00039_Transf.Const,"7,018,798,328.68","7,018,798,328.68",0.00


# Consolidando

In [ ]:
d4 = pd.concat([d4_00001, d4_00002, d4_00003, d4_00004, d4_00005, d4_00006, d4_00007, d4_00009, d4_00011,
                d4_00017, d4_00019, d4_00020, d4_00021, d4_00023, d4_00025, d4_00026, d4_00027, d4_00028,
                d4_00029, d4_00030, d4_00031, d4_00032, d4_00033, d4_00034, d4_00035, d4_00036, d4_00037, d4_00039
                ]).reset_index(drop=True)

d4

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D4_00001,OK,Igualdade da receita realizada,Anexo I-C da DCA e o Anexo 01 do RREO 6ºB
1,D4_00002,OK,Igualdade da execução da despesa,Anexo I-D da DCA e o Anexo 01 do RREO 6ºB
2,D4_00003,OK,Igualdade da execução da despesa por função,Anexo I-E da DCA e o Anexo 02 do RREO 6ºB (exceto intraorçamentária)
3,D4_00004,OK,Igualdade da execução da despesa por função,Anexo I-E da DCA e o Anexo 02 do RREO 6ºB (intraorçamentária)
4,D4_00005,OK,Igualdade dos restos a pagar não processados e processados,Anexo I-F da DCA e o Anexo 07 do RREO
5,D4_00006,OK,Igualdade dos restos a pagar não processados,Anexo I-G da DCA e o Anexo 07 do RREO
6,D4_00007,OK,Igualdade dos restos a pagar processados,Anexo I-G da DCA e o Anexo 07 do RREO
7,D4_00009,OK,Igualdade das receitas com tributos estaduais,Anexo I-C da DCA e o Anexo 11 do RREO
8,D4_00011,OK,Igualdade das transferências estaduais,Anexo I-C da DCA e o Anexo 03 do RREO
9,D4_00017,OK,Igualdade das contrib. dos servidores e compensações financeiras,Anexo I-C da DCA e o Anexo 03 do RREO


In [ ]:
d4.query('Resposta == "ERRO"')

,Dimensão,Resposta,Descrição da Dimensão,OBS
17,D4_00028,ERRO,"Disponibilidade de Caixa Bruta do Anexo 5 do RGF deve ser menor ou igual a Caixa e Equivalentes (111), do Anexo I-AB, da DCA",RGF 5 e DCA AB
24,D4_00035,ERRO,"Disponibilidade de Caixa Bruta do Anexo 5 do RGF deve ser menor ou igual a Caixa e Equivalentes (111), da Matriz de Encerramrento",RGF 5 e MSC ENCERR


In [ ]:
contagem_total_dim = d4.Dimensão.unique()
contagem_total_dim = len(contagem_total_dim)
contagem_total_dim

28

In [ ]:
total_dim = 29
dim_falta = total_dim - contagem_total_dim
dim_falta

1

# FINAL

In [ ]:
final = pd.concat([d1, d2, d3, d4])
final

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00017,OK,Verifica a quantidade de matrizes com valores negativos,Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0
1,D1_00018,OK,Verifica a quantidade de matrizes com diferenças na movimentação (SI+ MOV <> SF),Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0
2,D1_00019,OK,Verifica a quantidade de matrizes com códigos de Poderes incorretos,Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0
3,D1_00020,OK,Verifica a quantidade de matrizes com SI <> do SF da MSC do mês anterior,Cada MSC vale 1/12 - Erros: 0 - Pontos: 1.0
4,D1_00021,OK,Verifica a quantidade de matrizes com contas dos grupos de ATIVO com natureza diferente do PCASP,Cada MSC vale 1/13 - Erros: 0 - Pontos: 1.0
...,...,...,...,...
23,D4_00034,OK,Avalia a igualdade entre os saldos finais de RPP pagos e RPNP pagos,MSC de Dezembro e o Anexo 07 do RREO
24,D4_00035,ERRO,"Disponibilidade de Caixa Bruta do Anexo 5 do RGF deve ser menor ou igual a Caixa e Equivalentes (111), da Matriz de Encerramrento",RGF 5 e MSC ENCERR
25,D4_00036,OK,"Disponibilidade de Caixa Bruta do Anexo 2 do RGF deve ser menor ou igual a Caixa e Equivalentes (111), da Matriz de Encerramrento",RGF 2 e MSC ENCERR
26,D4_00037,OK,Igualdade das receitas com tributos estaduais,MSC Dezembro e o Anexo 6 do RREO


In [ ]:
final.query('Resposta == "ERRO"')

,Dimensão,Resposta,Descrição da Dimensão,OBS
8,D1_00025,ERRO,Verifica a quantidade de matrizes com contas dos grupos de PASSIVO com natureza diferente do PCASP,Cada MSC vale 1/13 - Erros: 1 - Pontos: 0.92
21,D1_00038,ERRO,Verifica a quantidade de matrizes com contas de classe 5 e 6 cujo saldo final está com natureza diferente da padrão do PCASP Estendido,Cada MSC vale 1/13 da pontuação - Erros: 13 - Pontos: 0.0
63,D2_00084,ERRO,Avalia se o total de passivo financeiro + permanente na DCA é maior ou igual ao total de ativo circulante + passivo circulante na MSC de dezembro,Anexo I-AB da DCA x MSC DEZ
65,D2_00090,ERRO,"Verifica a relação entre o valor de ajuste para perdas dos ""Demais créditos e valores a curto e longo prazo"" e suas contas principais.",Anexo I-AB da DCA
67,D2_00098,ERRO,Verificar se o valor total transferido pela União por meio de emendas parlamentares de bancada é maior ou igual ao valor total de receitas decorrentes dessas emendas registrados pelo ente.,MSC DEZ X Relatório de Transferências da União
11,D3_00017,ERRO,Igualdade entre os Restos a Pagar (Processados e Não-Processados) Pagos no Exercício,Anexo 6 do RREO e Anexo 7 do RREO
17,D3_00027,ERRO,"Verifica a igualdade entre Dotação atualizada, Despesas Empenhadas e Despesas Liquidadas",Anexos 1 e 6 do RREO
18,D3_00028,ERRO,Verifica a igualdade entre Previsão atualizada e Receitas Realizadas,Anexos 1 e 6 do RREO
17,D4_00028,ERRO,"Disponibilidade de Caixa Bruta do Anexo 5 do RGF deve ser menor ou igual a Caixa e Equivalentes (111), do Anexo I-AB, da DCA",RGF 5 e DCA AB
24,D4_00035,ERRO,"Disponibilidade de Caixa Bruta do Anexo 5 do RGF deve ser menor ou igual a Caixa e Equivalentes (111), da Matriz de Encerramrento",RGF 5 e MSC ENCERR
